### Fabio Valerio Massoli, Lucia Vadicamo, Fabrizio Falchi, Giuseppe Amato (ISTI-CNR)

Jupyter Notebook to create MarkDown files from Excel file that contains article details

In [51]:
import pandas as pd 
import numpy as np
from mdutils.mdutils import MdUtils
from mdutils import Html
from IPython.display import Markdown, display

title="A Leap among Quantum Computing and Quantum Neural Networks: A Survey"
authors="Fabio Valerio Massoli, Lucia Vadicamo, Fabrizio Falchi, Giuseppe Amato"
our_paper_url="https://arxiv.org/abs/2107.03313"
k=2 #minimum occurrece of a topic used to create the 'papersSortedByTopics' MarkDown file

# Loading  "LeapIntoQuantumComputationsPaperList.xlsx" data

In [52]:
df = pd.read_excel("LeapIntoQuantumComputationsPaperList.xlsx",sheet_name='all', usecols= 'A:N',engine='openpyxl') 
df = df.dropna(subset=['Title'])
df=df[['bibTex','URL','year','Author','Title','NameJournalConf','EntryType','Topic','Model']]
df.fillna('na', inplace=True)
surveys=df[df["EntryType"]=="survey"]
papers= df[df["EntryType"]!="survey"]
df['year']=pd.to_numeric(df['year'], downcast='signed')
print("*********************************************************")
print("**DataFrame details **")
print(df.info())
print("*********************************************************")
print(f"**last document in the list:\n{df.iloc[-1]}**")


#TOPIC LIST
allTopic=[t.strip() for  tt in df["Topic"]for t in tt.split(";") if tt!='na' and t!=' ' and t!='']
topicList_with_occurrences =list(zip(*np.unique(allTopic, return_counts=True))) #
print("*********************************************************")
print("**TOPICS with occurrences**")
for t in topicList_with_occurrences:
    print (t)
topk_topics=np.sort([t for t,count in topicList_with_occurrences if count>k])
print("*********************************************************")
print(f"**TOPICS with occurrence>{k} (total={len(topk_topics)}) **")
print(topk_topics)

*********************************************************
**DataFrame details **
<class 'pandas.core.frame.DataFrame'>
Int64Index: 184 entries, 0 to 183
Data columns (total 9 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   bibTex           184 non-null    object
 1   URL              184 non-null    object
 2   year             184 non-null    int16 
 3   Author           184 non-null    object
 4   Title            184 non-null    object
 5   NameJournalConf  184 non-null    object
 6   EntryType        184 non-null    object
 7   Topic            184 non-null    object
 8   Model            184 non-null    object
dtypes: int16(1), object(8)
memory usage: 13.3+ KB
None
*********************************************************
**last document in the list:
bibTex             @article{zoufal2019quantum,\n  title={Quantum ...
URL                https://www.nature.com/articles/s41534-019-0223-2
year                                   

# Creating MD files



In [54]:
def write_item (row,mdFile):
    topics= " ".join(['__`['+ s.strip() +']`__' for s in row['Topic'].split(';') if s!='na'] )
    model=" ".join(['_`('+ s.strip() +')`_' for s in row['Model'].split(';') if s!='na'])
    #topics= " ".join(['<code style="background-color: Gray; color: white">['+ s.strip() +']</code>' for s in row['Topic'].split(';') if s!='na'] )
    #model=" ".join(['<code style="background-color: DarkCyan; color: white">['+ s.strip() +']</code>' for s in row['Model'].split(';') if s!='na'])
    newentry=f"- [{row['Author']}] {mdFile.new_inline_link(link=row['URL'] , text='**'+row['Title'].strip()+'.**')} _{row['NameJournalConf']}_. {topics} {model}" 
    bib=f"<details><summary>Bibtex</summary><pre><code>{row['bibTex']}</code></pre></details>\n "
    mdFile.write(newentry+bib)

def write_item2 (row,file):
    prefix="<details><summary>"
    topics= " ".join(['<code><b>['+ s.strip() +']</b></code>' for s in row['Topic'].split(';') if s!='na'] )
    model=" ".join(['<code><i>('+ s.strip() +')</i></code>' for s in row['Model'].split(';') if s!='na'])
    newentry=f"[{row['Author']}] <b><a href={row['URL']}> {row['Title'].strip()}. </a></b> <i>{row['NameJournalConf']}.</i> {topics} {model}" 
    bib=f"</summary><pre><code>{row['bibTex']}</code></pre>"
    suffix="</details>\n"
    file.write(prefix+newentry+bib+suffix)
    
def writeTOC (values, file, sep='\n'):
    for y in values:
        ss="-".join(str(y).split())
        file.write(f"* [{str(y)}](#{ss}){sep}")

def writePaperByTopics (topics, data, file):
    for t in topics:
        file.new_paragraph(f"# {str(t)}\n")
        selIndex=[i  for i in data.index if t in data.loc[i]["Topic"]]        
        sel=data.loc[selIndex]
        years=np.sort(sel["year"].astype('int32').unique())[::-1]
        for y in years:
            file.new_paragraph(f"### {str(y)}\n")
            sy=sel[sel["year"]==y]
            for index, row in sy.iterrows():
                write_item2(row,file)

def writePaperByYear (years, data, file):
    for y in years:
        file.new_paragraph(f"### {str(y)}\n")
        sy=data[data["year"]==y]
        for index, row in sy.iterrows():
            write_item2(row,file)

    
################### README ############################
mdFile = MdUtils(file_name='README', title=title)

mdFile.new_paragraph("This repo reports a collection of relevant papers in the fields of Quantum Computing, Quantum Machine Learning and Quantum Deep Learning.")
mdFile.new_paragraph()

mdFile.new_paragraph(f"These articles represented the starting point for the paper "+ mdFile.new_inline_link(link=our_paper_url, text=title) +" and for anyone who is approaching such a field. ")
mdFile.write("We are continuously improving the project. Welcome to Pull Request the works (papers, repositories, topics) that are missed by the repo")

mdFile.new_paragraph('To cite our work, please use the following reference')
mdFile.new_paragraph("<pre><code>@article{massoli2021leap,\n title = {"+title+"},\n author = {"+authors+"},\n year={2021},\n eprint={2107.03313},\n archivePrefix={arXiv},\n primaryClass={quant-ph}\n }</code></pre>")
mdFile.new_paragraph()

mdFile.new_paragraph("# Table of Contents")
mdFile.new_paragraph("* [All the papers (flat view)](https://github.com/fvmassoli/survey-quantum-computations/blob/master/allPapers.md)")
mdFile.new_paragraph("* [Papers divided by topics](https://github.com/fvmassoli/survey-quantum-computations/blob/master/papersSortedByTopics.md) [WIP]")
mdFile.new_paragraph("## Contacts") 
mdFile.new_paragraph('Have you noticed a missing or misclassified article? We would greatly appreciate it if you could report it to us with a Pull Request or using the form the form at the following link:')
mdFile.new_paragraph('')
mdFile.new_paragraph('')
mdFile.new_paragraph('If you have any question about our work, please contact [Dr. Fabio Valerio Massoli](mailto:fabio.massoli@isti.cnr.it) and [Dr. Lucia Vadicamo](mailto:lucia.vadicamo@isti.cnr.it)')
mdFile.new_paragraph('Have fun! :-D')

mdFile.create_md_file()
###############################################


################### FLATVIEW ############################
mdFile = MdUtils(file_name='allPapers', title=title)
mdFile.new_paragraph("# Table of Contents\n")
mdFile.new_paragraph("* [Surveys](#Surveys)\n")
yearsSurvey=np.sort(surveys["year"].astype('int32').unique())[::-1]
mdFile.write("\t")
writeTOC(yearsSurvey,mdFile,'\t')
mdFile.new_paragraph("* [Papers](#Papers)\n")
years=np.sort(papers["year"].astype('int32').unique())[::-1]
mdFile.write("\t")
writeTOC(years,mdFile, '\t')
mdFile.new_paragraph()

#surveys
mdFile.new_paragraph("# Surveys")#mdFile.new_header(level=1, title="Surveys")
mdFile.new_paragraph()

writePaperByYear (yearsSurvey, surveys, mdFile)

#papers
mdFile.new_paragraph("# Papers")#mdFile.new_header(level=2, title="Papers")
mdFile.new_paragraph()

writePaperByYear (years, papers, mdFile)

mdFile.create_md_file()
###############################################

################### papersSortedByTopics ############################
#only topics with more than 2 papers are used
mdFile = MdUtils(file_name='papersSortedByTopics', title=title)
mdFile.new_paragraph('--Work in Progress--')
mdFile.new_paragraph("# Table of Contents\n")
writeTOC(topk_topics,mdFile)
mdFile.write("\n")
writePaperByTopics(topk_topics, df, mdFile)
mdFile.create_md_file()


In [56]:
display(Markdown("README.md"))


A Leap among Quantum Computing and Quantum Neural Networks: A Survey
====================================================================


This repo reports a collection of relevant papers in the fields of Quantum Computing, Quantum Machine Learning and Quantum Deep Learning.



These articles represented the starting point for the paper [A Leap among Quantum Computing and Quantum Neural Networks: A Survey](https://arxiv.org/abs/2107.03313) and for anyone who is approaching such a field. We are continuously improving the project. Welcome to Pull Request the works (papers, repositories, topics) that are missed by the repo

To cite our work, please use the following reference

<pre><code>@article{massoli2021leap,
 title = {A Leap among Quantum Computing and Quantum Neural Networks: A Survey},
 author = {Fabio Valerio Massoli, Lucia Vadicamo, Fabrizio Falchi, Giuseppe Amato},
 year={2021},
 eprint={2107.03313},
 archivePrefix={arXiv},
 primaryClass={quant-ph}
 }</code></pre>



# Table of Contents

* [All the papers (flat view)](https://github.com/fvmassoli/survey-quantum-computations/blob/master/allPapers.md)

* [Papers divided by topics](https://github.com/fvmassoli/survey-quantum-computations/blob/master/papersSortedByTopics.md) [WIP]

## Contacts

Have you noticed a missing or misclassified article? We would greatly appreciate it if you could report it to us with a Pull Request or using the form the form at the following link:





If you have any question about our work, please contact [Dr. Fabio Valerio Massoli](mailto:fabio.massoli@isti.cnr.it) and [Dr. Lucia Vadicamo](mailto:lucia.vadicamo@isti.cnr.it)

Have fun! :-D

In [57]:
display(Markdown(open('allPapers.md', encoding='utf-8').read()))


A Leap among Quantum Computing and Quantum Neural Networks: A Survey
====================================================================


# Table of Contents


* [Surveys](#Surveys)
	* [2021](#2021)	* [2020](#2020)	* [2019](#2019)	* [2018](#2018)	* [2017](#2017)	* [2016](#2016)	* [2015](#2015)	* [2013](#2013)	* [2012](#2012)	

* [Papers](#Papers)
	* [2021](#2021)	* [2020](#2020)	* [2019](#2019)	* [2018](#2018)	* [2017](#2017)	* [2016](#2016)	* [2015](#2015)	* [2014](#2014)	* [2013](#2013)	* [2012](#2012)	* [2011](#2011)	* [2010](#2010)	* [2009](#2009)	* [2008](#2008)	* [2007](#2007)	* [2006](#2006)	* [2005](#2005)	* [2004](#2004)	* [2003](#2003)	* [2002](#2002)	* [2001](#2001)	* [2000](#2000)	* [1999](#1999)	* [1998](#1998)	* [1997](#1997)	* [1996](#1996)	* [1995](#1995)	* [1994](#1994)	* [1992](#1992)	* [1989](#1989)	* [1986](#1986)	* [1985](#1985)	* [1983](#1983)	



# Surveys



### 2021
<details><summary>[Bharti et al] <b><a href=https://arxiv.org/abs/2101.08448> Noisy intermediate-scale quantum (NISQ) algorithms. </a></b> <i>arXiv.</i> <code><b>[Variational Quantum Algorithms]</b></code> <code><b>[Parameterized Quantum Circuits]</b></code> <code><b>[Quantum Optimization]</b></code> <code><b>[Quantum Machine Learning]</b></code> <code><i>(Gate Model)</i></code> <code><i>(Adiabatic Model)</i></code></summary><pre><code>@article{bharti2021noisy,
  title={Noisy intermediate-scale quantum (NISQ) algorithms},
  author={Bharti, Kishor and Cervera-Lierta, Alba and Kyaw, Thi Ha and Haug, Tobias and Alperin-Lea, Sumner and Anand, Abhinav and Degroote, Matthias and Heimonen, Hermanni and Kottmann, Jakob S and Menke, Tim and others},
  journal={arXiv preprint arXiv:2101.08448},
  year={2021}
}</code></pre></details>
<details><summary>[Mangini et al] <b><a href=https://iopscience.iop.org/article/10.1209/0295-5075/134/10002/meta> Quantum computing models for artificial neural networks. </a></b> <i>EPL (Europhysics Letters).</i> <code><b>[Quantum Neural Networks]</b></code> <code><b>[Parameterized Quantum Circuits]</b></code> <code><b>[Variational Quantum Algorithms]</b></code> <code><b>[Quantum Machine Learning]</b></code> </summary><pre><code>@article{mangini2021quantum,
  title={Quantum computing models for artificial neural networks},
  author={Mangini, Stefano and Tacchino, Francesco and Gerace, Dario and Bajoni, Daniele and Macchiavello, Chiara},
  journal={EPL (Europhysics Letters)},
  volume={134},
  number={1},
  pages={10002},
  year={2021},
  publisher={IOP Publishing}
}</code></pre></details>


### 2020
<details><summary>[Ablayev et al] <b><a href=https://ieeexplore.ieee.org/stamp/stamp.jsp?arnumber=8935094> On quantum methods for machine learning problems part I: Quantum tools. </a></b> <i>Big Data Mining and Analytics.</i> <code><b>[Quantum Machine Learning]</b></code> <code><b>[Quantum Classification]</b></code> <code><i>(Gate Model)</i></code></summary><pre><code>@Article{ablayev2020quantum1,
  Title                    = {{On quantum methods for machine learning problems part I: Quantum tools}},
  Author                   = {Ablayev, Farid and Ablayev, Marat and Huang, Joshua Zhexue and Khadiev, Kamil and Salikhova, Nailya and Wu, Dingming},
  Journal                  = {Big Data Mining and Analytics},
  Year                     = {2020},
  Number                   = {1},
  Pages                    = {41--55},
  Volume                   = {3},
  URL                      = {https://ieeexplore.ieee.org/stamp/stamp.jsp?arnumber=8935094}
}
</code></pre></details>
<details><summary>[Ablayev et al] <b><a href=https://ieeexplore.ieee.org/abstract/document/8935095> On quantum methods for machine learning problems part II: Quantum classification algorithms. </a></b> <i>Big Data Mining and Analytics.</i> <code><b>[Quantum Machine Learning]</b></code> <code><b>[Quantum Classification]</b></code> <code><b>[Quantum Nearest Neighbors Algorithm]</b></code> <code><i>(Gate Model)</i></code></summary><pre><code>@Article{ablayev2020quantum2,
  Title                    = {{On quantum methods for machine learning problems part II: Quantum classification algorithms}},
  Author                   = {Ablayev, Farid and Ablayev, Marat and Huang, Joshua Zhexue and Khadiev, Kamil and Salikhova, Nailya and Wu, Dingming},
  Journal                  = {Big Data Mining and Analytics},
  Year                     = {2020},
  Number                   = {1},
  Pages                    = {56--67},
  Volume                   = {3},
  URL                      = {https://ieeexplore.ieee.org/abstract/document/8935095}
}
</code></pre></details>
<details><summary>[Abohashima et al] <b><a href=https://arxiv.org/abs/2006.12270> Classification with Quantum Machine Learning: A Survey. </a></b> <i>arXiv.</i> <code><b>[Quantum Machine Learning]</b></code> <code><b>[Quantum Classification]</b></code> <code><i>(Gate Model)</i></code></summary><pre><code>@article{abohashima2020classification,
  title={Classification with quantum machine learning: A survey},
  author={Abohashima, Zainab and Elhosen, Mohamed and Houssein, Essam H and Mohamed, Waleed M},
  journal={arXiv preprint arXiv:2006.12270},
  year={2020}
}</code></pre></details>
<details><summary>[Bharti et al] <b><a href=https://avs.scitation.org/doi/10.1116/5.0007529> Machine learning meets quantum foundations: A brief survey. </a></b> <i>AVS Quantum Science.</i> <code><b>[Quantum Machine Learning]</b></code> <code><i>(Gate Model)</i></code></summary><pre><code>@Article{bharti2020machine,
  Title                    = {Machine learning meets quantum foundations: A brief survey},
  Author                   = {Bharti, Kishor and Haug, Tobias and Vedral, Vlatko and Kwek, Leong-Chuan},
  Journal                  = {AVS Quantum Science},
  Year                     = {2020},
  Number                   = {3},
  Pages                    = {034101},
  Volume                   = {2},
  Publisher                = {American Vacuum Society},
  URL                      = {https://avs.scitation.org/doi/10.1116/5.0007529}
}
</code></pre></details>
<details><summary>[Chakraborty et al] <b><a href=https://ieeexplore.ieee.org/document/9137960> An Analytical Review of Quantum Neural Network Models and Relevant Research. </a></b> <i>International Conference on Communication and Electronics Systems.</i> <code><b>[Quantum Neural Networks]</b></code> <code><b>[Quantum Machine Learning]</b></code> </summary><pre><code>@inproceedings{chakraborty2020analytical,
  title={An Analytical Review of Quantum Neural Network Models and Relevant Research},
  author={Chakraborty, Simantini and Das, Tamal and Sutradhar, Saurav and Das, Mrinmoy and Deb, Suman},
  booktitle={2020 5th International Conference on Communication and Electronics Systems (ICCES)},
  pages={1395--1400},
  year={2020},
  organization={IEEE}
}</code></pre></details>
<details><summary>[Duan & Guo] <b><a href=https://www.sciencedirect.com/science/article/pii/S037596012030462X> A survey on HHL algorithm: From theory to application in quantum machine learning. </a></b> <i>Physics Letters A.</i> <code><b>[Quantum Machine Learning]</b></code> <code><b>[Quantum Speedup-Advantage-Supremacy]</b></code> </summary><pre><code>@Article{duan1998probabilistic,
  Title                    = {Probabilistic cloning and identification of linearly independent quantum states},
  Author                   = {Duan, Lu-Ming and Guo, Guang-Can},
  Journal                  = {Physical review letters},
  Year                     = {1998},
  Number                   = {22},
  Pages                    = {4999},
  Volume                   = {80},
  Publisher                = {APS},
  URL                      = {https://www.sciencedirect.com/science/article/pii/S037596012030462X}
}
</code></pre></details>
<details><summary>[Pande et Mulay] <b><a href=https://www.tandfonline.com/doi/abs/10.1080/0194262X.2020.1776193> Bibliometric Survey of Quantum Machine Learning. </a></b> <i>Science & Technology Libraries.</i> <code><b>[Quantum Machine Learning]</b></code> </summary><pre><code>@article{pande2020bibliometric,
  title={Bibliometric survey of quantum machine learning},
  author={Pande, Mandaar and Mulay, Preeti},
  journal={Science \& Technology Libraries},
  volume={39},
  number={4},
  pages={369--382},
  year={2020},
  publisher={Taylor \& Francis}
}</code></pre></details>
<details><summary>[Ramezani et al] <b><a href=https://ieeexplore.ieee.org/abstract/document/9207714/?casa_token=086gCg9_PHoAAAAA:drZYsucyKd7gT4q2d0kxFc5X9rcStHITEq7ReXA-aw8lNIx0s7d51pIz2NdwwxOzFFCYsSsh_r0r> Machine Learning Algorithms in Quantum Computing: A Survey. </a></b> <i>International Joint Conference on Neural Networks.</i> <code><b>[Quantum Information and Computing]</b></code> <code><b>[Quantum Machine Learning]</b></code> </summary><pre><code>@inproceedings{ramezani2020machine,
  title={Machine learning algorithms in quantum computing: A survey},
  author={Ramezani, Somayeh Bakhtiari and Sommers, Alexander and Manchukonda, Harish Kumar and Rahimi, Shahram and Amirlatifi, Amin},
  booktitle={2020 International Joint Conference on Neural Networks (IJCNN)},
  pages={1--8},
  year={2020},
  organization={IEEE}
}</code></pre></details>
<details><summary>[Sergioli] <b><a href=https://link.springer.com/article/10.1007/s00500-019-04429-x> Quantum and quantum-like machine learning: a note on differences and similarities. </a></b> <i>Soft Computing.</i> <code><b>[Quantum Machine Learning]</b></code> <code><b>[Quantum Information and Computing]</b></code> <code><b>[Quantum Classification]</b></code> </summary><pre><code>@article{sergioli2020quantum,
  title={Quantum and quantum-like machine learning: a note on differences and similarities},
  author={Sergioli, Giuseppe},
  journal={Soft Computing},
  volume={24},
  number={14},
  pages={10247--10255},
  year={2020},
  publisher={Springer}
}</code></pre></details>
<details><summary>[Zhang & Ni] <b><a href=https://onlinelibrary.wiley.com/doi/abs/10.1002/que2.34?casa_token=t0KD3pYj7-wAAAAA:cbEXtTV6lJd1EGc4na3-akrlbLfw9jZM6r3lHfCwOIs5eP1Ma0ZxUit3wN-8Q5rkz3TmVYBdYrtayw> Recent advances in quantum machine learning. </a></b> <i>Quantum Engineering.</i> <code><b>[Quantum Machine Learning]</b></code> <code><b>[Quantum Theory]</b></code> </summary><pre><code>@article{zhang2020recent,
  title={Recent advances in quantum machine learning},
  author={Zhang, Yao and Ni, Qiang},
  journal={Quantum Engineering},
  volume={2},
  number={1},
  pages={e34},
  year={2020},
  publisher={Wiley Online Library}
}</code></pre></details>


### 2019
<details><summary>[Benedetti et al] <b><a href=https://iopscience.iop.org/article/10.1088/2058-9565/ab4eb5/meta> Parameterized quantum circuits as machine learning models. </a></b> <i>Quantum Science and Technology.</i> <code><b>[Variational Quantum Algorithms]</b></code> <code><b>[Quantum Machine Learning]</b></code> </summary><pre><code>@Article{benedetti2019parameterized,
  Title                    = {Parameterized quantum circuits as machine learning models},
  Author                   = {Benedetti, Marcello and Lloyd, Erika and Sack, Stefan and Fiorentini, Mattia},
  Journal                  = {Quantum Science and Technology},
  Year                     = {2019},
  Number                   = {4},
  Pages                    = {043001},
  Volume                   = {4},
  Publisher                = {IOP Publishing},
  URL                      = {https://iopscience.iop.org/article/10.1088/2058-9565/ab4eb5/meta}
}
</code></pre></details>
<details><summary>[Kamruzzaman et al] <b><a href=https://link.springer.com/chapter/10.1007/978-3-030-12385-7_24> Quantum Deep Learning Neural Networks. </a></b> <i>FICC 2019.</i> <code><b>[Quantum Neural Networks]</b></code> </summary><pre><code>@inproceedings{kamruzzaman2019quantum,
  title={Quantum deep learning neural networks},
  author={Kamruzzaman, Abu and Alhwaiti, Yousef and Leider, Avery and Tappert, Charles C},
  booktitle={Future of Information and Communication Conference},
  pages={299--311},
  year={2019},
  organization={Springer}
}</code></pre></details>


### 2018
<details><summary>[Allcock & Zhang] <b><a href=https://academic.oup.com/nsr/article/6/1/26/5222655?login=true> Quantum machine learning. </a></b> <i>National Science Review.</i> <code><b>[Quantum Machine Learning]</b></code> <code><b>[Quantum Boltzmann machines]</b></code> <code><b>[Quantum Generative Adversarial Networks]</b></code> </summary><pre><code>@Article{allcock2019quantum,
  Title                    = {Quantum machine learning},
  Author                   = {Allcock, Jonathan and Zhang, Shengyu},
  Journal                  = {National Science Review},
  Year                     = {2019},
  Number                   = {1},
  Pages                    = {26--28},
  Volume                   = {6},
  Publisher                = {Oxford University Press},
  URL                      = {https://academic.oup.com/nsr/article/6/1/26/5222655?login=true}
}
</code></pre></details>
<details><summary>[Ciliberto et al] <b><a href=https://doi.org/10.1098/rspa.2017.0551> Quantum machine learning: a classical perspective. </a></b> <i>Proceedings of the Royal Society A: Mathematical, Physical and Engineering Sciences.</i> <code><b>[Quantum Machine Learning]</b></code> <code><b>[Quantum Optimization]</b></code> <code><b>[Quantum Neural Networks]</b></code> </summary><pre><code>@Article{ciliberto2018quantum,
  Title                    = {Quantum machine learning: a classical perspective},
  Author                   = {Ciliberto, Carlo and Herbster, Mark and Ialongo, Alessandro Davide and Pontil, Massimiliano and Rocchetto, Andrea and Severini, Simone and Wossnig, Leonard},
  Journal                  = {Proceedings of the Royal Society A: Mathematical, Physical and Engineering Sciences},
  Year                     = {2018},
  Number                   = {2209},
  Pages                    = {20170551},
  Volume                   = {474},
  Publisher                = {The Royal Society Publishing},
  URL                      = {https://doi.org/10.1098/rspa.2017.0551}
}
</code></pre></details>


### 2017
<details><summary>[Arunachalam & de Wolf] <b><a href=https://dl.acm.org/doi/abs/10.1145/3106700.3106710> Guest column: A survey of quantum learning theory. </a></b> <i>ACM SIGACT News.</i> <code><b>[Quantum Machine Learning]</b></code> <code><b>[Probably Approximately Correct Learning]</b></code> <code><b>[Complexity Theory]</b></code> <code><i>(Gate Model)</i></code></summary><pre><code>@Article{arunachalam2017guest,
  Title                    = {Guest column: A survey of quantum learning theory},
  Author                   = {Arunachalam, Srinivasan and de Wolf, Ronald},
  Journal                  = {ACM SIGACT News},
  Year                     = {2017},
  Number                   = {2},
  Pages                    = {41--67},
  Volume                   = {48},
  Publisher                = {ACM New York, NY, USA},
  URL                      = {https://dl.acm.org/doi/abs/10.1145/3106700.3106710}
}
</code></pre></details>


### 2016
<details><summary>[Oshurko] <b><a href=https://www.worldscientific.com/doi/abs/10.1142/9781786348210_0010> Quantum machine learning. </a></b> <i>Quantum Computing.</i> <code><b>[Quantum Machine Learning]</b></code> </summary><pre><code>@article{oshurko2016quantum,
  title={Quantum machine learning},
  author={Oshurko, Ievgeniia},
  journal={Quantum Information and Computation Course},
  year={2016}
}</code></pre></details>


### 2015
<details><summary>[Adcock et al] <b><a href=https://arxiv.org/abs/1512.02900> Advances in quantum machine learning. </a></b> <i>arXiv.</i> <code><b>[Quantum Machine Learning]</b></code> <code><b>[Quantum Classification]</b></code> <code><b>[Quantum Clustering]</b></code> <code><b>[Quantum Nearest Neighbors Algorithm]</b></code> <code><b>[Quantum Neural Networks]</b></code> <code><i>(Gate Model)</i></code> <code><i>(Adiabatic Model)</i></code></summary><pre><code>@Article{adcock2015advances,
  Title                    = {Advances in quantum machine learning},
  Author                   = {Adcock, Jeremy and Allen, Euan and Day, Matthew and Frick, Stefan and Hinchliff, Janna and Johnson, Mack and Morley-Short, Sam and Pallister, Sam and Price, Alasdair and Stanisic, Stasja},
  Journal                  = {arXiv preprint arXiv:1512.02900},
  Year                     = {2015},
  URL                      = {https://arxiv.org/abs/1512.02900}
}
</code></pre></details>
<details><summary>[Schuld et al] <b><a href=https://www.tandfonline.com/doi/abs/10.1080/00107514.2014.964942> An introduction to quantum machine learning. </a></b> <i>Contemporary Physics.</i> <code><b>[Quantum Machine Learning]</b></code> <code><i>(Gate Model)</i></code> <code><i>(Adiabatic Model)</i></code></summary><pre><code>@article{schuld2015introduction,
  title={An introduction to quantum machine learning},
  author={Schuld, Maria and Sinayskiy, Ilya and Petruccione, Francesco},
  journal={Contemporary Physics},
  volume={56},
  number={2},
  pages={172--185},
  year={2015},
  publisher={Taylor \& Francis}
}</code></pre></details>


### 2013
<details><summary>[Devoret & Schoelkopf] <b><a href=https://science.sciencemag.org/content/339/6124/1169.abstract> Superconducting Circuits for Quantum Information: An Outlook. </a></b> <i>Science.</i> <code><b>[Quantum Information and Computing]</b></code> <code><b>[Superconducting Qubits]</b></code> <code><i>(Gate Model)</i></code></summary><pre><code>@Article{devoret2013superconducting,
  Title                    = {Superconducting circuits for quantum information: an outlook},
  Author                   = {Devoret, Michel H and Schoelkopf, Robert J},
  Journal                  = {Science},
  Year                     = {2013},
  Number                   = {6124},
  Pages                    = {1169--1174},
  Volume                   = {339},
  Publisher                = {American Association for the Advancement of Science},
  URL                      = {https://science.sciencemag.org/content/339/6124/1169.abstract}
}
</code></pre></details>


### 2012
<details><summary>[Preskill] <b><a href=https://arxiv.org/pdf/1203.5813.pdf> Quantum computing and the entanglement frontier. </a></b> <i>arXiv.</i> <code><b>[Quantum Information and Computing]</b></code> </summary><pre><code>@article{preskill2012quantum,
  title={Quantum computing and the entanglement frontier},
  author={Preskill, John},
  journal={arXiv preprint arXiv:1203.5813},
  year={2012}
}</code></pre></details>


# Papers



### 2021
<details><summary>[Amaro et al.] <b><a href=https://arxiv.org/abs/2106.10055> Filtering variational quantum algorithms for combinatorial optimization. </a></b> <i>arXiv.</i> <code><b>[Variational Quantum Algorithms]</b></code> <code><b>[Quantum Optimization]</b></code> <code><b>[Parameterized Quantum Circuits]</b></code> </summary><pre><code>@article{amaro2021filtering,
  title={Filtering variational quantum algorithms for combinatorial optimization},
  author={Amaro, David and Modica, Carlo and Rosenkranz, Matthias and Fiorentini, Mattia and Benedetti, Marcello and Lubasch, Michael},
  journal={Quantum Science and Technology},
  year={2021},
  publisher={IOP Publishing}
}
</code></pre></details>
<details><summary>[Ban et al] <b><a href=https://www.Nature.com/articles/s41598-021-85208-3> Speeding up quantum perceptron via shortcuts to adiabaticity. </a></b> <i>Scientific reports.</i> <code><b>[Quantum Perceptron]</b></code> <code><b>[Quantum Machine Learning]</b></code> <code><i>(Adiabatic Model)</i></code></summary><pre><code>@Article{ban2021speeding,
  Title                    = {Speeding up quantum perceptron via shortcuts to adiabaticity},
  Author                   = {Ban, Yue and Chen, Xi and Torrontegui, E and Solano, Enrique and Casanova, Jorge},
  Journal                  = {Scientific reports},
  Year                     = {2021},
  Number                   = {1},
  Pages                    = {1--8},
  Volume                   = {11},
  Publisher                = {Nature Publishing Group},
  URL                      = {https://www.Nature.com/articles/s41598-021-85208-3}
}
</code></pre></details>
<details><summary>[Benedetti et al] <b><a href=https://journals.aps.org/prresearch/abstract/10.1103/PhysRevResearch.3.033083> Hardware-efficient variational quantum algorithms for time evolution. </a></b> <i>Physical Review Research.</i> <code><b>[Time Evolution (Hamiltonian Simulation)]</b></code> <code><b>[Parameterized Quantum Circuits]</b></code> <code><b>[Quantum Machine Learning]</b></code> </summary><pre><code>@article{benedetti2021hardware,
  title={Hardware-efficient variational quantum algorithms for time evolution},
  author={Benedetti, Marcello and Fiorentini, Mattia and Lubasch, Michael},
  journal={Physical Review Research},
  volume={3},
  number={3},
  pages={033083},
  year={2021},
  publisher={APS}
}
</code></pre></details>
<details><summary>[Benedetti et al] <b><a href=https://journals.aps.org/prapplied/abstract/10.1103/PhysRevApplied.16.044057> Variational Inference with a Quantum Computer. </a></b> <i>Physical Review Applied.</i> <code><b>[Variational Inference]</b></code> <code><b>[Parameterized Quantum Circuits]</b></code> <code><b>[Quantum Machine Learning]</b></code> </summary><pre><code>@article{benedetti2021variational,
  title={Variational inference with a quantum computer},
  author={Benedetti, Marcello and Coyle, Brian and Fiorentini, Mattia and Lubasch, Michael and Rosenkranz, Matthias},
  journal={Physical Review Applied},
  volume={16},
  number={4},
  pages={044057},
  year={2021},
  publisher={APS}
}
</code></pre></details>
<details><summary>[Cerezo et al] <b><a href=https://www.nature.com/articles/s41467-021-21728-w> Cost Function Dependent Barren Plateaus in Shallow Parametrized Quantum Circuits. </a></b> <i>Nature communications.</i> <code><b>[Barren Plateaus]</b></code> <code><b>[Parameterized Quantum Circuits]</b></code> <code><b>[Quantum Neural Networks]</b></code> <code><b>[Quantum Machine Learning]</b></code> </summary><pre><code>@article{cerezo2021cost,
  title={Cost function dependent barren plateaus in shallow parametrized quantum circuits},
  author={Cerezo, Marco and Sone, Akira and Volkoff, Tyler and Cincio, Lukasz and Coles, Patrick J},
  journal={Nature communications},
  volume={12},
  number={1},
  pages={1--12},
  year={2021},
  publisher={Nature Publishing Group}
}</code></pre></details>
<details><summary>[Chen et al] <b><a href=https://arxiv.org/abs/1805.08654> Universal discriminative quantum neural networks. </a></b> <i>Quantum Machine Intelligence.</i> <code><b>[Quantum Neural Networks]</b></code> <code><b>[Quantum Machine Learning]</b></code> <code><i>(Gate Model)</i></code></summary><pre><code>@Article{chen2021universal,
  Title                    = {Universal discriminative quantum neural networks},
  Author                   = {Chen, Hongxiang and Wossnig, Leonard and Severini, Simone and Neven, Hartmut and Mohseni, Masoud},
  Journal                  = {Quantum Machine Intelligence},
  Year                     = {2021},
  Number                   = {1},
  Pages                    = {1--11},
  Volume                   = {3},
  Publisher                = {Springer},
  URL                      = {https://arxiv.org/abs/1805.08654}
}
</code></pre></details>
<details><summary>[Egger et al] <b><a href=https://quantum-journal.org/papers/q-2021-06-17-479/> Warm-starting quantum optimization. </a></b> <i>Quantum.</i> <code><b>[Quantum Optimization]</b></code> <code><b>[Parameterized Quantum Circuits]</b></code> <code><b>[Quantum Machine Learning]</b></code> </summary><pre><code>@article{egger2021warm,
  title={Warm-starting quantum optimization},
  author={Egger, Daniel J and Mare{\v{c}}ek, Jakub and Woerner, Stefan},
  journal={Quantum},
  volume={5},
  pages={479},
  year={2021},
  publisher={Verein zur F{\"o}rderung des Open Access Publizierens in den Quantenwissenschaften}
}</code></pre></details>
<details><summary>[Holmes et al ] <b><a href=https://arxiv.org/abs/2101.02138> Connecting ansatz expressibility to gradient magnitudes and barren plateaus. </a></b> <i>arXiv.</i> <code><b>[Parameterized Quantum Circuits]</b></code> <code><b>[Quantum Neural Networks]</b></code> <code><b>[Barren Plateaus]</b></code> <code><b>[Quantum Machine Learning]</b></code> </summary><pre><code>@article{holmes2021connecting,
  title={Connecting ansatz expressibility to gradient magnitudes and barren plateaus},
  author={Holmes, Zo{\"e} and Sharma, Kunal and Cerezo, M and Coles, Patrick J},
  journal={arXiv preprint arXiv:2101.02138},
  year={2021}
}</code></pre></details>
<details><summary>[Jaderberg et al] <b><a href=https://arxiv.org/abs/2103.14653> Quantum Self-Supervised Learning. </a></b> <i>arXiv.</i> <code><b>[Quantum Neural Networks]</b></code> <code><b>[Quantum Machine Learning]</b></code> <code><i>(Gate Model)</i></code></summary><pre><code>@article{jaderberg2021quantum,
  title={Quantum Self-Supervised Learning},
  author={Jaderberg, Ben and Anderson, Lewis W and Xie, Weidi and Albanie, Samuel and Kiffner, Martin and Jaksch, Dieter},
  journal={arXiv preprint arXiv:2103.14653},
  year={2021}
}</code></pre></details>
<details><summary>[Liu et al] <b><a href=https://www.nature.com/articles/s41567-021-01287-z> A rigorous and robust quantum speed-up in supervised machine learning. </a></b> <i>Nature Physics.</i> <code><b>[Quantum Machine Learning]</b></code> <code><b>[Quantum Classification]</b></code> <code><b>[Quantum Speedup-Advantage-Supremacy]</b></code> </summary><pre><code>@article{liu2021rigorous,
  title={A rigorous and robust quantum speed-up in supervised machine learning},
  author={Liu, Yunchao and Arunachalam, Srinivasan and Temme, Kristan},
  journal={Nature Physics},
  pages={1--5},
  year={2021},
  publisher={Nature Publishing Group}
}</code></pre></details>
<details><summary>[Lubsch et al] <b><a href=https://journals.aps.org/pra/abstract/10.1103/PhysRevA.101.010301> Variational quantum algorithms for nonlinear problems. </a></b> <i>Physical Review A.</i> <code><b>[Nonlinear Problems]</b></code> <code><b>[Partial Differential Equations]</b></code> <code><b>[Quantum Neural Networks]</b></code> <code><b>[Quantum Machine Learning]</b></code> </summary><pre><code>@article{lubasch2020variational,
  title={Variational quantum algorithms for nonlinear problems},
  author={Lubasch, Michael and Joo, Jaewoo and Moinier, Pierre and Kiffner, Martin and Jaksch, Dieter},
  journal={Physical Review A},
  volume={101},
  number={1},
  pages={010301},
  year={2020},
  publisher={APS}
}
</code></pre></details>
<details><summary>[Patti et al] <b><a href=https://arxiv.org/abs/2012.12658> Entanglement devised barren plateau mitigation. </a></b> <i>arXiv.</i> <code><b>[Barren Plateaus]</b></code> <code><b>[Parameterized Quantum Circuits]</b></code> <code><b>[Quantum Neural Networks]</b></code> <code><b>[Quantum Machine Learning]</b></code> </summary><pre><code>@article{patti2020entanglement,
  title={Entanglement devised barren plateau mitigation},
  author={Patti, Taylor L and Najafi, Khadijeh and Gao, Xun and Yelin, Susanne F},
  journal={arXiv preprint arXiv:2012.12658},
  year={2020}
}</code></pre></details>
<details><summary>[Perez-Salinas et al] <b><a href=https://arxiv.org/abs/2102.04032> One qubit as a Universal Approximant. </a></b> <i>arXiv.</i> <code><b>[Parameterized Quantum Circuits]</b></code> <code><b>[Quantum Neural Networks]</b></code> <code><b>[Quantum Machine Learning]</b></code> </summary><pre><code>@article{perez2021one,
  title={One qubit as a Universal Approximant},
  author={P{\'e}rez-Salinas, Adri{\'a}n and L{\'o}pez-N{\'u}{\~n}ez, David and Garc{\'\i}a-S{\'a}ez, Artur and Forn-D{\'\i}az, P and Latorre, Jos{\'e} I},
  journal={arXiv preprint arXiv:2102.04032},
  year={2021}
}</code></pre></details>
<details><summary>[Plekhanov et al] <b><a href=https://arxiv.org/abs/2109.03687> Variational quantum amplitude estimation. </a></b> <i>arXiv.</i> <code><b>[Variational Quantum Algorithms]</b></code> <code><b>[Amplitude estimation]</b></code> <code><b>[Parameterized Quantum Circuits]</b></code> </summary><pre><code>@article{plekhanov2021variational,
  title={Variational quantum amplitude estimation},
  author={Plekhanov, Kirill and Rosenkranz, Matthias and Fiorentini, Mattia and Lubasch, Michael},
  journal={arXiv preprint arXiv:2109.03687},
  year={2021}
}
</code></pre></details>
<details><summary>[Schuld et al] <b><a href=https://journals.aps.org/pra/abstract/10.1103/PhysRevA.103.032430> Effect of data encoding on the expressive power of variational quantum-machine-learning models. </a></b> <i>Physical Review A.</i> <code><b>[Parameterized Quantum Circuits]</b></code> <code><b>[Variational Quantum Algorithms]</b></code> <code><b>[Quantum Machine Learning]</b></code> </summary><pre><code>@article{schuld2021effect,
  title={Effect of data encoding on the expressive power of variational quantum-machine-learning models},
  author={Schuld, Maria and Sweke, Ryan and Meyer, Johannes Jakob},
  journal={Physical Review A},
  volume={103},
  number={3},
  pages={032430},
  year={2021},
  publisher={APS}
}</code></pre></details>
<details><summary>[Skolik et al] <b><a href=https://link.springer.com/article/10.1007/s42484-020-00036-4> Layerwise learning for quantum neural networks. </a></b> <i>Quantum Machine Intelligence.</i> <code><b>[Barren Plateaus]</b></code> <code><b>[Parameterized Quantum Circuits]</b></code> <code><b>[Quantum Neural Networks]</b></code> <code><b>[Quantum Machine Learning]</b></code> </summary><pre><code>@article{skolik2021layerwise,
  title={Layerwise learning for quantum neural networks},
  author={Skolik, Andrea and McClean, Jarrod R and Mohseni, Masoud and van der Smagt, Patrick and Leib, Martin},
  journal={Quantum Machine Intelligence},
  volume={3},
  number={1},
  pages={1--11},
  year={2021},
  publisher={Springer}
}</code></pre></details>
<details><summary>[Volkoff & Coles] <b><a href=https://arxiv.org/abs/2005.12200> Large gradients via correlation in random parameterized quantum circuits. </a></b> <i>arXiv.</i> <code><b>[Barren Plateaus]</b></code> <code><b>[Parameterized Quantum Circuits]</b></code> <code><b>[Quantum Neural Networks]</b></code> <code><b>[Variational Quantum Algorithms]</b></code> <code><b>[Quantum Machine Learning]</b></code> </summary><pre><code>@article{volkoff2021large,
  title={Large gradients via correlation in random parameterized quantum circuits},
  author={Volkoff, Tyler and Coles, Patrick J},
  journal={Quantum Science and Technology},
  volume={6},
  number={2},
  pages={025008},
  year={2021},
  publisher={IOP Publishing}
}</code></pre></details>


### 2020
<details><summary>[Barkoutsos et al] <b><a href=https://quantum-journal.org/papers/q-2020-04-20-256/> Improving Variational Quantum Optimization Using CVaR. </a></b> <i>Quantum.</i> <code><b>[Variational Quantum Algorithms]</b></code> <code><b>[Quantum Optimization]</b></code> <code><b>[Quantum Machine Learning]</b></code> </summary><pre><code>@article{barkoutsos2020improving,
  title={Improving variational quantum optimization using cvar},
  author={Barkoutsos, Panagiotis Kl and Nannicini, Giacomo and Robert, Anton and Tavernelli, Ivano and Woerner, Stefan},
  journal={Quantum},
  volume={4},
  pages={256},
  year={2020},
  publisher={Verein zur F{\"o}rderung des Open Access Publizierens in den Quantenwissenschaften}
}</code></pre></details>
<details><summary>[Beer et al] <b><a href=https://www.nature.com/articles/s41467-020-14454-2.pdf> Training deep quantum neural networks. </a></b> <i>Nature communications.</i> <code><b>[Quantum Neural Networks]</b></code> <code><b>[Quantum Perceptron]</b></code> <code><b>[Quantum Machine Learning]</b></code> <code><i>(Gate Model)</i></code></summary><pre><code>@Article{beer2020training,
  Title                    = {Training deep quantum neural networks},
  Author                   = {Beer, Kerstin and Bondarenko, Dmytro and Farrelly, Terry and Osborne, Tobias J and Salzmann, Robert and Scheiermann, Daniel and Wolf, Ramona},
  Journal                  = {Nature communications},
  Year                     = {2020},
  Number                   = {1},
  Pages                    = {1--6},
  Volume                   = {11},
  Publisher                = {Nature Publishing Group},
  URL                      = {https://www.nature.com/articles/s41467-020-14454-2.pdf}
}
</code></pre></details>
<details><summary>[Carolan et al ] <b><a href=https://www.nature.com/articles/s41567-019-0747-6> Variational quantum unsampling on a quantum photonic processor. </a></b> <i>Nature Physics.</i> <code><b>[Variational Quantum Algorithms]</b></code> <code><b>[Quantum Neural Networks]</b></code> </summary><pre><code>@article{carolan2020variational,
  title={Variational quantum unsampling on a quantum photonic processor},
  author={Carolan, Jacques and Mohseni, Masoud and Olson, Jonathan P and Prabhu, Mihika and Chen, Changchen and Bunandar, Darius and Niu, Murphy Yuezhen and Harris, Nicholas C and Wong, Franco NC and Hochberg, Michael and others},
  journal={Nature Physics},
  volume={16},
  number={3},
  pages={322--327},
  year={2020},
  publisher={Nature Publishing Group}
}</code></pre></details>
<details><summary>[Cerezo et al] <b><a href=https://arxiv.org/pdf/2012.09265.pdf> Variational Quantum Algorithms. </a></b> <i>arXiv.</i> <code><b>[Variational Quantum Algorithms]</b></code> <code><b>[Quantum Machine Learning]</b></code> <code><i>(Gate Model)</i></code> <code><i>(Adiabatic Model)</i></code></summary><pre><code>@Article{cerezo2020variational,
  Title                    = {Variational quantum algorithms},
  Author                   = {Cerezo, Marco and Arrasmith, Andrew and Babbush, Ryan and Benjamin, Simon C and Endo, Suguru and Fujii, Keisuke and McClean, Jarrod R and Mitarai, Kosuke and Yuan, Xiao and Cincio, Lukasz and others},
  Journal                  = {arXiv preprint arXiv:2012.09265},
  Year                     = {2020},
  URL                      = {https://arxiv.org/pdf/2012.09265.pdf}
}
</code></pre></details>
<details><summary>[Dunjko & Wittek] <b><a href=https://quantum-journal.org/views/qv-2020-03-17-32/> A non-review of Quantum Machine Learning: trends and explorations. </a></b> <i>Quantum Views.</i> <code><b>[Quantum Machine Learning]</b></code> </summary><pre><code>@Article{dunjko2020non,
  Title                    = {A non-review of Quantum Machine Learning: trends and explorations},
  Author                   = {Dunjko, Vedran and Wittek, Peter},
  Journal                  = {Quantum Views},
  Year                     = {2020},
  Pages                    = {32},
  Volume                   = {4},
  Publisher                = {Verein zur F{\"o}rderung des Open Access Publizierens in den Quantenwissenschaften},
  URL                      = {https://quantum-journal.org/views/qv-2020-03-17-32/}
}
</code></pre></details>
<details><summary>[Gil Vidal & Theis] <b><a href=https://doi.org/10.3389/fphy.2020.00297> Input redundancy for parameterized quantum circuits. </a></b> <i>Frontiers in Physics.</i> <code><b>[Parameterized Quantum Circuits]</b></code> <code><b>[Quantum Neural Networks]</b></code> <code><b>[Quantum Machine Learning]</b></code> </summary><pre><code>@article{gil2020input,
  title={Input redundancy for parameterized quantum circuits},
  author={Gil Vidal, Francisco Javier and Theis, Dirk Oliver},
  journal={Frontiers in Physics},
  volume={8},
  pages={297},
  year={2020},
  publisher={Frontiers}
}</code></pre></details>
<details><summary>[Pjilipp  et al] <b><a href=https://iopscience.iop.org/article/10.1088/1361-6633/ab85b8> Perspectives of quantum annealing: Methods andimplementations. </a></b> <i>Reports on Progress in Physics.</i> <code><b>[Quantum Information and Computing]</b></code> <code><b>[Quantum Annealing]</b></code> <code><i>(Adiabatic Model)</i></code></summary><pre><code>@article{hauke2020perspectives,
  title={Perspectives of quantum annealing: Methods and implementations},
  author={Hauke, Philipp and Katzgraber, Helmut G and Lechner, Wolfgang and Nishimori, Hidetoshi and Oliver, William D},
  journal={Reports on Progress in Physics},
  volume={83},
  number={5},
  pages={054401},
  year={2020},
  publisher={IOP Publishing}
}</code></pre></details>
<details><summary>[Henderson et al] <b><a href=https://link.springer.com/content/pdf/10.1007/s42484-020-00012-y.pdf> Quanvolutional neural networks: powering image recognition with quantum circuits. </a></b> <i>Quantum Machine Intelligence.</i> <code><b>[Quantum Convolutional Neural Networks]</b></code> <code><b>[Quantum Neural Networks]</b></code> <code><b>[Quantum Machine Learning]</b></code> <code><i>(Gate Model)</i></code></summary><pre><code>@article{henderson2020quanvolutional,
  title={Quanvolutional neural networks: powering image recognition with quantum circuits},
  author={Henderson, Maxwell and Shakya, Samriddhi and Pradhan, Shashindra and Cook, Tristan},
  journal={Quantum Machine Intelligence},
  volume={2},
  number={1},
  pages={1--9},
  year={2020},
  publisher={Springer}
}</code></pre></details>
<details><summary>[Kerenidis et al] <b><a href=https://openreview.net/pdf?id=Hygab1rKDS> Quantum algorithms for deep convolutional neural networks. </a></b> <i>ICLR.</i> <code><b>[Quantum Neural Networks]</b></code> <code><b>[Quantum Convolutional Neural Networks]</b></code> <code><b>[Quantum Machine Learning]</b></code> <code><i>(Gate Model)</i></code></summary><pre><code>@inproceedings{kerenidis2019quantum,
  title={Quantum Algorithms for Deep Convolutional Neural Networks},
  author={Kerenidis, Iordanis and Landman, Jonas and Prakash, Anupam},
  booktitle={International Conference on Learning Representations},
  year={2019}
}</code></pre></details>
<details><summary>[Kjaergaard et al] <b><a href=https://www.annualreviews.org/doi/pdf/10.1146/annurev-conmatphys-031119-050605> Superconducting Qubits:
Current State of Play. </a></b> <i>Annual review of condensed matter physics.</i> <code><b>[Quantum Information and Computing]</b></code> <code><b>[Physical Realization of Qubits]</b></code> <code><b>[Superconducting Qubits]</b></code> </summary><pre><code>@article{kjaergaard2020superconducting,
  title={Superconducting qubits: Current state of play},
  author={Kjaergaard, Morten and Schwartz, Mollie E and Braum{\"u}ller, Jochen and Krantz, Philip and Wang, Joel I-J and Gustavsson, Simon and Oliver, William D},
  journal={Annual Review of Condensed Matter Physics},
  volume={11},
  pages={369--395},
  year={2020},
  publisher={Annual Reviews}
}</code></pre></details>
<details><summary>[Li et al] <b><a href=https://iopscience.iop.org/article/10.1088/2058-9565/ab9f93> A quantum deep convolutional neural network for image recognition. </a></b> <i>Quantum Science and Technology.</i> <code><b>[Quantum Convolutional Neural Networks]</b></code> <code><b>[Quantum Neural Networks]</b></code> <code><b>[Quantum Machine Learning]</b></code> </summary><pre><code>@article{li2020quantum,
  title={A quantum deep convolutional neural network for image recognition},
  author={Li, YaoChong and Zhou, Ri-Gui and Xu, RuQing and Luo, Jia and Hu, WenWen},
  journal={Quantum Science and Technology},
  volume={5},
  number={4},
  pages={044003},
  year={2020},
  publisher={IOP Publishing}
}</code></pre></details>
<details><summary>[Macaluso et al] <b><a href=https://link.springer.com/content/pdf/10.1007%2F978-3-030-50433-5_45.pdf> A Variational Algorithm for QuantumNeural Networks. </a></b> <i>International Conference on Computational Science.</i> <code><b>[Quantum Neural Networks]</b></code> <code><b>[Quantum Perceptron]</b></code> <code><b>[Quantum Machine Learning]</b></code> <code><i>(Gate Model)</i></code></summary><pre><code>@inproceedings{macaluso2020variational,
  title={A Variational Algorithm for Quantum Neural Networks},
  author={Macaluso, Antonio and Clissa, Luca and Lodi, Stefano and Sartori, Claudio},
  booktitle={International Conference on Computational Science},
  pages={591--604},
  year={2020},
  organization={Springer}
}</code></pre></details>
<details><summary>[Mari et al] <b><a href=https://quantum-journal.org/papers/q-2020-10-09-340/> Transfer learning in hybrid classical-quantum neural networks. </a></b> <i>Quantum.</i> <code><b>[Quantum Neural Networks]</b></code> <code><b>[Quantum Transfer Learning]</b></code> <code><b>[Quantum Machine Learning]</b></code> <code><i>(Gate Model)</i></code></summary><pre><code>@article{mari2020transfer,
  title={Transfer learning in hybrid classical-quantum neural networks},
  author={Mari, Andrea and Bromley, Thomas R and Izaac, Josh and Schuld, Maria and Killoran, Nathan},
  journal={Quantum},
  volume={4},
  pages={340},
  year={2020},
  publisher={Verein zur F{\"o}rderung des Open Access Publizierens in den Quantenwissenschaften}
}</code></pre></details>
<details><summary>[Marrero et al] <b><a href=https://arxiv.org/abs/2010.15968> Entanglement Induced Barren Plateaus. </a></b> <i>arXiv.</i> <code><b>[Barren Plateaus]</b></code> <code><b>[Parameterized Quantum Circuits]</b></code> <code><b>[Quantum Neural Networks]</b></code> <code><b>[Quantum Machine Learning]</b></code> </summary><pre><code>@article{marrero2020entanglement,
  title={Entanglement induced barren plateaus},
  author={Marrero, Carlos Ortiz and Kieferov{\'a}, M{\'a}ria and Wiebe, Nathan},
  journal={arXiv preprint arXiv:2010.15968},
  year={2020}
}</code></pre></details>
<details><summary>[McGeoch] <b><a href=https://www.sciencedirect.com/science/article/abs/pii/S0304397520300529> Theory versus practice in annealing-based quantum computing. </a></b> <i>Theoretical Computer Science.</i> <code><b>[Quantum Information and Computing]</b></code> <code><i>(Adiabatic Model)</i></code></summary><pre><code>@article{mcgeoch2020theory,
  title={Theory versus practice in annealing-based quantum computing},
  author={McGeoch, Catherine C},
  journal={Theoretical Computer Science},
  volume={816},
  pages={169--183},
  year={2020},
  publisher={Elsevier}
}</code></pre></details>
<details><summary>[Mezquita et al] <b><a href=https://link.springer.com/chapter/10.1007/978-3-030-53829-3_20> A Review of k-NN Algorithm Based on Classical and Quantum Machine Learning. </a></b> <i>DCAI 2020.</i> <code><b>[Quantum Machine Learning]</b></code> </summary><pre><code>@inproceedings{mezquita2020review,
  title={A Review of k-NN Algorithm Based on Classical and Quantum Machine Learning},
  author={Mezquita, Yeray and Alonso, Ricardo S and Casado-Vara, Roberto and Prieto, Javier and Corchado, Juan Manuel},
  booktitle={International Symposium on Distributed Computing and Artificial Intelligence},
  pages={189--198},
  year={2020},
  organization={Springer}
}</code></pre></details>
<details><summary>[Perez-Salinas et al] <b><a href=https://quantum-journal.org/papers/q-2020-02-06-226/> Data re-uploading for a universal quantum classifier. </a></b> <i>Quantum.</i> <code><b>[Quantum Neural Networks]</b></code> <code><b>[Quantum Classification]</b></code> <code><b>[Quantum Machine Learning]</b></code> <code><i>(Gate Model)</i></code></summary><pre><code>@article{perez2020data,
  title={Data re-uploading for a universal quantum classifier},
  author={P{\'e}rez-Salinas, Adri{\'a}n and Cervera-Lierta, Alba and Gil-Fuster, Elies and Latorre, Jos{\'e} I},
  journal={Quantum},
  volume={4},
  pages={226},
  year={2020},
  publisher={Verein zur F{\"o}rderung des Open Access Publizierens in den Quantenwissenschaften}
}</code></pre></details>
<details><summary>[Pesah et al] <b><a href=https://arxiv.org/abs/2011.02966> Absence of Barren Plateaus in Quantum Convolutional Neural Networks. </a></b> <i>arXiv.</i> <code><b>[Barren Plateaus]</b></code> <code><b>[Quantum Convolutional Neural Networks]</b></code> <code><b>[Parameterized Quantum Circuits]</b></code> <code><b>[Quantum Neural Networks]</b></code> <code><b>[Quantum Machine Learning]</b></code> </summary><pre><code>@article{pesah2020absence,
  title={Absence of barren plateaus in quantum convolutional neural networks},
  author={Pesah, Arthur and Cerezo, M and Wang, Samson and Volkoff, Tyler and Sornborger, Andrew T and Coles, Patrick J},
  journal={arXiv preprint arXiv:2011.02966},
  year={2020}
}</code></pre></details>
<details><summary>[Schuld et al] <b><a href=https://journals.aps.org/pra/abstract/10.1103/PhysRevA.101.032308> Circuit-centric quantum classifiers. </a></b> <i>Physical Review A.</i> <code><b>[Quantum Neural Networks]</b></code> <code><b>[Quantum Classification]</b></code> <code><b>[Quantum Machine Learning]</b></code> <code><i>(Gate Model)</i></code></summary><pre><code>@article{schuld2020circuit,
  title={Circuit-centric quantum classifiers},
  author={Schuld, Maria and Bocharov, Alex and Svore, Krysta M and Wiebe, Nathan},
  journal={Physical Review A},
  volume={101},
  number={3},
  pages={032308},
  year={2020},
  publisher={APS}
}</code></pre></details>
<details><summary>[Sharma et al] <b><a href=https://arxiv.org/abs/2005.12458> Trainability of Dissipative Perceptron-Based Quantum Neural Networks. </a></b> <i>arXiv.</i> <code><b>[Parameterized Quantum Circuits]</b></code> <code><b>[Quantum Neural Networks]</b></code> <code><b>[Quantum Machine Learning]</b></code> <code><b>[Barren Plateaus]</b></code> </summary><pre><code>@article{sharma2020trainability,
  title={Trainability of dissipative perceptron-based quantum neural networks},
  author={Sharma, Kunal and Cerezo, Marco and Cincio, Lukasz and Coles, Patrick J},
  journal={arXiv preprint arXiv:2005.12458},
  year={2020}
}</code></pre></details>
<details><summary>[Tacchino et al] <b><a href=https://ieeexplore.ieee.org/abstract/document/9364892> Variational Learning for Quantum Artificial Neural Networks. </a></b> <i>IEEE International Conference on Quantum Computing and Engineering.</i> <code><b>[Parameterized Quantum Circuits]</b></code> <code><b>[Quantum Neural Networks]</b></code> <code><b>[Quantum Machine Learning]</b></code> </summary><pre><code>@inproceedings{tacchino2020variational,
  title={Variational learning for quantum artificial neural networks},
  author={Tacchino, Francesco and Barkoutsos, Panagiotis Kl and Macchiavello, Chiara and Gerace, Dario and Tavernelli, Ivano and Bajoni, Daniele},
  booktitle={2020 IEEE International Conference on Quantum Computing and Engineering (QCE)},
  pages={130--136},
  year={2020},
  organization={IEEE}
}</code></pre></details>
<details><summary>[Wang et al] <b><a href=https://arxiv.org/abs/2007.14384> Noise-Induced Barren Plateaus in Variational Quantum Algorithms. </a></b> <i>arXiv.</i> <code><b>[Barren Plateaus]</b></code> <code><b>[Parameterized Quantum Circuits]</b></code> <code><b>[Variational Quantum Algorithms]</b></code> <code><b>[Quantum Machine Learning]</b></code> </summary><pre><code>@article{wang2020noise,
  title={Noise-induced barren plateaus in variational quantum algorithms},
  author={Wang, Samson and Fontana, Enrico and Cerezo, Marco and Sharma, Kunal and Sone, Akira and Cincio, Lukasz and Coles, Patrick J},
  journal={arXiv preprint arXiv:2007.14384},
  year={2020}
}</code></pre></details>
<details><summary>[Willsch et al] <b><a href=https://www.sciencedirect.com/science/article/pii/S001046551930342X> Support vector machines on the D-Wave quantum annealer. </a></b> <i>Computer Physics Communications.</i> <code><b>[Quantum Support Vector Machine]</b></code> <code><b>[Quantum Machine Learning]</b></code> <code><i>(Adiabatic Model)</i></code></summary><pre><code>@article{willsch2020support,
  title={Support vector machines on the D-Wave quantum annealer},
  author={Willsch, Dennis and Willsch, Madita and De Raedt, Hans and Michielsen, Kristel},
  journal={Computer physics communications},
  volume={248},
  pages={107006},
  year={2020},
  publisher={Elsevier}
}</code></pre></details>


### 2019
<details><summary>[Ajagekar & You                                                                                                                                                                                                                         ] <b><a href=https://doi.org/10.1016/j.energy.2019.04.186> Quantum computing for energy systems optimization: Challenges and opportunities. </a></b> <i>Energy.</i> <code><b>[Quantum Energy Systems Optimization]</b></code> <code><i>(Gate Model)</i></code> <code><i>(Adiabatic Model)</i></code></summary><pre><code>@Article{ajagekar2019quantum,
  Title                    = {Quantum computing for energy systems optimization: Challenges and opportunities},
  Author                   = {Ajagekar, Akshay and You, Fengqi},
  Journal                  = {Energy},
  Year                     = {2019},
  Pages                    = {76--89},
  Volume                   = {179},
  Publisher                = {Elsevier},
  URL                      = {https://doi.org/10.1016/j.energy.2019.04.186}
}
</code></pre></details>
<details><summary>[Arute et al] <b><a href=https://www.Nature.com/articles/s41586-019-1666-5> Quantum supremacy using a programmable superconducting processor. </a></b> <i>Nature.</i> <code><b>[Quantum Speedup-Advantage-Supremacy]</b></code> <code><b>[Superconducting Qubits]</b></code> <code><i>(Gate Model)</i></code></summary><pre><code>@Article{arute2019quantum,
  Title                    = {Quantum supremacy using a programmable superconducting processor},
  Author                   = {Arute, Frank and Arya, Kunal and Babbush, Ryan and Bacon, Dave and Bardin, Joseph C and Barends, Rami and Biswas, Rupak and Boixo, Sergio and Brandao, Fernando GSL and Buell, David A and others},
  Journal                  = {Nature},
  Year                     = {2019},
  Number                   = {7779},
  Pages                    = {505--510},
  Volume                   = {574},
  Publisher                = {Nature Publishing Group},
  URL                      = {https://www.Nature.com/articles/s41586-019-1666-5}
}
</code></pre></details>
<details><summary>[Brandão et al] <b><a href=https://arxiv.org/abs/1710.02581> Quantum SDP Solvers: Large Speed-ups, Optimality, and Applications to Quantum Learning. </a></b> <i>arXiv.</i> <code><b>[Quantum Speedup-Advantage-Supremacy]</b></code> <code><b>[Semidefinite programming]</b></code> </summary><pre><code>@Article{brandao2019QuantumSDP,
  Title                    = {Quantum SDP Solvers: Large Speed-ups, Optimality, and Applications to Quantum Learning},
  Author                   = {Brand{\~a}o, Fernando GSL and Kalev, Amir and Li, Tongyang and Lin, Cedric Yen-Yu and Svore, Krysta M and Wu, Xiaodi},
  Journal                  = {arXiv preprint arXiv:1710.02581},
  Year                     = {2019},
  Primaryclass             = {quant-ph},
  URL                      = {https://arxiv.org/abs/1710.02581}
}
</code></pre></details>
<details><summary>[Cong et al] <b><a href=https://www.nature.com/articles/s41567-019-0648-8.pdf> Quantum convolutional neural networks. </a></b> <i>Nature Physics.</i> <code><b>[Quantum Neural Networks]</b></code> <code><b>[Quantum Convolutional Neural Networks]</b></code> <code><b>[Quantum Machine Learning]</b></code> <code><i>(Gate Model)</i></code></summary><pre><code>@Article{cong2019quantum,
  Title                    = {Quantum convolutional neural networks},
  Author                   = {Cong, Iris and Choi, Soonwon and Lukin, Mikhail D},
  Journal                  = {Nature Physics},
  Year                     = {2019},
  Number                   = {12},
  Pages                    = {1273--1278},
  Volume                   = {15},
  Publisher                = {Nature Publishing Group},
  URL                      = {https://www.nature.com/articles/s41567-019-0648-8.pdf}
}
</code></pre></details>
<details><summary>[Grant et al] <b><a href=https://quantum-journal.org/papers/q-2019-12-09-214/> An initialization strategy for addressing barren plateaus in parametrized quantum circuits. </a></b> <i>Quantum.</i> <code><b>[Barren Plateaus]</b></code> <code><b>[Parameterized Quantum Circuits]</b></code> <code><b>[Quantum Neural Networks]</b></code> <code><b>[Quantum Machine Learning]</b></code> </summary><pre><code>@article{grant2019initialization,
  title={An initialization strategy for addressing barren plateaus in parametrized quantum circuits},
  author={Grant, Edward and Wossnig, Leonard and Ostaszewski, Mateusz and Benedetti, Marcello},
  journal={Quantum},
  volume={3},
  pages={214},
  year={2019},
  publisher={Verein zur F{\"o}rderung des Open Access Publizierens in den Quantenwissenschaften}
}</code></pre></details>
<details><summary>[Gyongyosi & Imre] <b><a href=https://www.sciencedirect.com/science/article/abs/pii/S1574013718301709> A Survey on quantum computing technology. </a></b> <i>Computer Science Review.</i> <code><b>[Quantum Information and Computing]</b></code> </summary><pre><code>@Article{gyongyosi2019survey,
  Title                    = {A survey on quantum computing technology},
  Author                   = {Gyongyosi, Laszlo and Imre, Sandor},
  Journal                  = {Computer Science Review},
  Year                     = {2019},
  Pages                    = {51--71},
  Volume                   = {31},
  Publisher                = {Elsevier},
  URL                      = {https://www.sciencedirect.com/science/article/abs/pii/S1574013718301709}
}
</code></pre></details>
<details><summary>[Havlíček et al] <b><a href=https://www.nature.com/articles/s41586-019-0980-2> Supervised learning with quantum-enhanced feature spaces. </a></b> <i>Nature.</i> <code><b>[Variational Quantum Algorithms]</b></code> <code><b>[Quantum Support Vector Machine]</b></code> <code><b>[Quantum Classification]</b></code> <code><b>[Quantum Machine Learning]</b></code> <code><i>(Gate Model)</i></code></summary><pre><code>@Article{havlivcek2019supervised,
  Title                    = {Supervised learning with quantum-enhanced feature spaces},
  Author                   = {Havl{\'\i}{\v{c}}ek, Vojt{\v{e}}ch and C{\'o}rcoles, Antonio D and Temme, Kristan and Harrow, Aram W and Kandala, Abhinav and Chow, Jerry M and Gambetta, Jay M},
  Journal                  = {Nature},
  Year                     = {2019},
  Number                   = {7747},
  Pages                    = {209--212},
  Volume                   = {567},
  Publisher                = {Nature Publishing Group},
  URL                      = {https://www.nature.com/articles/s41586-019-0980-2}
}
</code></pre></details>
<details><summary>[Killoran et al] <b><a href=https://journals.aps.org/prresearch/abstract/10.1103/PhysRevResearch.1.033063> Continuous-variable quantum neural networks. </a></b> <i>Physical Review Research.</i> <code><b>[Quantum Neural Networks]</b></code> <code><b>[Quantum Machine Learning]</b></code> <code><i>(Gate Model)</i></code></summary><pre><code>@article{killoran2019continuous,
  title={Continuous-variable quantum neural networks},
  author={Killoran, Nathan and Bromley, Thomas R and Arrazola, Juan Miguel and Schuld, Maria and Quesada, Nicol{\'a}s and Lloyd, Seth},
  journal={Physical Review Research},
  volume={1},
  number={3},
  pages={033063},
  year={2019},
  publisher={APS}
}</code></pre></details>
<details><summary>[Krantz et al] <b><a href=https://aip.scitation.org/doi/full/10.1063/1.5089550> A quantum engineer's guide to superconducting qubits. </a></b> <i>Applied Physics Reviews .</i> <code><b>[Physical Realization of Qubits]</b></code> <code><b>[Superconducting Qubits]</b></code> </summary><pre><code>@article{krantz2019quantum,
  title={A quantum engineer's guide to superconducting qubits},
  author={Krantz, Philip and Kjaergaard, Morten and Yan, Fei and Orlando, Terry P and Gustavsson, Simon and Oliver, William D},
  journal={Applied Physics Reviews},
  volume={6},
  number={2},
  pages={021318},
  year={2019},
  publisher={AIP Publishing LLC}
}</code></pre></details>
<details><summary>[Liu et al] <b><a href=https://ieeexplore.ieee.org/document/8631025> A Unitary Weights Based One-Iteration Quantum Perceptron Algorithm for Non-Ideal Training Sets. </a></b> <i>IEEE Access.</i> <code><b>[Quantum Perceptron]</b></code> <code><b>[Quantum Machine Learning]</b></code> <code><i>(Gate Model)</i></code></summary><pre><code>@article{liu2019unitary,
  title={A unitary weights based one-iteration quantum perceptron algorithm for non-ideal training sets},
  author={Liu, Wenjie and Gao, Peipei and Wang, Yuxiang and Yu, Wenbin and Zhang, Maojun},
  journal={IEEE Access},
  volume={7},
  pages={36854--36865},
  year={2019},
  publisher={IEEE}
}</code></pre></details>
<details><summary>[Savchuk & Fesenko] <b><a href=https://link.springer.com/content/pdf/10.1007/s10559-019-00107-w.pdf> Quantum Computing: Survey and Analysis. </a></b> <i>Cybernetics and Systems Analysis.</i> <code><b>[Quantum Information and Computing]</b></code> </summary><pre><code>@article{savchuk2019quantum,
  title={Quantum Computing: Survey and Analysis},
  author={Savchuk, MM and Fesenko, AV},
  journal={Cybernetics and Systems Analysis},
  volume={55},
  number={1},
  pages={10--21},
  year={2019},
  publisher={Springer}
}</code></pre></details>
<details><summary>[Schuld & Killoran] <b><a href=https://journals.aps.org/prl/abstract/10.1103/PhysRevLett.122.040504> Quantum machine learning in feature Hilbert spaces. </a></b> <i>Physical Review Letters.</i> <code><b>[Quantum Machine Learning]</b></code> <code><b>[Quantum Information and Computing]</b></code> </summary><pre><code>@article{schuld2019quantum,
  title={Quantum machine learning in feature Hilbert spaces},
  author={Schuld, Maria and Killoran, Nathan},
  journal={Physical review letters},
  volume={122},
  number={4},
  pages={040504},
  year={2019},
  publisher={APS}
}</code></pre></details>
<details><summary>[Sim et al] <b><a href=https://onlinelibrary.wiley.com/doi/10.1002/qute.201900070> Expressibility and Entangling Capability of Parameterized Quantum Circuits for Hybrid Quantum-Classical Algorithms. </a></b> <i>Advanced Quantum Technologies.</i> <code><b>[Variational Quantum Algorithms]</b></code> <code><b>[Parameterized Quantum Circuits]</b></code> <code><i>(Gate Model)</i></code></summary><pre><code>@article{sim2019expressibility,
  title={Expressibility and entangling capability of parameterized quantum circuits for hybrid quantum-classical algorithms},
  author={Sim, Sukin and Johnson, Peter D and Aspuru-Guzik, Al{\'a}n},
  journal={Advanced Quantum Technologies},
  volume={2},
  number={12},
  pages={1900070},
  year={2019},
  publisher={Wiley Online Library}
}</code></pre></details>
<details><summary>[Tacchino et al] <b><a href=https://www.Nature.com/articles/s41534-019-0140-4.pdf> An artificial neuron implemented on an actual quantumprocessor. </a></b> <i>Nature.</i> <code><b>[Quantum Perceptron]</b></code> <code><b>[Quantum Machine Learning]</b></code> <code><b>[Quantum Information and Computing]</b></code> <code><i>(Gate Model)</i></code></summary><pre><code>@article{tacchino2019artificial,
  title={An artificial neuron implemented on an actual quantum processor},
  author={Tacchino, Francesco and Macchiavello, Chiara and Gerace, Dario and Bajoni, Daniele},
  journal={npj Quantum Information},
  volume={5},
  number={1},
  pages={1--8},
  year={2019},
  publisher={Nature Publishing Group}
}</code></pre></details>
<details><summary>[Torrontegui et al] <b><a href=https://iopscience.iop.org/article/10.1209/0295-5075/125/30004/pdf> Unitary quantum perceptron as efficient universal approximator. </a></b> <i>Europhysics Letters.</i> <code><b>[Quantum Perceptron]</b></code> <code><b>[Quantum Machine Learning]</b></code> <code><b>[Quantum Information and Computing]</b></code> </summary><pre><code>@article{torrontegui2019unitary,
  title={Unitary quantum perceptron as efficient universal approximator},
  author={Torrontegui, Erik and Garc{\'\i}a-Ripoll, Juan Jos{\'e}},
  journal={EPL (Europhysics Letters)},
  volume={125},
  number={3},
  pages={30004},
  year={2019},
  publisher={IOP Publishing}
}</code></pre></details>
<details><summary>[Verdon et al] <b><a href=https://arxiv.org/abs/1907.05415> Learning to learn with quantum neural networks via classical neural networks. </a></b> <i>arXiv.</i> <code><b>[Quantum Perceptron]</b></code> <code><b>[Parameterized Quantum Circuits]</b></code> <code><b>[Quantum Neural Networks]</b></code> <code><b>[Quantum Machine Learning]</b></code> <code><b>[Variational Quantum Algorithms]</b></code> </summary><pre><code>@article{verdon2019learning,
  title={Learning to learn with quantum neural networks via classical neural networks},
  author={Verdon, Guillaume and Broughton, Michael and McClean, Jarrod R and Sung, Kevin J and Babbush, Ryan and Jiang, Zhang and Neven, Hartmut and Mohseni, Masoud},
  journal={arXiv preprint arXiv:1907.05415},
  year={2019}
}</code></pre></details>
<details><summary>[Verdon et al] <b><a href=https://arxiv.org/abs/1909.12264> Quantum graph neural networks. </a></b> <i>arXiv.</i> <code><b>[Quantum Neural Networks]</b></code> <code><b>[Quantum Machine Learning]</b></code> <code><i>(Gate Model)</i></code></summary><pre><code>@article{verdon2019quantum,
  title={Quantum graph neural networks},
  author={Verdon, Guillaume and McCourt, Trevor and Luzhnica, Enxhell and Singh, Vikash and Leichenauer, Stefan and Hidary, Jack},
  journal={arXiv preprint arXiv:1909.12264},
  year={2019}
</code></pre></details>
<details><summary>[Wang et al ] <b><a href=https://journals.aps.org/prl/abstract/10.1103/PhysRevLett.122.140504> Accelerated Variational Quantum Eigensolver. </a></b> <i>Physical review letters.</i> <code><b>[Variational Quantum Algorithms]</b></code> </summary><pre><code>@article{wang2019accelerated,
  title={Accelerated variational quantum eigensolver},
  author={Wang, Daochen and Higgott, Oscar and Brierley, Stephen},
  journal={Physical review letters},
  volume={122},
  number={14},
  pages={140504},
  year={2019},
  publisher={APS}
}</code></pre></details>
<details><summary>[Wiebe & Wossnig] <b><a href=https://arxiv.org/abs/1905.09902> Generative training of quantum Boltzmann machines with hidden units. </a></b> <i>arXiv.</i> <code><b>[Quantum Boltzmann machines]</b></code> <code><b>[Quantum Machine Learning]</b></code> </summary><pre><code>@article{wiebe2019generative,
  title={Generative training of quantum Boltzmann machines with hidden units},
  author={Wiebe, Nathan and Wossnig, Leonard},
  journal={arXiv preprint arXiv:1905.09902},
  year={2019}
}</code></pre></details>
<details><summary>[Wiersema et al] <b><a href=https://journals.aps.org/pra/abstract/10.1103/PhysRevA.100.020301> Implementing perceptron models with qubits. </a></b> <i>Physical Review A.</i> <code><b>[Quantum Perceptron]</b></code> <code><b>[Quantum Machine Learning]</b></code> <code><b>[Quantum Information and Computing]</b></code> <code><i>(Gate Model)</i></code></summary><pre><code>@article{wiersema2019implementing,
  title={Implementing perceptron models with qubits},
  author={Wiersema, RC and Kappen, HJ},
  journal={Physical Review A},
  volume={100},
  number={2},
  pages={020301},
  year={2019},
  publisher={APS}
}</code></pre></details>
<details><summary>[Zoufal et al] <b><a href=https://www.nature.com/articles/s41534-019-0223-2> Quantum Generative Adversarial Networks for Learning and Loading Random
Distributions. </a></b> <i>npj Quantum Information.</i> <code><b>[Quantum Generative Adversarial Networks]</b></code> <code><b>[Variational Quantum Algorithms]</b></code> <code><b>[Quantum Machine Learning]</b></code> <code><i>(Gate Model)</i></code></summary><pre><code>@article{zoufal2019quantum,
  title={Quantum generative adversarial networks for learning and loading random distributions},
  author={Zoufal, Christa and Lucchi, Aur{\'e}lien and Woerner, Stefan},
  journal={npj Quantum Information},
  volume={5},
  number={1},
  pages={1--9},
  year={2019},
  publisher={Nature Publishing Group}
}</code></pre></details>


### 2018
<details><summary>[Albash & Lidar] <b><a href=https://journals.aps.org/rmp/abstract/10.1103/RevModPhys.90.015002> Adiabatic Quantum Computing. </a></b> <i>Reviews of Modern Physics.</i> <code><b>[Quantum Information and Computing]</b></code> <code><b>[Adiabatic Quantum Computing]</b></code> <code><i>(Adiabatic Model)</i></code></summary><pre><code>@Article{albash2018adiabatic,
  Title                    = {Adiabatic quantum computation},
  Author                   = {Albash, Tameem and Lidar, Daniel A},
  Journal                  = {Reviews of Modern Physics},
  Year                     = {2018},
  Number                   = {1},
  Pages                    = {015002},
  Volume                   = {90},
  Publisher                = {APS},
  URL                      = {https://journals.aps.org/rmp/abstract/10.1103/RevModPhys.90.015002}
}
</code></pre></details>
<details><summary>[Amin et al] <b><a href=https://journals.aps.org/prx/abstract/10.1103/PhysRevX.8.021050> Quantum boltzmann machine. </a></b> <i>Physical Review X.</i> <code><b>[Quantum Boltzmann machines]</b></code> <code><b>[Quantum Machine Learning]</b></code> <code><i>(Gate Model)</i></code> <code><i>(Adiabatic Model)</i></code></summary><pre><code>@Article{amin2018quantum,
  Title                    = {Quantum boltzmann machine},
  Author                   = {Amin, Mohammad H and Andriyash, Evgeny and Rolfe, Jason and Kulchytskyy, Bohdan and Melko, Roger},
  Journal                  = {Physical Review X},
  Year                     = {2018},
  Number                   = {2},
  Pages                    = {021050},
  Volume                   = {8},
  Publisher                = {APS},
  URL                      = {https://journals.aps.org/prx/abstract/10.1103/PhysRevX.8.021050}
}
</code></pre></details>
<details><summary>[Boixo et al] <b><a href=https://www.nature.com/articles/s41567-018-0124-x> Characterizing quantum supremacy in near-term devices. </a></b> <i>Nature Physics.</i> <code><b>[Quantum Speedup-Advantage-Supremacy]</b></code> </summary><pre><code>@Article{boixo2018characterizing,
  Title                    = {Characterizing quantum supremacy in near-term devices},
  Author                   = {Boixo, Sergio and Isakov, Sergei V and Smelyanskiy, Vadim N and Babbush, Ryan and Ding, Nan and Jiang, Zhang and Bremner, Michael J and Martinis, John M and Neven, Hartmut},
  Journal                  = {Nature Physics},
  Year                     = {2018},
  Number                   = {6},
  Pages                    = {595--600},
  Volume                   = {14},
  Publisher                = {Nature Publishing Group},
  URL                      = {https://www.nature.com/articles/s41567-018-0124-x}
}
</code></pre></details>
<details><summary>[Dallaire-Demers & Killoran] <b><a href=https://journals.aps.org/pra/abstract/10.1103/PhysRevA.98.012324> Quantum generative adversarial networks. </a></b> <i>Physical Review A.</i> <code><b>[Quantum Generative Adversarial Networks]</b></code> <code><b>[Quantum Neural Networks]</b></code> <code><b>[Variational Quantum Algorithms]</b></code> <code><b>[Quantum Machine Learning]</b></code> <code><i>(Gate Model)</i></code></summary><pre><code>@article{dallaire2018quantum,
  title={Quantum generative adversarial networks},
  author={Dallaire-Demers, Pierre-Luc and Killoran, Nathan},
  journal={Physical Review A},
  volume={98},
  number={1},
  pages={012324},
  year={2018},
  publisher={APS}
}</code></pre></details>
<details><summary>[Du et al] <b><a href=https://arxiv.org/abs/1809.06056> Implementable quantum classifier for nonlinear data. </a></b> <i>arXiv.</i> <code><b>[Quantum Perceptron]</b></code> <code><b>[Quantum Machine Learning]</b></code> <code><b>[Quantum Classification]</b></code> <code><b>[Quantum Machine Learning]</b></code> <code><i>(Gate Model)</i></code></summary><pre><code>@Article{du2018implementable,
  Title                    = {Implementable quantum classifier for nonlinear data},
  Author                   = {Du, Yuxuan and Hsieh, Min-Hsiu and Liu, Tongliang and Tao, Dacheng},
  Journal                  = {arXiv preprint arXiv:1809.06056},
  Year                     = {2018},
  URL                      = {https://arxiv.org/abs/1809.06056}
}
</code></pre></details>
<details><summary>[Dunjko & Briegel] <b><a href=https://iopscience.iop.org/article/10.1088/1361-6633/aab406/meta> Machine learning & artificial intelligence in the quantum domain: a review of recent progress. </a></b> <i>Reports on Progress in Physics.</i> <code><b>[Quantum Machine Learning]</b></code> <code><b>[Probably Approximately Correct Learning]</b></code> <code><b>[Quantum Information and Computing]</b></code> <code><i>(Gate Model)</i></code> <code><i>(Adiabatic Model)</i></code></summary><pre><code>@Article{dunjko2018machine,
  Title                    = {Machine learning \& artificial intelligence in the quantum domain: a review of recent progress},
  Author                   = {Dunjko, Vedran and Briegel, Hans J},
  Journal                  = {Reports on Progress in Physics},
  Year                     = {2018},
  Number                   = {7},
  Pages                    = {074001},
  Volume                   = {81},
  Publisher                = {IOP Publishing},
  URL                      = {https://iopscience.iop.org/article/10.1088/1361-6633/aab406/meta}
}
</code></pre></details>
<details><summary>[Fahri & Neven] <b><a href=https://arxiv.org/abs/1802.06002> Classification with Quantum Neural Networks on Near Term Processors. </a></b> <i>arXiv.</i> <code><b>[Quantum Neural Networks]</b></code> <code><b>[Quantum Machine Learning]</b></code> <code><b>[Quantum Classification]</b></code> <code><i>(Gate Model)</i></code></summary><pre><code>@Article{farhi2018classification,
  Title                    = {Classification with quantum neural networks on near term processors},
  Author                   = {Farhi, Edward and Neven, Hartmut},
  Journal                  = {arXiv preprint arXiv:1802.06002},
  Year                     = {2018},
  URL                      = {https://arxiv.org/abs/1802.06002}
}
</code></pre></details>
<details><summary>[Grant et al] <b><a href=https://www.nature.com/articles/s41534-018-0116-9> Hierarchical quantum classifiers. </a></b> <i>npj Quantum information.</i> <code><b>[Quantum Neural Networks]</b></code> <code><b>[Quantum Machine Learning]</b></code> <code><b>[Quantum Classification]</b></code> <code><i>(Gate Model)</i></code></summary><pre><code>@Article{grant2018hierarchical,
  Title                    = {Hierarchical quantum classifiers},
  Author                   = {Grant, Edward and Benedetti, Marcello and Cao, Shuxiang and Hallam, Andrew and Lockhart, Joshua and Stojevic, Vid and Green, Andrew G and Severini, Simone},
  Journal                  = {npj Quantum Information},
  Year                     = {2018},
  Number                   = {1},
  Pages                    = {1--8},
  Volume                   = {4},
  Publisher                = {Nature Publishing Group},
  URL                      = {https://www.nature.com/articles/s41534-018-0116-9}
}
</code></pre></details>
<details><summary>[Hu Wei] <b><a href=https://www.scirp.org/pdf/NS_2018031517004350.pdf> Towards a real quantum neuron. </a></b> <i>Natural Science.</i> <code><b>[Quantum Perceptron]</b></code> <code><b>[Quantum Machine Learning]</b></code> <code><i>(Gate Model)</i></code></summary><pre><code>@article{hu2018towards,
  title={Towards a real quantum neuron},
  author={Hu, Wei},
  journal={Natural Science},
  volume={10},
  number={3},
  pages={99--109},
  year={2018},
  publisher={Scientific Research Publishing}
}</code></pre></details>
<details><summary>[Katzgraber et al] <b><a href=https://iopscience.iop.org/article/10.1088/2058-9565/aab6ba> Viewing vanilla quantum annealing through spin glasses. </a></b> <i>Quantum Science and Technology.</i> <code><b>[Quantum Annealing]</b></code> <code><i>(Adiabatic Model)</i></code></summary><pre><code>@article{katzgraber2018viewing,
  title={Viewing vanilla quantum annealing through spin glasses},
  author={Katzgraber, Helmut G},
  journal={Quantum Science and Technology},
  volume={3},
  number={3},
  pages={030505},
  year={2018},
  publisher={IOP Publishing}
}</code></pre></details>
<details><summary>[McClean et al] <b><a href=https://www.nature.com/articles/s41467-018-07090-4> Barren plateaus in quantum neural network training landscapes. </a></b> <i>Nature communications.</i> <code><b>[Barren Plateaus]</b></code> <code><b>[Parameterized Quantum Circuits]</b></code> <code><b>[Quantum Neural Networks]</b></code> <code><b>[Quantum Machine Learning]</b></code> </summary><pre><code>@article{mcclean2018barren,
  title={Barren plateaus in quantum neural network training landscapes},
  author={McClean, Jarrod R and Boixo, Sergio and Smelyanskiy, Vadim N and Babbush, Ryan and Neven, Hartmut},
  journal={Nature communications},
  volume={9},
  number={1},
  pages={1--6},
  year={2018},
  publisher={Nature Publishing Group}
}</code></pre></details>
<details><summary>[Mitarai et al] <b><a href=https://journals.aps.org/pra/abstract/10.1103/PhysRevA.98.032309> Quantum circuit learning. </a></b> <i>Physical Review A.</i> <code><b>[Parameterized Quantum Circuits]</b></code> <code><b>[Quantum Neural Networks]</b></code> <code><b>[Quantum Machine Learning]</b></code> </summary><pre><code>@article{mitarai2018quantum,
  title={Quantum circuit learning},
  author={Mitarai, Kosuke and Negoro, Makoto and Kitagawa, Masahiro and Fujii, Keisuke},
  journal={Physical Review A},
  volume={98},
  number={3},
  pages={032309},
  year={2018},
  publisher={APS}
}</code></pre></details>
<details><summary>[Moll et al] <b><a href=https://iopscience.iop.org/article/10.1088/2058-9565/aab822/pdf> Quantum optimization using variational algorithms on near-term quantum devices. </a></b> <i>Quantum Science and Technology.</i> <code><b>[Variational Quantum Algorithms]</b></code> <code><b>[Quantum Optimization]</b></code> <code><b>[Quantum Machine Learning]</b></code> <code><b>[Quantum Information and Computing]</b></code> <code><i>(Gate Model)</i></code></summary><pre><code>@Article{moll2018quantum,
  Title                    = {Quantum optimization using variational algorithms on near-term quantum devices},
  Author                   = {Moll, Nikolaj and Barkoutsos, Panagiotis and Bishop, Lev S and Chow, Jerry M and Cross, Andrew and Egger, Daniel J and Filipp, Stefan and Fuhrer, Andreas and Gambetta, Jay M and Ganzhorn, Marc and others},
  Journal                  = {Quantum Science and Technology},
  Year                     = {2018},
  Number                   = {3},
  Pages                    = {030503},
  Volume                   = {3},
  Publisher                = {IOP Publishing},
  URL                      = {https://iopscience.iop.org/article/10.1088/2058-9565/aab822/meta}
}
</code></pre></details>
<details><summary>[Neukart et al] <b><a href=https://www.frontiersin.org/articles/10.3389/fphy.2017.00071/full> Quantum-enhanced reinforcement learning for finite-episode games with discrete state
spaces. </a></b> <i>Frontiers in physics.</i> <code><b>[Quantum Reinforcement Learning]</b></code> <code><b>[Quantum Machine Learning]</b></code> <code><i>(Adiabatic Model)</i></code></summary><pre><code>@article{neukart2018quantum,
  title={Quantum-enhanced reinforcement learning for finite-episode games with discrete state spaces},
  author={Neukart, Florian and Von Dollen, David and Seidel, Christian and Compostella, Gabriele},
  journal={Frontiers in physics},
  volume={5},
  pages={71},
  year={2018},
  publisher={Frontiers}
}</code></pre></details>
<details><summary>[Preskill] <b><a href=https://quantum-journal.org/papers/q-2018-08-06-79/pdf/> Quantum Computing in the NISQ era and beyond. </a></b> <i>Quantum Journal.</i> <code><b>[Quantum Information and Computing]</b></code> </summary><pre><code>@article{preskill2018quantum,
  title={Quantum computing in the NISQ era and beyond},
  author={Preskill, John},
  journal={Quantum},
  volume={2},
  pages={79},
  year={2018},
  publisher={Verein zur F{\"o}rderung des Open Access Publizierens in den Quantenwissenschaften}
}</code></pre></details>
<details><summary>[Soloviev et al] <b><a href=https://aip.scitation.org/doi/10.1063/1.5042147> Adiabatic superconducting artificial neural network: Basic cells. </a></b> <i>Journal of Applied Physics.</i> <code><b>[Physical Realization of Qubits]</b></code> <code><i>(Adiabatic Model)</i></code></summary><pre><code>@article{soloviev2018adiabatic,
  title={Adiabatic superconducting artificial neural network: Basic cells},
  author={Soloviev, Igor I and Schegolev, Andrey E and Klenov, Nikolay V and Bakurskiy, Sergey V and Kupriyanov, Mikhail Yu and Tereshonok, Maxim V and Shadrin, Anton V and Stolyarov, Vasily S and Golubov, Alexander A},
  journal={Journal of Applied Physics},
  volume={124},
  number={15},
  pages={152113},
  year={2018},
  publisher={AIP Publishing LLC}
}</code></pre></details>
<details><summary>[Wiebe et al] <b><a href=https://papers.nips.cc/paper/2016/file/d47268e9db2e9aa3827bba3afb7ff94a-Paper.pdf> Quantum Perceptron Models. </a></b> <i>NIPS .</i> <code><b>[Quantum Perceptron]</b></code> <code><b>[Quantum Machine Learning]</b></code> <code><b>[Quantum Information and Computing]</b></code> <code><i>(Gate Model)</i></code></summary><pre><code>@article{wiebe2016quantum,
  title={Quantum perceptron models},
  author={Wiebe, Nathan and Kapoor, Ashish and Svore, Krysta M},
  journal={arXiv preprint arXiv:1602.04799},
  year={2016}
}</code></pre></details>


### 2017
<details><summary>[Andriyash et al] <b><a href=https://arxiv.org/abs/1703.09277> Can quantum Monte Carlo simulate quantum annealing?. </a></b> <i>arXiv.</i> <code><b>[Adiabatic Quantum Computing]</b></code> <code><b>[Quantum Annealing]</b></code> <code><b>[Quantum Monte Carlo]</b></code> <code><i>(Adiabatic Model)</i></code></summary><pre><code>@Article{andriyash2017can,
  Title                    = {Can quantum Monte Carlo simulate quantum annealing?},
  Author                   = {Andriyash, Evgeny and Amin, Mohammad H},
  Journal                  = {arXiv preprint arXiv:1703.09277},
  Year                     = {2017},
  URL                      = {https://arxiv.org/abs/1703.09277}
}
</code></pre></details>
<details><summary>[Biamonte et al] <b><a href=https://www.Nature.com/articles/Nature23474> Quantum machine learning. </a></b> <i>Nature.</i> <code><b>[Quantum Machine Learning]</b></code> <code><i>(Gate Model)</i></code> <code><i>(Adiabatic Model)</i></code></summary><pre><code>@Article{biamonte2017quantum,
  Title                    = {Quantum machine learning},
  Author                   = {Biamonte, Jacob and Wittek, Peter and Pancotti, Nicola and Rebentrost, Patrick and Wiebe, Nathan and Lloyd, Seth},
  Journal                  = {Nature},
  Year                     = {2017},
  Number                   = {7671},
  Pages                    = {195--202},
  Volume                   = {549},
  Publisher                = {Nature Publishing Group},
  URL                      = {https://www.Nature.com/articles/Nature23474}
}
</code></pre></details>
<details><summary>[Bremner et al] <b><a href=https://doi.org/10.22331/q-2017-04-25-8> Achieving quantum supremacy with sparse and noisy commuting quantum computations. </a></b> <i>Quantum.</i> <code><b>[Quantum Speedup-Advantage-Supremacy]</b></code> <code><i>(Gate Model)</i></code> <code><i>(Adiabatic Model)</i></code></summary><pre><code>@Article{bremner2017achieving,
  Title                    = {Achieving quantum supremacy with sparse and noisy commuting quantum computations},
  Author                   = {Bremner, Michael J and Montanaro, Ashley and Shepherd, Dan J},
  Journal                  = {Quantum},
  Year                     = {2017},
  Pages                    = {8},
  Volume                   = {1},
  DOI                      = {https://doi.org/10.22331/q-2017-04-25-8},
  Publisher                = {Verein zur F{\"o}rderung des Open Access Publizierens in den Quantenwissenschaften}
}
</code></pre></details>
<details><summary>[Cao et al] <b><a href=https://arxiv.org/pdf/1711.11240.pdf> Quantum Neuron:  an elementary building block for machine learningon quantum computers. </a></b> <i>arXiv.</i> <code><b>[Quantum Perceptron]</b></code> <code><b>[Quantum Information and Computing]</b></code> <code><b>[Quantum Neural Networks]</b></code> <code><b>[Quantum Machine Learning]</b></code> <code><i>(Gate Model)</i></code></summary><pre><code>@Article{cao2017quantum,
  Title                    = {Quantum neuron: an elementary building block for machine learning on quantum computers},
  Author                   = {Cao, Yudong and Guerreschi, Gian Giacomo and Aspuru-Guzik, Al{\'a}n},
  Journal                  = {arXiv preprint arXiv:1711.11240},
  Year                     = {2017},
  URL                      = {https://arxiv.org/pdf/1711.11240.pdf}
}
</code></pre></details>
<details><summary>[Gambetta et al] <b><a href=https://www.nature.com/articles/s41534-016-0004-0> Building logical qubits in a superconducting quantum computing system. </a></b> <i>Physical Review Letters.</i> <code><b>[Physical Realization of Qubits]</b></code> <code><b>[Superconducting Qubits]</b></code> </summary><pre><code>@Article{gambetta2017building,
  Title                    = {Building logical qubits in a superconducting quantum computing system},
  Author                   = {Gambetta, Jay M and Chow, Jerry M and Steffen, Matthias},
  Journal                  = {npj Quantum Information},
  Year                     = {2017},
  Number                   = {1},
  Pages                    = {1--7},
  Volume                   = {3},
  Publisher                = {Nature Publishing Group},
  URL                      = {https://www.nature.com/articles/s41534-016-0004-0}
}
</code></pre></details>
<details><summary>[Harrow & Montanaro] <b><a href=https://www.Nature.com/articles/Nature23458.pdf> Quantum computational supremacy. </a></b> <i>Nature.</i> <code><b>[Quantum Information and Computing]</b></code> </summary><pre><code>@article{harrow2017quantum,
  title={Quantum computational supremacy},
  author={Harrow, Aram W and Montanaro, Ashley},
  journal={Nature},
  volume={549},
  number={7671},
  pages={203--209},
  year={2017},
  publisher={Nature Publishing Group}
}</code></pre></details>
<details><summary>[Kandala et al ] <b><a href=https://www.nature.com/articles/nature23879?sf114016447=1> Hardware-efficient variational quantum eigensolver for small molecules and quantum magnets. </a></b> <i>Nature.</i> <code><b>[Variational Quantum Algorithms]</b></code> </summary><pre><code>@article{kandala2017hardware,
  title={Hardware-efficient variational quantum eigensolver for small molecules and quantum magnets},
  author={Kandala, Abhinav and Mezzacapo, Antonio and Temme, Kristan and Takita, Maika and Brink, Markus and Chow, Jerry M and Gambetta, Jay M},
  journal={Nature},
  volume={549},
  number={7671},
  pages={242--246},
  year={2017},
  publisher={Nature Publishing Group}
}</code></pre></details>
<details><summary>[Mandra' et al] <b><a href=https://iopscience.iop.org/article/10.1088/2058-9565/aa7877> The pitfalls of planar spin-glass benchmarks: raising the bar for quantum annealers (again). </a></b> <i>Quantum Science and Technology.</i> <code><b>[Quantum Annealing]</b></code> <code><b>[Quantum Speedup-Advantage-Supremacy]</b></code> <code><i>(Adiabatic Model)</i></code></summary><pre><code>@article{mandra2017pitfalls,
  title={The pitfalls of planar spin-glass benchmarks: raising the bar for quantum annealers (again)},
  author={Mandra, Salvatore and Katzgraber, Helmut G and Thomas, Creighton},
  journal={Quantum Science and Technology},
  volume={2},
  number={3},
  pages={038501},
  year={2017},
  publisher={IOP Publishing}
}</code></pre></details>
<details><summary>[Romero et al] <b><a href=https://arxiv.org/pdf/1612.02806.pdf> Quantum autoencoders for efficient compression of quantum data. </a></b> <i>Quantum Science and Technology.</i> <code><b>[Quantum Neural Networks]</b></code> <code><b>[Quantum Autoencoders]</b></code> <code><b>[Quantum Machine Learning]</b></code> <code><i>(Gate Model)</i></code></summary><pre><code>@article{romero2017quantum,
  title={Quantum autoencoders for efficient compression of quantum data},
  author={Romero, Jonathan and Olson, Jonathan P and Aspuru-Guzik, Alan},
  journal={Quantum Science and Technology},
  volume={2},
  number={4},
  pages={045001},
  year={2017},
  publisher={IOP Publishing}
}</code></pre></details>
<details><summary>[Ruan et al] <b><a href=https://idp.springer.com/authorize/casa?redirect_uri=https://link.springer.com/article/10.1007/s10773-017-3514-4&casa_token=CO3V9sMtWRcAAAAA:9hw3GSs7B7yTOJZoSEACk9y4uoIrHM2RuXAeY_ThKvnmvCTLd5lkn-ova5xZ4Do4xVczdIiy_YTuI8JC> Quantum algorithm for k-nearest neighbors classification based on the metric of hamming distance. </a></b> <i>International Journal of Theoretical Physics.</i> <code><b>[Quantum Machine Learning]</b></code> <code><b>[Quantum Classification]</b></code> </summary><pre><code>@article{ruan2017quantum,
  title={Quantum algorithm for k-nearest neighbors classification based on the metric of hamming distance},
  author={Ruan, Yue and Xue, Xiling and Liu, Heng and Tan, Jianing and Li, Xi},
  journal={International Journal of Theoretical Physics},
  volume={56},
  number={11},
  pages={3496--3507},
  year={2017},
  publisher={Springer}
}</code></pre></details>
<details><summary>[Wan et al] <b><a href=https://www.nature.com/articles/s41534-017-0032-4> Quantum generalisation of feedforward neural networks. </a></b> <i>npj Quantum information.</i> <code><b>[Quantum Neural Networks]</b></code> <code><b>[Quantum Autoencoders]</b></code> <code><b>[Quantum Machine Learning]</b></code> <code><i>(Gate Model)</i></code></summary><pre><code>@Article{wan2017quantum,
  Title                    = {Quantum generalisation of feedforward neural networks},
  Author                   = {Wan, Kwok Ho and Dahlsten, Oscar and Kristj{\'a}nsson, Hl{\'e}r and Gardner, Robert and Kim, MS},
  Journal                  = {npj Quantum information},
  Year                     = {2017},
  Number                   = {1},
  Pages                    = {1--8},
  Volume                   = {3},
  Publisher                = {Nature Publishing Group},
  URL                      = {https://www.nature.com/articles/s41534-017-0032-4}
}
</code></pre></details>
<details><summary>[Wendin et al] <b><a href=https://iopscience.iop.org/article/10.1088/1361-6633/aa7e1a/meta> Quantum information processing with superconducting circuits: a review. </a></b> <i>Reports on Progress in Physics.</i> <code><b>[Physical Realization of Qubits]</b></code> </summary><pre><code>@article{wendin2017quantum,
  title={Quantum information processing with superconducting circuits: a review},
  author={Wendin, G{\"o}ran},
  journal={Reports on Progress in Physics},
  volume={80},
  number={10},
  pages={106001},
  year={2017},
  publisher={IOP Publishing}
}</code></pre></details>


### 2016
<details><summary>[Batle et al] <b><a href=https://link.springer.com/article/10.1007/s11128-016-1324-x> Do multipartite correlations speed up adiabatic quantum computation or quantum annealing?. </a></b> <i>Quantum Information Processing.</i> <code><b>[Adiabatic Quantum Computing]</b></code> <code><b>[Quantum Annealing]</b></code> <code><b>[Quantum Speedup-Advantage-Supremacy]</b></code> <code><i>(Adiabatic Model)</i></code></summary><pre><code>@Article{batle2016multipartite,
  Title                    = {Do multipartite correlations speed up adiabatic quantum computation or quantum annealing?},
  Author                   = {Batle, Josep and Ooi, CH Raymond and Farouk, Ahmed and Abutalib, M and Abdalla, S},
  Journal                  = {Quantum Information Processing},
  Year                     = {2016},
  Number                   = {8},
  Pages                    = {3081--3099},
  Volume                   = {15},
  Publisher                = {Springer},
  URL                      = {https://link.springer.com/article/10.1007/s11128-016-1324-x}
}
</code></pre></details>
<details><summary>[Isakov et al] <b><a href=https://journals.aps.org/prl/abstract/10.1103/PhysRevLett.117.180402> Understanding Quantum Tunneling through Quantum Monte Carlo Simulations. </a></b> <i>Physical Review Letters.</i> <code><b>[Quantum Annealing]</b></code> <code><i>(Adiabatic Model)</i></code></summary><pre><code>@article{isakov2016understanding,
  title={Understanding quantum tunneling through quantum Monte Carlo simulations},
  author={Isakov, Sergei V and Mazzola, Guglielmo and Smelyanskiy, Vadim N and Jiang, Zhang and Boixo, Sergio and Neven, Hartmut and Troyer, Matthias},
  journal={Physical review letters},
  volume={117},
  number={18},
  pages={180402},
  year={2016},
  publisher={APS}
}</code></pre></details>
<details><summary>[Sergey et al] <b><a href=https://www.Nature.com/articles/ncomms12370> Zero-temperature quantum annealing bottlenecks in the spin-glass phase. </a></b> <i>Nature.</i> <code><b>[Quantum Annealing]</b></code> <code><i>(Adiabatic Model)</i></code></summary><pre><code>@article{knysh2016zero,
  title={Zero-temperature quantum annealing bottlenecks in the spin-glass phase},
  author={Knysh, Sergey},
  journal={Nature communications},
  volume={7},
  number={1},
  pages={1--9},
  year={2016},
  publisher={Nature Publishing Group}
}</code></pre></details>
<details><summary>[Mandra' et al] <b><a href=https://journals.aps.org/pra/abstract/10.1103/PhysRevA.94.022337> Strengths and weaknesses of weak-strong cluster problems: A detailed overview of state-of-the-art classical heuristics versus quantum approaches. </a></b> <i>Physical Review A.</i> <code><b>[Quantum Annealing]</b></code> <code><b>[Quantum Speedup-Advantage-Supremacy]</b></code> <code><i>(Adiabatic Model)</i></code></summary><pre><code>@article{mandra2016strengths,
  title={Strengths and weaknesses of weak-strong cluster problems: A detailed overview of state-of-the-art classical heuristics versus quantum approaches},
  author={Mandra, Salvatore and Zhu, Zheng and Wang, Wenlong and Perdomo-Ortiz, Alejandro and Katzgraber, Helmut G},
  journal={Physical Review A},
  volume={94},
  number={2},
  pages={022337},
  year={2016},
  publisher={APS}
}</code></pre></details>


### 2015
<details><summary>[Aaronson ] <b><a href=https://www.nature.com/articles/nphys3272> Quantum Machine Learning Algorithms: Read the Fine Print. </a></b> <i>Nature Physics.</i> <code><b>[Quantum Speedup-Advantage-Supremacy]</b></code> </summary><pre><code>@Article{aaronson2015read,
  Title                    = {Read the fine print},
  Author                   = {Aaronson, Scott},
  Journal                  = {Nature Physics},
  Year                     = {2015},
  Number                   = {4},
  Pages                    = {291--293},
  Volume                   = {11},
  Publisher                = {Nature Publishing Group},
  URL                      = {https://www.nature.com/articles/nphys3272}
}
</code></pre></details>
<details><summary>[Bocharov et al] <b><a href=https://journals.aps.org/prl/abstract/10.1103/PhysRevLett.114.080502> Efficient synthesis of universal repeat-until-success quantum circuits. </a></b> <i>Physical Review Letters.</i> <code><b>[Probabilistic Quantum Circuits]</b></code> <code><b>[Repeat-Until-Success Circuits]</b></code> <code><i>(Gate Model)</i></code></summary><pre><code>@Article{bocharov2015efficient,
  Title                    = {Efficient synthesis of universal repeat-until-success quantum circuits},
  Author                   = {Bocharov, Alex and Roetteler, Martin and Svore, Krysta M},
  Journal                  = {Physical review letters},
  Year                     = {2015},
  Number                   = {8},
  Pages                    = {080502},
  Volume                   = {114},
  Publisher                = {APS},
  URL                      = {https://journals.aps.org/prl/abstract/10.1103/PhysRevLett.114.080502}
}
</code></pre></details>
<details><summary>[Itay ert al] <b><a href=https://journals.aps.org/pra/abstract/10.1103/PhysRevA.92.042325> Probing for quantum speedup in spin-glass problems with planted solutions. </a></b> <i>Physical Review A.</i> <code><b>[Quantum Annealing]</b></code> <code><b>[Quantum Speedup-Advantage-Supremacy]</b></code> <code><i>(Adiabatic Model)</i></code></summary><pre><code>@article{hen2015probing,
  title={Probing for quantum speedup in spin-glass problems with planted solutions},
  author={Hen, Itay and Job, Joshua and Albash, Tameem and R{\o}nnow, Troels F and Troyer, Matthias and Lidar, Daniel A},
  journal={Physical Review A},
  volume={92},
  number={4},
  pages={042325},
  year={2015},
  publisher={APS}
}</code></pre></details>
<details><summary>[Katzgraber et al] <b><a href=https://journals.aps.org/prx/abstract/10.1103/PhysRevX.5.031026> Seeking Quantum Speedup Through Spin Glasses: The Good, the Bad, and the Ugly. </a></b> <i>Physical Review X.</i> <code><b>[Quantum Annealing]</b></code> <code><b>[Quantum Speedup-Advantage-Supremacy]</b></code> <code><i>(Adiabatic Model)</i></code></summary><pre><code>@article{katzgraber2015seeking,
  title={Seeking quantum speedup through spin glasses: The good, the bad, and the ugly},
  author={Katzgraber, Helmut G and Hamze, Firas and Zhu, Zheng and Ochoa, Andrew J and Munoz-Bauza, Humberto},
  journal={Physical Review X},
  volume={5},
  number={3},
  pages={031026},
  year={2015},
  publisher={APS}
}</code></pre></details>
<details><summary>[Schuld et al] <b><a href=https://doi.org/10.1016/j.physleta.2014.11.061> Simulating a perceptron on a quantum computer. </a></b> <i>Physics Letters A.</i> <code><b>[Quantum Perceptron]</b></code> <code><b>[Quantum Machine Learning]</b></code> </summary><pre><code>@article{schuld2015simulating,
  title={Simulating a perceptron on a quantum computer},
  author={Schuld, Maria and Sinayskiy, Ilya and Petruccione, Francesco},
  journal={Physics Letters A},
  volume={379},
  number={7},
  pages={660--663},
  year={2015},
  publisher={Elsevier}
}</code></pre></details>


### 2014
<details><summary>[Bunyk et al] <b><a href=https://ieeexplore.ieee.org/document/6802426> Architectural considerations in the design of a superconducting quantum annealing processor. </a></b> <i>IEEE Transactions on Applied Superconductivity.</i> <code><b>[Quantum Annealing]</b></code> <code><b>[Quantum Information and Computing]</b></code> <code><b>[QPU]</b></code> <code><b>[Superconducting Qubits]</b></code> <code><i>(Adiabatic Model)</i></code></summary><pre><code>@Article{bunyk2014architectural,
  Title                    = {Architectural considerations in the design of a superconducting quantum annealing processor},
  Author                   = {Bunyk, Paul I and Hoskinson, Emile M and Johnson, Mark W and Tolkacheva, Elena and Altomare, Fabio and Berkley, Andrew J and Harris, Richard and Hilton, Jeremy P and Lanting, Trevor and Przybysz, Anthony J and others},
  Journal                  = {IEEE Transactions on Applied Superconductivity},
  Year                     = {2014},
  Number                   = {4},
  Pages                    = {1--10},
  Volume                   = {24},
  Publisher                = {IEEE},
  URL                      = {https://ieeexplore.ieee.org/document/6802426}
}
</code></pre></details>
<details><summary>[Boixo et al] <b><a href=https://www.Nature.com/articles/nphys2900> Evidence for quantum annealing with more than one hundred qubits. </a></b> <i>Nature.</i> <code><b>[Quantum Annealing]</b></code> <code><i>(Adiabatic Model)</i></code></summary><pre><code>@Article{evidenceboixo2014,
  Title                    = {Evidence for quantum annealing with more than one hundred qubits},
  Author                   = {Boixo, Sergio and R{\o}nnow, Troels F and Isakov, Sergei V and Wang, Zhihui and Wecker, David and Lidar, Daniel A and Martinis, John M and Troyer, Matthias},
  Journal                  = {Nature physics},
  Year                     = {2014},
  Number                   = {3},
  Pages                    = {218--224},
  Volume                   = {10},
  Publisher                = {Nature Publishing Group},
  URL                      = {https://www.Nature.com/articles/nphys2900}
}
</code></pre></details>
<details><summary>[Farhi et al] <b><a href=https://arxiv.org/abs/1411.4028> A Quantum Approximate Optimization Algorithm. </a></b> <i>arXiv.</i> <code><b>[Variational Quantum Algorithms]</b></code> <code><b>[Parameterized Quantum Circuits]</b></code> <code><b>[Quantum Optimization]</b></code> <code><b>[Quantum Machine Learning]</b></code> <code><b>[]</b></code> <code><i>(Gate Model)</i></code></summary><pre><code>@Article{farhi2014quantum,
  Title                    = {A quantum approximate optimization algorithm},
  Author                   = {Farhi, Edward and Goldstone, Jeffrey and Gutmann, Sam},
  Journal                  = {arXiv preprint arXiv:1411.4028},
  Year                     = {2014},
  URL                      = {https://arxiv.org/abs/1411.4028}
}
</code></pre></details>
<details><summary>[Lanting et al] <b><a href=https://journals.aps.org/prx/abstract/10.1103/PhysRevX.4.021041> Entanglement in a Quantum Annealing Processor. </a></b> <i>PHYSICAL REVIEW X.</i> <code><b>[Quantum Annealing]</b></code> <code><i>(Adiabatic Model)</i></code></summary><pre><code>@article{lanting2014entanglement,
  title={Entanglement in a quantum annealing processor},
  author={Lanting, Trevor and Przybysz, Anthony J and Smirnov, A Yu and Spedalieri, Federico M and Amin, Mohammad H and Berkley, Andrew J and Harris, Richard and Altomare, Fabio and Boixo, Sergio and Bunyk, Paul and others},
  journal={Physical Review X},
  volume={4},
  number={2},
  pages={021041},
  year={2014},
  publisher={APS}
}</code></pre></details>
<details><summary>[McGeoch] <b><a href=https://ieeexplore.ieee.org/document/7055969> Adiabatic Quantum Computation and Quantum Annealing: Theory and Practice. </a></b> <i>Synthesis Lectures on Quantum Computing.</i> <code><b>[Quantum Annealing]</b></code> <code><i>(Adiabatic Model)</i></code></summary><pre><code>@article{mcgeoch2014adiabatic,
  title={Adiabatic quantum computation and quantum annealing: Theory and practice},
  author={McGeoch, Catherine C},
  journal={Synthesis Lectures on Quantum Computing},
  volume={5},
  number={2},
  pages={1--93},
  year={2014},
  publisher={Morgan \& Claypool Publishers}
}</code></pre></details>
<details><summary>[Peruzzo et al] <b><a href=https://www.nature.com/articles/ncomms5213> A variational eigenvalue solver on a photonic quantum processor. </a></b> <i>Nature communications.</i> <code><b>[Variational Quantum Algorithms]</b></code> </summary><pre><code>@article{peruzzo2014variational,
  title={A variational eigenvalue solver on a photonic quantum processor},
  author={Peruzzo, Alberto and McClean, Jarrod and Shadbolt, Peter and Yung, Man-Hong and Zhou, Xiao-Qi and Love, Peter J and Aspuru-Guzik, Al{\'a}n and O’brien, Jeremy L},
  journal={Nature communications},
  volume={5},
  number={1},
  pages={1--7},
  year={2014},
  publisher={Nature Publishing Group}
}</code></pre></details>
<details><summary>[Rønnow et al] <b><a href=https://pubmed.ncbi.nlm.nih.gov/25061205/> Quantum computing Defining and detecting quantum speedup. </a></b> <i>Science.</i> <code><b>[Quantum Annealing]</b></code> <code><i>(Adiabatic Model)</i></code></summary><pre><code>@article{ronnow2014defining,
  title={Defining and detecting quantum speedup},
  author={R{\o}nnow, Troels F and Wang, Zhihui and Job, Joshua and Boixo, Sergio and Isakov, Sergei V and Wecker, David and Martinis, John M and Lidar, Daniel A and Troyer, Matthias},
  journal={science},
  volume={345},
  number={6195},
  pages={420--424},
  year={2014},
  publisher={American Association for the Advancement of Science}
}</code></pre></details>
<details><summary>[Rønnow et al] <b><a href=https://arxiv.org/pdf/1401.2910.pdf> Defining and detecting quantum speedup. </a></b> <i>Science.</i> <code><b>[Quantum Information and Computing]</b></code> <code><b>[Quantum Speedup-Advantage-Supremacy]</b></code> <code><i>(Adiabatic Model)</i></code></summary><pre><code>@article{ronnow2014defining,
  title={Defining and detecting quantum speedup},
  author={R{\o}nnow, Troels F and Wang, Zhihui and Job, Joshua and Boixo, Sergio and Isakov, Sergei V and Wecker, David and Martinis, John M and Lidar, Daniel A and Troyer, Matthias},
  journal={science},
  volume={345},
  number={6195},
  pages={420--424},
  year={2014},
  publisher={American Association for the Advancement of Science}
}</code></pre></details>
<details><summary>[Schuld et al] <b><a href=https://link.springer.com/chapter/10.1007/978-3-319-13560-1_17> Quantum computing for pattern classification. </a></b> <i>Pacific Rim International Conference on Artificial Intelligence.</i> <code><b>[Quantum Machine Learning]</b></code> </summary><pre><code>@inproceedings{schuld2014quantum,
  title={Quantum computing for pattern classification},
  author={Schuld, Maria and Sinayskiy, Ilya and Petruccione, Francesco},
  booktitle={Pacific Rim International Conference on Artificial Intelligence},
  pages={208--220},
  year={2014},
  organization={Springer}
}</code></pre></details>
<details><summary>[Schuld et al] <b><a href=https://link.springer.com/article/10.1007%2Fs11128-014-0809-8> The quest for a Quantum Neural Networt. </a></b> <i>Quantum Information Processing.</i> <code><b>[Quantum Neural Networks]</b></code> <code><b>[Quantum Machine Learning]</b></code> <code><i>(Gate Model)</i></code></summary><pre><code>@article{schuld2014quest,
  title={The quest for a quantum neural network},
  author={Schuld, Maria and Sinayskiy, Ilya and Petruccione, Francesco},
  journal={Quantum Information Processing},
  volume={13},
  number={11},
  pages={2567--2586},
  year={2014},
  publisher={Springer}
}</code></pre></details>
<details><summary>[Wiebe et al] <b><a href=https://dl.acm.org/doi/abs/10.5555/2871393.2871400> Quantum Algorithms for Nearest-Neighbour Methods for Supervised and Unsupervised Learning. </a></b> <i>Quantum Information & Computation.</i> <code><b>[Quantum Nearest Neighbors Algorithm]</b></code> <code><b>[Quantum Classification]</b></code> <code><b>[Quantum Machine Learning]</b></code> <code><i>(Gate Model)</i></code></summary><pre><code>@article{wiebe2015quantum,
  title={Quantum algorithms for nearest-neighbor methods for supervised and unsupervised learning},
  author={Wiebe, Nathan and Kapoor, Ashish and Svore, Krysta M},
  journal={Quantum Information \& Computation},
  volume={15},
  number={3-4},
  pages={316--356},
  year={2015},
  publisher={Rinton Press, Incorporated Paramus, NJ}
}</code></pre></details>


### 2013
<details><summary>[Aïmeur et al] <b><a href=https://link.springer.com/article/10.1007/s10994-012-5316-5> Quantum speed-up for unsupervised learning. </a></b> <i>Machine Learning.</i> <code><b>[Quantum Machine Learning]</b></code> <code><b>[Unsupervised Learning]</b></code> <code><b>[Quantum Clustering]</b></code> <code><i>(Gate Model)</i></code></summary><pre><code>@Article{aimeur2013quantum,
  Title                    = {Quantum speed-up for unsupervised learning},
  Author                   = {A{\"\i}meur, Esma and Brassard, Gilles and Gambs, S{\'e}bastien},
  Journal                  = {Machine Learning},
  Year                     = {2013},
  Number                   = {2},
  Pages                    = {261--287},
  Volume                   = {90},
  Publisher                = {Springer},
  URL                      = {https://link.springer.com/article/10.1007/s10994-012-5316-5}
}
</code></pre></details>
<details><summary>[Boixo et al] <b><a href=https://www.Nature.com/articles/ncomms3067> Experimental signature of programmable quantum annealing. </a></b> <i>Nature communications.</i> <code><b>[Quantum Annealing]</b></code> <code><i>(Adiabatic Model)</i></code></summary><pre><code>@Article{boixo2013experimental,
  Title                    = {Experimental signature of programmable quantum annealing},
  Author                   = {Boixo, Sergio and Albash, Tameem and Spedalieri, Federico M and Chancellor, Nicholas and Lidar, Daniel A},
  Journal                  = {Nature communications},
  Year                     = {2013},
  Number                   = {1},
  Pages                    = {1--8},
  Volume                   = {4},
  Publisher                = {Nature Publishing Group},
  URL                      = {https://www.Nature.com/articles/ncomms3067}
}
</code></pre></details>
<details><summary>[Lloyd et al] <b><a href=https://arxiv.org/abs/1307.0411> Quantum algorithms for supervised and unsupervised machine learning. </a></b> <i>arXiv.</i> <code><b>[Quantum Machine Learning]</b></code> <code><b>[Quantum Clustering]</b></code> <code><i>(Adiabatic Model)</i></code></summary><pre><code>@article{lloyd2013quantum,
  title={Quantum algorithms for supervised and unsupervised machine learning},
  author={Lloyd, Seth and Mohseni, Masoud and Rebentrost, Patrick},
  journal={arXiv preprint arXiv:1307.0411},
  year={2013}
}</code></pre></details>


### 2012
<details><summary>[Da Silva et al] <b><a href=https://www.sciencedirect.com/science/article/abs/pii/S0925231211004127?via%3Dihub> Classical and superposed learning for quantum weightless neural networks. </a></b> <i>Neurocomputing.</i> <code><b>[Quantum Neural Networks]</b></code> <code><b>[Quantum Machine Learning]</b></code> <code><i>(Gate Model)</i></code></summary><pre><code>@Article{da2012classical,
  Title                    = {Classical and superposed learning for quantum weightless neural networks},
  Author                   = {Da Silva, Adenilton J and De Oliveira, Wilson R and Ludermir, Teresa B},
  Journal                  = {Neurocomputing},
  Year                     = {2012},
  Number                   = {1},
  Pages                    = {52--60},
  Volume                   = {75},
  Publisher                = {Elsevier},
  URL                      = {https://www.sciencedirect.com/science/article/abs/pii/S0925231211004127?via%3Dihub}
}
</code></pre></details>
<details><summary>[Gaitan & Clark] <b><a href=https://journals.aps.org/prl/abstract/10.1103/PhysRevLett.108.010501> Ramsey Numbers and Adiabatic Quantum Computing. </a></b> <i>Physical Review Letters.</i> <code><b>[Adiabatic Quantum Computing]</b></code> <code><b>[Quantum Annealing]</b></code> <code><b>[Classical Implementation]</b></code> <code><i>(Adiabatic Model)</i></code></summary><pre><code>@Article{gaitan2012ramsey,
  Title                    = {Ramsey numbers and adiabatic quantum computing},
  Author                   = {Gaitan, Frank and Clark, Lane},
  Journal                  = {Physical review letters},
  Year                     = {2012},
  Number                   = {1},
  Pages                    = {010501},
  Volume                   = {108},
  Publisher                = {APS},
  URL                      = {https://journals.aps.org/prl/abstract/10.1103/PhysRevLett.108.010501}
}
</code></pre></details>


### 2011
<details><summary>[Aaronson and Arkhipov] <b><a href=https://dl.acm.org/doi/pdf/10.1145/1993636.1993682> The computational complexity of linear optics. </a></b> <i>Annual ACM symposium on Theory of computing.</i> <code><b>[Quantum Speedup-Advantage-Supremacy]</b></code> <code><b>[Complexity Theory]</b></code> </summary><pre><code>@InProceedings{aaronson2011computational,
  Title                    = {The computational complexity of linear optics},
  Author                   = {Aaronson, Scott and Arkhipov, Alex},
  Booktitle                = {Proceedings of the forty-third annual ACM symposium on Theory of computing},
  Year                     = {2011},
  Pages                    = {333--342},
  Publisher                = {Association for Computing Machinery},
  URL                      = {https://dl.acm.org/doi/pdf/10.1145/1993636.1993682}
}
</code></pre></details>
<details><summary>[Bremner et al] <b><a href=https://doi.org/10.1098/rspa.2010.0301> Classical simulation of commuting quantum computations implies collapse of the polynomial hierarchy. </a></b> <i>Proceedings of the Royal Society A: Mathematical, Physical and Engineering Sciences.</i> <code><b>[Quantum Speedup-Advantage-Supremacy]</b></code> <code><b>[Quantum Information and Computing]</b></code> <code><i>(Gate Model)</i></code></summary><pre><code>@Article{bremner2011classical,
  Title                    = {Classical simulation of commuting quantum computations implies collapse of the polynomial hierarchy},
  Author                   = {Bremner, Michael J and Jozsa, Richard and Shepherd, Dan J},
  Journal                  = {Proceedings of the Royal Society A: Mathematical, Physical and Engineering Sciences},
  Year                     = {2011},
  Number                   = {2126},
  Pages                    = {459--472},
  Volume                   = {467},
  Publisher                = {The Royal Society Publishing},
  URL                      = {https://doi.org/10.1098/rspa.2010.0301}
}
</code></pre></details>
<details><summary>[Johnson et al] <b><a href=https://www.Nature.com/articles/Nature10012.pdff> Quantum annealing with manufactured spins. </a></b> <i>Nature.</i> <code><b>[Quantum Annealing]</b></code> <code><i>(Adiabatic Model)</i></code></summary><pre><code>@article{johnson2011quantum,
  title={Quantum annealing with manufactured spins},
  author={Johnson, Mark W and Amin, Mohammad HS and Gildert, Suzanne and Lanting, Trevor and Hamze, Firas and Dickson, Neil and Harris, Richard and Berkley, Andrew J and Johansson, Jan and Bunyk, Paul and others},
  journal={Nature},
  volume={473},
  number={7346},
  pages={194--198},
  year={2011},
  publisher={Nature Publishing Group}
}</code></pre></details>
<details><summary>[Johnson et al] <b><a href=https://www.Nature.com/articles/Nature10012.pdf> Quantum annealing with manufactured spins. </a></b> <i>Nature.</i> <code><b>[Quantum Annealing]</b></code> <code><i>(Adiabatic Model)</i></code></summary><pre><code>@article{johnson2011quantum,
  title={Quantum annealing with manufactured spins},
  author={Johnson, Mark W and Amin, Mohammad HS and Gildert, Suzanne and Lanting, Trevor and Hamze, Firas and Dickson, Neil and Harris, Richard and Berkley, Andrew J and Johansson, Jan and Bunyk, Paul and others},
  journal={Nature},
  volume={473},
  number={7346},
  pages={194--198},
  year={2011},
  publisher={Nature Publishing Group}
}</code></pre></details>
<details><summary>[Panella & Martinelli] <b><a href=https://onlinelibrary.wiley.com/doi/10.1002/cta.619> Neural networks with quantum. </a></b> <i>International Journal of Circuit Theory and Applications.</i> <code><b>[Quantum Neurofuzzy Network]</b></code> <code><b>[Quantum Neural Networks]</b></code> <code><b>[Quantum Machine Learning]</b></code> </summary><pre><code>@article{panella2011neural,
  title={Neural networks with quantum architecture and quantum learning},
  author={Panella, Massimo and Martinelli, Giuseppe},
  journal={International Journal of Circuit Theory and Applications},
  volume={39},
  number={1},
  pages={61--77},
  year={2011},
  publisher={Wiley Online Library}
}</code></pre></details>


### 2010
<details><summary>[Altshuler et al] <b><a href=https://www.pnas.org/content/107/28/12446> Anderson localization makes adiabatic quantum optimization fail. </a></b> <i>PNAS.</i> <code><b>[Adiabatic Quantum Computing]</b></code> <code><b>[Quantum Optimization]</b></code> <code><i>(Adiabatic Model)</i></code></summary><pre><code>@Article{altshuler2010anderson,
  Title                    = {Anderson localization makes adiabatic quantum optimization fail},
  Author                   = {Altshuler, Boris and Krovi, Hari and Roland, J{\'e}r{\'e}mie},
  Journal                  = {Proceedings of the National Academy of Sciences},
  Year                     = {2010},
  Number                   = {28},
  Pages                    = {12446--12450},
  Volume                   = {107},
  Publisher                = {National Acad Sciences},
  URL                      = {https://www.pnas.org/content/107/28/12446}
}
</code></pre></details>
<details><summary>[Bravyi & Terhal] <b><a href=https://arxiv.org/abs/0806.1746> Complexity of stoquastic frustration-free Hamiltonians. </a></b> <i>Siam journal on computing.</i> <code><b>[Adiabatic Quantum Computing]</b></code> <code><b>[Quantum Annealing]</b></code> <code><i>(Adiabatic Model)</i></code></summary><pre><code>@Article{bravyi2010complexity,
  Title                    = {Complexity of stoquastic frustration-free Hamiltonians},
  Author                   = {Bravyi, Sergey and Terhal, Barbara},
  Journal                  = {Siam journal on computing},
  Year                     = {2010},
  Number                   = {4},
  Pages                    = {1462--1485},
  Volume                   = {39},
  Publisher                = {SIAM},
  URL                      = {https://arxiv.org/abs/0806.1746}
}
</code></pre></details>
<details><summary>[Harris et al] <b><a href=https://journals.aps.org/prb/abstract/10.1103/PhysRevB.81.134510> Experimental demonstration of a robust and scalable flux qubit. </a></b> <i>Physical Review B.</i> <code><b>[Quantum Annealing]</b></code> <code><i>(Adiabatic Model)</i></code></summary><pre><code>@Article{harris2010demonstration,
  Title                    = {Experimental demonstration of a robust and scalable flux qubit},
  Author                   = {Harris, R and Johansson, J and Berkley, AJ and Johnson, MW and Lanting, T and Han, Siyuan and Bunyk, P and Ladizinsky, E and Oh, T and Perminov, I and others},
  Journal                  = {Physical Review B},
  Year                     = {2010},
  Number                   = {13},
  Pages                    = {134510},
  Volume                   = {81},
  Publisher                = {APS},
  URL                      = {https://journals.aps.org/prb/abstract/10.1103/PhysRevB.81.134510}
}
</code></pre></details>
<details><summary>[Harris et al] <b><a href=https://journals.aps.org/prb/abstract/10.1103/PhysRevB.82.024511> Experimental investigation of an eight-qubit unit cell in a superconducting optimization processor. </a></b> <i>Physical Review B.</i> <code><b>[Quantum Annealing]</b></code> <code><b>[Superconducting Qubits]</b></code> <code><i>(Adiabatic Model)</i></code></summary><pre><code>@article{harris2010experimental,
  title={Experimental investigation of an eight-qubit unit cell in a superconducting optimization processor},
  author={Harris, Richard and Johnson, Mark W and Lanting, T and Berkley, AJ and Johansson, J and Bunyk, P and Tolkacheva, E and Ladizinsky, E and Ladizinsky, N and Oh, T and others},
  journal={Physical Review B},
  volume={82},
  number={2},
  pages={024511},
  year={2010},
  publisher={APS}
}</code></pre></details>
<details><summary>[Da Silva et al] <b><a href=https://ieeexplore.ieee.org/document/5715204> Superposition Based Learning Algorithm. </a></b> <i>Brazilian Symposium on Neural Networks.</i> <code><b>[Quantum Neural Networks]</b></code> <code><b>[Quantum Machine Learning]</b></code> <code><i>(Gate Model)</i></code></summary><pre><code>@inproceedings{silva2010superposition,
  title={Superposition based learning algorithm},
  author={Silva, Adenilton J and Ludermir, Teresa B and de Oliveira Jr, Wilson R},
  booktitle={2010 Eleventh Brazilian Symposium on Neural Networks},
  pages={1--6},
  year={2010},
  organization={IEEE}
}</code></pre></details>


### 2009
<details><summary>[Barnett & Croke] <b><a href=https://www.osapublishing.org/abstract.cfm?uri=aop-1-2-238> Quantum state discrimination. </a></b> <i>Advances in Optics and Photonics.</i> <code><b>[State Discrimination]</b></code> </summary><pre><code>@Article{barnett2009quantum,
  Title                    = {Quantum state discrimination},
  Author                   = {Barnett, Stephen M and Croke, Sarah},
  Journal                  = {Advances in Optics and Photonics},
  Year                     = {2009},
  Number                   = {2},
  Pages                    = {238--278},
  Volume                   = {1},
  Publisher                = {Optical Society of America},
  URL                      = {https://www.osapublishing.org/abstract.cfm?uri=aop-1-2-238}
}
</code></pre></details>
<details><summary>[de Oliveira] <b><a href=https://citeseerx.ist.psu.edu/viewdoc/download?doi=10.1.1.227.4363&rep=rep1&type=pdf> Quantum RAM based neural networks. </a></b> <i>European Symposium on Artificial Neural Network.</i> <code><b>[QRAM]</b></code> <code><b>[Quantum Neural Networks]</b></code> <code><b>[Quantum Machine Learning]</b></code> <code><i>(Gate Model)</i></code></summary><pre><code>@InProceedings{de2009quantum,
  Title                    = {Quantum RAM Based Neural Netoworks.},
  Author                   = {de Oliveira, Wilson Rosa},
  Booktitle                = {ESANN},
  Year                     = {2009},
  Organization             = {Citeseer},
  Pages                    = {331--336},
  Volume                   = {9},
  URL                      = {https://citeseerx.ist.psu.edu/viewdoc/download?doi=10.1.1.227.4363&rep=rep1&type=pdf}
}
</code></pre></details>
<details><summary>[Harrow et al] <b><a href=https://journals.aps.org/prl/abstract/10.1103/PhysRevLett.103.150502> Quantum Algorithm for Linear Systems of Equations. </a></b> <i>Physical Review Letters.</i> <code><b>[Quantum Machine Learning]</b></code> <code><b>[Quantum Speedup-Advantage-Supremacy]</b></code> </summary><pre><code>@Article{harrow2009quantum,
  Title                    = {Quantum algorithm for linear systems of equations},
  Author                   = {Harrow, Aram W and Hassidim, Avinatan and Lloyd, Seth},
  Journal                  = {Physical review letters},
  Year                     = {2009},
  Number                   = {15},
  Pages                    = {150502},
  Volume                   = {103},
  Publisher                = {APS},
  URL                      = {https://journals.aps.org/prl/abstract/10.1103/PhysRevLett.103.150502}
}
</code></pre></details>
<details><summary>[Lidar et al] <b><a href=https://aip.scitation.org/doi/full/10.1063/1.3236685> Adiabatic approximation with exponential accuracy for many-body systems and quantum computation. </a></b> <i>Journal of Mathematical Physics.</i> <code><b>[Approximation to Adiabatic Theorem]</b></code> <code><b>[Adiabatic Quantum Computing]</b></code> <code><i>(Adiabatic Model)</i></code></summary><pre><code>@article{lidar2009adiabatic,
  title={Adiabatic approximation with exponential accuracy for many-body systems and quantum computation},
  author={Lidar, Daniel A and Rezakhani, Ali T and Hamma, Alioscia},
  journal={Journal of Mathematical Physics},
  volume={50},
  number={10},
  pages={102106},
  year={2009},
  publisher={American Institute of Physics}
}</code></pre></details>
<details><summary>[Matsuda et al] <b><a href=https://iopscience.iop.org/article/10.1088/1367-2630/11/7/073021> Ground-state statistics from annealing algorithms: quantum versus classical approaches. </a></b> <i>New Journal of Physics.</i> <code><b>[Classical Implementation]</b></code> <code><b>[Quantum Annealing]</b></code> </summary><pre><code>@article{matsuda2009ground,
  title={Ground-state statistics from annealing algorithms: quantum versus classical approaches},
  author={Matsuda, Yoshiki and Nishimori, Hidetoshi and Katzgraber, Helmut G},
  journal={New Journal of Physics},
  volume={11},
  number={7},
  pages={073021},
  year={2009},
  publisher={IOP Publishing}
}</code></pre></details>


### 2008
<details><summary>[Aharonov et al] <b><a href=https://epubs.siam.org/doi/10.1137/080734479> Adiabatic quantum computation is equivalent to standard quantum computation. </a></b> <i>SIAM review.</i> <code><b>[Adiabatic Quantum Computing]</b></code> <code><b>[Adiabatic Simulation]</b></code> <code><i>(Adiabatic Model)</i></code></summary><pre><code>@Article{aharonov2008adiabatic,
  Title                    = {Adiabatic quantum computation is equivalent to standard quantum computation},
  Author                   = {Aharonov, Dorit and Van Dam, Wim and Kempe, Julia and Landau, Zeph and Lloyd, Seth and Regev, Oded},
  Journal                  = {SIAM review},
  Year                     = {2008},
  Number                   = {4},
  Pages                    = {755--787},
  Volume                   = {50},
  Publisher                = {SIAM},
  URL                      = {https://epubs.siam.org/doi/10.1137/080734479}
}
</code></pre></details>
<details><summary>[Choi ] <b><a href=https://link.springer.com/article/10.1007/s11128-008-0082-9> Minor-Embedding in Adiabatic Quantum Computation: I The Parameter Setting Problem. </a></b> <i>Quantum Information Processing.</i> <code><b>[Quantum Annealing]</b></code> <code><i>(Adiabatic Model)</i></code></summary><pre><code>@Article{minorchoi2011,
  Title                    = {Minor-embedding in adiabatic quantum computation: II. Minor-universal graph design},
  Author                   = {Choi, Vicky},
  Journal                  = {Quantum Information Processing},
  Year                     = {2011},
  Number                   = {3},
  Pages                    = {343--353},
  Volume                   = {10},
  Publisher                = {Springer},
  URL                      = {https://link.springer.com/article/10.1007/s11128-008-0082-9}
}
</code></pre></details>
<details><summary>[Clarke et al] <b><a href=https://www.Nature.com/articles/Nature07128> Superconducting quantum bits. </a></b> <i>Nature.</i> <code><b>[Quantum Information and Computing]</b></code> </summary><pre><code>@Article{clarke2008superconducting,
  Title                    = {Superconducting quantum bits},
  Author                   = {Clarke, John and Wilhelm, Frank K},
  Journal                  = {Nature},
  Year                     = {2008},
  Number                   = {7198},
  Pages                    = {1031--1042},
  Volume                   = {453},
  Publisher                = {Nature Publishing Group},
  URL                      = {https://www.Nature.com/articles/Nature07128}
}
</code></pre></details>
<details><summary>[Dong et al] <b><a href=https://ieeexplore.ieee.org/document/4579244> Quantum Reinforcement Learning. </a></b> <i>IEEE Transactions on Systems, Man, and Cybernetics, Part B (Cybernetics).</i> <code><b>[Quantum Reinforcement Learning]</b></code> <code><b>[Quantum Machine Learning]</b></code> <code><i>(Gate Model)</i></code></summary><pre><code>@article{dong2008quantum,
  title={Quantum reinforcement learning},
  author={Dong, Daoyi and Chen, Chunlin and Li, Hanxiong and Tarn, Tzyh-Jong},
  journal={IEEE Transactions on Systems, Man, and Cybernetics, Part B (Cybernetics)},
  volume={38},
  number={5},
  pages={1207--1220},
  year={2008},
  publisher={IEEE}
}</code></pre></details>
<details><summary>[Giovannetti et al] <b><a href=https://journals.aps.org/prl/abstract/10.1103/PhysRevLett.100.160501> Quantum Random Access Memory. </a></b> <i>Physical Review Letters.</i> <code><b>[QRAM]</b></code> </summary><pre><code>@Article{giovannetti2008quantum,
  Title                    = {Quantum random access memory},
  Author                   = {Giovannetti, Vittorio and Lloyd, Seth and Maccone, Lorenzo},
  Journal                  = {Physical review letters},
  Year                     = {2008},
  Number                   = {16},
  Pages                    = {160501},
  Volume                   = {100},
  Publisher                = {APS},
  URL                      = {https://journals.aps.org/prl/abstract/10.1103/PhysRevLett.100.160501}
}
</code></pre></details>
<details><summary>[Panella & Martinelli] <b><a href=https://ieeexplore.ieee.org/document/4565676> Neurofuzzy networks with nonlinear quantum learning. </a></b> <i>IEEE Transactions on Fuzzy Systems.</i> <code><b>[Quantum Neurofuzzy Network]</b></code> <code><b>[Quantum Neural Networks]</b></code> <code><b>[Quantum Machine Learning]</b></code> </summary><pre><code>@article{panella2008neurofuzzy,
  title={Neurofuzzy networks with nonlinear quantum learning},
  author={Panella, Massimo and Martinelli, Giuseppe},
  journal={IEEE Transactions on Fuzzy Systems},
  volume={17},
  number={3},
  pages={698--710},
  year={2008},
  publisher={IEEE}
}</code></pre></details>


### 2007
<details><summary>[Aïmeur et al] <b><a href=https://dl.acm.org/doi/abs/10.1145/1273496.1273497> Quantum Clustering Algorithms. </a></b> <i>ICML '07.</i> <code><b>[Quantum Machine Learning]</b></code> <code><b>[Unsupervised Learning]</b></code> <code><b>[Quantum Clustering]</b></code> <code><i>(Gate Model)</i></code></summary><pre><code>@inproceedings{aimeur2007quantum,
  title={Quantum clustering algorithms},
  author={A{\"\i}meur, Esma and Brassard, Gilles and Gambs, S{\'e}bastien},
  booktitle={Proceedings of the 24th international conference on machine learning},
  pages={1--8},
  year={2007}
}</code></pre></details>


### 2006
<details><summary>[Aïmeur et al] <b><a href=https://link.springer.com/chapter/10.1007/11766247_37> Machine Learning in a Quantum World. </a></b> <i>Canadian AI 2006.</i> <code><b>[Quantum Machine Learning]</b></code> <code><b>[Unsupervised Learning]</b></code> <code><b>[Quantum Clustering]</b></code> <code><i>(Gate Model)</i></code></summary><pre><code>@InProceedings{aimeur2006machine,
  Title                    = {Machine learning in a quantum world},
  Author                   = {A{\"\i}meur, Esma and Brassard, Gilles and Gambs, S{\'e}bastien},
  Booktitle                = {Conference of the Canadian Society for Computational Studies of Intelligence},
  Year                     = {2006},
  Organization             = {Springer},
  Pages                    = {431--442},
  URL                      = {https://link.springer.com/chapter/10.1007/11766247_37}
}
</code></pre></details>


### 2005
<details><summary>[Battaglia et al] <b><a href=https://link.springer.com/chapter/10.1007%2F11526216_7> Deterministic and stochastic quantum annealing approaches. </a></b> <i>Lecture Notes in Physics .</i> <code><b>[Adiabatic Quantum Computing]</b></code> <code><b>[Quantum Annealing]</b></code> <code><b>[Simulated Annealing]</b></code> <code><i>(Adiabatic Model)</i></code></summary><pre><code>@InCollection{battaglia2005deterministic,
  Title                    = {Deterministic and stochastic quantum annealing approaches},
  Author                   = {Battaglia, Demian and Stella, Lorenzo and Zagordi, Osvaldo and Santoro, Giuseppe E and Tosatti, Erio},
  Booktitle                = {Quantum Annealing and Other Optimization Methods},
  Publisher                = {Springer},
  Year                     = {2005},
  Pages                    = {171--206},
  URL                      = {https://link.springer.com/chapter/10.1007%2F11526216_7}
}
</code></pre></details>
<details><summary>[Battaglia et al] <b><a href=https://journals.aps.org/pre/abstract/10.1103/PhysRevE.71.066707> Optimization by Quantum Annealing: Lessons from hard 3-SAT cases,. </a></b> <i>Physical Review E.</i> <code><b>[Adiabatic Quantum Computing]</b></code> <code><b>[Quantum Annealing]</b></code> <code><b>[Simulated Annealing]</b></code> <code><b>[Quantum Monte Carlo]</b></code> <code><i>(Adiabatic Model)</i></code></summary><pre><code>@Article{battaglia2005optimization,
  Title                    = {Optimization by quantum annealing: Lessons from hard satisfiability problems},
  Author                   = {Battaglia, Demian A and Santoro, Giuseppe E and Tosatti, Erio},
  Journal                  = {Physical Review E},
  Year                     = {2005},
  Number                   = {6},
  Pages                    = {066707},
  Volume                   = {71},
  Publisher                = {APS},
  URL                      = {https://journals.aps.org/pre/abstract/10.1103/PhysRevE.71.066707}
}
</code></pre></details>
<details><summary>[Olivera et al] <b><a href=https://dl.acm.org/doi/10.5555/2016985.2016987> The complexity of quantum spin systems on a two-dimensional square lattice. </a></b> <i>Quantum Information & Computing.</i> <code><b>[Complexity Theory]</b></code> <code><b>[Adiabatic Quantum Computing]</b></code> </summary><pre><code>@article{oliveira2005complexity,
  title={The complexity of quantum spin systems on a two-dimensional square lattice},
  author={Oliveira, Roberto and Terhal, Barbara M},
  journal={arXiv preprint quant-ph/0504050},
  year={2005}
}</code></pre></details>


### 2004
<details><summary>[Aharonov et al] <b><a href=https://arxiv.org/pdf/quant-ph/0405098.pdf> Adiabatic computation is equivalent to standard quantum computation. </a></b> <i>arXiv.</i> <code><b>[Adiabatic Quantum Computing]</b></code> <code><b>[Adiabatic Simulation]</b></code> <code><i>(Adiabatic Model)</i></code></summary><pre><code>@Article{aharonov2004adiabatic,
  Title                    = {Adiabatic computation is equivalent to standard quantum computation},
  Author                   = {Aharonov, D and van Dam, W and Kempe, J and Landau, Z and Lloyd, S and Regev, O},
  Journal                  = {arXiv preprint quant-ph/0405098},
  Year                     = {2004},
  URL                      = {https://arxiv.org/pdf/quant-ph/0405098.pdf}
}
</code></pre></details>
<details><summary>[Martonak et al] <b><a href=https://journals.aps.org/pre/abstract/10.1103/PhysRevE.70.057701> Quantum annealing of the traveling-salesman problem. </a></b> <i>Physical Review E.</i> <code><b>[Classical Implementation]</b></code> <code><b>[Quantum Annealing]</b></code> </summary><pre><code>@article{martovnak2004quantum,
  title={Quantum annealing of the traveling-salesman problem},
  author={Marto{\v{n}}{\'a}k, Roman and Santoro, Giuseppe E and Tosatti, Erio},
  journal={Physical Review E},
  volume={70},
  number={5},
  pages={057701},
  year={2004},
  publisher={APS}
}</code></pre></details>


### 2003
<details><summary>[Ricks & Ventura] <b><a href=https://dl.acm.org/doi/abs/10.5555/2981345.2981472> Training a Quantum Neural Network. </a></b> <i>NIPS.</i> <code><b>[Quantum Neural Networks]</b></code> </summary><pre><code>@article{ricks2003training,
  title={Training a quantum neural network},
  author={Ricks, Bob and Ventura, Dan},
  journal={Advances in neural information processing systems},
  volume={16},
  pages={1019--1026},
  year={2003}
}</code></pre></details>


### 2002
<details><summary>[Brassard et al ] <b><a href=http://dx.doi.org/10.1090/conm/305/05215> Quantum amplitude amplification and estimation. </a></b> <i>Contemporary Mathematics.</i> <code><b>[Quantum Information and Computing]</b></code> <code><i>(Gate Model)</i></code></summary><pre><code>@Article{brassard2002quantum,
  Title                    = {Quantum amplitude amplification and estimation},
  Author                   = {Brassard, Gilles and Hoyer, Peter and Mosca, Michele and Tapp, Alain},
  Journal                  = {Contemporary Mathematics},
  Year                     = {2002},
  Pages                    = {53--74},
  Volume                   = {305},
  Publisher                = {Providence, RI; American Mathematical Society; 1999},
  URL                      = {http://dx.doi.org/10.1090/conm/305/05215}
}
</code></pre></details>


### 2001
<details><summary>[Altaisky] <b><a href=https://arxiv.org/abs/quant-ph/0107012> Quantum neural network. </a></b> <i>arXiv.</i> <code><b>[Quantum Neural Networks]</b></code> <code><b>[Quantum Perceptron]</b></code> <code><b>[Quantum Machine Learning]</b></code> </summary><pre><code>@Article{altaisky2001quantum,
  Title                    = {Quantum neural network},
  Author                   = {Altaisky, MV},
  Journal                  = {arXiv preprint quant-ph/0107012},
  Year                     = {2001},
  URL                      = {https://arxiv.org/abs/quant-ph/0107012}
}
</code></pre></details>
<details><summary>[Farhi et al] <b><a href=https://science.sciencemag.org/content/292/5516/472> A Quantum Adiabatic Evolution Algorithm Applied to Random Instances of an NP-Complete Problem. </a></b> <i>Science.</i> <code><b>[Adiabatic Quantum Computing]</b></code> <code><b>[Classical Implementation]</b></code> <code><b>[Quantum Annealing]</b></code> <code><i>(Adiabatic Model)</i></code></summary><pre><code>@Article{farhi2001quantum,
  Title                    = {{A quantum adiabatic evolution algorithm applied to random instances of an NP-complete problem}},
  Author                   = {Farhi, Edward and Goldstone, Jeffrey and Gutmann, Sam and Lapan, Joshua and Lundgren, Andrew and Preda, Daniel},
  Journal                  = {Science},
  Year                     = {2001},
  Number                   = {5516},
  Pages                    = {472--475},
  Volume                   = {292},
  Publisher                = {American Association for the Advancement of Science},
  URL                      = {https://science.sciencemag.org/content/292/5516/472}
}
</code></pre></details>


### 2000
<details><summary>[Farhi et al] <b><a href=https://arxiv.org/pdf/quant-ph/0001106.pdf> Quantum computation by adiabatic evolution. </a></b> <i>arXiv.</i> <code><b>[Adiabatic Quantum Computing]</b></code> <code><i>(Adiabatic Model)</i></code></summary><pre><code>@Article{farhi2000quantum,
  Title                    = {Quantum computation by adiabatic evolution},
  Author                   = {Farhi, Edward and Goldstone, Jeffrey and Gutmann, Sam and Sipser, Michael},
  Journal                  = {arXiv preprint quant-ph/0001106},
  Year                     = {2000},
  URL                      = {https://arxiv.org/pdf/quant-ph/0001106.pdf}
}
</code></pre></details>
<details><summary>[Narayanan & Menneer] <b><a href=https://www.sciencedirect.com/science/article/abs/pii/S0020025500000554> Quantum artificial neural network architectures and components. </a></b> <i>Information Sciences.</i> <code><b>[Quantum Neural Networks]</b></code> <code><b>[Quantum Machine Learning]</b></code> </summary><pre><code>@article{narayanan2000quantum,
  title={Quantum artificial neural network architectures and components},
  author={Narayanan, Ajit and Menneer, Tammy},
  journal={Information Sciences},
  volume={128},
  number={3-4},
  pages={231--255},
  year={2000},
  publisher={Elsevier}
}</code></pre></details>
<details><summary>[Ventura & Martinez] <b><a href=https://www.sciencedirect.com/science/article/abs/pii/S0020025599001012> Quantum associative memory. </a></b> <i>Information Sciences.</i> <code><b>[Quantum Machine Learning]</b></code> <code><b>[Quantum Information and Computing]</b></code> <code><b>[Quantum Classification]</b></code> </summary><pre><code>@article{ventura2000quantum,
  title={Quantum associative memory},
  author={Ventura, Dan and Martinez, Tony},
  journal={Information Sciences},
  volume={124},
  number={1-4},
  pages={273--296},
  year={2000},
  publisher={Elsevier}
}</code></pre></details>


### 1999
<details><summary>[Abrams &Lloyd] <b><a href=https://journals.aps.org/prl/abstract/10.1103/PhysRevLett.83.5162> Quantum algorithm providing exponential speed increase for finding eigenvalues and eigenvectors. </a></b> <i>Physical Review Letters.</i> <code><b>[Quantum Speedup-Advantage-Supremacy]</b></code> <code><i>(Gate Model)</i></code></summary><pre><code>@Article{abrams1999quantum,
  Title                    = {Quantum algorithm providing exponential speed increase for finding eigenvalues and eigenvectors},
  Author                   = {Abrams, Daniel S and Lloyd, Seth},
  Journal                  = {Physical review letters},
  Year                     = {1999},
  Number                   = {24},
  Pages                    = {5162},
  Volume                   = {83},
  Publisher                = {APS},
  URL                      = {https://journals.aps.org/prl/abstract/10.1103/PhysRevLett.83.5162}
}
</code></pre></details>


### 1998
<details><summary>[Kadowaki &Nishimori] <b><a href=https://journals.aps.org/pre/abstract/10.1103/PhysRevE.58.5355> Quantum annealing in the transverse Ising model. </a></b> <i>Physical Review E.</i> <code><b>[Quantum Annealing]</b></code> <code><i>(Adiabatic Model)</i></code></summary><pre><code>@article{kadowaki1998quantum,
  title={Quantum annealing in the transverse Ising model},
  author={Kadowaki, Tadashi and Nishimori, Hidetoshi},
  journal={Physical Review E},
  volume={58},
  number={5},
  pages={5355},
  year={1998},
  publisher={APS}
}</code></pre></details>


### 1997
<details><summary>[Bernstein & Vazirani] <b><a href=https://epubs.siam.org/doi/abs/10.1137/S0097539796300921> Quantum complexity theory. </a></b> <i>SIAM Journal on computing.</i> <code><b>[Quantum Information and Computing]</b></code> <code><b>[Quantum Speedup-Advantage-Supremacy]</b></code> </summary><pre><code>@article{bernstein1997quantum,
  title={Quantum complexity theory},
  author={Bernstein, Ethan and Vazirani, Umesh},
  journal={SIAM Journal on computing},
  volume={26},
  number={5},
  pages={1411--1473},
  year={1997},
  publisher={SIAM}
}</code></pre></details>
<details><summary>[Simon] <b><a href=https://epubs.siam.org/doi/10.1137/S0097539796298637> On the Power of Quantum Computation. </a></b> <i>SIAM Journal on Computing.</i> <code><b>[Quantum Information and Computing]</b></code> <code><b>[Quantum Speedup-Advantage-Supremacy]</b></code> </summary><pre><code>@article{simon1997power,
  title={On the power of quantum computation},
  author={Simon, Daniel R},
  journal={SIAM journal on computing},
  volume={26},
  number={5},
  pages={1474--1483},
  year={1997},
  publisher={SIAM}
}</code></pre></details>


### 1996
<details><summary>[Behrman et al ] <b><a href=http://citeseerx.ist.psu.edu/viewdoc/download?doi=10.1.1.56.1507&rep=rep1&type=pdf> A quantum dot neural network. </a></b> <i>Workshop on Physics of Computation.</i> <code><b>[Quantum Neural Networks]</b></code> <code><b>[Quantum Machine Learning]</b></code> <code><i>(Gate Model)</i></code></summary><pre><code>@InProceedings{behrman1996quantum,
  Title                    = {A quantum dot neural network},
  Author                   = {Behrman, Elizabeth C and Niemel, John and Steck, James E and Skinner, Steve R},
  Booktitle                = {Proceedings of the 4th Workshop on Physics of Computation},
  Year                     = {1996},
  Pages                    = {22--24},
  URL                      = {http://citeseerx.ist.psu.edu/viewdoc/download?doi=10.1.1.56.1507&rep=rep1&type=pdf}
}
</code></pre></details>


### 1995
<details><summary>[Barenco et al] <b><a href=https://journals.aps.org/pra/abstract/10.1103/PhysRevA.52.3457> Elementary gates for quantum computation. </a></b> <i>Physical Review A.</i> <code><b>[Quantum Information and Computing]</b></code> <code><b>[Quantum Gates]</b></code> <code><i>(Gate Model)</i></code></summary><pre><code>@Article{barenco1995elementary,
  Title                    = {Elementary gates for quantum computation},
  Author                   = {Barenco, Adriano and Bennett, Charles H and Cleve, Richard and DiVincenzo, David P and Margolus, Norman and Shor, Peter and Sleator, Tycho and Smolin, John A and Weinfurter, Harald},
  Journal                  = {Physical review A},
  Year                     = {1995},
  Number                   = {5},
  Pages                    = {3457},
  Volume                   = {52},
  Publisher                = {APS},
  URL                      = {https://journals.aps.org/pra/abstract/10.1103/PhysRevA.52.3457}
}
</code></pre></details>


### 1994
<details><summary>[Finnila et al] <b><a href=https://www.sciencedirect.com/science/article/abs/pii/0009261494001170?via%3Dihub> Quantum annealing: A new method for minimizing multidimensional functions. </a></b> <i>Chemical Physics Letters.</i> <code><b>[Quantum Annealing]</b></code> <code><i>(Adiabatic Model)</i></code></summary><pre><code>@Article{finnila1994quantum,
  Title                    = {Quantum annealing: A new method for minimizing multidimensional functions},
  Author                   = {Finnila, Aleta Berk and Gomez, MA and Sebenik, C and Stenson, Catherine and Doll, Jimmie D},
  Journal                  = {Chemical physics letters},
  Year                     = {1994},
  Number                   = {5-6},
  Pages                    = {343--348},
  Volume                   = {219},
  Publisher                = {Elsevier},
  URL                      = {https://www.sciencedirect.com/science/article/abs/pii/0009261494001170?via%3Dihub}
}
</code></pre></details>


### 1992
<details><summary>[Deutsch & Josza] <b><a href=https://royalsocietypublishing.org/doi/10.1098/rspa.1992.0167> Rapid solution of problems by quantum computation. </a></b> <i>Proceedings of the Royal Society A.</i> <code><b>[Quantum Information and Computing]</b></code> <code><b>[Quantum Speedup-Advantage-Supremacy]</b></code> </summary><pre><code>@Article{deutsch1992rapid,
  Title                    = {Rapid solution of problems by quantum computation},
  Author                   = {Deutsch, David and Jozsa, Richard},
  Journal                  = {Proceedings of the Royal Society of London. Series A: Mathematical and Physical Sciences},
  Year                     = {1992},
  Number                   = {1907},
  Pages                    = {553--558},
  Volume                   = {439},
  Publisher                = {The Royal Society London},
  URL                      = {https://royalsocietypublishing.org/doi/10.1098/rspa.1992.0167}
}
</code></pre></details>


### 1989
<details><summary>[Apolloni et al] <b><a href=https://www.sciencedirect.com/science/article/pii/0304414989900409?via%3Dihub> Quantum stochastic optimization. </a></b> <i>Stochastic Processes and their Applications.</i> <code><b>[Adiabatic Quantum Computing]</b></code> <code><b>[Quantum Annealing]</b></code> <code><b>[Quantum Optimization]</b></code> <code><i>(Adiabatic Model)</i></code></summary><pre><code>@Article{apolloni1989quantum,
  Title                    = {Quantum stochastic optimization},
  Author                   = {Apolloni, Bruno and Carvalho, C and De Falco, Diego},
  Journal                  = {Stochastic Processes and their Applications},
  Year                     = {1989},
  Number                   = {2},
  Pages                    = {233--244},
  Volume                   = {33},
  Publisher                = {Elsevier},
  URL                      = {https://www.sciencedirect.com/science/article/pii/0304414989900409?via%3Dihub}
}
</code></pre></details>


### 1986
<details><summary>[Fu & Anderson] <b><a href=https://iopscience.iop.org/article/10.1088/0305-4470/19/9/033> Application of statistical mechanics to NP-complete problems in combinatorial optimisation. </a></b> <i>Journal of Physics A: Mathematical and General
.</i> <code><b>[Statistical Mechanics]</b></code> </summary><pre><code>@Article{fu1986application,
  Title                    = {{Application of statistical mechanics to NP-complete problems in combinatorial optimisation}},
  Author                   = {Fu, Yaotian and Anderson, Philip W},
  Journal                  = {Journal of Physics A: Mathematical and General},
  Year                     = {1986},
  Number                   = {9},
  Pages                    = {1605},
  Volume                   = {19},
  Publisher                = {IOP Publishing},
  URL                      = {https://iopscience.iop.org/article/10.1088/0305-4470/19/9/033}
}
</code></pre></details>


### 1985
<details><summary>[Deutsch] <b><a href=https://royalsocietypublishing.org/doi/10.1098/rspa.1985.0070> Quantum theory, the Church–Turing principle and the universal quantum computer. </a></b> <i>Proceedings of the Royal Society A.</i> <code><b>[Quantum Information and Computing]</b></code> </summary><pre><code>@Article{deutsch1985quantum,
  Title                    = {{Quantum theory, the Church--Turing principle and the universal quantum computer}},
  Author                   = {Deutsch, David},
  Journal                  = {Proceedings of the Royal Society of London. A. Mathematical and Physical Sciences},
  Year                     = {1985},
  Number                   = {1818},
  Pages                    = {97--117},
  Volume                   = {400},
  Publisher                = {The Royal Society London},
  URL                      = {https://royalsocietypublishing.org/doi/10.1098/rspa.1985.0070}
}
</code></pre></details>


### 1983
<details><summary>[Kirkpatrick et al] <b><a href=http://wexler.free.fr/library/files/kirkpatrick%20(1983)%20optimization%20by%20simulated%20annealing.pdf> Optimization by Simulated Annealing. </a></b> <i>Science.</i> <code><b>[Simulated Annealing]</b></code> </summary><pre><code>@article{kirkpatrick1983optimization,
  title={Optimization by simulated annealing},
  author={Kirkpatrick, Scott and Gelatt, C Daniel and Vecchi, Mario P},
  journal={science},
  volume={220},
  number={4598},
  pages={671--680},
  year={1983},
  publisher={American association for the advancement of science}
}</code></pre></details>


In [58]:
display(Markdown(open('papersSortedByTopics.md', encoding='utf-8').read()))


A Leap among Quantum Computing and Quantum Neural Networks: A Survey
====================================================================


--Work in Progress--

# Table of Contents
* [Adiabatic Quantum Computing](#Adiabatic-Quantum-Computing)
* [Barren Plateaus](#Barren-Plateaus)
* [Classical Implementation](#Classical-Implementation)
* [Complexity Theory](#Complexity-Theory)
* [Parameterized Quantum Circuits](#Parameterized-Quantum-Circuits)
* [Physical Realization of Qubits](#Physical-Realization-of-Qubits)
* [Quantum Annealing](#Quantum-Annealing)
* [Quantum Boltzmann machines](#Quantum-Boltzmann-machines)
* [Quantum Classification](#Quantum-Classification)
* [Quantum Clustering](#Quantum-Clustering)
* [Quantum Convolutional Neural Networks](#Quantum-Convolutional-Neural-Networks)
* [Quantum Generative Adversarial Networks](#Quantum-Generative-Adversarial-Networks)
* [Quantum Information and Computing](#Quantum-Information-and-Computing)
* [Quantum Machine Learning](#Quantum-Machine-Learning)
* [Quantum Nearest Neighbors Algorithm](#Quantum-Nearest-Neighbors-Algorithm)
* [Quantum Neural Networks](#Quantum-Neural-Networks)
* [Quantum Optimization](#Quantum-Optimization)
* [Quantum Perceptron](#Quantum-Perceptron)
* [Quantum Speedup-Advantage-Supremacy](#Quantum-Speedup-Advantage-Supremacy)
* [Simulated Annealing](#Simulated-Annealing)
* [Superconducting Qubits](#Superconducting-Qubits)
* [Unsupervised Learning](#Unsupervised-Learning)
* [Variational Quantum Algorithms](#Variational-Quantum-Algorithms)



# Adiabatic Quantum Computing


### 2018
<details><summary>[Albash & Lidar] <b><a href=https://journals.aps.org/rmp/abstract/10.1103/RevModPhys.90.015002> Adiabatic Quantum Computing. </a></b> <i>Reviews of Modern Physics.</i> <code><b>[Quantum Information and Computing]</b></code> <code><b>[Adiabatic Quantum Computing]</b></code> <code><i>(Adiabatic Model)</i></code></summary><pre><code>@Article{albash2018adiabatic,
  Title                    = {Adiabatic quantum computation},
  Author                   = {Albash, Tameem and Lidar, Daniel A},
  Journal                  = {Reviews of Modern Physics},
  Year                     = {2018},
  Number                   = {1},
  Pages                    = {015002},
  Volume                   = {90},
  Publisher                = {APS},
  URL                      = {https://journals.aps.org/rmp/abstract/10.1103/RevModPhys.90.015002}
}
</code></pre></details>


### 2017
<details><summary>[Andriyash et al] <b><a href=https://arxiv.org/abs/1703.09277> Can quantum Monte Carlo simulate quantum annealing?. </a></b> <i>arXiv.</i> <code><b>[Adiabatic Quantum Computing]</b></code> <code><b>[Quantum Annealing]</b></code> <code><b>[Quantum Monte Carlo]</b></code> <code><i>(Adiabatic Model)</i></code></summary><pre><code>@Article{andriyash2017can,
  Title                    = {Can quantum Monte Carlo simulate quantum annealing?},
  Author                   = {Andriyash, Evgeny and Amin, Mohammad H},
  Journal                  = {arXiv preprint arXiv:1703.09277},
  Year                     = {2017},
  URL                      = {https://arxiv.org/abs/1703.09277}
}
</code></pre></details>


### 2016
<details><summary>[Batle et al] <b><a href=https://link.springer.com/article/10.1007/s11128-016-1324-x> Do multipartite correlations speed up adiabatic quantum computation or quantum annealing?. </a></b> <i>Quantum Information Processing.</i> <code><b>[Adiabatic Quantum Computing]</b></code> <code><b>[Quantum Annealing]</b></code> <code><b>[Quantum Speedup-Advantage-Supremacy]</b></code> <code><i>(Adiabatic Model)</i></code></summary><pre><code>@Article{batle2016multipartite,
  Title                    = {Do multipartite correlations speed up adiabatic quantum computation or quantum annealing?},
  Author                   = {Batle, Josep and Ooi, CH Raymond and Farouk, Ahmed and Abutalib, M and Abdalla, S},
  Journal                  = {Quantum Information Processing},
  Year                     = {2016},
  Number                   = {8},
  Pages                    = {3081--3099},
  Volume                   = {15},
  Publisher                = {Springer},
  URL                      = {https://link.springer.com/article/10.1007/s11128-016-1324-x}
}
</code></pre></details>


### 2012
<details><summary>[Gaitan & Clark] <b><a href=https://journals.aps.org/prl/abstract/10.1103/PhysRevLett.108.010501> Ramsey Numbers and Adiabatic Quantum Computing. </a></b> <i>Physical Review Letters.</i> <code><b>[Adiabatic Quantum Computing]</b></code> <code><b>[Quantum Annealing]</b></code> <code><b>[Classical Implementation]</b></code> <code><i>(Adiabatic Model)</i></code></summary><pre><code>@Article{gaitan2012ramsey,
  Title                    = {Ramsey numbers and adiabatic quantum computing},
  Author                   = {Gaitan, Frank and Clark, Lane},
  Journal                  = {Physical review letters},
  Year                     = {2012},
  Number                   = {1},
  Pages                    = {010501},
  Volume                   = {108},
  Publisher                = {APS},
  URL                      = {https://journals.aps.org/prl/abstract/10.1103/PhysRevLett.108.010501}
}
</code></pre></details>


### 2010
<details><summary>[Altshuler et al] <b><a href=https://www.pnas.org/content/107/28/12446> Anderson localization makes adiabatic quantum optimization fail. </a></b> <i>PNAS.</i> <code><b>[Adiabatic Quantum Computing]</b></code> <code><b>[Quantum Optimization]</b></code> <code><i>(Adiabatic Model)</i></code></summary><pre><code>@Article{altshuler2010anderson,
  Title                    = {Anderson localization makes adiabatic quantum optimization fail},
  Author                   = {Altshuler, Boris and Krovi, Hari and Roland, J{\'e}r{\'e}mie},
  Journal                  = {Proceedings of the National Academy of Sciences},
  Year                     = {2010},
  Number                   = {28},
  Pages                    = {12446--12450},
  Volume                   = {107},
  Publisher                = {National Acad Sciences},
  URL                      = {https://www.pnas.org/content/107/28/12446}
}
</code></pre></details>
<details><summary>[Bravyi & Terhal] <b><a href=https://arxiv.org/abs/0806.1746> Complexity of stoquastic frustration-free Hamiltonians. </a></b> <i>Siam journal on computing.</i> <code><b>[Adiabatic Quantum Computing]</b></code> <code><b>[Quantum Annealing]</b></code> <code><i>(Adiabatic Model)</i></code></summary><pre><code>@Article{bravyi2010complexity,
  Title                    = {Complexity of stoquastic frustration-free Hamiltonians},
  Author                   = {Bravyi, Sergey and Terhal, Barbara},
  Journal                  = {Siam journal on computing},
  Year                     = {2010},
  Number                   = {4},
  Pages                    = {1462--1485},
  Volume                   = {39},
  Publisher                = {SIAM},
  URL                      = {https://arxiv.org/abs/0806.1746}
}
</code></pre></details>


### 2009
<details><summary>[Lidar et al] <b><a href=https://aip.scitation.org/doi/full/10.1063/1.3236685> Adiabatic approximation with exponential accuracy for many-body systems and quantum computation. </a></b> <i>Journal of Mathematical Physics.</i> <code><b>[Approximation to Adiabatic Theorem]</b></code> <code><b>[Adiabatic Quantum Computing]</b></code> <code><i>(Adiabatic Model)</i></code></summary><pre><code>@article{lidar2009adiabatic,
  title={Adiabatic approximation with exponential accuracy for many-body systems and quantum computation},
  author={Lidar, Daniel A and Rezakhani, Ali T and Hamma, Alioscia},
  journal={Journal of Mathematical Physics},
  volume={50},
  number={10},
  pages={102106},
  year={2009},
  publisher={American Institute of Physics}
}</code></pre></details>


### 2008
<details><summary>[Aharonov et al] <b><a href=https://epubs.siam.org/doi/10.1137/080734479> Adiabatic quantum computation is equivalent to standard quantum computation. </a></b> <i>SIAM review.</i> <code><b>[Adiabatic Quantum Computing]</b></code> <code><b>[Adiabatic Simulation]</b></code> <code><i>(Adiabatic Model)</i></code></summary><pre><code>@Article{aharonov2008adiabatic,
  Title                    = {Adiabatic quantum computation is equivalent to standard quantum computation},
  Author                   = {Aharonov, Dorit and Van Dam, Wim and Kempe, Julia and Landau, Zeph and Lloyd, Seth and Regev, Oded},
  Journal                  = {SIAM review},
  Year                     = {2008},
  Number                   = {4},
  Pages                    = {755--787},
  Volume                   = {50},
  Publisher                = {SIAM},
  URL                      = {https://epubs.siam.org/doi/10.1137/080734479}
}
</code></pre></details>


### 2005
<details><summary>[Battaglia et al] <b><a href=https://link.springer.com/chapter/10.1007%2F11526216_7> Deterministic and stochastic quantum annealing approaches. </a></b> <i>Lecture Notes in Physics .</i> <code><b>[Adiabatic Quantum Computing]</b></code> <code><b>[Quantum Annealing]</b></code> <code><b>[Simulated Annealing]</b></code> <code><i>(Adiabatic Model)</i></code></summary><pre><code>@InCollection{battaglia2005deterministic,
  Title                    = {Deterministic and stochastic quantum annealing approaches},
  Author                   = {Battaglia, Demian and Stella, Lorenzo and Zagordi, Osvaldo and Santoro, Giuseppe E and Tosatti, Erio},
  Booktitle                = {Quantum Annealing and Other Optimization Methods},
  Publisher                = {Springer},
  Year                     = {2005},
  Pages                    = {171--206},
  URL                      = {https://link.springer.com/chapter/10.1007%2F11526216_7}
}
</code></pre></details>
<details><summary>[Battaglia et al] <b><a href=https://journals.aps.org/pre/abstract/10.1103/PhysRevE.71.066707> Optimization by Quantum Annealing: Lessons from hard 3-SAT cases,. </a></b> <i>Physical Review E.</i> <code><b>[Adiabatic Quantum Computing]</b></code> <code><b>[Quantum Annealing]</b></code> <code><b>[Simulated Annealing]</b></code> <code><b>[Quantum Monte Carlo]</b></code> <code><i>(Adiabatic Model)</i></code></summary><pre><code>@Article{battaglia2005optimization,
  Title                    = {Optimization by quantum annealing: Lessons from hard satisfiability problems},
  Author                   = {Battaglia, Demian A and Santoro, Giuseppe E and Tosatti, Erio},
  Journal                  = {Physical Review E},
  Year                     = {2005},
  Number                   = {6},
  Pages                    = {066707},
  Volume                   = {71},
  Publisher                = {APS},
  URL                      = {https://journals.aps.org/pre/abstract/10.1103/PhysRevE.71.066707}
}
</code></pre></details>
<details><summary>[Olivera et al] <b><a href=https://dl.acm.org/doi/10.5555/2016985.2016987> The complexity of quantum spin systems on a two-dimensional square lattice. </a></b> <i>Quantum Information & Computing.</i> <code><b>[Complexity Theory]</b></code> <code><b>[Adiabatic Quantum Computing]</b></code> </summary><pre><code>@article{oliveira2005complexity,
  title={The complexity of quantum spin systems on a two-dimensional square lattice},
  author={Oliveira, Roberto and Terhal, Barbara M},
  journal={arXiv preprint quant-ph/0504050},
  year={2005}
}</code></pre></details>


### 2004
<details><summary>[Aharonov et al] <b><a href=https://arxiv.org/pdf/quant-ph/0405098.pdf> Adiabatic computation is equivalent to standard quantum computation. </a></b> <i>arXiv.</i> <code><b>[Adiabatic Quantum Computing]</b></code> <code><b>[Adiabatic Simulation]</b></code> <code><i>(Adiabatic Model)</i></code></summary><pre><code>@Article{aharonov2004adiabatic,
  Title                    = {Adiabatic computation is equivalent to standard quantum computation},
  Author                   = {Aharonov, D and van Dam, W and Kempe, J and Landau, Z and Lloyd, S and Regev, O},
  Journal                  = {arXiv preprint quant-ph/0405098},
  Year                     = {2004},
  URL                      = {https://arxiv.org/pdf/quant-ph/0405098.pdf}
}
</code></pre></details>


### 2001
<details><summary>[Farhi et al] <b><a href=https://science.sciencemag.org/content/292/5516/472> A Quantum Adiabatic Evolution Algorithm Applied to Random Instances of an NP-Complete Problem. </a></b> <i>Science.</i> <code><b>[Adiabatic Quantum Computing]</b></code> <code><b>[Classical Implementation]</b></code> <code><b>[Quantum Annealing]</b></code> <code><i>(Adiabatic Model)</i></code></summary><pre><code>@Article{farhi2001quantum,
  Title                    = {{A quantum adiabatic evolution algorithm applied to random instances of an NP-complete problem}},
  Author                   = {Farhi, Edward and Goldstone, Jeffrey and Gutmann, Sam and Lapan, Joshua and Lundgren, Andrew and Preda, Daniel},
  Journal                  = {Science},
  Year                     = {2001},
  Number                   = {5516},
  Pages                    = {472--475},
  Volume                   = {292},
  Publisher                = {American Association for the Advancement of Science},
  URL                      = {https://science.sciencemag.org/content/292/5516/472}
}
</code></pre></details>


### 2000
<details><summary>[Farhi et al] <b><a href=https://arxiv.org/pdf/quant-ph/0001106.pdf> Quantum computation by adiabatic evolution. </a></b> <i>arXiv.</i> <code><b>[Adiabatic Quantum Computing]</b></code> <code><i>(Adiabatic Model)</i></code></summary><pre><code>@Article{farhi2000quantum,
  Title                    = {Quantum computation by adiabatic evolution},
  Author                   = {Farhi, Edward and Goldstone, Jeffrey and Gutmann, Sam and Sipser, Michael},
  Journal                  = {arXiv preprint quant-ph/0001106},
  Year                     = {2000},
  URL                      = {https://arxiv.org/pdf/quant-ph/0001106.pdf}
}
</code></pre></details>


### 1989
<details><summary>[Apolloni et al] <b><a href=https://www.sciencedirect.com/science/article/pii/0304414989900409?via%3Dihub> Quantum stochastic optimization. </a></b> <i>Stochastic Processes and their Applications.</i> <code><b>[Adiabatic Quantum Computing]</b></code> <code><b>[Quantum Annealing]</b></code> <code><b>[Quantum Optimization]</b></code> <code><i>(Adiabatic Model)</i></code></summary><pre><code>@Article{apolloni1989quantum,
  Title                    = {Quantum stochastic optimization},
  Author                   = {Apolloni, Bruno and Carvalho, C and De Falco, Diego},
  Journal                  = {Stochastic Processes and their Applications},
  Year                     = {1989},
  Number                   = {2},
  Pages                    = {233--244},
  Volume                   = {33},
  Publisher                = {Elsevier},
  URL                      = {https://www.sciencedirect.com/science/article/pii/0304414989900409?via%3Dihub}
}
</code></pre></details>


# Barren Plateaus


### 2021
<details><summary>[Cerezo et al] <b><a href=https://www.nature.com/articles/s41467-021-21728-w> Cost Function Dependent Barren Plateaus in Shallow Parametrized Quantum Circuits. </a></b> <i>Nature communications.</i> <code><b>[Barren Plateaus]</b></code> <code><b>[Parameterized Quantum Circuits]</b></code> <code><b>[Quantum Neural Networks]</b></code> <code><b>[Quantum Machine Learning]</b></code> </summary><pre><code>@article{cerezo2021cost,
  title={Cost function dependent barren plateaus in shallow parametrized quantum circuits},
  author={Cerezo, Marco and Sone, Akira and Volkoff, Tyler and Cincio, Lukasz and Coles, Patrick J},
  journal={Nature communications},
  volume={12},
  number={1},
  pages={1--12},
  year={2021},
  publisher={Nature Publishing Group}
}</code></pre></details>
<details><summary>[Holmes et al ] <b><a href=https://arxiv.org/abs/2101.02138> Connecting ansatz expressibility to gradient magnitudes and barren plateaus. </a></b> <i>arXiv.</i> <code><b>[Parameterized Quantum Circuits]</b></code> <code><b>[Quantum Neural Networks]</b></code> <code><b>[Barren Plateaus]</b></code> <code><b>[Quantum Machine Learning]</b></code> </summary><pre><code>@article{holmes2021connecting,
  title={Connecting ansatz expressibility to gradient magnitudes and barren plateaus},
  author={Holmes, Zo{\"e} and Sharma, Kunal and Cerezo, M and Coles, Patrick J},
  journal={arXiv preprint arXiv:2101.02138},
  year={2021}
}</code></pre></details>
<details><summary>[Patti et al] <b><a href=https://arxiv.org/abs/2012.12658> Entanglement devised barren plateau mitigation. </a></b> <i>arXiv.</i> <code><b>[Barren Plateaus]</b></code> <code><b>[Parameterized Quantum Circuits]</b></code> <code><b>[Quantum Neural Networks]</b></code> <code><b>[Quantum Machine Learning]</b></code> </summary><pre><code>@article{patti2020entanglement,
  title={Entanglement devised barren plateau mitigation},
  author={Patti, Taylor L and Najafi, Khadijeh and Gao, Xun and Yelin, Susanne F},
  journal={arXiv preprint arXiv:2012.12658},
  year={2020}
}</code></pre></details>
<details><summary>[Skolik et al] <b><a href=https://link.springer.com/article/10.1007/s42484-020-00036-4> Layerwise learning for quantum neural networks. </a></b> <i>Quantum Machine Intelligence.</i> <code><b>[Barren Plateaus]</b></code> <code><b>[Parameterized Quantum Circuits]</b></code> <code><b>[Quantum Neural Networks]</b></code> <code><b>[Quantum Machine Learning]</b></code> </summary><pre><code>@article{skolik2021layerwise,
  title={Layerwise learning for quantum neural networks},
  author={Skolik, Andrea and McClean, Jarrod R and Mohseni, Masoud and van der Smagt, Patrick and Leib, Martin},
  journal={Quantum Machine Intelligence},
  volume={3},
  number={1},
  pages={1--11},
  year={2021},
  publisher={Springer}
}</code></pre></details>
<details><summary>[Volkoff & Coles] <b><a href=https://arxiv.org/abs/2005.12200> Large gradients via correlation in random parameterized quantum circuits. </a></b> <i>arXiv.</i> <code><b>[Barren Plateaus]</b></code> <code><b>[Parameterized Quantum Circuits]</b></code> <code><b>[Quantum Neural Networks]</b></code> <code><b>[Variational Quantum Algorithms]</b></code> <code><b>[Quantum Machine Learning]</b></code> </summary><pre><code>@article{volkoff2021large,
  title={Large gradients via correlation in random parameterized quantum circuits},
  author={Volkoff, Tyler and Coles, Patrick J},
  journal={Quantum Science and Technology},
  volume={6},
  number={2},
  pages={025008},
  year={2021},
  publisher={IOP Publishing}
}</code></pre></details>


### 2020
<details><summary>[Marrero et al] <b><a href=https://arxiv.org/abs/2010.15968> Entanglement Induced Barren Plateaus. </a></b> <i>arXiv.</i> <code><b>[Barren Plateaus]</b></code> <code><b>[Parameterized Quantum Circuits]</b></code> <code><b>[Quantum Neural Networks]</b></code> <code><b>[Quantum Machine Learning]</b></code> </summary><pre><code>@article{marrero2020entanglement,
  title={Entanglement induced barren plateaus},
  author={Marrero, Carlos Ortiz and Kieferov{\'a}, M{\'a}ria and Wiebe, Nathan},
  journal={arXiv preprint arXiv:2010.15968},
  year={2020}
}</code></pre></details>
<details><summary>[Pesah et al] <b><a href=https://arxiv.org/abs/2011.02966> Absence of Barren Plateaus in Quantum Convolutional Neural Networks. </a></b> <i>arXiv.</i> <code><b>[Barren Plateaus]</b></code> <code><b>[Quantum Convolutional Neural Networks]</b></code> <code><b>[Parameterized Quantum Circuits]</b></code> <code><b>[Quantum Neural Networks]</b></code> <code><b>[Quantum Machine Learning]</b></code> </summary><pre><code>@article{pesah2020absence,
  title={Absence of barren plateaus in quantum convolutional neural networks},
  author={Pesah, Arthur and Cerezo, M and Wang, Samson and Volkoff, Tyler and Sornborger, Andrew T and Coles, Patrick J},
  journal={arXiv preprint arXiv:2011.02966},
  year={2020}
}</code></pre></details>
<details><summary>[Sharma et al] <b><a href=https://arxiv.org/abs/2005.12458> Trainability of Dissipative Perceptron-Based Quantum Neural Networks. </a></b> <i>arXiv.</i> <code><b>[Parameterized Quantum Circuits]</b></code> <code><b>[Quantum Neural Networks]</b></code> <code><b>[Quantum Machine Learning]</b></code> <code><b>[Barren Plateaus]</b></code> </summary><pre><code>@article{sharma2020trainability,
  title={Trainability of dissipative perceptron-based quantum neural networks},
  author={Sharma, Kunal and Cerezo, Marco and Cincio, Lukasz and Coles, Patrick J},
  journal={arXiv preprint arXiv:2005.12458},
  year={2020}
}</code></pre></details>
<details><summary>[Wang et al] <b><a href=https://arxiv.org/abs/2007.14384> Noise-Induced Barren Plateaus in Variational Quantum Algorithms. </a></b> <i>arXiv.</i> <code><b>[Barren Plateaus]</b></code> <code><b>[Parameterized Quantum Circuits]</b></code> <code><b>[Variational Quantum Algorithms]</b></code> <code><b>[Quantum Machine Learning]</b></code> </summary><pre><code>@article{wang2020noise,
  title={Noise-induced barren plateaus in variational quantum algorithms},
  author={Wang, Samson and Fontana, Enrico and Cerezo, Marco and Sharma, Kunal and Sone, Akira and Cincio, Lukasz and Coles, Patrick J},
  journal={arXiv preprint arXiv:2007.14384},
  year={2020}
}</code></pre></details>


### 2019
<details><summary>[Grant et al] <b><a href=https://quantum-journal.org/papers/q-2019-12-09-214/> An initialization strategy for addressing barren plateaus in parametrized quantum circuits. </a></b> <i>Quantum.</i> <code><b>[Barren Plateaus]</b></code> <code><b>[Parameterized Quantum Circuits]</b></code> <code><b>[Quantum Neural Networks]</b></code> <code><b>[Quantum Machine Learning]</b></code> </summary><pre><code>@article{grant2019initialization,
  title={An initialization strategy for addressing barren plateaus in parametrized quantum circuits},
  author={Grant, Edward and Wossnig, Leonard and Ostaszewski, Mateusz and Benedetti, Marcello},
  journal={Quantum},
  volume={3},
  pages={214},
  year={2019},
  publisher={Verein zur F{\"o}rderung des Open Access Publizierens in den Quantenwissenschaften}
}</code></pre></details>


### 2018
<details><summary>[McClean et al] <b><a href=https://www.nature.com/articles/s41467-018-07090-4> Barren plateaus in quantum neural network training landscapes. </a></b> <i>Nature communications.</i> <code><b>[Barren Plateaus]</b></code> <code><b>[Parameterized Quantum Circuits]</b></code> <code><b>[Quantum Neural Networks]</b></code> <code><b>[Quantum Machine Learning]</b></code> </summary><pre><code>@article{mcclean2018barren,
  title={Barren plateaus in quantum neural network training landscapes},
  author={McClean, Jarrod R and Boixo, Sergio and Smelyanskiy, Vadim N and Babbush, Ryan and Neven, Hartmut},
  journal={Nature communications},
  volume={9},
  number={1},
  pages={1--6},
  year={2018},
  publisher={Nature Publishing Group}
}</code></pre></details>


# Classical Implementation


### 2012
<details><summary>[Gaitan & Clark] <b><a href=https://journals.aps.org/prl/abstract/10.1103/PhysRevLett.108.010501> Ramsey Numbers and Adiabatic Quantum Computing. </a></b> <i>Physical Review Letters.</i> <code><b>[Adiabatic Quantum Computing]</b></code> <code><b>[Quantum Annealing]</b></code> <code><b>[Classical Implementation]</b></code> <code><i>(Adiabatic Model)</i></code></summary><pre><code>@Article{gaitan2012ramsey,
  Title                    = {Ramsey numbers and adiabatic quantum computing},
  Author                   = {Gaitan, Frank and Clark, Lane},
  Journal                  = {Physical review letters},
  Year                     = {2012},
  Number                   = {1},
  Pages                    = {010501},
  Volume                   = {108},
  Publisher                = {APS},
  URL                      = {https://journals.aps.org/prl/abstract/10.1103/PhysRevLett.108.010501}
}
</code></pre></details>


### 2009
<details><summary>[Matsuda et al] <b><a href=https://iopscience.iop.org/article/10.1088/1367-2630/11/7/073021> Ground-state statistics from annealing algorithms: quantum versus classical approaches. </a></b> <i>New Journal of Physics.</i> <code><b>[Classical Implementation]</b></code> <code><b>[Quantum Annealing]</b></code> </summary><pre><code>@article{matsuda2009ground,
  title={Ground-state statistics from annealing algorithms: quantum versus classical approaches},
  author={Matsuda, Yoshiki and Nishimori, Hidetoshi and Katzgraber, Helmut G},
  journal={New Journal of Physics},
  volume={11},
  number={7},
  pages={073021},
  year={2009},
  publisher={IOP Publishing}
}</code></pre></details>


### 2004
<details><summary>[Martonak et al] <b><a href=https://journals.aps.org/pre/abstract/10.1103/PhysRevE.70.057701> Quantum annealing of the traveling-salesman problem. </a></b> <i>Physical Review E.</i> <code><b>[Classical Implementation]</b></code> <code><b>[Quantum Annealing]</b></code> </summary><pre><code>@article{martovnak2004quantum,
  title={Quantum annealing of the traveling-salesman problem},
  author={Marto{\v{n}}{\'a}k, Roman and Santoro, Giuseppe E and Tosatti, Erio},
  journal={Physical Review E},
  volume={70},
  number={5},
  pages={057701},
  year={2004},
  publisher={APS}
}</code></pre></details>


### 2001
<details><summary>[Farhi et al] <b><a href=https://science.sciencemag.org/content/292/5516/472> A Quantum Adiabatic Evolution Algorithm Applied to Random Instances of an NP-Complete Problem. </a></b> <i>Science.</i> <code><b>[Adiabatic Quantum Computing]</b></code> <code><b>[Classical Implementation]</b></code> <code><b>[Quantum Annealing]</b></code> <code><i>(Adiabatic Model)</i></code></summary><pre><code>@Article{farhi2001quantum,
  Title                    = {{A quantum adiabatic evolution algorithm applied to random instances of an NP-complete problem}},
  Author                   = {Farhi, Edward and Goldstone, Jeffrey and Gutmann, Sam and Lapan, Joshua and Lundgren, Andrew and Preda, Daniel},
  Journal                  = {Science},
  Year                     = {2001},
  Number                   = {5516},
  Pages                    = {472--475},
  Volume                   = {292},
  Publisher                = {American Association for the Advancement of Science},
  URL                      = {https://science.sciencemag.org/content/292/5516/472}
}
</code></pre></details>


# Complexity Theory


### 2017
<details><summary>[Arunachalam & de Wolf] <b><a href=https://dl.acm.org/doi/abs/10.1145/3106700.3106710> Guest column: A survey of quantum learning theory. </a></b> <i>ACM SIGACT News.</i> <code><b>[Quantum Machine Learning]</b></code> <code><b>[Probably Approximately Correct Learning]</b></code> <code><b>[Complexity Theory]</b></code> <code><i>(Gate Model)</i></code></summary><pre><code>@Article{arunachalam2017guest,
  Title                    = {Guest column: A survey of quantum learning theory},
  Author                   = {Arunachalam, Srinivasan and de Wolf, Ronald},
  Journal                  = {ACM SIGACT News},
  Year                     = {2017},
  Number                   = {2},
  Pages                    = {41--67},
  Volume                   = {48},
  Publisher                = {ACM New York, NY, USA},
  URL                      = {https://dl.acm.org/doi/abs/10.1145/3106700.3106710}
}
</code></pre></details>


### 2011
<details><summary>[Aaronson and Arkhipov] <b><a href=https://dl.acm.org/doi/pdf/10.1145/1993636.1993682> The computational complexity of linear optics. </a></b> <i>Annual ACM symposium on Theory of computing.</i> <code><b>[Quantum Speedup-Advantage-Supremacy]</b></code> <code><b>[Complexity Theory]</b></code> </summary><pre><code>@InProceedings{aaronson2011computational,
  Title                    = {The computational complexity of linear optics},
  Author                   = {Aaronson, Scott and Arkhipov, Alex},
  Booktitle                = {Proceedings of the forty-third annual ACM symposium on Theory of computing},
  Year                     = {2011},
  Pages                    = {333--342},
  Publisher                = {Association for Computing Machinery},
  URL                      = {https://dl.acm.org/doi/pdf/10.1145/1993636.1993682}
}
</code></pre></details>


### 2005
<details><summary>[Olivera et al] <b><a href=https://dl.acm.org/doi/10.5555/2016985.2016987> The complexity of quantum spin systems on a two-dimensional square lattice. </a></b> <i>Quantum Information & Computing.</i> <code><b>[Complexity Theory]</b></code> <code><b>[Adiabatic Quantum Computing]</b></code> </summary><pre><code>@article{oliveira2005complexity,
  title={The complexity of quantum spin systems on a two-dimensional square lattice},
  author={Oliveira, Roberto and Terhal, Barbara M},
  journal={arXiv preprint quant-ph/0504050},
  year={2005}
}</code></pre></details>


# Parameterized Quantum Circuits


### 2021
<details><summary>[Amaro et al.] <b><a href=https://arxiv.org/abs/2106.10055> Filtering variational quantum algorithms for combinatorial optimization. </a></b> <i>arXiv.</i> <code><b>[Variational Quantum Algorithms]</b></code> <code><b>[Quantum Optimization]</b></code> <code><b>[Parameterized Quantum Circuits]</b></code> </summary><pre><code>@article{amaro2021filtering,
  title={Filtering variational quantum algorithms for combinatorial optimization},
  author={Amaro, David and Modica, Carlo and Rosenkranz, Matthias and Fiorentini, Mattia and Benedetti, Marcello and Lubasch, Michael},
  journal={Quantum Science and Technology},
  year={2021},
  publisher={IOP Publishing}
}
</code></pre></details>
<details><summary>[Benedetti et al] <b><a href=https://journals.aps.org/prresearch/abstract/10.1103/PhysRevResearch.3.033083> Hardware-efficient variational quantum algorithms for time evolution. </a></b> <i>Physical Review Research.</i> <code><b>[Time Evolution (Hamiltonian Simulation)]</b></code> <code><b>[Parameterized Quantum Circuits]</b></code> <code><b>[Quantum Machine Learning]</b></code> </summary><pre><code>@article{benedetti2021hardware,
  title={Hardware-efficient variational quantum algorithms for time evolution},
  author={Benedetti, Marcello and Fiorentini, Mattia and Lubasch, Michael},
  journal={Physical Review Research},
  volume={3},
  number={3},
  pages={033083},
  year={2021},
  publisher={APS}
}
</code></pre></details>
<details><summary>[Benedetti et al] <b><a href=https://journals.aps.org/prapplied/abstract/10.1103/PhysRevApplied.16.044057> Variational Inference with a Quantum Computer. </a></b> <i>Physical Review Applied.</i> <code><b>[Variational Inference]</b></code> <code><b>[Parameterized Quantum Circuits]</b></code> <code><b>[Quantum Machine Learning]</b></code> </summary><pre><code>@article{benedetti2021variational,
  title={Variational inference with a quantum computer},
  author={Benedetti, Marcello and Coyle, Brian and Fiorentini, Mattia and Lubasch, Michael and Rosenkranz, Matthias},
  journal={Physical Review Applied},
  volume={16},
  number={4},
  pages={044057},
  year={2021},
  publisher={APS}
}
</code></pre></details>
<details><summary>[Bharti et al] <b><a href=https://arxiv.org/abs/2101.08448> Noisy intermediate-scale quantum (NISQ) algorithms. </a></b> <i>arXiv.</i> <code><b>[Variational Quantum Algorithms]</b></code> <code><b>[Parameterized Quantum Circuits]</b></code> <code><b>[Quantum Optimization]</b></code> <code><b>[Quantum Machine Learning]</b></code> <code><i>(Gate Model)</i></code> <code><i>(Adiabatic Model)</i></code></summary><pre><code>@article{bharti2021noisy,
  title={Noisy intermediate-scale quantum (NISQ) algorithms},
  author={Bharti, Kishor and Cervera-Lierta, Alba and Kyaw, Thi Ha and Haug, Tobias and Alperin-Lea, Sumner and Anand, Abhinav and Degroote, Matthias and Heimonen, Hermanni and Kottmann, Jakob S and Menke, Tim and others},
  journal={arXiv preprint arXiv:2101.08448},
  year={2021}
}</code></pre></details>
<details><summary>[Cerezo et al] <b><a href=https://www.nature.com/articles/s41467-021-21728-w> Cost Function Dependent Barren Plateaus in Shallow Parametrized Quantum Circuits. </a></b> <i>Nature communications.</i> <code><b>[Barren Plateaus]</b></code> <code><b>[Parameterized Quantum Circuits]</b></code> <code><b>[Quantum Neural Networks]</b></code> <code><b>[Quantum Machine Learning]</b></code> </summary><pre><code>@article{cerezo2021cost,
  title={Cost function dependent barren plateaus in shallow parametrized quantum circuits},
  author={Cerezo, Marco and Sone, Akira and Volkoff, Tyler and Cincio, Lukasz and Coles, Patrick J},
  journal={Nature communications},
  volume={12},
  number={1},
  pages={1--12},
  year={2021},
  publisher={Nature Publishing Group}
}</code></pre></details>
<details><summary>[Egger et al] <b><a href=https://quantum-journal.org/papers/q-2021-06-17-479/> Warm-starting quantum optimization. </a></b> <i>Quantum.</i> <code><b>[Quantum Optimization]</b></code> <code><b>[Parameterized Quantum Circuits]</b></code> <code><b>[Quantum Machine Learning]</b></code> </summary><pre><code>@article{egger2021warm,
  title={Warm-starting quantum optimization},
  author={Egger, Daniel J and Mare{\v{c}}ek, Jakub and Woerner, Stefan},
  journal={Quantum},
  volume={5},
  pages={479},
  year={2021},
  publisher={Verein zur F{\"o}rderung des Open Access Publizierens in den Quantenwissenschaften}
}</code></pre></details>
<details><summary>[Holmes et al ] <b><a href=https://arxiv.org/abs/2101.02138> Connecting ansatz expressibility to gradient magnitudes and barren plateaus. </a></b> <i>arXiv.</i> <code><b>[Parameterized Quantum Circuits]</b></code> <code><b>[Quantum Neural Networks]</b></code> <code><b>[Barren Plateaus]</b></code> <code><b>[Quantum Machine Learning]</b></code> </summary><pre><code>@article{holmes2021connecting,
  title={Connecting ansatz expressibility to gradient magnitudes and barren plateaus},
  author={Holmes, Zo{\"e} and Sharma, Kunal and Cerezo, M and Coles, Patrick J},
  journal={arXiv preprint arXiv:2101.02138},
  year={2021}
}</code></pre></details>
<details><summary>[Mangini et al] <b><a href=https://iopscience.iop.org/article/10.1209/0295-5075/134/10002/meta> Quantum computing models for artificial neural networks. </a></b> <i>EPL (Europhysics Letters).</i> <code><b>[Quantum Neural Networks]</b></code> <code><b>[Parameterized Quantum Circuits]</b></code> <code><b>[Variational Quantum Algorithms]</b></code> <code><b>[Quantum Machine Learning]</b></code> </summary><pre><code>@article{mangini2021quantum,
  title={Quantum computing models for artificial neural networks},
  author={Mangini, Stefano and Tacchino, Francesco and Gerace, Dario and Bajoni, Daniele and Macchiavello, Chiara},
  journal={EPL (Europhysics Letters)},
  volume={134},
  number={1},
  pages={10002},
  year={2021},
  publisher={IOP Publishing}
}</code></pre></details>
<details><summary>[Patti et al] <b><a href=https://arxiv.org/abs/2012.12658> Entanglement devised barren plateau mitigation. </a></b> <i>arXiv.</i> <code><b>[Barren Plateaus]</b></code> <code><b>[Parameterized Quantum Circuits]</b></code> <code><b>[Quantum Neural Networks]</b></code> <code><b>[Quantum Machine Learning]</b></code> </summary><pre><code>@article{patti2020entanglement,
  title={Entanglement devised barren plateau mitigation},
  author={Patti, Taylor L and Najafi, Khadijeh and Gao, Xun and Yelin, Susanne F},
  journal={arXiv preprint arXiv:2012.12658},
  year={2020}
}</code></pre></details>
<details><summary>[Perez-Salinas et al] <b><a href=https://arxiv.org/abs/2102.04032> One qubit as a Universal Approximant. </a></b> <i>arXiv.</i> <code><b>[Parameterized Quantum Circuits]</b></code> <code><b>[Quantum Neural Networks]</b></code> <code><b>[Quantum Machine Learning]</b></code> </summary><pre><code>@article{perez2021one,
  title={One qubit as a Universal Approximant},
  author={P{\'e}rez-Salinas, Adri{\'a}n and L{\'o}pez-N{\'u}{\~n}ez, David and Garc{\'\i}a-S{\'a}ez, Artur and Forn-D{\'\i}az, P and Latorre, Jos{\'e} I},
  journal={arXiv preprint arXiv:2102.04032},
  year={2021}
}</code></pre></details>
<details><summary>[Plekhanov et al] <b><a href=https://arxiv.org/abs/2109.03687> Variational quantum amplitude estimation. </a></b> <i>arXiv.</i> <code><b>[Variational Quantum Algorithms]</b></code> <code><b>[Amplitude estimation]</b></code> <code><b>[Parameterized Quantum Circuits]</b></code> </summary><pre><code>@article{plekhanov2021variational,
  title={Variational quantum amplitude estimation},
  author={Plekhanov, Kirill and Rosenkranz, Matthias and Fiorentini, Mattia and Lubasch, Michael},
  journal={arXiv preprint arXiv:2109.03687},
  year={2021}
}
</code></pre></details>
<details><summary>[Schuld et al] <b><a href=https://journals.aps.org/pra/abstract/10.1103/PhysRevA.103.032430> Effect of data encoding on the expressive power of variational quantum-machine-learning models. </a></b> <i>Physical Review A.</i> <code><b>[Parameterized Quantum Circuits]</b></code> <code><b>[Variational Quantum Algorithms]</b></code> <code><b>[Quantum Machine Learning]</b></code> </summary><pre><code>@article{schuld2021effect,
  title={Effect of data encoding on the expressive power of variational quantum-machine-learning models},
  author={Schuld, Maria and Sweke, Ryan and Meyer, Johannes Jakob},
  journal={Physical Review A},
  volume={103},
  number={3},
  pages={032430},
  year={2021},
  publisher={APS}
}</code></pre></details>
<details><summary>[Skolik et al] <b><a href=https://link.springer.com/article/10.1007/s42484-020-00036-4> Layerwise learning for quantum neural networks. </a></b> <i>Quantum Machine Intelligence.</i> <code><b>[Barren Plateaus]</b></code> <code><b>[Parameterized Quantum Circuits]</b></code> <code><b>[Quantum Neural Networks]</b></code> <code><b>[Quantum Machine Learning]</b></code> </summary><pre><code>@article{skolik2021layerwise,
  title={Layerwise learning for quantum neural networks},
  author={Skolik, Andrea and McClean, Jarrod R and Mohseni, Masoud and van der Smagt, Patrick and Leib, Martin},
  journal={Quantum Machine Intelligence},
  volume={3},
  number={1},
  pages={1--11},
  year={2021},
  publisher={Springer}
}</code></pre></details>
<details><summary>[Volkoff & Coles] <b><a href=https://arxiv.org/abs/2005.12200> Large gradients via correlation in random parameterized quantum circuits. </a></b> <i>arXiv.</i> <code><b>[Barren Plateaus]</b></code> <code><b>[Parameterized Quantum Circuits]</b></code> <code><b>[Quantum Neural Networks]</b></code> <code><b>[Variational Quantum Algorithms]</b></code> <code><b>[Quantum Machine Learning]</b></code> </summary><pre><code>@article{volkoff2021large,
  title={Large gradients via correlation in random parameterized quantum circuits},
  author={Volkoff, Tyler and Coles, Patrick J},
  journal={Quantum Science and Technology},
  volume={6},
  number={2},
  pages={025008},
  year={2021},
  publisher={IOP Publishing}
}</code></pre></details>


### 2020
<details><summary>[Gil Vidal & Theis] <b><a href=https://doi.org/10.3389/fphy.2020.00297> Input redundancy for parameterized quantum circuits. </a></b> <i>Frontiers in Physics.</i> <code><b>[Parameterized Quantum Circuits]</b></code> <code><b>[Quantum Neural Networks]</b></code> <code><b>[Quantum Machine Learning]</b></code> </summary><pre><code>@article{gil2020input,
  title={Input redundancy for parameterized quantum circuits},
  author={Gil Vidal, Francisco Javier and Theis, Dirk Oliver},
  journal={Frontiers in Physics},
  volume={8},
  pages={297},
  year={2020},
  publisher={Frontiers}
}</code></pre></details>
<details><summary>[Marrero et al] <b><a href=https://arxiv.org/abs/2010.15968> Entanglement Induced Barren Plateaus. </a></b> <i>arXiv.</i> <code><b>[Barren Plateaus]</b></code> <code><b>[Parameterized Quantum Circuits]</b></code> <code><b>[Quantum Neural Networks]</b></code> <code><b>[Quantum Machine Learning]</b></code> </summary><pre><code>@article{marrero2020entanglement,
  title={Entanglement induced barren plateaus},
  author={Marrero, Carlos Ortiz and Kieferov{\'a}, M{\'a}ria and Wiebe, Nathan},
  journal={arXiv preprint arXiv:2010.15968},
  year={2020}
}</code></pre></details>
<details><summary>[Pesah et al] <b><a href=https://arxiv.org/abs/2011.02966> Absence of Barren Plateaus in Quantum Convolutional Neural Networks. </a></b> <i>arXiv.</i> <code><b>[Barren Plateaus]</b></code> <code><b>[Quantum Convolutional Neural Networks]</b></code> <code><b>[Parameterized Quantum Circuits]</b></code> <code><b>[Quantum Neural Networks]</b></code> <code><b>[Quantum Machine Learning]</b></code> </summary><pre><code>@article{pesah2020absence,
  title={Absence of barren plateaus in quantum convolutional neural networks},
  author={Pesah, Arthur and Cerezo, M and Wang, Samson and Volkoff, Tyler and Sornborger, Andrew T and Coles, Patrick J},
  journal={arXiv preprint arXiv:2011.02966},
  year={2020}
}</code></pre></details>
<details><summary>[Sharma et al] <b><a href=https://arxiv.org/abs/2005.12458> Trainability of Dissipative Perceptron-Based Quantum Neural Networks. </a></b> <i>arXiv.</i> <code><b>[Parameterized Quantum Circuits]</b></code> <code><b>[Quantum Neural Networks]</b></code> <code><b>[Quantum Machine Learning]</b></code> <code><b>[Barren Plateaus]</b></code> </summary><pre><code>@article{sharma2020trainability,
  title={Trainability of dissipative perceptron-based quantum neural networks},
  author={Sharma, Kunal and Cerezo, Marco and Cincio, Lukasz and Coles, Patrick J},
  journal={arXiv preprint arXiv:2005.12458},
  year={2020}
}</code></pre></details>
<details><summary>[Tacchino et al] <b><a href=https://ieeexplore.ieee.org/abstract/document/9364892> Variational Learning for Quantum Artificial Neural Networks. </a></b> <i>IEEE International Conference on Quantum Computing and Engineering.</i> <code><b>[Parameterized Quantum Circuits]</b></code> <code><b>[Quantum Neural Networks]</b></code> <code><b>[Quantum Machine Learning]</b></code> </summary><pre><code>@inproceedings{tacchino2020variational,
  title={Variational learning for quantum artificial neural networks},
  author={Tacchino, Francesco and Barkoutsos, Panagiotis Kl and Macchiavello, Chiara and Gerace, Dario and Tavernelli, Ivano and Bajoni, Daniele},
  booktitle={2020 IEEE International Conference on Quantum Computing and Engineering (QCE)},
  pages={130--136},
  year={2020},
  organization={IEEE}
}</code></pre></details>
<details><summary>[Wang et al] <b><a href=https://arxiv.org/abs/2007.14384> Noise-Induced Barren Plateaus in Variational Quantum Algorithms. </a></b> <i>arXiv.</i> <code><b>[Barren Plateaus]</b></code> <code><b>[Parameterized Quantum Circuits]</b></code> <code><b>[Variational Quantum Algorithms]</b></code> <code><b>[Quantum Machine Learning]</b></code> </summary><pre><code>@article{wang2020noise,
  title={Noise-induced barren plateaus in variational quantum algorithms},
  author={Wang, Samson and Fontana, Enrico and Cerezo, Marco and Sharma, Kunal and Sone, Akira and Cincio, Lukasz and Coles, Patrick J},
  journal={arXiv preprint arXiv:2007.14384},
  year={2020}
}</code></pre></details>


### 2019
<details><summary>[Grant et al] <b><a href=https://quantum-journal.org/papers/q-2019-12-09-214/> An initialization strategy for addressing barren plateaus in parametrized quantum circuits. </a></b> <i>Quantum.</i> <code><b>[Barren Plateaus]</b></code> <code><b>[Parameterized Quantum Circuits]</b></code> <code><b>[Quantum Neural Networks]</b></code> <code><b>[Quantum Machine Learning]</b></code> </summary><pre><code>@article{grant2019initialization,
  title={An initialization strategy for addressing barren plateaus in parametrized quantum circuits},
  author={Grant, Edward and Wossnig, Leonard and Ostaszewski, Mateusz and Benedetti, Marcello},
  journal={Quantum},
  volume={3},
  pages={214},
  year={2019},
  publisher={Verein zur F{\"o}rderung des Open Access Publizierens in den Quantenwissenschaften}
}</code></pre></details>
<details><summary>[Sim et al] <b><a href=https://onlinelibrary.wiley.com/doi/10.1002/qute.201900070> Expressibility and Entangling Capability of Parameterized Quantum Circuits for Hybrid Quantum-Classical Algorithms. </a></b> <i>Advanced Quantum Technologies.</i> <code><b>[Variational Quantum Algorithms]</b></code> <code><b>[Parameterized Quantum Circuits]</b></code> <code><i>(Gate Model)</i></code></summary><pre><code>@article{sim2019expressibility,
  title={Expressibility and entangling capability of parameterized quantum circuits for hybrid quantum-classical algorithms},
  author={Sim, Sukin and Johnson, Peter D and Aspuru-Guzik, Al{\'a}n},
  journal={Advanced Quantum Technologies},
  volume={2},
  number={12},
  pages={1900070},
  year={2019},
  publisher={Wiley Online Library}
}</code></pre></details>
<details><summary>[Verdon et al] <b><a href=https://arxiv.org/abs/1907.05415> Learning to learn with quantum neural networks via classical neural networks. </a></b> <i>arXiv.</i> <code><b>[Quantum Perceptron]</b></code> <code><b>[Parameterized Quantum Circuits]</b></code> <code><b>[Quantum Neural Networks]</b></code> <code><b>[Quantum Machine Learning]</b></code> <code><b>[Variational Quantum Algorithms]</b></code> </summary><pre><code>@article{verdon2019learning,
  title={Learning to learn with quantum neural networks via classical neural networks},
  author={Verdon, Guillaume and Broughton, Michael and McClean, Jarrod R and Sung, Kevin J and Babbush, Ryan and Jiang, Zhang and Neven, Hartmut and Mohseni, Masoud},
  journal={arXiv preprint arXiv:1907.05415},
  year={2019}
}</code></pre></details>


### 2018
<details><summary>[McClean et al] <b><a href=https://www.nature.com/articles/s41467-018-07090-4> Barren plateaus in quantum neural network training landscapes. </a></b> <i>Nature communications.</i> <code><b>[Barren Plateaus]</b></code> <code><b>[Parameterized Quantum Circuits]</b></code> <code><b>[Quantum Neural Networks]</b></code> <code><b>[Quantum Machine Learning]</b></code> </summary><pre><code>@article{mcclean2018barren,
  title={Barren plateaus in quantum neural network training landscapes},
  author={McClean, Jarrod R and Boixo, Sergio and Smelyanskiy, Vadim N and Babbush, Ryan and Neven, Hartmut},
  journal={Nature communications},
  volume={9},
  number={1},
  pages={1--6},
  year={2018},
  publisher={Nature Publishing Group}
}</code></pre></details>
<details><summary>[Mitarai et al] <b><a href=https://journals.aps.org/pra/abstract/10.1103/PhysRevA.98.032309> Quantum circuit learning. </a></b> <i>Physical Review A.</i> <code><b>[Parameterized Quantum Circuits]</b></code> <code><b>[Quantum Neural Networks]</b></code> <code><b>[Quantum Machine Learning]</b></code> </summary><pre><code>@article{mitarai2018quantum,
  title={Quantum circuit learning},
  author={Mitarai, Kosuke and Negoro, Makoto and Kitagawa, Masahiro and Fujii, Keisuke},
  journal={Physical Review A},
  volume={98},
  number={3},
  pages={032309},
  year={2018},
  publisher={APS}
}</code></pre></details>


### 2014
<details><summary>[Farhi et al] <b><a href=https://arxiv.org/abs/1411.4028> A Quantum Approximate Optimization Algorithm. </a></b> <i>arXiv.</i> <code><b>[Variational Quantum Algorithms]</b></code> <code><b>[Parameterized Quantum Circuits]</b></code> <code><b>[Quantum Optimization]</b></code> <code><b>[Quantum Machine Learning]</b></code> <code><b>[]</b></code> <code><i>(Gate Model)</i></code></summary><pre><code>@Article{farhi2014quantum,
  Title                    = {A quantum approximate optimization algorithm},
  Author                   = {Farhi, Edward and Goldstone, Jeffrey and Gutmann, Sam},
  Journal                  = {arXiv preprint arXiv:1411.4028},
  Year                     = {2014},
  URL                      = {https://arxiv.org/abs/1411.4028}
}
</code></pre></details>


# Physical Realization of Qubits


### 2020
<details><summary>[Kjaergaard et al] <b><a href=https://www.annualreviews.org/doi/pdf/10.1146/annurev-conmatphys-031119-050605> Superconducting Qubits:
Current State of Play. </a></b> <i>Annual review of condensed matter physics.</i> <code><b>[Quantum Information and Computing]</b></code> <code><b>[Physical Realization of Qubits]</b></code> <code><b>[Superconducting Qubits]</b></code> </summary><pre><code>@article{kjaergaard2020superconducting,
  title={Superconducting qubits: Current state of play},
  author={Kjaergaard, Morten and Schwartz, Mollie E and Braum{\"u}ller, Jochen and Krantz, Philip and Wang, Joel I-J and Gustavsson, Simon and Oliver, William D},
  journal={Annual Review of Condensed Matter Physics},
  volume={11},
  pages={369--395},
  year={2020},
  publisher={Annual Reviews}
}</code></pre></details>


### 2019
<details><summary>[Krantz et al] <b><a href=https://aip.scitation.org/doi/full/10.1063/1.5089550> A quantum engineer's guide to superconducting qubits. </a></b> <i>Applied Physics Reviews .</i> <code><b>[Physical Realization of Qubits]</b></code> <code><b>[Superconducting Qubits]</b></code> </summary><pre><code>@article{krantz2019quantum,
  title={A quantum engineer's guide to superconducting qubits},
  author={Krantz, Philip and Kjaergaard, Morten and Yan, Fei and Orlando, Terry P and Gustavsson, Simon and Oliver, William D},
  journal={Applied Physics Reviews},
  volume={6},
  number={2},
  pages={021318},
  year={2019},
  publisher={AIP Publishing LLC}
}</code></pre></details>


### 2018
<details><summary>[Soloviev et al] <b><a href=https://aip.scitation.org/doi/10.1063/1.5042147> Adiabatic superconducting artificial neural network: Basic cells. </a></b> <i>Journal of Applied Physics.</i> <code><b>[Physical Realization of Qubits]</b></code> <code><i>(Adiabatic Model)</i></code></summary><pre><code>@article{soloviev2018adiabatic,
  title={Adiabatic superconducting artificial neural network: Basic cells},
  author={Soloviev, Igor I and Schegolev, Andrey E and Klenov, Nikolay V and Bakurskiy, Sergey V and Kupriyanov, Mikhail Yu and Tereshonok, Maxim V and Shadrin, Anton V and Stolyarov, Vasily S and Golubov, Alexander A},
  journal={Journal of Applied Physics},
  volume={124},
  number={15},
  pages={152113},
  year={2018},
  publisher={AIP Publishing LLC}
}</code></pre></details>


### 2017
<details><summary>[Gambetta et al] <b><a href=https://www.nature.com/articles/s41534-016-0004-0> Building logical qubits in a superconducting quantum computing system. </a></b> <i>Physical Review Letters.</i> <code><b>[Physical Realization of Qubits]</b></code> <code><b>[Superconducting Qubits]</b></code> </summary><pre><code>@Article{gambetta2017building,
  Title                    = {Building logical qubits in a superconducting quantum computing system},
  Author                   = {Gambetta, Jay M and Chow, Jerry M and Steffen, Matthias},
  Journal                  = {npj Quantum Information},
  Year                     = {2017},
  Number                   = {1},
  Pages                    = {1--7},
  Volume                   = {3},
  Publisher                = {Nature Publishing Group},
  URL                      = {https://www.nature.com/articles/s41534-016-0004-0}
}
</code></pre></details>
<details><summary>[Wendin et al] <b><a href=https://iopscience.iop.org/article/10.1088/1361-6633/aa7e1a/meta> Quantum information processing with superconducting circuits: a review. </a></b> <i>Reports on Progress in Physics.</i> <code><b>[Physical Realization of Qubits]</b></code> </summary><pre><code>@article{wendin2017quantum,
  title={Quantum information processing with superconducting circuits: a review},
  author={Wendin, G{\"o}ran},
  journal={Reports on Progress in Physics},
  volume={80},
  number={10},
  pages={106001},
  year={2017},
  publisher={IOP Publishing}
}</code></pre></details>


# Quantum Annealing


### 2020
<details><summary>[Pjilipp  et al] <b><a href=https://iopscience.iop.org/article/10.1088/1361-6633/ab85b8> Perspectives of quantum annealing: Methods andimplementations. </a></b> <i>Reports on Progress in Physics.</i> <code><b>[Quantum Information and Computing]</b></code> <code><b>[Quantum Annealing]</b></code> <code><i>(Adiabatic Model)</i></code></summary><pre><code>@article{hauke2020perspectives,
  title={Perspectives of quantum annealing: Methods and implementations},
  author={Hauke, Philipp and Katzgraber, Helmut G and Lechner, Wolfgang and Nishimori, Hidetoshi and Oliver, William D},
  journal={Reports on Progress in Physics},
  volume={83},
  number={5},
  pages={054401},
  year={2020},
  publisher={IOP Publishing}
}</code></pre></details>


### 2018
<details><summary>[Katzgraber et al] <b><a href=https://iopscience.iop.org/article/10.1088/2058-9565/aab6ba> Viewing vanilla quantum annealing through spin glasses. </a></b> <i>Quantum Science and Technology.</i> <code><b>[Quantum Annealing]</b></code> <code><i>(Adiabatic Model)</i></code></summary><pre><code>@article{katzgraber2018viewing,
  title={Viewing vanilla quantum annealing through spin glasses},
  author={Katzgraber, Helmut G},
  journal={Quantum Science and Technology},
  volume={3},
  number={3},
  pages={030505},
  year={2018},
  publisher={IOP Publishing}
}</code></pre></details>


### 2017
<details><summary>[Andriyash et al] <b><a href=https://arxiv.org/abs/1703.09277> Can quantum Monte Carlo simulate quantum annealing?. </a></b> <i>arXiv.</i> <code><b>[Adiabatic Quantum Computing]</b></code> <code><b>[Quantum Annealing]</b></code> <code><b>[Quantum Monte Carlo]</b></code> <code><i>(Adiabatic Model)</i></code></summary><pre><code>@Article{andriyash2017can,
  Title                    = {Can quantum Monte Carlo simulate quantum annealing?},
  Author                   = {Andriyash, Evgeny and Amin, Mohammad H},
  Journal                  = {arXiv preprint arXiv:1703.09277},
  Year                     = {2017},
  URL                      = {https://arxiv.org/abs/1703.09277}
}
</code></pre></details>
<details><summary>[Mandra' et al] <b><a href=https://iopscience.iop.org/article/10.1088/2058-9565/aa7877> The pitfalls of planar spin-glass benchmarks: raising the bar for quantum annealers (again). </a></b> <i>Quantum Science and Technology.</i> <code><b>[Quantum Annealing]</b></code> <code><b>[Quantum Speedup-Advantage-Supremacy]</b></code> <code><i>(Adiabatic Model)</i></code></summary><pre><code>@article{mandra2017pitfalls,
  title={The pitfalls of planar spin-glass benchmarks: raising the bar for quantum annealers (again)},
  author={Mandra, Salvatore and Katzgraber, Helmut G and Thomas, Creighton},
  journal={Quantum Science and Technology},
  volume={2},
  number={3},
  pages={038501},
  year={2017},
  publisher={IOP Publishing}
}</code></pre></details>


### 2016
<details><summary>[Batle et al] <b><a href=https://link.springer.com/article/10.1007/s11128-016-1324-x> Do multipartite correlations speed up adiabatic quantum computation or quantum annealing?. </a></b> <i>Quantum Information Processing.</i> <code><b>[Adiabatic Quantum Computing]</b></code> <code><b>[Quantum Annealing]</b></code> <code><b>[Quantum Speedup-Advantage-Supremacy]</b></code> <code><i>(Adiabatic Model)</i></code></summary><pre><code>@Article{batle2016multipartite,
  Title                    = {Do multipartite correlations speed up adiabatic quantum computation or quantum annealing?},
  Author                   = {Batle, Josep and Ooi, CH Raymond and Farouk, Ahmed and Abutalib, M and Abdalla, S},
  Journal                  = {Quantum Information Processing},
  Year                     = {2016},
  Number                   = {8},
  Pages                    = {3081--3099},
  Volume                   = {15},
  Publisher                = {Springer},
  URL                      = {https://link.springer.com/article/10.1007/s11128-016-1324-x}
}
</code></pre></details>
<details><summary>[Isakov et al] <b><a href=https://journals.aps.org/prl/abstract/10.1103/PhysRevLett.117.180402> Understanding Quantum Tunneling through Quantum Monte Carlo Simulations. </a></b> <i>Physical Review Letters.</i> <code><b>[Quantum Annealing]</b></code> <code><i>(Adiabatic Model)</i></code></summary><pre><code>@article{isakov2016understanding,
  title={Understanding quantum tunneling through quantum Monte Carlo simulations},
  author={Isakov, Sergei V and Mazzola, Guglielmo and Smelyanskiy, Vadim N and Jiang, Zhang and Boixo, Sergio and Neven, Hartmut and Troyer, Matthias},
  journal={Physical review letters},
  volume={117},
  number={18},
  pages={180402},
  year={2016},
  publisher={APS}
}</code></pre></details>
<details><summary>[Sergey et al] <b><a href=https://www.Nature.com/articles/ncomms12370> Zero-temperature quantum annealing bottlenecks in the spin-glass phase. </a></b> <i>Nature.</i> <code><b>[Quantum Annealing]</b></code> <code><i>(Adiabatic Model)</i></code></summary><pre><code>@article{knysh2016zero,
  title={Zero-temperature quantum annealing bottlenecks in the spin-glass phase},
  author={Knysh, Sergey},
  journal={Nature communications},
  volume={7},
  number={1},
  pages={1--9},
  year={2016},
  publisher={Nature Publishing Group}
}</code></pre></details>
<details><summary>[Mandra' et al] <b><a href=https://journals.aps.org/pra/abstract/10.1103/PhysRevA.94.022337> Strengths and weaknesses of weak-strong cluster problems: A detailed overview of state-of-the-art classical heuristics versus quantum approaches. </a></b> <i>Physical Review A.</i> <code><b>[Quantum Annealing]</b></code> <code><b>[Quantum Speedup-Advantage-Supremacy]</b></code> <code><i>(Adiabatic Model)</i></code></summary><pre><code>@article{mandra2016strengths,
  title={Strengths and weaknesses of weak-strong cluster problems: A detailed overview of state-of-the-art classical heuristics versus quantum approaches},
  author={Mandra, Salvatore and Zhu, Zheng and Wang, Wenlong and Perdomo-Ortiz, Alejandro and Katzgraber, Helmut G},
  journal={Physical Review A},
  volume={94},
  number={2},
  pages={022337},
  year={2016},
  publisher={APS}
}</code></pre></details>


### 2015
<details><summary>[Itay ert al] <b><a href=https://journals.aps.org/pra/abstract/10.1103/PhysRevA.92.042325> Probing for quantum speedup in spin-glass problems with planted solutions. </a></b> <i>Physical Review A.</i> <code><b>[Quantum Annealing]</b></code> <code><b>[Quantum Speedup-Advantage-Supremacy]</b></code> <code><i>(Adiabatic Model)</i></code></summary><pre><code>@article{hen2015probing,
  title={Probing for quantum speedup in spin-glass problems with planted solutions},
  author={Hen, Itay and Job, Joshua and Albash, Tameem and R{\o}nnow, Troels F and Troyer, Matthias and Lidar, Daniel A},
  journal={Physical Review A},
  volume={92},
  number={4},
  pages={042325},
  year={2015},
  publisher={APS}
}</code></pre></details>
<details><summary>[Katzgraber et al] <b><a href=https://journals.aps.org/prx/abstract/10.1103/PhysRevX.5.031026> Seeking Quantum Speedup Through Spin Glasses: The Good, the Bad, and the Ugly. </a></b> <i>Physical Review X.</i> <code><b>[Quantum Annealing]</b></code> <code><b>[Quantum Speedup-Advantage-Supremacy]</b></code> <code><i>(Adiabatic Model)</i></code></summary><pre><code>@article{katzgraber2015seeking,
  title={Seeking quantum speedup through spin glasses: The good, the bad, and the ugly},
  author={Katzgraber, Helmut G and Hamze, Firas and Zhu, Zheng and Ochoa, Andrew J and Munoz-Bauza, Humberto},
  journal={Physical Review X},
  volume={5},
  number={3},
  pages={031026},
  year={2015},
  publisher={APS}
}</code></pre></details>


### 2014
<details><summary>[Bunyk et al] <b><a href=https://ieeexplore.ieee.org/document/6802426> Architectural considerations in the design of a superconducting quantum annealing processor. </a></b> <i>IEEE Transactions on Applied Superconductivity.</i> <code><b>[Quantum Annealing]</b></code> <code><b>[Quantum Information and Computing]</b></code> <code><b>[QPU]</b></code> <code><b>[Superconducting Qubits]</b></code> <code><i>(Adiabatic Model)</i></code></summary><pre><code>@Article{bunyk2014architectural,
  Title                    = {Architectural considerations in the design of a superconducting quantum annealing processor},
  Author                   = {Bunyk, Paul I and Hoskinson, Emile M and Johnson, Mark W and Tolkacheva, Elena and Altomare, Fabio and Berkley, Andrew J and Harris, Richard and Hilton, Jeremy P and Lanting, Trevor and Przybysz, Anthony J and others},
  Journal                  = {IEEE Transactions on Applied Superconductivity},
  Year                     = {2014},
  Number                   = {4},
  Pages                    = {1--10},
  Volume                   = {24},
  Publisher                = {IEEE},
  URL                      = {https://ieeexplore.ieee.org/document/6802426}
}
</code></pre></details>
<details><summary>[Boixo et al] <b><a href=https://www.Nature.com/articles/nphys2900> Evidence for quantum annealing with more than one hundred qubits. </a></b> <i>Nature.</i> <code><b>[Quantum Annealing]</b></code> <code><i>(Adiabatic Model)</i></code></summary><pre><code>@Article{evidenceboixo2014,
  Title                    = {Evidence for quantum annealing with more than one hundred qubits},
  Author                   = {Boixo, Sergio and R{\o}nnow, Troels F and Isakov, Sergei V and Wang, Zhihui and Wecker, David and Lidar, Daniel A and Martinis, John M and Troyer, Matthias},
  Journal                  = {Nature physics},
  Year                     = {2014},
  Number                   = {3},
  Pages                    = {218--224},
  Volume                   = {10},
  Publisher                = {Nature Publishing Group},
  URL                      = {https://www.Nature.com/articles/nphys2900}
}
</code></pre></details>
<details><summary>[Lanting et al] <b><a href=https://journals.aps.org/prx/abstract/10.1103/PhysRevX.4.021041> Entanglement in a Quantum Annealing Processor. </a></b> <i>PHYSICAL REVIEW X.</i> <code><b>[Quantum Annealing]</b></code> <code><i>(Adiabatic Model)</i></code></summary><pre><code>@article{lanting2014entanglement,
  title={Entanglement in a quantum annealing processor},
  author={Lanting, Trevor and Przybysz, Anthony J and Smirnov, A Yu and Spedalieri, Federico M and Amin, Mohammad H and Berkley, Andrew J and Harris, Richard and Altomare, Fabio and Boixo, Sergio and Bunyk, Paul and others},
  journal={Physical Review X},
  volume={4},
  number={2},
  pages={021041},
  year={2014},
  publisher={APS}
}</code></pre></details>
<details><summary>[McGeoch] <b><a href=https://ieeexplore.ieee.org/document/7055969> Adiabatic Quantum Computation and Quantum Annealing: Theory and Practice. </a></b> <i>Synthesis Lectures on Quantum Computing.</i> <code><b>[Quantum Annealing]</b></code> <code><i>(Adiabatic Model)</i></code></summary><pre><code>@article{mcgeoch2014adiabatic,
  title={Adiabatic quantum computation and quantum annealing: Theory and practice},
  author={McGeoch, Catherine C},
  journal={Synthesis Lectures on Quantum Computing},
  volume={5},
  number={2},
  pages={1--93},
  year={2014},
  publisher={Morgan \& Claypool Publishers}
}</code></pre></details>
<details><summary>[Rønnow et al] <b><a href=https://pubmed.ncbi.nlm.nih.gov/25061205/> Quantum computing Defining and detecting quantum speedup. </a></b> <i>Science.</i> <code><b>[Quantum Annealing]</b></code> <code><i>(Adiabatic Model)</i></code></summary><pre><code>@article{ronnow2014defining,
  title={Defining and detecting quantum speedup},
  author={R{\o}nnow, Troels F and Wang, Zhihui and Job, Joshua and Boixo, Sergio and Isakov, Sergei V and Wecker, David and Martinis, John M and Lidar, Daniel A and Troyer, Matthias},
  journal={science},
  volume={345},
  number={6195},
  pages={420--424},
  year={2014},
  publisher={American Association for the Advancement of Science}
}</code></pre></details>


### 2013
<details><summary>[Boixo et al] <b><a href=https://www.Nature.com/articles/ncomms3067> Experimental signature of programmable quantum annealing. </a></b> <i>Nature communications.</i> <code><b>[Quantum Annealing]</b></code> <code><i>(Adiabatic Model)</i></code></summary><pre><code>@Article{boixo2013experimental,
  Title                    = {Experimental signature of programmable quantum annealing},
  Author                   = {Boixo, Sergio and Albash, Tameem and Spedalieri, Federico M and Chancellor, Nicholas and Lidar, Daniel A},
  Journal                  = {Nature communications},
  Year                     = {2013},
  Number                   = {1},
  Pages                    = {1--8},
  Volume                   = {4},
  Publisher                = {Nature Publishing Group},
  URL                      = {https://www.Nature.com/articles/ncomms3067}
}
</code></pre></details>


### 2012
<details><summary>[Gaitan & Clark] <b><a href=https://journals.aps.org/prl/abstract/10.1103/PhysRevLett.108.010501> Ramsey Numbers and Adiabatic Quantum Computing. </a></b> <i>Physical Review Letters.</i> <code><b>[Adiabatic Quantum Computing]</b></code> <code><b>[Quantum Annealing]</b></code> <code><b>[Classical Implementation]</b></code> <code><i>(Adiabatic Model)</i></code></summary><pre><code>@Article{gaitan2012ramsey,
  Title                    = {Ramsey numbers and adiabatic quantum computing},
  Author                   = {Gaitan, Frank and Clark, Lane},
  Journal                  = {Physical review letters},
  Year                     = {2012},
  Number                   = {1},
  Pages                    = {010501},
  Volume                   = {108},
  Publisher                = {APS},
  URL                      = {https://journals.aps.org/prl/abstract/10.1103/PhysRevLett.108.010501}
}
</code></pre></details>


### 2011
<details><summary>[Johnson et al] <b><a href=https://www.Nature.com/articles/Nature10012.pdff> Quantum annealing with manufactured spins. </a></b> <i>Nature.</i> <code><b>[Quantum Annealing]</b></code> <code><i>(Adiabatic Model)</i></code></summary><pre><code>@article{johnson2011quantum,
  title={Quantum annealing with manufactured spins},
  author={Johnson, Mark W and Amin, Mohammad HS and Gildert, Suzanne and Lanting, Trevor and Hamze, Firas and Dickson, Neil and Harris, Richard and Berkley, Andrew J and Johansson, Jan and Bunyk, Paul and others},
  journal={Nature},
  volume={473},
  number={7346},
  pages={194--198},
  year={2011},
  publisher={Nature Publishing Group}
}</code></pre></details>
<details><summary>[Johnson et al] <b><a href=https://www.Nature.com/articles/Nature10012.pdf> Quantum annealing with manufactured spins. </a></b> <i>Nature.</i> <code><b>[Quantum Annealing]</b></code> <code><i>(Adiabatic Model)</i></code></summary><pre><code>@article{johnson2011quantum,
  title={Quantum annealing with manufactured spins},
  author={Johnson, Mark W and Amin, Mohammad HS and Gildert, Suzanne and Lanting, Trevor and Hamze, Firas and Dickson, Neil and Harris, Richard and Berkley, Andrew J and Johansson, Jan and Bunyk, Paul and others},
  journal={Nature},
  volume={473},
  number={7346},
  pages={194--198},
  year={2011},
  publisher={Nature Publishing Group}
}</code></pre></details>


### 2010
<details><summary>[Bravyi & Terhal] <b><a href=https://arxiv.org/abs/0806.1746> Complexity of stoquastic frustration-free Hamiltonians. </a></b> <i>Siam journal on computing.</i> <code><b>[Adiabatic Quantum Computing]</b></code> <code><b>[Quantum Annealing]</b></code> <code><i>(Adiabatic Model)</i></code></summary><pre><code>@Article{bravyi2010complexity,
  Title                    = {Complexity of stoquastic frustration-free Hamiltonians},
  Author                   = {Bravyi, Sergey and Terhal, Barbara},
  Journal                  = {Siam journal on computing},
  Year                     = {2010},
  Number                   = {4},
  Pages                    = {1462--1485},
  Volume                   = {39},
  Publisher                = {SIAM},
  URL                      = {https://arxiv.org/abs/0806.1746}
}
</code></pre></details>
<details><summary>[Harris et al] <b><a href=https://journals.aps.org/prb/abstract/10.1103/PhysRevB.81.134510> Experimental demonstration of a robust and scalable flux qubit. </a></b> <i>Physical Review B.</i> <code><b>[Quantum Annealing]</b></code> <code><i>(Adiabatic Model)</i></code></summary><pre><code>@Article{harris2010demonstration,
  Title                    = {Experimental demonstration of a robust and scalable flux qubit},
  Author                   = {Harris, R and Johansson, J and Berkley, AJ and Johnson, MW and Lanting, T and Han, Siyuan and Bunyk, P and Ladizinsky, E and Oh, T and Perminov, I and others},
  Journal                  = {Physical Review B},
  Year                     = {2010},
  Number                   = {13},
  Pages                    = {134510},
  Volume                   = {81},
  Publisher                = {APS},
  URL                      = {https://journals.aps.org/prb/abstract/10.1103/PhysRevB.81.134510}
}
</code></pre></details>
<details><summary>[Harris et al] <b><a href=https://journals.aps.org/prb/abstract/10.1103/PhysRevB.82.024511> Experimental investigation of an eight-qubit unit cell in a superconducting optimization processor. </a></b> <i>Physical Review B.</i> <code><b>[Quantum Annealing]</b></code> <code><b>[Superconducting Qubits]</b></code> <code><i>(Adiabatic Model)</i></code></summary><pre><code>@article{harris2010experimental,
  title={Experimental investigation of an eight-qubit unit cell in a superconducting optimization processor},
  author={Harris, Richard and Johnson, Mark W and Lanting, T and Berkley, AJ and Johansson, J and Bunyk, P and Tolkacheva, E and Ladizinsky, E and Ladizinsky, N and Oh, T and others},
  journal={Physical Review B},
  volume={82},
  number={2},
  pages={024511},
  year={2010},
  publisher={APS}
}</code></pre></details>


### 2009
<details><summary>[Matsuda et al] <b><a href=https://iopscience.iop.org/article/10.1088/1367-2630/11/7/073021> Ground-state statistics from annealing algorithms: quantum versus classical approaches. </a></b> <i>New Journal of Physics.</i> <code><b>[Classical Implementation]</b></code> <code><b>[Quantum Annealing]</b></code> </summary><pre><code>@article{matsuda2009ground,
  title={Ground-state statistics from annealing algorithms: quantum versus classical approaches},
  author={Matsuda, Yoshiki and Nishimori, Hidetoshi and Katzgraber, Helmut G},
  journal={New Journal of Physics},
  volume={11},
  number={7},
  pages={073021},
  year={2009},
  publisher={IOP Publishing}
}</code></pre></details>


### 2008
<details><summary>[Choi ] <b><a href=https://link.springer.com/article/10.1007/s11128-008-0082-9> Minor-Embedding in Adiabatic Quantum Computation: I The Parameter Setting Problem. </a></b> <i>Quantum Information Processing.</i> <code><b>[Quantum Annealing]</b></code> <code><i>(Adiabatic Model)</i></code></summary><pre><code>@Article{minorchoi2011,
  Title                    = {Minor-embedding in adiabatic quantum computation: II. Minor-universal graph design},
  Author                   = {Choi, Vicky},
  Journal                  = {Quantum Information Processing},
  Year                     = {2011},
  Number                   = {3},
  Pages                    = {343--353},
  Volume                   = {10},
  Publisher                = {Springer},
  URL                      = {https://link.springer.com/article/10.1007/s11128-008-0082-9}
}
</code></pre></details>


### 2005
<details><summary>[Battaglia et al] <b><a href=https://link.springer.com/chapter/10.1007%2F11526216_7> Deterministic and stochastic quantum annealing approaches. </a></b> <i>Lecture Notes in Physics .</i> <code><b>[Adiabatic Quantum Computing]</b></code> <code><b>[Quantum Annealing]</b></code> <code><b>[Simulated Annealing]</b></code> <code><i>(Adiabatic Model)</i></code></summary><pre><code>@InCollection{battaglia2005deterministic,
  Title                    = {Deterministic and stochastic quantum annealing approaches},
  Author                   = {Battaglia, Demian and Stella, Lorenzo and Zagordi, Osvaldo and Santoro, Giuseppe E and Tosatti, Erio},
  Booktitle                = {Quantum Annealing and Other Optimization Methods},
  Publisher                = {Springer},
  Year                     = {2005},
  Pages                    = {171--206},
  URL                      = {https://link.springer.com/chapter/10.1007%2F11526216_7}
}
</code></pre></details>
<details><summary>[Battaglia et al] <b><a href=https://journals.aps.org/pre/abstract/10.1103/PhysRevE.71.066707> Optimization by Quantum Annealing: Lessons from hard 3-SAT cases,. </a></b> <i>Physical Review E.</i> <code><b>[Adiabatic Quantum Computing]</b></code> <code><b>[Quantum Annealing]</b></code> <code><b>[Simulated Annealing]</b></code> <code><b>[Quantum Monte Carlo]</b></code> <code><i>(Adiabatic Model)</i></code></summary><pre><code>@Article{battaglia2005optimization,
  Title                    = {Optimization by quantum annealing: Lessons from hard satisfiability problems},
  Author                   = {Battaglia, Demian A and Santoro, Giuseppe E and Tosatti, Erio},
  Journal                  = {Physical Review E},
  Year                     = {2005},
  Number                   = {6},
  Pages                    = {066707},
  Volume                   = {71},
  Publisher                = {APS},
  URL                      = {https://journals.aps.org/pre/abstract/10.1103/PhysRevE.71.066707}
}
</code></pre></details>


### 2004
<details><summary>[Martonak et al] <b><a href=https://journals.aps.org/pre/abstract/10.1103/PhysRevE.70.057701> Quantum annealing of the traveling-salesman problem. </a></b> <i>Physical Review E.</i> <code><b>[Classical Implementation]</b></code> <code><b>[Quantum Annealing]</b></code> </summary><pre><code>@article{martovnak2004quantum,
  title={Quantum annealing of the traveling-salesman problem},
  author={Marto{\v{n}}{\'a}k, Roman and Santoro, Giuseppe E and Tosatti, Erio},
  journal={Physical Review E},
  volume={70},
  number={5},
  pages={057701},
  year={2004},
  publisher={APS}
}</code></pre></details>


### 2001
<details><summary>[Farhi et al] <b><a href=https://science.sciencemag.org/content/292/5516/472> A Quantum Adiabatic Evolution Algorithm Applied to Random Instances of an NP-Complete Problem. </a></b> <i>Science.</i> <code><b>[Adiabatic Quantum Computing]</b></code> <code><b>[Classical Implementation]</b></code> <code><b>[Quantum Annealing]</b></code> <code><i>(Adiabatic Model)</i></code></summary><pre><code>@Article{farhi2001quantum,
  Title                    = {{A quantum adiabatic evolution algorithm applied to random instances of an NP-complete problem}},
  Author                   = {Farhi, Edward and Goldstone, Jeffrey and Gutmann, Sam and Lapan, Joshua and Lundgren, Andrew and Preda, Daniel},
  Journal                  = {Science},
  Year                     = {2001},
  Number                   = {5516},
  Pages                    = {472--475},
  Volume                   = {292},
  Publisher                = {American Association for the Advancement of Science},
  URL                      = {https://science.sciencemag.org/content/292/5516/472}
}
</code></pre></details>


### 1998
<details><summary>[Kadowaki &Nishimori] <b><a href=https://journals.aps.org/pre/abstract/10.1103/PhysRevE.58.5355> Quantum annealing in the transverse Ising model. </a></b> <i>Physical Review E.</i> <code><b>[Quantum Annealing]</b></code> <code><i>(Adiabatic Model)</i></code></summary><pre><code>@article{kadowaki1998quantum,
  title={Quantum annealing in the transverse Ising model},
  author={Kadowaki, Tadashi and Nishimori, Hidetoshi},
  journal={Physical Review E},
  volume={58},
  number={5},
  pages={5355},
  year={1998},
  publisher={APS}
}</code></pre></details>


### 1994
<details><summary>[Finnila et al] <b><a href=https://www.sciencedirect.com/science/article/abs/pii/0009261494001170?via%3Dihub> Quantum annealing: A new method for minimizing multidimensional functions. </a></b> <i>Chemical Physics Letters.</i> <code><b>[Quantum Annealing]</b></code> <code><i>(Adiabatic Model)</i></code></summary><pre><code>@Article{finnila1994quantum,
  Title                    = {Quantum annealing: A new method for minimizing multidimensional functions},
  Author                   = {Finnila, Aleta Berk and Gomez, MA and Sebenik, C and Stenson, Catherine and Doll, Jimmie D},
  Journal                  = {Chemical physics letters},
  Year                     = {1994},
  Number                   = {5-6},
  Pages                    = {343--348},
  Volume                   = {219},
  Publisher                = {Elsevier},
  URL                      = {https://www.sciencedirect.com/science/article/abs/pii/0009261494001170?via%3Dihub}
}
</code></pre></details>


### 1989
<details><summary>[Apolloni et al] <b><a href=https://www.sciencedirect.com/science/article/pii/0304414989900409?via%3Dihub> Quantum stochastic optimization. </a></b> <i>Stochastic Processes and their Applications.</i> <code><b>[Adiabatic Quantum Computing]</b></code> <code><b>[Quantum Annealing]</b></code> <code><b>[Quantum Optimization]</b></code> <code><i>(Adiabatic Model)</i></code></summary><pre><code>@Article{apolloni1989quantum,
  Title                    = {Quantum stochastic optimization},
  Author                   = {Apolloni, Bruno and Carvalho, C and De Falco, Diego},
  Journal                  = {Stochastic Processes and their Applications},
  Year                     = {1989},
  Number                   = {2},
  Pages                    = {233--244},
  Volume                   = {33},
  Publisher                = {Elsevier},
  URL                      = {https://www.sciencedirect.com/science/article/pii/0304414989900409?via%3Dihub}
}
</code></pre></details>


# Quantum Boltzmann machines


### 2019
<details><summary>[Wiebe & Wossnig] <b><a href=https://arxiv.org/abs/1905.09902> Generative training of quantum Boltzmann machines with hidden units. </a></b> <i>arXiv.</i> <code><b>[Quantum Boltzmann machines]</b></code> <code><b>[Quantum Machine Learning]</b></code> </summary><pre><code>@article{wiebe2019generative,
  title={Generative training of quantum Boltzmann machines with hidden units},
  author={Wiebe, Nathan and Wossnig, Leonard},
  journal={arXiv preprint arXiv:1905.09902},
  year={2019}
}</code></pre></details>


### 2018
<details><summary>[Allcock & Zhang] <b><a href=https://academic.oup.com/nsr/article/6/1/26/5222655?login=true> Quantum machine learning. </a></b> <i>National Science Review.</i> <code><b>[Quantum Machine Learning]</b></code> <code><b>[Quantum Boltzmann machines]</b></code> <code><b>[Quantum Generative Adversarial Networks]</b></code> </summary><pre><code>@Article{allcock2019quantum,
  Title                    = {Quantum machine learning},
  Author                   = {Allcock, Jonathan and Zhang, Shengyu},
  Journal                  = {National Science Review},
  Year                     = {2019},
  Number                   = {1},
  Pages                    = {26--28},
  Volume                   = {6},
  Publisher                = {Oxford University Press},
  URL                      = {https://academic.oup.com/nsr/article/6/1/26/5222655?login=true}
}
</code></pre></details>
<details><summary>[Amin et al] <b><a href=https://journals.aps.org/prx/abstract/10.1103/PhysRevX.8.021050> Quantum boltzmann machine. </a></b> <i>Physical Review X.</i> <code><b>[Quantum Boltzmann machines]</b></code> <code><b>[Quantum Machine Learning]</b></code> <code><i>(Gate Model)</i></code> <code><i>(Adiabatic Model)</i></code></summary><pre><code>@Article{amin2018quantum,
  Title                    = {Quantum boltzmann machine},
  Author                   = {Amin, Mohammad H and Andriyash, Evgeny and Rolfe, Jason and Kulchytskyy, Bohdan and Melko, Roger},
  Journal                  = {Physical Review X},
  Year                     = {2018},
  Number                   = {2},
  Pages                    = {021050},
  Volume                   = {8},
  Publisher                = {APS},
  URL                      = {https://journals.aps.org/prx/abstract/10.1103/PhysRevX.8.021050}
}
</code></pre></details>


# Quantum Classification


### 2021
<details><summary>[Liu et al] <b><a href=https://www.nature.com/articles/s41567-021-01287-z> A rigorous and robust quantum speed-up in supervised machine learning. </a></b> <i>Nature Physics.</i> <code><b>[Quantum Machine Learning]</b></code> <code><b>[Quantum Classification]</b></code> <code><b>[Quantum Speedup-Advantage-Supremacy]</b></code> </summary><pre><code>@article{liu2021rigorous,
  title={A rigorous and robust quantum speed-up in supervised machine learning},
  author={Liu, Yunchao and Arunachalam, Srinivasan and Temme, Kristan},
  journal={Nature Physics},
  pages={1--5},
  year={2021},
  publisher={Nature Publishing Group}
}</code></pre></details>


### 2020
<details><summary>[Ablayev et al] <b><a href=https://ieeexplore.ieee.org/stamp/stamp.jsp?arnumber=8935094> On quantum methods for machine learning problems part I: Quantum tools. </a></b> <i>Big Data Mining and Analytics.</i> <code><b>[Quantum Machine Learning]</b></code> <code><b>[Quantum Classification]</b></code> <code><i>(Gate Model)</i></code></summary><pre><code>@Article{ablayev2020quantum1,
  Title                    = {{On quantum methods for machine learning problems part I: Quantum tools}},
  Author                   = {Ablayev, Farid and Ablayev, Marat and Huang, Joshua Zhexue and Khadiev, Kamil and Salikhova, Nailya and Wu, Dingming},
  Journal                  = {Big Data Mining and Analytics},
  Year                     = {2020},
  Number                   = {1},
  Pages                    = {41--55},
  Volume                   = {3},
  URL                      = {https://ieeexplore.ieee.org/stamp/stamp.jsp?arnumber=8935094}
}
</code></pre></details>
<details><summary>[Ablayev et al] <b><a href=https://ieeexplore.ieee.org/abstract/document/8935095> On quantum methods for machine learning problems part II: Quantum classification algorithms. </a></b> <i>Big Data Mining and Analytics.</i> <code><b>[Quantum Machine Learning]</b></code> <code><b>[Quantum Classification]</b></code> <code><b>[Quantum Nearest Neighbors Algorithm]</b></code> <code><i>(Gate Model)</i></code></summary><pre><code>@Article{ablayev2020quantum2,
  Title                    = {{On quantum methods for machine learning problems part II: Quantum classification algorithms}},
  Author                   = {Ablayev, Farid and Ablayev, Marat and Huang, Joshua Zhexue and Khadiev, Kamil and Salikhova, Nailya and Wu, Dingming},
  Journal                  = {Big Data Mining and Analytics},
  Year                     = {2020},
  Number                   = {1},
  Pages                    = {56--67},
  Volume                   = {3},
  URL                      = {https://ieeexplore.ieee.org/abstract/document/8935095}
}
</code></pre></details>
<details><summary>[Abohashima et al] <b><a href=https://arxiv.org/abs/2006.12270> Classification with Quantum Machine Learning: A Survey. </a></b> <i>arXiv.</i> <code><b>[Quantum Machine Learning]</b></code> <code><b>[Quantum Classification]</b></code> <code><i>(Gate Model)</i></code></summary><pre><code>@article{abohashima2020classification,
  title={Classification with quantum machine learning: A survey},
  author={Abohashima, Zainab and Elhosen, Mohamed and Houssein, Essam H and Mohamed, Waleed M},
  journal={arXiv preprint arXiv:2006.12270},
  year={2020}
}</code></pre></details>
<details><summary>[Perez-Salinas et al] <b><a href=https://quantum-journal.org/papers/q-2020-02-06-226/> Data re-uploading for a universal quantum classifier. </a></b> <i>Quantum.</i> <code><b>[Quantum Neural Networks]</b></code> <code><b>[Quantum Classification]</b></code> <code><b>[Quantum Machine Learning]</b></code> <code><i>(Gate Model)</i></code></summary><pre><code>@article{perez2020data,
  title={Data re-uploading for a universal quantum classifier},
  author={P{\'e}rez-Salinas, Adri{\'a}n and Cervera-Lierta, Alba and Gil-Fuster, Elies and Latorre, Jos{\'e} I},
  journal={Quantum},
  volume={4},
  pages={226},
  year={2020},
  publisher={Verein zur F{\"o}rderung des Open Access Publizierens in den Quantenwissenschaften}
}</code></pre></details>
<details><summary>[Schuld et al] <b><a href=https://journals.aps.org/pra/abstract/10.1103/PhysRevA.101.032308> Circuit-centric quantum classifiers. </a></b> <i>Physical Review A.</i> <code><b>[Quantum Neural Networks]</b></code> <code><b>[Quantum Classification]</b></code> <code><b>[Quantum Machine Learning]</b></code> <code><i>(Gate Model)</i></code></summary><pre><code>@article{schuld2020circuit,
  title={Circuit-centric quantum classifiers},
  author={Schuld, Maria and Bocharov, Alex and Svore, Krysta M and Wiebe, Nathan},
  journal={Physical Review A},
  volume={101},
  number={3},
  pages={032308},
  year={2020},
  publisher={APS}
}</code></pre></details>
<details><summary>[Sergioli] <b><a href=https://link.springer.com/article/10.1007/s00500-019-04429-x> Quantum and quantum-like machine learning: a note on differences and similarities. </a></b> <i>Soft Computing.</i> <code><b>[Quantum Machine Learning]</b></code> <code><b>[Quantum Information and Computing]</b></code> <code><b>[Quantum Classification]</b></code> </summary><pre><code>@article{sergioli2020quantum,
  title={Quantum and quantum-like machine learning: a note on differences and similarities},
  author={Sergioli, Giuseppe},
  journal={Soft Computing},
  volume={24},
  number={14},
  pages={10247--10255},
  year={2020},
  publisher={Springer}
}</code></pre></details>


### 2019
<details><summary>[Havlíček et al] <b><a href=https://www.nature.com/articles/s41586-019-0980-2> Supervised learning with quantum-enhanced feature spaces. </a></b> <i>Nature.</i> <code><b>[Variational Quantum Algorithms]</b></code> <code><b>[Quantum Support Vector Machine]</b></code> <code><b>[Quantum Classification]</b></code> <code><b>[Quantum Machine Learning]</b></code> <code><i>(Gate Model)</i></code></summary><pre><code>@Article{havlivcek2019supervised,
  Title                    = {Supervised learning with quantum-enhanced feature spaces},
  Author                   = {Havl{\'\i}{\v{c}}ek, Vojt{\v{e}}ch and C{\'o}rcoles, Antonio D and Temme, Kristan and Harrow, Aram W and Kandala, Abhinav and Chow, Jerry M and Gambetta, Jay M},
  Journal                  = {Nature},
  Year                     = {2019},
  Number                   = {7747},
  Pages                    = {209--212},
  Volume                   = {567},
  Publisher                = {Nature Publishing Group},
  URL                      = {https://www.nature.com/articles/s41586-019-0980-2}
}
</code></pre></details>


### 2018
<details><summary>[Du et al] <b><a href=https://arxiv.org/abs/1809.06056> Implementable quantum classifier for nonlinear data. </a></b> <i>arXiv.</i> <code><b>[Quantum Perceptron]</b></code> <code><b>[Quantum Machine Learning]</b></code> <code><b>[Quantum Classification]</b></code> <code><b>[Quantum Machine Learning]</b></code> <code><i>(Gate Model)</i></code></summary><pre><code>@Article{du2018implementable,
  Title                    = {Implementable quantum classifier for nonlinear data},
  Author                   = {Du, Yuxuan and Hsieh, Min-Hsiu and Liu, Tongliang and Tao, Dacheng},
  Journal                  = {arXiv preprint arXiv:1809.06056},
  Year                     = {2018},
  URL                      = {https://arxiv.org/abs/1809.06056}
}
</code></pre></details>
<details><summary>[Fahri & Neven] <b><a href=https://arxiv.org/abs/1802.06002> Classification with Quantum Neural Networks on Near Term Processors. </a></b> <i>arXiv.</i> <code><b>[Quantum Neural Networks]</b></code> <code><b>[Quantum Machine Learning]</b></code> <code><b>[Quantum Classification]</b></code> <code><i>(Gate Model)</i></code></summary><pre><code>@Article{farhi2018classification,
  Title                    = {Classification with quantum neural networks on near term processors},
  Author                   = {Farhi, Edward and Neven, Hartmut},
  Journal                  = {arXiv preprint arXiv:1802.06002},
  Year                     = {2018},
  URL                      = {https://arxiv.org/abs/1802.06002}
}
</code></pre></details>
<details><summary>[Grant et al] <b><a href=https://www.nature.com/articles/s41534-018-0116-9> Hierarchical quantum classifiers. </a></b> <i>npj Quantum information.</i> <code><b>[Quantum Neural Networks]</b></code> <code><b>[Quantum Machine Learning]</b></code> <code><b>[Quantum Classification]</b></code> <code><i>(Gate Model)</i></code></summary><pre><code>@Article{grant2018hierarchical,
  Title                    = {Hierarchical quantum classifiers},
  Author                   = {Grant, Edward and Benedetti, Marcello and Cao, Shuxiang and Hallam, Andrew and Lockhart, Joshua and Stojevic, Vid and Green, Andrew G and Severini, Simone},
  Journal                  = {npj Quantum Information},
  Year                     = {2018},
  Number                   = {1},
  Pages                    = {1--8},
  Volume                   = {4},
  Publisher                = {Nature Publishing Group},
  URL                      = {https://www.nature.com/articles/s41534-018-0116-9}
}
</code></pre></details>


### 2017
<details><summary>[Ruan et al] <b><a href=https://idp.springer.com/authorize/casa?redirect_uri=https://link.springer.com/article/10.1007/s10773-017-3514-4&casa_token=CO3V9sMtWRcAAAAA:9hw3GSs7B7yTOJZoSEACk9y4uoIrHM2RuXAeY_ThKvnmvCTLd5lkn-ova5xZ4Do4xVczdIiy_YTuI8JC> Quantum algorithm for k-nearest neighbors classification based on the metric of hamming distance. </a></b> <i>International Journal of Theoretical Physics.</i> <code><b>[Quantum Machine Learning]</b></code> <code><b>[Quantum Classification]</b></code> </summary><pre><code>@article{ruan2017quantum,
  title={Quantum algorithm for k-nearest neighbors classification based on the metric of hamming distance},
  author={Ruan, Yue and Xue, Xiling and Liu, Heng and Tan, Jianing and Li, Xi},
  journal={International Journal of Theoretical Physics},
  volume={56},
  number={11},
  pages={3496--3507},
  year={2017},
  publisher={Springer}
}</code></pre></details>


### 2015
<details><summary>[Adcock et al] <b><a href=https://arxiv.org/abs/1512.02900> Advances in quantum machine learning. </a></b> <i>arXiv.</i> <code><b>[Quantum Machine Learning]</b></code> <code><b>[Quantum Classification]</b></code> <code><b>[Quantum Clustering]</b></code> <code><b>[Quantum Nearest Neighbors Algorithm]</b></code> <code><b>[Quantum Neural Networks]</b></code> <code><i>(Gate Model)</i></code> <code><i>(Adiabatic Model)</i></code></summary><pre><code>@Article{adcock2015advances,
  Title                    = {Advances in quantum machine learning},
  Author                   = {Adcock, Jeremy and Allen, Euan and Day, Matthew and Frick, Stefan and Hinchliff, Janna and Johnson, Mack and Morley-Short, Sam and Pallister, Sam and Price, Alasdair and Stanisic, Stasja},
  Journal                  = {arXiv preprint arXiv:1512.02900},
  Year                     = {2015},
  URL                      = {https://arxiv.org/abs/1512.02900}
}
</code></pre></details>


### 2014
<details><summary>[Wiebe et al] <b><a href=https://dl.acm.org/doi/abs/10.5555/2871393.2871400> Quantum Algorithms for Nearest-Neighbour Methods for Supervised and Unsupervised Learning. </a></b> <i>Quantum Information & Computation.</i> <code><b>[Quantum Nearest Neighbors Algorithm]</b></code> <code><b>[Quantum Classification]</b></code> <code><b>[Quantum Machine Learning]</b></code> <code><i>(Gate Model)</i></code></summary><pre><code>@article{wiebe2015quantum,
  title={Quantum algorithms for nearest-neighbor methods for supervised and unsupervised learning},
  author={Wiebe, Nathan and Kapoor, Ashish and Svore, Krysta M},
  journal={Quantum Information \& Computation},
  volume={15},
  number={3-4},
  pages={316--356},
  year={2015},
  publisher={Rinton Press, Incorporated Paramus, NJ}
}</code></pre></details>


### 2000
<details><summary>[Ventura & Martinez] <b><a href=https://www.sciencedirect.com/science/article/abs/pii/S0020025599001012> Quantum associative memory. </a></b> <i>Information Sciences.</i> <code><b>[Quantum Machine Learning]</b></code> <code><b>[Quantum Information and Computing]</b></code> <code><b>[Quantum Classification]</b></code> </summary><pre><code>@article{ventura2000quantum,
  title={Quantum associative memory},
  author={Ventura, Dan and Martinez, Tony},
  journal={Information Sciences},
  volume={124},
  number={1-4},
  pages={273--296},
  year={2000},
  publisher={Elsevier}
}</code></pre></details>


# Quantum Clustering


### 2015
<details><summary>[Adcock et al] <b><a href=https://arxiv.org/abs/1512.02900> Advances in quantum machine learning. </a></b> <i>arXiv.</i> <code><b>[Quantum Machine Learning]</b></code> <code><b>[Quantum Classification]</b></code> <code><b>[Quantum Clustering]</b></code> <code><b>[Quantum Nearest Neighbors Algorithm]</b></code> <code><b>[Quantum Neural Networks]</b></code> <code><i>(Gate Model)</i></code> <code><i>(Adiabatic Model)</i></code></summary><pre><code>@Article{adcock2015advances,
  Title                    = {Advances in quantum machine learning},
  Author                   = {Adcock, Jeremy and Allen, Euan and Day, Matthew and Frick, Stefan and Hinchliff, Janna and Johnson, Mack and Morley-Short, Sam and Pallister, Sam and Price, Alasdair and Stanisic, Stasja},
  Journal                  = {arXiv preprint arXiv:1512.02900},
  Year                     = {2015},
  URL                      = {https://arxiv.org/abs/1512.02900}
}
</code></pre></details>


### 2013
<details><summary>[Aïmeur et al] <b><a href=https://link.springer.com/article/10.1007/s10994-012-5316-5> Quantum speed-up for unsupervised learning. </a></b> <i>Machine Learning.</i> <code><b>[Quantum Machine Learning]</b></code> <code><b>[Unsupervised Learning]</b></code> <code><b>[Quantum Clustering]</b></code> <code><i>(Gate Model)</i></code></summary><pre><code>@Article{aimeur2013quantum,
  Title                    = {Quantum speed-up for unsupervised learning},
  Author                   = {A{\"\i}meur, Esma and Brassard, Gilles and Gambs, S{\'e}bastien},
  Journal                  = {Machine Learning},
  Year                     = {2013},
  Number                   = {2},
  Pages                    = {261--287},
  Volume                   = {90},
  Publisher                = {Springer},
  URL                      = {https://link.springer.com/article/10.1007/s10994-012-5316-5}
}
</code></pre></details>
<details><summary>[Lloyd et al] <b><a href=https://arxiv.org/abs/1307.0411> Quantum algorithms for supervised and unsupervised machine learning. </a></b> <i>arXiv.</i> <code><b>[Quantum Machine Learning]</b></code> <code><b>[Quantum Clustering]</b></code> <code><i>(Adiabatic Model)</i></code></summary><pre><code>@article{lloyd2013quantum,
  title={Quantum algorithms for supervised and unsupervised machine learning},
  author={Lloyd, Seth and Mohseni, Masoud and Rebentrost, Patrick},
  journal={arXiv preprint arXiv:1307.0411},
  year={2013}
}</code></pre></details>


### 2007
<details><summary>[Aïmeur et al] <b><a href=https://dl.acm.org/doi/abs/10.1145/1273496.1273497> Quantum Clustering Algorithms. </a></b> <i>ICML '07.</i> <code><b>[Quantum Machine Learning]</b></code> <code><b>[Unsupervised Learning]</b></code> <code><b>[Quantum Clustering]</b></code> <code><i>(Gate Model)</i></code></summary><pre><code>@inproceedings{aimeur2007quantum,
  title={Quantum clustering algorithms},
  author={A{\"\i}meur, Esma and Brassard, Gilles and Gambs, S{\'e}bastien},
  booktitle={Proceedings of the 24th international conference on machine learning},
  pages={1--8},
  year={2007}
}</code></pre></details>


### 2006
<details><summary>[Aïmeur et al] <b><a href=https://link.springer.com/chapter/10.1007/11766247_37> Machine Learning in a Quantum World. </a></b> <i>Canadian AI 2006.</i> <code><b>[Quantum Machine Learning]</b></code> <code><b>[Unsupervised Learning]</b></code> <code><b>[Quantum Clustering]</b></code> <code><i>(Gate Model)</i></code></summary><pre><code>@InProceedings{aimeur2006machine,
  Title                    = {Machine learning in a quantum world},
  Author                   = {A{\"\i}meur, Esma and Brassard, Gilles and Gambs, S{\'e}bastien},
  Booktitle                = {Conference of the Canadian Society for Computational Studies of Intelligence},
  Year                     = {2006},
  Organization             = {Springer},
  Pages                    = {431--442},
  URL                      = {https://link.springer.com/chapter/10.1007/11766247_37}
}
</code></pre></details>


# Quantum Convolutional Neural Networks


### 2020
<details><summary>[Henderson et al] <b><a href=https://link.springer.com/content/pdf/10.1007/s42484-020-00012-y.pdf> Quanvolutional neural networks: powering image recognition with quantum circuits. </a></b> <i>Quantum Machine Intelligence.</i> <code><b>[Quantum Convolutional Neural Networks]</b></code> <code><b>[Quantum Neural Networks]</b></code> <code><b>[Quantum Machine Learning]</b></code> <code><i>(Gate Model)</i></code></summary><pre><code>@article{henderson2020quanvolutional,
  title={Quanvolutional neural networks: powering image recognition with quantum circuits},
  author={Henderson, Maxwell and Shakya, Samriddhi and Pradhan, Shashindra and Cook, Tristan},
  journal={Quantum Machine Intelligence},
  volume={2},
  number={1},
  pages={1--9},
  year={2020},
  publisher={Springer}
}</code></pre></details>
<details><summary>[Kerenidis et al] <b><a href=https://openreview.net/pdf?id=Hygab1rKDS> Quantum algorithms for deep convolutional neural networks. </a></b> <i>ICLR.</i> <code><b>[Quantum Neural Networks]</b></code> <code><b>[Quantum Convolutional Neural Networks]</b></code> <code><b>[Quantum Machine Learning]</b></code> <code><i>(Gate Model)</i></code></summary><pre><code>@inproceedings{kerenidis2019quantum,
  title={Quantum Algorithms for Deep Convolutional Neural Networks},
  author={Kerenidis, Iordanis and Landman, Jonas and Prakash, Anupam},
  booktitle={International Conference on Learning Representations},
  year={2019}
}</code></pre></details>
<details><summary>[Li et al] <b><a href=https://iopscience.iop.org/article/10.1088/2058-9565/ab9f93> A quantum deep convolutional neural network for image recognition. </a></b> <i>Quantum Science and Technology.</i> <code><b>[Quantum Convolutional Neural Networks]</b></code> <code><b>[Quantum Neural Networks]</b></code> <code><b>[Quantum Machine Learning]</b></code> </summary><pre><code>@article{li2020quantum,
  title={A quantum deep convolutional neural network for image recognition},
  author={Li, YaoChong and Zhou, Ri-Gui and Xu, RuQing and Luo, Jia and Hu, WenWen},
  journal={Quantum Science and Technology},
  volume={5},
  number={4},
  pages={044003},
  year={2020},
  publisher={IOP Publishing}
}</code></pre></details>
<details><summary>[Pesah et al] <b><a href=https://arxiv.org/abs/2011.02966> Absence of Barren Plateaus in Quantum Convolutional Neural Networks. </a></b> <i>arXiv.</i> <code><b>[Barren Plateaus]</b></code> <code><b>[Quantum Convolutional Neural Networks]</b></code> <code><b>[Parameterized Quantum Circuits]</b></code> <code><b>[Quantum Neural Networks]</b></code> <code><b>[Quantum Machine Learning]</b></code> </summary><pre><code>@article{pesah2020absence,
  title={Absence of barren plateaus in quantum convolutional neural networks},
  author={Pesah, Arthur and Cerezo, M and Wang, Samson and Volkoff, Tyler and Sornborger, Andrew T and Coles, Patrick J},
  journal={arXiv preprint arXiv:2011.02966},
  year={2020}
}</code></pre></details>


### 2019
<details><summary>[Cong et al] <b><a href=https://www.nature.com/articles/s41567-019-0648-8.pdf> Quantum convolutional neural networks. </a></b> <i>Nature Physics.</i> <code><b>[Quantum Neural Networks]</b></code> <code><b>[Quantum Convolutional Neural Networks]</b></code> <code><b>[Quantum Machine Learning]</b></code> <code><i>(Gate Model)</i></code></summary><pre><code>@Article{cong2019quantum,
  Title                    = {Quantum convolutional neural networks},
  Author                   = {Cong, Iris and Choi, Soonwon and Lukin, Mikhail D},
  Journal                  = {Nature Physics},
  Year                     = {2019},
  Number                   = {12},
  Pages                    = {1273--1278},
  Volume                   = {15},
  Publisher                = {Nature Publishing Group},
  URL                      = {https://www.nature.com/articles/s41567-019-0648-8.pdf}
}
</code></pre></details>


# Quantum Generative Adversarial Networks


### 2019
<details><summary>[Zoufal et al] <b><a href=https://www.nature.com/articles/s41534-019-0223-2> Quantum Generative Adversarial Networks for Learning and Loading Random
Distributions. </a></b> <i>npj Quantum Information.</i> <code><b>[Quantum Generative Adversarial Networks]</b></code> <code><b>[Variational Quantum Algorithms]</b></code> <code><b>[Quantum Machine Learning]</b></code> <code><i>(Gate Model)</i></code></summary><pre><code>@article{zoufal2019quantum,
  title={Quantum generative adversarial networks for learning and loading random distributions},
  author={Zoufal, Christa and Lucchi, Aur{\'e}lien and Woerner, Stefan},
  journal={npj Quantum Information},
  volume={5},
  number={1},
  pages={1--9},
  year={2019},
  publisher={Nature Publishing Group}
}</code></pre></details>


### 2018
<details><summary>[Allcock & Zhang] <b><a href=https://academic.oup.com/nsr/article/6/1/26/5222655?login=true> Quantum machine learning. </a></b> <i>National Science Review.</i> <code><b>[Quantum Machine Learning]</b></code> <code><b>[Quantum Boltzmann machines]</b></code> <code><b>[Quantum Generative Adversarial Networks]</b></code> </summary><pre><code>@Article{allcock2019quantum,
  Title                    = {Quantum machine learning},
  Author                   = {Allcock, Jonathan and Zhang, Shengyu},
  Journal                  = {National Science Review},
  Year                     = {2019},
  Number                   = {1},
  Pages                    = {26--28},
  Volume                   = {6},
  Publisher                = {Oxford University Press},
  URL                      = {https://academic.oup.com/nsr/article/6/1/26/5222655?login=true}
}
</code></pre></details>
<details><summary>[Dallaire-Demers & Killoran] <b><a href=https://journals.aps.org/pra/abstract/10.1103/PhysRevA.98.012324> Quantum generative adversarial networks. </a></b> <i>Physical Review A.</i> <code><b>[Quantum Generative Adversarial Networks]</b></code> <code><b>[Quantum Neural Networks]</b></code> <code><b>[Variational Quantum Algorithms]</b></code> <code><b>[Quantum Machine Learning]</b></code> <code><i>(Gate Model)</i></code></summary><pre><code>@article{dallaire2018quantum,
  title={Quantum generative adversarial networks},
  author={Dallaire-Demers, Pierre-Luc and Killoran, Nathan},
  journal={Physical Review A},
  volume={98},
  number={1},
  pages={012324},
  year={2018},
  publisher={APS}
}</code></pre></details>


# Quantum Information and Computing


### 2020
<details><summary>[Pjilipp  et al] <b><a href=https://iopscience.iop.org/article/10.1088/1361-6633/ab85b8> Perspectives of quantum annealing: Methods andimplementations. </a></b> <i>Reports on Progress in Physics.</i> <code><b>[Quantum Information and Computing]</b></code> <code><b>[Quantum Annealing]</b></code> <code><i>(Adiabatic Model)</i></code></summary><pre><code>@article{hauke2020perspectives,
  title={Perspectives of quantum annealing: Methods and implementations},
  author={Hauke, Philipp and Katzgraber, Helmut G and Lechner, Wolfgang and Nishimori, Hidetoshi and Oliver, William D},
  journal={Reports on Progress in Physics},
  volume={83},
  number={5},
  pages={054401},
  year={2020},
  publisher={IOP Publishing}
}</code></pre></details>
<details><summary>[Kjaergaard et al] <b><a href=https://www.annualreviews.org/doi/pdf/10.1146/annurev-conmatphys-031119-050605> Superconducting Qubits:
Current State of Play. </a></b> <i>Annual review of condensed matter physics.</i> <code><b>[Quantum Information and Computing]</b></code> <code><b>[Physical Realization of Qubits]</b></code> <code><b>[Superconducting Qubits]</b></code> </summary><pre><code>@article{kjaergaard2020superconducting,
  title={Superconducting qubits: Current state of play},
  author={Kjaergaard, Morten and Schwartz, Mollie E and Braum{\"u}ller, Jochen and Krantz, Philip and Wang, Joel I-J and Gustavsson, Simon and Oliver, William D},
  journal={Annual Review of Condensed Matter Physics},
  volume={11},
  pages={369--395},
  year={2020},
  publisher={Annual Reviews}
}</code></pre></details>
<details><summary>[McGeoch] <b><a href=https://www.sciencedirect.com/science/article/abs/pii/S0304397520300529> Theory versus practice in annealing-based quantum computing. </a></b> <i>Theoretical Computer Science.</i> <code><b>[Quantum Information and Computing]</b></code> <code><i>(Adiabatic Model)</i></code></summary><pre><code>@article{mcgeoch2020theory,
  title={Theory versus practice in annealing-based quantum computing},
  author={McGeoch, Catherine C},
  journal={Theoretical Computer Science},
  volume={816},
  pages={169--183},
  year={2020},
  publisher={Elsevier}
}</code></pre></details>
<details><summary>[Ramezani et al] <b><a href=https://ieeexplore.ieee.org/abstract/document/9207714/?casa_token=086gCg9_PHoAAAAA:drZYsucyKd7gT4q2d0kxFc5X9rcStHITEq7ReXA-aw8lNIx0s7d51pIz2NdwwxOzFFCYsSsh_r0r> Machine Learning Algorithms in Quantum Computing: A Survey. </a></b> <i>International Joint Conference on Neural Networks.</i> <code><b>[Quantum Information and Computing]</b></code> <code><b>[Quantum Machine Learning]</b></code> </summary><pre><code>@inproceedings{ramezani2020machine,
  title={Machine learning algorithms in quantum computing: A survey},
  author={Ramezani, Somayeh Bakhtiari and Sommers, Alexander and Manchukonda, Harish Kumar and Rahimi, Shahram and Amirlatifi, Amin},
  booktitle={2020 International Joint Conference on Neural Networks (IJCNN)},
  pages={1--8},
  year={2020},
  organization={IEEE}
}</code></pre></details>
<details><summary>[Sergioli] <b><a href=https://link.springer.com/article/10.1007/s00500-019-04429-x> Quantum and quantum-like machine learning: a note on differences and similarities. </a></b> <i>Soft Computing.</i> <code><b>[Quantum Machine Learning]</b></code> <code><b>[Quantum Information and Computing]</b></code> <code><b>[Quantum Classification]</b></code> </summary><pre><code>@article{sergioli2020quantum,
  title={Quantum and quantum-like machine learning: a note on differences and similarities},
  author={Sergioli, Giuseppe},
  journal={Soft Computing},
  volume={24},
  number={14},
  pages={10247--10255},
  year={2020},
  publisher={Springer}
}</code></pre></details>


### 2019
<details><summary>[Gyongyosi & Imre] <b><a href=https://www.sciencedirect.com/science/article/abs/pii/S1574013718301709> A Survey on quantum computing technology. </a></b> <i>Computer Science Review.</i> <code><b>[Quantum Information and Computing]</b></code> </summary><pre><code>@Article{gyongyosi2019survey,
  Title                    = {A survey on quantum computing technology},
  Author                   = {Gyongyosi, Laszlo and Imre, Sandor},
  Journal                  = {Computer Science Review},
  Year                     = {2019},
  Pages                    = {51--71},
  Volume                   = {31},
  Publisher                = {Elsevier},
  URL                      = {https://www.sciencedirect.com/science/article/abs/pii/S1574013718301709}
}
</code></pre></details>
<details><summary>[Savchuk & Fesenko] <b><a href=https://link.springer.com/content/pdf/10.1007/s10559-019-00107-w.pdf> Quantum Computing: Survey and Analysis. </a></b> <i>Cybernetics and Systems Analysis.</i> <code><b>[Quantum Information and Computing]</b></code> </summary><pre><code>@article{savchuk2019quantum,
  title={Quantum Computing: Survey and Analysis},
  author={Savchuk, MM and Fesenko, AV},
  journal={Cybernetics and Systems Analysis},
  volume={55},
  number={1},
  pages={10--21},
  year={2019},
  publisher={Springer}
}</code></pre></details>
<details><summary>[Schuld & Killoran] <b><a href=https://journals.aps.org/prl/abstract/10.1103/PhysRevLett.122.040504> Quantum machine learning in feature Hilbert spaces. </a></b> <i>Physical Review Letters.</i> <code><b>[Quantum Machine Learning]</b></code> <code><b>[Quantum Information and Computing]</b></code> </summary><pre><code>@article{schuld2019quantum,
  title={Quantum machine learning in feature Hilbert spaces},
  author={Schuld, Maria and Killoran, Nathan},
  journal={Physical review letters},
  volume={122},
  number={4},
  pages={040504},
  year={2019},
  publisher={APS}
}</code></pre></details>
<details><summary>[Tacchino et al] <b><a href=https://www.Nature.com/articles/s41534-019-0140-4.pdf> An artificial neuron implemented on an actual quantumprocessor. </a></b> <i>Nature.</i> <code><b>[Quantum Perceptron]</b></code> <code><b>[Quantum Machine Learning]</b></code> <code><b>[Quantum Information and Computing]</b></code> <code><i>(Gate Model)</i></code></summary><pre><code>@article{tacchino2019artificial,
  title={An artificial neuron implemented on an actual quantum processor},
  author={Tacchino, Francesco and Macchiavello, Chiara and Gerace, Dario and Bajoni, Daniele},
  journal={npj Quantum Information},
  volume={5},
  number={1},
  pages={1--8},
  year={2019},
  publisher={Nature Publishing Group}
}</code></pre></details>
<details><summary>[Torrontegui et al] <b><a href=https://iopscience.iop.org/article/10.1209/0295-5075/125/30004/pdf> Unitary quantum perceptron as efficient universal approximator. </a></b> <i>Europhysics Letters.</i> <code><b>[Quantum Perceptron]</b></code> <code><b>[Quantum Machine Learning]</b></code> <code><b>[Quantum Information and Computing]</b></code> </summary><pre><code>@article{torrontegui2019unitary,
  title={Unitary quantum perceptron as efficient universal approximator},
  author={Torrontegui, Erik and Garc{\'\i}a-Ripoll, Juan Jos{\'e}},
  journal={EPL (Europhysics Letters)},
  volume={125},
  number={3},
  pages={30004},
  year={2019},
  publisher={IOP Publishing}
}</code></pre></details>
<details><summary>[Wiersema et al] <b><a href=https://journals.aps.org/pra/abstract/10.1103/PhysRevA.100.020301> Implementing perceptron models with qubits. </a></b> <i>Physical Review A.</i> <code><b>[Quantum Perceptron]</b></code> <code><b>[Quantum Machine Learning]</b></code> <code><b>[Quantum Information and Computing]</b></code> <code><i>(Gate Model)</i></code></summary><pre><code>@article{wiersema2019implementing,
  title={Implementing perceptron models with qubits},
  author={Wiersema, RC and Kappen, HJ},
  journal={Physical Review A},
  volume={100},
  number={2},
  pages={020301},
  year={2019},
  publisher={APS}
}</code></pre></details>


### 2018
<details><summary>[Albash & Lidar] <b><a href=https://journals.aps.org/rmp/abstract/10.1103/RevModPhys.90.015002> Adiabatic Quantum Computing. </a></b> <i>Reviews of Modern Physics.</i> <code><b>[Quantum Information and Computing]</b></code> <code><b>[Adiabatic Quantum Computing]</b></code> <code><i>(Adiabatic Model)</i></code></summary><pre><code>@Article{albash2018adiabatic,
  Title                    = {Adiabatic quantum computation},
  Author                   = {Albash, Tameem and Lidar, Daniel A},
  Journal                  = {Reviews of Modern Physics},
  Year                     = {2018},
  Number                   = {1},
  Pages                    = {015002},
  Volume                   = {90},
  Publisher                = {APS},
  URL                      = {https://journals.aps.org/rmp/abstract/10.1103/RevModPhys.90.015002}
}
</code></pre></details>
<details><summary>[Dunjko & Briegel] <b><a href=https://iopscience.iop.org/article/10.1088/1361-6633/aab406/meta> Machine learning & artificial intelligence in the quantum domain: a review of recent progress. </a></b> <i>Reports on Progress in Physics.</i> <code><b>[Quantum Machine Learning]</b></code> <code><b>[Probably Approximately Correct Learning]</b></code> <code><b>[Quantum Information and Computing]</b></code> <code><i>(Gate Model)</i></code> <code><i>(Adiabatic Model)</i></code></summary><pre><code>@Article{dunjko2018machine,
  Title                    = {Machine learning \& artificial intelligence in the quantum domain: a review of recent progress},
  Author                   = {Dunjko, Vedran and Briegel, Hans J},
  Journal                  = {Reports on Progress in Physics},
  Year                     = {2018},
  Number                   = {7},
  Pages                    = {074001},
  Volume                   = {81},
  Publisher                = {IOP Publishing},
  URL                      = {https://iopscience.iop.org/article/10.1088/1361-6633/aab406/meta}
}
</code></pre></details>
<details><summary>[Moll et al] <b><a href=https://iopscience.iop.org/article/10.1088/2058-9565/aab822/pdf> Quantum optimization using variational algorithms on near-term quantum devices. </a></b> <i>Quantum Science and Technology.</i> <code><b>[Variational Quantum Algorithms]</b></code> <code><b>[Quantum Optimization]</b></code> <code><b>[Quantum Machine Learning]</b></code> <code><b>[Quantum Information and Computing]</b></code> <code><i>(Gate Model)</i></code></summary><pre><code>@Article{moll2018quantum,
  Title                    = {Quantum optimization using variational algorithms on near-term quantum devices},
  Author                   = {Moll, Nikolaj and Barkoutsos, Panagiotis and Bishop, Lev S and Chow, Jerry M and Cross, Andrew and Egger, Daniel J and Filipp, Stefan and Fuhrer, Andreas and Gambetta, Jay M and Ganzhorn, Marc and others},
  Journal                  = {Quantum Science and Technology},
  Year                     = {2018},
  Number                   = {3},
  Pages                    = {030503},
  Volume                   = {3},
  Publisher                = {IOP Publishing},
  URL                      = {https://iopscience.iop.org/article/10.1088/2058-9565/aab822/meta}
}
</code></pre></details>
<details><summary>[Preskill] <b><a href=https://quantum-journal.org/papers/q-2018-08-06-79/pdf/> Quantum Computing in the NISQ era and beyond. </a></b> <i>Quantum Journal.</i> <code><b>[Quantum Information and Computing]</b></code> </summary><pre><code>@article{preskill2018quantum,
  title={Quantum computing in the NISQ era and beyond},
  author={Preskill, John},
  journal={Quantum},
  volume={2},
  pages={79},
  year={2018},
  publisher={Verein zur F{\"o}rderung des Open Access Publizierens in den Quantenwissenschaften}
}</code></pre></details>
<details><summary>[Wiebe et al] <b><a href=https://papers.nips.cc/paper/2016/file/d47268e9db2e9aa3827bba3afb7ff94a-Paper.pdf> Quantum Perceptron Models. </a></b> <i>NIPS .</i> <code><b>[Quantum Perceptron]</b></code> <code><b>[Quantum Machine Learning]</b></code> <code><b>[Quantum Information and Computing]</b></code> <code><i>(Gate Model)</i></code></summary><pre><code>@article{wiebe2016quantum,
  title={Quantum perceptron models},
  author={Wiebe, Nathan and Kapoor, Ashish and Svore, Krysta M},
  journal={arXiv preprint arXiv:1602.04799},
  year={2016}
}</code></pre></details>


### 2017
<details><summary>[Cao et al] <b><a href=https://arxiv.org/pdf/1711.11240.pdf> Quantum Neuron:  an elementary building block for machine learningon quantum computers. </a></b> <i>arXiv.</i> <code><b>[Quantum Perceptron]</b></code> <code><b>[Quantum Information and Computing]</b></code> <code><b>[Quantum Neural Networks]</b></code> <code><b>[Quantum Machine Learning]</b></code> <code><i>(Gate Model)</i></code></summary><pre><code>@Article{cao2017quantum,
  Title                    = {Quantum neuron: an elementary building block for machine learning on quantum computers},
  Author                   = {Cao, Yudong and Guerreschi, Gian Giacomo and Aspuru-Guzik, Al{\'a}n},
  Journal                  = {arXiv preprint arXiv:1711.11240},
  Year                     = {2017},
  URL                      = {https://arxiv.org/pdf/1711.11240.pdf}
}
</code></pre></details>
<details><summary>[Harrow & Montanaro] <b><a href=https://www.Nature.com/articles/Nature23458.pdf> Quantum computational supremacy. </a></b> <i>Nature.</i> <code><b>[Quantum Information and Computing]</b></code> </summary><pre><code>@article{harrow2017quantum,
  title={Quantum computational supremacy},
  author={Harrow, Aram W and Montanaro, Ashley},
  journal={Nature},
  volume={549},
  number={7671},
  pages={203--209},
  year={2017},
  publisher={Nature Publishing Group}
}</code></pre></details>


### 2014
<details><summary>[Bunyk et al] <b><a href=https://ieeexplore.ieee.org/document/6802426> Architectural considerations in the design of a superconducting quantum annealing processor. </a></b> <i>IEEE Transactions on Applied Superconductivity.</i> <code><b>[Quantum Annealing]</b></code> <code><b>[Quantum Information and Computing]</b></code> <code><b>[QPU]</b></code> <code><b>[Superconducting Qubits]</b></code> <code><i>(Adiabatic Model)</i></code></summary><pre><code>@Article{bunyk2014architectural,
  Title                    = {Architectural considerations in the design of a superconducting quantum annealing processor},
  Author                   = {Bunyk, Paul I and Hoskinson, Emile M and Johnson, Mark W and Tolkacheva, Elena and Altomare, Fabio and Berkley, Andrew J and Harris, Richard and Hilton, Jeremy P and Lanting, Trevor and Przybysz, Anthony J and others},
  Journal                  = {IEEE Transactions on Applied Superconductivity},
  Year                     = {2014},
  Number                   = {4},
  Pages                    = {1--10},
  Volume                   = {24},
  Publisher                = {IEEE},
  URL                      = {https://ieeexplore.ieee.org/document/6802426}
}
</code></pre></details>
<details><summary>[Rønnow et al] <b><a href=https://arxiv.org/pdf/1401.2910.pdf> Defining and detecting quantum speedup. </a></b> <i>Science.</i> <code><b>[Quantum Information and Computing]</b></code> <code><b>[Quantum Speedup-Advantage-Supremacy]</b></code> <code><i>(Adiabatic Model)</i></code></summary><pre><code>@article{ronnow2014defining,
  title={Defining and detecting quantum speedup},
  author={R{\o}nnow, Troels F and Wang, Zhihui and Job, Joshua and Boixo, Sergio and Isakov, Sergei V and Wecker, David and Martinis, John M and Lidar, Daniel A and Troyer, Matthias},
  journal={science},
  volume={345},
  number={6195},
  pages={420--424},
  year={2014},
  publisher={American Association for the Advancement of Science}
}</code></pre></details>


### 2013
<details><summary>[Devoret & Schoelkopf] <b><a href=https://science.sciencemag.org/content/339/6124/1169.abstract> Superconducting Circuits for Quantum Information: An Outlook. </a></b> <i>Science.</i> <code><b>[Quantum Information and Computing]</b></code> <code><b>[Superconducting Qubits]</b></code> <code><i>(Gate Model)</i></code></summary><pre><code>@Article{devoret2013superconducting,
  Title                    = {Superconducting circuits for quantum information: an outlook},
  Author                   = {Devoret, Michel H and Schoelkopf, Robert J},
  Journal                  = {Science},
  Year                     = {2013},
  Number                   = {6124},
  Pages                    = {1169--1174},
  Volume                   = {339},
  Publisher                = {American Association for the Advancement of Science},
  URL                      = {https://science.sciencemag.org/content/339/6124/1169.abstract}
}
</code></pre></details>


### 2012
<details><summary>[Preskill] <b><a href=https://arxiv.org/pdf/1203.5813.pdf> Quantum computing and the entanglement frontier. </a></b> <i>arXiv.</i> <code><b>[Quantum Information and Computing]</b></code> </summary><pre><code>@article{preskill2012quantum,
  title={Quantum computing and the entanglement frontier},
  author={Preskill, John},
  journal={arXiv preprint arXiv:1203.5813},
  year={2012}
}</code></pre></details>


### 2011
<details><summary>[Bremner et al] <b><a href=https://doi.org/10.1098/rspa.2010.0301> Classical simulation of commuting quantum computations implies collapse of the polynomial hierarchy. </a></b> <i>Proceedings of the Royal Society A: Mathematical, Physical and Engineering Sciences.</i> <code><b>[Quantum Speedup-Advantage-Supremacy]</b></code> <code><b>[Quantum Information and Computing]</b></code> <code><i>(Gate Model)</i></code></summary><pre><code>@Article{bremner2011classical,
  Title                    = {Classical simulation of commuting quantum computations implies collapse of the polynomial hierarchy},
  Author                   = {Bremner, Michael J and Jozsa, Richard and Shepherd, Dan J},
  Journal                  = {Proceedings of the Royal Society A: Mathematical, Physical and Engineering Sciences},
  Year                     = {2011},
  Number                   = {2126},
  Pages                    = {459--472},
  Volume                   = {467},
  Publisher                = {The Royal Society Publishing},
  URL                      = {https://doi.org/10.1098/rspa.2010.0301}
}
</code></pre></details>


### 2008
<details><summary>[Clarke et al] <b><a href=https://www.Nature.com/articles/Nature07128> Superconducting quantum bits. </a></b> <i>Nature.</i> <code><b>[Quantum Information and Computing]</b></code> </summary><pre><code>@Article{clarke2008superconducting,
  Title                    = {Superconducting quantum bits},
  Author                   = {Clarke, John and Wilhelm, Frank K},
  Journal                  = {Nature},
  Year                     = {2008},
  Number                   = {7198},
  Pages                    = {1031--1042},
  Volume                   = {453},
  Publisher                = {Nature Publishing Group},
  URL                      = {https://www.Nature.com/articles/Nature07128}
}
</code></pre></details>


### 2002
<details><summary>[Brassard et al ] <b><a href=http://dx.doi.org/10.1090/conm/305/05215> Quantum amplitude amplification and estimation. </a></b> <i>Contemporary Mathematics.</i> <code><b>[Quantum Information and Computing]</b></code> <code><i>(Gate Model)</i></code></summary><pre><code>@Article{brassard2002quantum,
  Title                    = {Quantum amplitude amplification and estimation},
  Author                   = {Brassard, Gilles and Hoyer, Peter and Mosca, Michele and Tapp, Alain},
  Journal                  = {Contemporary Mathematics},
  Year                     = {2002},
  Pages                    = {53--74},
  Volume                   = {305},
  Publisher                = {Providence, RI; American Mathematical Society; 1999},
  URL                      = {http://dx.doi.org/10.1090/conm/305/05215}
}
</code></pre></details>


### 2000
<details><summary>[Ventura & Martinez] <b><a href=https://www.sciencedirect.com/science/article/abs/pii/S0020025599001012> Quantum associative memory. </a></b> <i>Information Sciences.</i> <code><b>[Quantum Machine Learning]</b></code> <code><b>[Quantum Information and Computing]</b></code> <code><b>[Quantum Classification]</b></code> </summary><pre><code>@article{ventura2000quantum,
  title={Quantum associative memory},
  author={Ventura, Dan and Martinez, Tony},
  journal={Information Sciences},
  volume={124},
  number={1-4},
  pages={273--296},
  year={2000},
  publisher={Elsevier}
}</code></pre></details>


### 1997
<details><summary>[Bernstein & Vazirani] <b><a href=https://epubs.siam.org/doi/abs/10.1137/S0097539796300921> Quantum complexity theory. </a></b> <i>SIAM Journal on computing.</i> <code><b>[Quantum Information and Computing]</b></code> <code><b>[Quantum Speedup-Advantage-Supremacy]</b></code> </summary><pre><code>@article{bernstein1997quantum,
  title={Quantum complexity theory},
  author={Bernstein, Ethan and Vazirani, Umesh},
  journal={SIAM Journal on computing},
  volume={26},
  number={5},
  pages={1411--1473},
  year={1997},
  publisher={SIAM}
}</code></pre></details>
<details><summary>[Simon] <b><a href=https://epubs.siam.org/doi/10.1137/S0097539796298637> On the Power of Quantum Computation. </a></b> <i>SIAM Journal on Computing.</i> <code><b>[Quantum Information and Computing]</b></code> <code><b>[Quantum Speedup-Advantage-Supremacy]</b></code> </summary><pre><code>@article{simon1997power,
  title={On the power of quantum computation},
  author={Simon, Daniel R},
  journal={SIAM journal on computing},
  volume={26},
  number={5},
  pages={1474--1483},
  year={1997},
  publisher={SIAM}
}</code></pre></details>


### 1995
<details><summary>[Barenco et al] <b><a href=https://journals.aps.org/pra/abstract/10.1103/PhysRevA.52.3457> Elementary gates for quantum computation. </a></b> <i>Physical Review A.</i> <code><b>[Quantum Information and Computing]</b></code> <code><b>[Quantum Gates]</b></code> <code><i>(Gate Model)</i></code></summary><pre><code>@Article{barenco1995elementary,
  Title                    = {Elementary gates for quantum computation},
  Author                   = {Barenco, Adriano and Bennett, Charles H and Cleve, Richard and DiVincenzo, David P and Margolus, Norman and Shor, Peter and Sleator, Tycho and Smolin, John A and Weinfurter, Harald},
  Journal                  = {Physical review A},
  Year                     = {1995},
  Number                   = {5},
  Pages                    = {3457},
  Volume                   = {52},
  Publisher                = {APS},
  URL                      = {https://journals.aps.org/pra/abstract/10.1103/PhysRevA.52.3457}
}
</code></pre></details>


### 1992
<details><summary>[Deutsch & Josza] <b><a href=https://royalsocietypublishing.org/doi/10.1098/rspa.1992.0167> Rapid solution of problems by quantum computation. </a></b> <i>Proceedings of the Royal Society A.</i> <code><b>[Quantum Information and Computing]</b></code> <code><b>[Quantum Speedup-Advantage-Supremacy]</b></code> </summary><pre><code>@Article{deutsch1992rapid,
  Title                    = {Rapid solution of problems by quantum computation},
  Author                   = {Deutsch, David and Jozsa, Richard},
  Journal                  = {Proceedings of the Royal Society of London. Series A: Mathematical and Physical Sciences},
  Year                     = {1992},
  Number                   = {1907},
  Pages                    = {553--558},
  Volume                   = {439},
  Publisher                = {The Royal Society London},
  URL                      = {https://royalsocietypublishing.org/doi/10.1098/rspa.1992.0167}
}
</code></pre></details>


### 1985
<details><summary>[Deutsch] <b><a href=https://royalsocietypublishing.org/doi/10.1098/rspa.1985.0070> Quantum theory, the Church–Turing principle and the universal quantum computer. </a></b> <i>Proceedings of the Royal Society A.</i> <code><b>[Quantum Information and Computing]</b></code> </summary><pre><code>@Article{deutsch1985quantum,
  Title                    = {{Quantum theory, the Church--Turing principle and the universal quantum computer}},
  Author                   = {Deutsch, David},
  Journal                  = {Proceedings of the Royal Society of London. A. Mathematical and Physical Sciences},
  Year                     = {1985},
  Number                   = {1818},
  Pages                    = {97--117},
  Volume                   = {400},
  Publisher                = {The Royal Society London},
  URL                      = {https://royalsocietypublishing.org/doi/10.1098/rspa.1985.0070}
}
</code></pre></details>


# Quantum Machine Learning


### 2021
<details><summary>[Ban et al] <b><a href=https://www.Nature.com/articles/s41598-021-85208-3> Speeding up quantum perceptron via shortcuts to adiabaticity. </a></b> <i>Scientific reports.</i> <code><b>[Quantum Perceptron]</b></code> <code><b>[Quantum Machine Learning]</b></code> <code><i>(Adiabatic Model)</i></code></summary><pre><code>@Article{ban2021speeding,
  Title                    = {Speeding up quantum perceptron via shortcuts to adiabaticity},
  Author                   = {Ban, Yue and Chen, Xi and Torrontegui, E and Solano, Enrique and Casanova, Jorge},
  Journal                  = {Scientific reports},
  Year                     = {2021},
  Number                   = {1},
  Pages                    = {1--8},
  Volume                   = {11},
  Publisher                = {Nature Publishing Group},
  URL                      = {https://www.Nature.com/articles/s41598-021-85208-3}
}
</code></pre></details>
<details><summary>[Benedetti et al] <b><a href=https://journals.aps.org/prresearch/abstract/10.1103/PhysRevResearch.3.033083> Hardware-efficient variational quantum algorithms for time evolution. </a></b> <i>Physical Review Research.</i> <code><b>[Time Evolution (Hamiltonian Simulation)]</b></code> <code><b>[Parameterized Quantum Circuits]</b></code> <code><b>[Quantum Machine Learning]</b></code> </summary><pre><code>@article{benedetti2021hardware,
  title={Hardware-efficient variational quantum algorithms for time evolution},
  author={Benedetti, Marcello and Fiorentini, Mattia and Lubasch, Michael},
  journal={Physical Review Research},
  volume={3},
  number={3},
  pages={033083},
  year={2021},
  publisher={APS}
}
</code></pre></details>
<details><summary>[Benedetti et al] <b><a href=https://journals.aps.org/prapplied/abstract/10.1103/PhysRevApplied.16.044057> Variational Inference with a Quantum Computer. </a></b> <i>Physical Review Applied.</i> <code><b>[Variational Inference]</b></code> <code><b>[Parameterized Quantum Circuits]</b></code> <code><b>[Quantum Machine Learning]</b></code> </summary><pre><code>@article{benedetti2021variational,
  title={Variational inference with a quantum computer},
  author={Benedetti, Marcello and Coyle, Brian and Fiorentini, Mattia and Lubasch, Michael and Rosenkranz, Matthias},
  journal={Physical Review Applied},
  volume={16},
  number={4},
  pages={044057},
  year={2021},
  publisher={APS}
}
</code></pre></details>
<details><summary>[Bharti et al] <b><a href=https://arxiv.org/abs/2101.08448> Noisy intermediate-scale quantum (NISQ) algorithms. </a></b> <i>arXiv.</i> <code><b>[Variational Quantum Algorithms]</b></code> <code><b>[Parameterized Quantum Circuits]</b></code> <code><b>[Quantum Optimization]</b></code> <code><b>[Quantum Machine Learning]</b></code> <code><i>(Gate Model)</i></code> <code><i>(Adiabatic Model)</i></code></summary><pre><code>@article{bharti2021noisy,
  title={Noisy intermediate-scale quantum (NISQ) algorithms},
  author={Bharti, Kishor and Cervera-Lierta, Alba and Kyaw, Thi Ha and Haug, Tobias and Alperin-Lea, Sumner and Anand, Abhinav and Degroote, Matthias and Heimonen, Hermanni and Kottmann, Jakob S and Menke, Tim and others},
  journal={arXiv preprint arXiv:2101.08448},
  year={2021}
}</code></pre></details>
<details><summary>[Cerezo et al] <b><a href=https://www.nature.com/articles/s41467-021-21728-w> Cost Function Dependent Barren Plateaus in Shallow Parametrized Quantum Circuits. </a></b> <i>Nature communications.</i> <code><b>[Barren Plateaus]</b></code> <code><b>[Parameterized Quantum Circuits]</b></code> <code><b>[Quantum Neural Networks]</b></code> <code><b>[Quantum Machine Learning]</b></code> </summary><pre><code>@article{cerezo2021cost,
  title={Cost function dependent barren plateaus in shallow parametrized quantum circuits},
  author={Cerezo, Marco and Sone, Akira and Volkoff, Tyler and Cincio, Lukasz and Coles, Patrick J},
  journal={Nature communications},
  volume={12},
  number={1},
  pages={1--12},
  year={2021},
  publisher={Nature Publishing Group}
}</code></pre></details>
<details><summary>[Chen et al] <b><a href=https://arxiv.org/abs/1805.08654> Universal discriminative quantum neural networks. </a></b> <i>Quantum Machine Intelligence.</i> <code><b>[Quantum Neural Networks]</b></code> <code><b>[Quantum Machine Learning]</b></code> <code><i>(Gate Model)</i></code></summary><pre><code>@Article{chen2021universal,
  Title                    = {Universal discriminative quantum neural networks},
  Author                   = {Chen, Hongxiang and Wossnig, Leonard and Severini, Simone and Neven, Hartmut and Mohseni, Masoud},
  Journal                  = {Quantum Machine Intelligence},
  Year                     = {2021},
  Number                   = {1},
  Pages                    = {1--11},
  Volume                   = {3},
  Publisher                = {Springer},
  URL                      = {https://arxiv.org/abs/1805.08654}
}
</code></pre></details>
<details><summary>[Egger et al] <b><a href=https://quantum-journal.org/papers/q-2021-06-17-479/> Warm-starting quantum optimization. </a></b> <i>Quantum.</i> <code><b>[Quantum Optimization]</b></code> <code><b>[Parameterized Quantum Circuits]</b></code> <code><b>[Quantum Machine Learning]</b></code> </summary><pre><code>@article{egger2021warm,
  title={Warm-starting quantum optimization},
  author={Egger, Daniel J and Mare{\v{c}}ek, Jakub and Woerner, Stefan},
  journal={Quantum},
  volume={5},
  pages={479},
  year={2021},
  publisher={Verein zur F{\"o}rderung des Open Access Publizierens in den Quantenwissenschaften}
}</code></pre></details>
<details><summary>[Holmes et al ] <b><a href=https://arxiv.org/abs/2101.02138> Connecting ansatz expressibility to gradient magnitudes and barren plateaus. </a></b> <i>arXiv.</i> <code><b>[Parameterized Quantum Circuits]</b></code> <code><b>[Quantum Neural Networks]</b></code> <code><b>[Barren Plateaus]</b></code> <code><b>[Quantum Machine Learning]</b></code> </summary><pre><code>@article{holmes2021connecting,
  title={Connecting ansatz expressibility to gradient magnitudes and barren plateaus},
  author={Holmes, Zo{\"e} and Sharma, Kunal and Cerezo, M and Coles, Patrick J},
  journal={arXiv preprint arXiv:2101.02138},
  year={2021}
}</code></pre></details>
<details><summary>[Jaderberg et al] <b><a href=https://arxiv.org/abs/2103.14653> Quantum Self-Supervised Learning. </a></b> <i>arXiv.</i> <code><b>[Quantum Neural Networks]</b></code> <code><b>[Quantum Machine Learning]</b></code> <code><i>(Gate Model)</i></code></summary><pre><code>@article{jaderberg2021quantum,
  title={Quantum Self-Supervised Learning},
  author={Jaderberg, Ben and Anderson, Lewis W and Xie, Weidi and Albanie, Samuel and Kiffner, Martin and Jaksch, Dieter},
  journal={arXiv preprint arXiv:2103.14653},
  year={2021}
}</code></pre></details>
<details><summary>[Liu et al] <b><a href=https://www.nature.com/articles/s41567-021-01287-z> A rigorous and robust quantum speed-up in supervised machine learning. </a></b> <i>Nature Physics.</i> <code><b>[Quantum Machine Learning]</b></code> <code><b>[Quantum Classification]</b></code> <code><b>[Quantum Speedup-Advantage-Supremacy]</b></code> </summary><pre><code>@article{liu2021rigorous,
  title={A rigorous and robust quantum speed-up in supervised machine learning},
  author={Liu, Yunchao and Arunachalam, Srinivasan and Temme, Kristan},
  journal={Nature Physics},
  pages={1--5},
  year={2021},
  publisher={Nature Publishing Group}
}</code></pre></details>
<details><summary>[Lubsch et al] <b><a href=https://journals.aps.org/pra/abstract/10.1103/PhysRevA.101.010301> Variational quantum algorithms for nonlinear problems. </a></b> <i>Physical Review A.</i> <code><b>[Nonlinear Problems]</b></code> <code><b>[Partial Differential Equations]</b></code> <code><b>[Quantum Neural Networks]</b></code> <code><b>[Quantum Machine Learning]</b></code> </summary><pre><code>@article{lubasch2020variational,
  title={Variational quantum algorithms for nonlinear problems},
  author={Lubasch, Michael and Joo, Jaewoo and Moinier, Pierre and Kiffner, Martin and Jaksch, Dieter},
  journal={Physical Review A},
  volume={101},
  number={1},
  pages={010301},
  year={2020},
  publisher={APS}
}
</code></pre></details>
<details><summary>[Mangini et al] <b><a href=https://iopscience.iop.org/article/10.1209/0295-5075/134/10002/meta> Quantum computing models for artificial neural networks. </a></b> <i>EPL (Europhysics Letters).</i> <code><b>[Quantum Neural Networks]</b></code> <code><b>[Parameterized Quantum Circuits]</b></code> <code><b>[Variational Quantum Algorithms]</b></code> <code><b>[Quantum Machine Learning]</b></code> </summary><pre><code>@article{mangini2021quantum,
  title={Quantum computing models for artificial neural networks},
  author={Mangini, Stefano and Tacchino, Francesco and Gerace, Dario and Bajoni, Daniele and Macchiavello, Chiara},
  journal={EPL (Europhysics Letters)},
  volume={134},
  number={1},
  pages={10002},
  year={2021},
  publisher={IOP Publishing}
}</code></pre></details>
<details><summary>[Patti et al] <b><a href=https://arxiv.org/abs/2012.12658> Entanglement devised barren plateau mitigation. </a></b> <i>arXiv.</i> <code><b>[Barren Plateaus]</b></code> <code><b>[Parameterized Quantum Circuits]</b></code> <code><b>[Quantum Neural Networks]</b></code> <code><b>[Quantum Machine Learning]</b></code> </summary><pre><code>@article{patti2020entanglement,
  title={Entanglement devised barren plateau mitigation},
  author={Patti, Taylor L and Najafi, Khadijeh and Gao, Xun and Yelin, Susanne F},
  journal={arXiv preprint arXiv:2012.12658},
  year={2020}
}</code></pre></details>
<details><summary>[Perez-Salinas et al] <b><a href=https://arxiv.org/abs/2102.04032> One qubit as a Universal Approximant. </a></b> <i>arXiv.</i> <code><b>[Parameterized Quantum Circuits]</b></code> <code><b>[Quantum Neural Networks]</b></code> <code><b>[Quantum Machine Learning]</b></code> </summary><pre><code>@article{perez2021one,
  title={One qubit as a Universal Approximant},
  author={P{\'e}rez-Salinas, Adri{\'a}n and L{\'o}pez-N{\'u}{\~n}ez, David and Garc{\'\i}a-S{\'a}ez, Artur and Forn-D{\'\i}az, P and Latorre, Jos{\'e} I},
  journal={arXiv preprint arXiv:2102.04032},
  year={2021}
}</code></pre></details>
<details><summary>[Schuld et al] <b><a href=https://journals.aps.org/pra/abstract/10.1103/PhysRevA.103.032430> Effect of data encoding on the expressive power of variational quantum-machine-learning models. </a></b> <i>Physical Review A.</i> <code><b>[Parameterized Quantum Circuits]</b></code> <code><b>[Variational Quantum Algorithms]</b></code> <code><b>[Quantum Machine Learning]</b></code> </summary><pre><code>@article{schuld2021effect,
  title={Effect of data encoding on the expressive power of variational quantum-machine-learning models},
  author={Schuld, Maria and Sweke, Ryan and Meyer, Johannes Jakob},
  journal={Physical Review A},
  volume={103},
  number={3},
  pages={032430},
  year={2021},
  publisher={APS}
}</code></pre></details>
<details><summary>[Skolik et al] <b><a href=https://link.springer.com/article/10.1007/s42484-020-00036-4> Layerwise learning for quantum neural networks. </a></b> <i>Quantum Machine Intelligence.</i> <code><b>[Barren Plateaus]</b></code> <code><b>[Parameterized Quantum Circuits]</b></code> <code><b>[Quantum Neural Networks]</b></code> <code><b>[Quantum Machine Learning]</b></code> </summary><pre><code>@article{skolik2021layerwise,
  title={Layerwise learning for quantum neural networks},
  author={Skolik, Andrea and McClean, Jarrod R and Mohseni, Masoud and van der Smagt, Patrick and Leib, Martin},
  journal={Quantum Machine Intelligence},
  volume={3},
  number={1},
  pages={1--11},
  year={2021},
  publisher={Springer}
}</code></pre></details>
<details><summary>[Volkoff & Coles] <b><a href=https://arxiv.org/abs/2005.12200> Large gradients via correlation in random parameterized quantum circuits. </a></b> <i>arXiv.</i> <code><b>[Barren Plateaus]</b></code> <code><b>[Parameterized Quantum Circuits]</b></code> <code><b>[Quantum Neural Networks]</b></code> <code><b>[Variational Quantum Algorithms]</b></code> <code><b>[Quantum Machine Learning]</b></code> </summary><pre><code>@article{volkoff2021large,
  title={Large gradients via correlation in random parameterized quantum circuits},
  author={Volkoff, Tyler and Coles, Patrick J},
  journal={Quantum Science and Technology},
  volume={6},
  number={2},
  pages={025008},
  year={2021},
  publisher={IOP Publishing}
}</code></pre></details>


### 2020
<details><summary>[Ablayev et al] <b><a href=https://ieeexplore.ieee.org/stamp/stamp.jsp?arnumber=8935094> On quantum methods for machine learning problems part I: Quantum tools. </a></b> <i>Big Data Mining and Analytics.</i> <code><b>[Quantum Machine Learning]</b></code> <code><b>[Quantum Classification]</b></code> <code><i>(Gate Model)</i></code></summary><pre><code>@Article{ablayev2020quantum1,
  Title                    = {{On quantum methods for machine learning problems part I: Quantum tools}},
  Author                   = {Ablayev, Farid and Ablayev, Marat and Huang, Joshua Zhexue and Khadiev, Kamil and Salikhova, Nailya and Wu, Dingming},
  Journal                  = {Big Data Mining and Analytics},
  Year                     = {2020},
  Number                   = {1},
  Pages                    = {41--55},
  Volume                   = {3},
  URL                      = {https://ieeexplore.ieee.org/stamp/stamp.jsp?arnumber=8935094}
}
</code></pre></details>
<details><summary>[Ablayev et al] <b><a href=https://ieeexplore.ieee.org/abstract/document/8935095> On quantum methods for machine learning problems part II: Quantum classification algorithms. </a></b> <i>Big Data Mining and Analytics.</i> <code><b>[Quantum Machine Learning]</b></code> <code><b>[Quantum Classification]</b></code> <code><b>[Quantum Nearest Neighbors Algorithm]</b></code> <code><i>(Gate Model)</i></code></summary><pre><code>@Article{ablayev2020quantum2,
  Title                    = {{On quantum methods for machine learning problems part II: Quantum classification algorithms}},
  Author                   = {Ablayev, Farid and Ablayev, Marat and Huang, Joshua Zhexue and Khadiev, Kamil and Salikhova, Nailya and Wu, Dingming},
  Journal                  = {Big Data Mining and Analytics},
  Year                     = {2020},
  Number                   = {1},
  Pages                    = {56--67},
  Volume                   = {3},
  URL                      = {https://ieeexplore.ieee.org/abstract/document/8935095}
}
</code></pre></details>
<details><summary>[Abohashima et al] <b><a href=https://arxiv.org/abs/2006.12270> Classification with Quantum Machine Learning: A Survey. </a></b> <i>arXiv.</i> <code><b>[Quantum Machine Learning]</b></code> <code><b>[Quantum Classification]</b></code> <code><i>(Gate Model)</i></code></summary><pre><code>@article{abohashima2020classification,
  title={Classification with quantum machine learning: A survey},
  author={Abohashima, Zainab and Elhosen, Mohamed and Houssein, Essam H and Mohamed, Waleed M},
  journal={arXiv preprint arXiv:2006.12270},
  year={2020}
}</code></pre></details>
<details><summary>[Barkoutsos et al] <b><a href=https://quantum-journal.org/papers/q-2020-04-20-256/> Improving Variational Quantum Optimization Using CVaR. </a></b> <i>Quantum.</i> <code><b>[Variational Quantum Algorithms]</b></code> <code><b>[Quantum Optimization]</b></code> <code><b>[Quantum Machine Learning]</b></code> </summary><pre><code>@article{barkoutsos2020improving,
  title={Improving variational quantum optimization using cvar},
  author={Barkoutsos, Panagiotis Kl and Nannicini, Giacomo and Robert, Anton and Tavernelli, Ivano and Woerner, Stefan},
  journal={Quantum},
  volume={4},
  pages={256},
  year={2020},
  publisher={Verein zur F{\"o}rderung des Open Access Publizierens in den Quantenwissenschaften}
}</code></pre></details>
<details><summary>[Beer et al] <b><a href=https://www.nature.com/articles/s41467-020-14454-2.pdf> Training deep quantum neural networks. </a></b> <i>Nature communications.</i> <code><b>[Quantum Neural Networks]</b></code> <code><b>[Quantum Perceptron]</b></code> <code><b>[Quantum Machine Learning]</b></code> <code><i>(Gate Model)</i></code></summary><pre><code>@Article{beer2020training,
  Title                    = {Training deep quantum neural networks},
  Author                   = {Beer, Kerstin and Bondarenko, Dmytro and Farrelly, Terry and Osborne, Tobias J and Salzmann, Robert and Scheiermann, Daniel and Wolf, Ramona},
  Journal                  = {Nature communications},
  Year                     = {2020},
  Number                   = {1},
  Pages                    = {1--6},
  Volume                   = {11},
  Publisher                = {Nature Publishing Group},
  URL                      = {https://www.nature.com/articles/s41467-020-14454-2.pdf}
}
</code></pre></details>
<details><summary>[Bharti et al] <b><a href=https://avs.scitation.org/doi/10.1116/5.0007529> Machine learning meets quantum foundations: A brief survey. </a></b> <i>AVS Quantum Science.</i> <code><b>[Quantum Machine Learning]</b></code> <code><i>(Gate Model)</i></code></summary><pre><code>@Article{bharti2020machine,
  Title                    = {Machine learning meets quantum foundations: A brief survey},
  Author                   = {Bharti, Kishor and Haug, Tobias and Vedral, Vlatko and Kwek, Leong-Chuan},
  Journal                  = {AVS Quantum Science},
  Year                     = {2020},
  Number                   = {3},
  Pages                    = {034101},
  Volume                   = {2},
  Publisher                = {American Vacuum Society},
  URL                      = {https://avs.scitation.org/doi/10.1116/5.0007529}
}
</code></pre></details>
<details><summary>[Cerezo et al] <b><a href=https://arxiv.org/pdf/2012.09265.pdf> Variational Quantum Algorithms. </a></b> <i>arXiv.</i> <code><b>[Variational Quantum Algorithms]</b></code> <code><b>[Quantum Machine Learning]</b></code> <code><i>(Gate Model)</i></code> <code><i>(Adiabatic Model)</i></code></summary><pre><code>@Article{cerezo2020variational,
  Title                    = {Variational quantum algorithms},
  Author                   = {Cerezo, Marco and Arrasmith, Andrew and Babbush, Ryan and Benjamin, Simon C and Endo, Suguru and Fujii, Keisuke and McClean, Jarrod R and Mitarai, Kosuke and Yuan, Xiao and Cincio, Lukasz and others},
  Journal                  = {arXiv preprint arXiv:2012.09265},
  Year                     = {2020},
  URL                      = {https://arxiv.org/pdf/2012.09265.pdf}
}
</code></pre></details>
<details><summary>[Chakraborty et al] <b><a href=https://ieeexplore.ieee.org/document/9137960> An Analytical Review of Quantum Neural Network Models and Relevant Research. </a></b> <i>International Conference on Communication and Electronics Systems.</i> <code><b>[Quantum Neural Networks]</b></code> <code><b>[Quantum Machine Learning]</b></code> </summary><pre><code>@inproceedings{chakraborty2020analytical,
  title={An Analytical Review of Quantum Neural Network Models and Relevant Research},
  author={Chakraborty, Simantini and Das, Tamal and Sutradhar, Saurav and Das, Mrinmoy and Deb, Suman},
  booktitle={2020 5th International Conference on Communication and Electronics Systems (ICCES)},
  pages={1395--1400},
  year={2020},
  organization={IEEE}
}</code></pre></details>
<details><summary>[Duan & Guo] <b><a href=https://www.sciencedirect.com/science/article/pii/S037596012030462X> A survey on HHL algorithm: From theory to application in quantum machine learning. </a></b> <i>Physics Letters A.</i> <code><b>[Quantum Machine Learning]</b></code> <code><b>[Quantum Speedup-Advantage-Supremacy]</b></code> </summary><pre><code>@Article{duan1998probabilistic,
  Title                    = {Probabilistic cloning and identification of linearly independent quantum states},
  Author                   = {Duan, Lu-Ming and Guo, Guang-Can},
  Journal                  = {Physical review letters},
  Year                     = {1998},
  Number                   = {22},
  Pages                    = {4999},
  Volume                   = {80},
  Publisher                = {APS},
  URL                      = {https://www.sciencedirect.com/science/article/pii/S037596012030462X}
}
</code></pre></details>
<details><summary>[Dunjko & Wittek] <b><a href=https://quantum-journal.org/views/qv-2020-03-17-32/> A non-review of Quantum Machine Learning: trends and explorations. </a></b> <i>Quantum Views.</i> <code><b>[Quantum Machine Learning]</b></code> </summary><pre><code>@Article{dunjko2020non,
  Title                    = {A non-review of Quantum Machine Learning: trends and explorations},
  Author                   = {Dunjko, Vedran and Wittek, Peter},
  Journal                  = {Quantum Views},
  Year                     = {2020},
  Pages                    = {32},
  Volume                   = {4},
  Publisher                = {Verein zur F{\"o}rderung des Open Access Publizierens in den Quantenwissenschaften},
  URL                      = {https://quantum-journal.org/views/qv-2020-03-17-32/}
}
</code></pre></details>
<details><summary>[Gil Vidal & Theis] <b><a href=https://doi.org/10.3389/fphy.2020.00297> Input redundancy for parameterized quantum circuits. </a></b> <i>Frontiers in Physics.</i> <code><b>[Parameterized Quantum Circuits]</b></code> <code><b>[Quantum Neural Networks]</b></code> <code><b>[Quantum Machine Learning]</b></code> </summary><pre><code>@article{gil2020input,
  title={Input redundancy for parameterized quantum circuits},
  author={Gil Vidal, Francisco Javier and Theis, Dirk Oliver},
  journal={Frontiers in Physics},
  volume={8},
  pages={297},
  year={2020},
  publisher={Frontiers}
}</code></pre></details>
<details><summary>[Henderson et al] <b><a href=https://link.springer.com/content/pdf/10.1007/s42484-020-00012-y.pdf> Quanvolutional neural networks: powering image recognition with quantum circuits. </a></b> <i>Quantum Machine Intelligence.</i> <code><b>[Quantum Convolutional Neural Networks]</b></code> <code><b>[Quantum Neural Networks]</b></code> <code><b>[Quantum Machine Learning]</b></code> <code><i>(Gate Model)</i></code></summary><pre><code>@article{henderson2020quanvolutional,
  title={Quanvolutional neural networks: powering image recognition with quantum circuits},
  author={Henderson, Maxwell and Shakya, Samriddhi and Pradhan, Shashindra and Cook, Tristan},
  journal={Quantum Machine Intelligence},
  volume={2},
  number={1},
  pages={1--9},
  year={2020},
  publisher={Springer}
}</code></pre></details>
<details><summary>[Kerenidis et al] <b><a href=https://openreview.net/pdf?id=Hygab1rKDS> Quantum algorithms for deep convolutional neural networks. </a></b> <i>ICLR.</i> <code><b>[Quantum Neural Networks]</b></code> <code><b>[Quantum Convolutional Neural Networks]</b></code> <code><b>[Quantum Machine Learning]</b></code> <code><i>(Gate Model)</i></code></summary><pre><code>@inproceedings{kerenidis2019quantum,
  title={Quantum Algorithms for Deep Convolutional Neural Networks},
  author={Kerenidis, Iordanis and Landman, Jonas and Prakash, Anupam},
  booktitle={International Conference on Learning Representations},
  year={2019}
}</code></pre></details>
<details><summary>[Li et al] <b><a href=https://iopscience.iop.org/article/10.1088/2058-9565/ab9f93> A quantum deep convolutional neural network for image recognition. </a></b> <i>Quantum Science and Technology.</i> <code><b>[Quantum Convolutional Neural Networks]</b></code> <code><b>[Quantum Neural Networks]</b></code> <code><b>[Quantum Machine Learning]</b></code> </summary><pre><code>@article{li2020quantum,
  title={A quantum deep convolutional neural network for image recognition},
  author={Li, YaoChong and Zhou, Ri-Gui and Xu, RuQing and Luo, Jia and Hu, WenWen},
  journal={Quantum Science and Technology},
  volume={5},
  number={4},
  pages={044003},
  year={2020},
  publisher={IOP Publishing}
}</code></pre></details>
<details><summary>[Macaluso et al] <b><a href=https://link.springer.com/content/pdf/10.1007%2F978-3-030-50433-5_45.pdf> A Variational Algorithm for QuantumNeural Networks. </a></b> <i>International Conference on Computational Science.</i> <code><b>[Quantum Neural Networks]</b></code> <code><b>[Quantum Perceptron]</b></code> <code><b>[Quantum Machine Learning]</b></code> <code><i>(Gate Model)</i></code></summary><pre><code>@inproceedings{macaluso2020variational,
  title={A Variational Algorithm for Quantum Neural Networks},
  author={Macaluso, Antonio and Clissa, Luca and Lodi, Stefano and Sartori, Claudio},
  booktitle={International Conference on Computational Science},
  pages={591--604},
  year={2020},
  organization={Springer}
}</code></pre></details>
<details><summary>[Mari et al] <b><a href=https://quantum-journal.org/papers/q-2020-10-09-340/> Transfer learning in hybrid classical-quantum neural networks. </a></b> <i>Quantum.</i> <code><b>[Quantum Neural Networks]</b></code> <code><b>[Quantum Transfer Learning]</b></code> <code><b>[Quantum Machine Learning]</b></code> <code><i>(Gate Model)</i></code></summary><pre><code>@article{mari2020transfer,
  title={Transfer learning in hybrid classical-quantum neural networks},
  author={Mari, Andrea and Bromley, Thomas R and Izaac, Josh and Schuld, Maria and Killoran, Nathan},
  journal={Quantum},
  volume={4},
  pages={340},
  year={2020},
  publisher={Verein zur F{\"o}rderung des Open Access Publizierens in den Quantenwissenschaften}
}</code></pre></details>
<details><summary>[Marrero et al] <b><a href=https://arxiv.org/abs/2010.15968> Entanglement Induced Barren Plateaus. </a></b> <i>arXiv.</i> <code><b>[Barren Plateaus]</b></code> <code><b>[Parameterized Quantum Circuits]</b></code> <code><b>[Quantum Neural Networks]</b></code> <code><b>[Quantum Machine Learning]</b></code> </summary><pre><code>@article{marrero2020entanglement,
  title={Entanglement induced barren plateaus},
  author={Marrero, Carlos Ortiz and Kieferov{\'a}, M{\'a}ria and Wiebe, Nathan},
  journal={arXiv preprint arXiv:2010.15968},
  year={2020}
}</code></pre></details>
<details><summary>[Mezquita et al] <b><a href=https://link.springer.com/chapter/10.1007/978-3-030-53829-3_20> A Review of k-NN Algorithm Based on Classical and Quantum Machine Learning. </a></b> <i>DCAI 2020.</i> <code><b>[Quantum Machine Learning]</b></code> </summary><pre><code>@inproceedings{mezquita2020review,
  title={A Review of k-NN Algorithm Based on Classical and Quantum Machine Learning},
  author={Mezquita, Yeray and Alonso, Ricardo S and Casado-Vara, Roberto and Prieto, Javier and Corchado, Juan Manuel},
  booktitle={International Symposium on Distributed Computing and Artificial Intelligence},
  pages={189--198},
  year={2020},
  organization={Springer}
}</code></pre></details>
<details><summary>[Pande et Mulay] <b><a href=https://www.tandfonline.com/doi/abs/10.1080/0194262X.2020.1776193> Bibliometric Survey of Quantum Machine Learning. </a></b> <i>Science & Technology Libraries.</i> <code><b>[Quantum Machine Learning]</b></code> </summary><pre><code>@article{pande2020bibliometric,
  title={Bibliometric survey of quantum machine learning},
  author={Pande, Mandaar and Mulay, Preeti},
  journal={Science \& Technology Libraries},
  volume={39},
  number={4},
  pages={369--382},
  year={2020},
  publisher={Taylor \& Francis}
}</code></pre></details>
<details><summary>[Perez-Salinas et al] <b><a href=https://quantum-journal.org/papers/q-2020-02-06-226/> Data re-uploading for a universal quantum classifier. </a></b> <i>Quantum.</i> <code><b>[Quantum Neural Networks]</b></code> <code><b>[Quantum Classification]</b></code> <code><b>[Quantum Machine Learning]</b></code> <code><i>(Gate Model)</i></code></summary><pre><code>@article{perez2020data,
  title={Data re-uploading for a universal quantum classifier},
  author={P{\'e}rez-Salinas, Adri{\'a}n and Cervera-Lierta, Alba and Gil-Fuster, Elies and Latorre, Jos{\'e} I},
  journal={Quantum},
  volume={4},
  pages={226},
  year={2020},
  publisher={Verein zur F{\"o}rderung des Open Access Publizierens in den Quantenwissenschaften}
}</code></pre></details>
<details><summary>[Pesah et al] <b><a href=https://arxiv.org/abs/2011.02966> Absence of Barren Plateaus in Quantum Convolutional Neural Networks. </a></b> <i>arXiv.</i> <code><b>[Barren Plateaus]</b></code> <code><b>[Quantum Convolutional Neural Networks]</b></code> <code><b>[Parameterized Quantum Circuits]</b></code> <code><b>[Quantum Neural Networks]</b></code> <code><b>[Quantum Machine Learning]</b></code> </summary><pre><code>@article{pesah2020absence,
  title={Absence of barren plateaus in quantum convolutional neural networks},
  author={Pesah, Arthur and Cerezo, M and Wang, Samson and Volkoff, Tyler and Sornborger, Andrew T and Coles, Patrick J},
  journal={arXiv preprint arXiv:2011.02966},
  year={2020}
}</code></pre></details>
<details><summary>[Ramezani et al] <b><a href=https://ieeexplore.ieee.org/abstract/document/9207714/?casa_token=086gCg9_PHoAAAAA:drZYsucyKd7gT4q2d0kxFc5X9rcStHITEq7ReXA-aw8lNIx0s7d51pIz2NdwwxOzFFCYsSsh_r0r> Machine Learning Algorithms in Quantum Computing: A Survey. </a></b> <i>International Joint Conference on Neural Networks.</i> <code><b>[Quantum Information and Computing]</b></code> <code><b>[Quantum Machine Learning]</b></code> </summary><pre><code>@inproceedings{ramezani2020machine,
  title={Machine learning algorithms in quantum computing: A survey},
  author={Ramezani, Somayeh Bakhtiari and Sommers, Alexander and Manchukonda, Harish Kumar and Rahimi, Shahram and Amirlatifi, Amin},
  booktitle={2020 International Joint Conference on Neural Networks (IJCNN)},
  pages={1--8},
  year={2020},
  organization={IEEE}
}</code></pre></details>
<details><summary>[Schuld et al] <b><a href=https://journals.aps.org/pra/abstract/10.1103/PhysRevA.101.032308> Circuit-centric quantum classifiers. </a></b> <i>Physical Review A.</i> <code><b>[Quantum Neural Networks]</b></code> <code><b>[Quantum Classification]</b></code> <code><b>[Quantum Machine Learning]</b></code> <code><i>(Gate Model)</i></code></summary><pre><code>@article{schuld2020circuit,
  title={Circuit-centric quantum classifiers},
  author={Schuld, Maria and Bocharov, Alex and Svore, Krysta M and Wiebe, Nathan},
  journal={Physical Review A},
  volume={101},
  number={3},
  pages={032308},
  year={2020},
  publisher={APS}
}</code></pre></details>
<details><summary>[Sergioli] <b><a href=https://link.springer.com/article/10.1007/s00500-019-04429-x> Quantum and quantum-like machine learning: a note on differences and similarities. </a></b> <i>Soft Computing.</i> <code><b>[Quantum Machine Learning]</b></code> <code><b>[Quantum Information and Computing]</b></code> <code><b>[Quantum Classification]</b></code> </summary><pre><code>@article{sergioli2020quantum,
  title={Quantum and quantum-like machine learning: a note on differences and similarities},
  author={Sergioli, Giuseppe},
  journal={Soft Computing},
  volume={24},
  number={14},
  pages={10247--10255},
  year={2020},
  publisher={Springer}
}</code></pre></details>
<details><summary>[Sharma et al] <b><a href=https://arxiv.org/abs/2005.12458> Trainability of Dissipative Perceptron-Based Quantum Neural Networks. </a></b> <i>arXiv.</i> <code><b>[Parameterized Quantum Circuits]</b></code> <code><b>[Quantum Neural Networks]</b></code> <code><b>[Quantum Machine Learning]</b></code> <code><b>[Barren Plateaus]</b></code> </summary><pre><code>@article{sharma2020trainability,
  title={Trainability of dissipative perceptron-based quantum neural networks},
  author={Sharma, Kunal and Cerezo, Marco and Cincio, Lukasz and Coles, Patrick J},
  journal={arXiv preprint arXiv:2005.12458},
  year={2020}
}</code></pre></details>
<details><summary>[Tacchino et al] <b><a href=https://ieeexplore.ieee.org/abstract/document/9364892> Variational Learning for Quantum Artificial Neural Networks. </a></b> <i>IEEE International Conference on Quantum Computing and Engineering.</i> <code><b>[Parameterized Quantum Circuits]</b></code> <code><b>[Quantum Neural Networks]</b></code> <code><b>[Quantum Machine Learning]</b></code> </summary><pre><code>@inproceedings{tacchino2020variational,
  title={Variational learning for quantum artificial neural networks},
  author={Tacchino, Francesco and Barkoutsos, Panagiotis Kl and Macchiavello, Chiara and Gerace, Dario and Tavernelli, Ivano and Bajoni, Daniele},
  booktitle={2020 IEEE International Conference on Quantum Computing and Engineering (QCE)},
  pages={130--136},
  year={2020},
  organization={IEEE}
}</code></pre></details>
<details><summary>[Wang et al] <b><a href=https://arxiv.org/abs/2007.14384> Noise-Induced Barren Plateaus in Variational Quantum Algorithms. </a></b> <i>arXiv.</i> <code><b>[Barren Plateaus]</b></code> <code><b>[Parameterized Quantum Circuits]</b></code> <code><b>[Variational Quantum Algorithms]</b></code> <code><b>[Quantum Machine Learning]</b></code> </summary><pre><code>@article{wang2020noise,
  title={Noise-induced barren plateaus in variational quantum algorithms},
  author={Wang, Samson and Fontana, Enrico and Cerezo, Marco and Sharma, Kunal and Sone, Akira and Cincio, Lukasz and Coles, Patrick J},
  journal={arXiv preprint arXiv:2007.14384},
  year={2020}
}</code></pre></details>
<details><summary>[Willsch et al] <b><a href=https://www.sciencedirect.com/science/article/pii/S001046551930342X> Support vector machines on the D-Wave quantum annealer. </a></b> <i>Computer Physics Communications.</i> <code><b>[Quantum Support Vector Machine]</b></code> <code><b>[Quantum Machine Learning]</b></code> <code><i>(Adiabatic Model)</i></code></summary><pre><code>@article{willsch2020support,
  title={Support vector machines on the D-Wave quantum annealer},
  author={Willsch, Dennis and Willsch, Madita and De Raedt, Hans and Michielsen, Kristel},
  journal={Computer physics communications},
  volume={248},
  pages={107006},
  year={2020},
  publisher={Elsevier}
}</code></pre></details>
<details><summary>[Zhang & Ni] <b><a href=https://onlinelibrary.wiley.com/doi/abs/10.1002/que2.34?casa_token=t0KD3pYj7-wAAAAA:cbEXtTV6lJd1EGc4na3-akrlbLfw9jZM6r3lHfCwOIs5eP1Ma0ZxUit3wN-8Q5rkz3TmVYBdYrtayw> Recent advances in quantum machine learning. </a></b> <i>Quantum Engineering.</i> <code><b>[Quantum Machine Learning]</b></code> <code><b>[Quantum Theory]</b></code> </summary><pre><code>@article{zhang2020recent,
  title={Recent advances in quantum machine learning},
  author={Zhang, Yao and Ni, Qiang},
  journal={Quantum Engineering},
  volume={2},
  number={1},
  pages={e34},
  year={2020},
  publisher={Wiley Online Library}
}</code></pre></details>


### 2019
<details><summary>[Benedetti et al] <b><a href=https://iopscience.iop.org/article/10.1088/2058-9565/ab4eb5/meta> Parameterized quantum circuits as machine learning models. </a></b> <i>Quantum Science and Technology.</i> <code><b>[Variational Quantum Algorithms]</b></code> <code><b>[Quantum Machine Learning]</b></code> </summary><pre><code>@Article{benedetti2019parameterized,
  Title                    = {Parameterized quantum circuits as machine learning models},
  Author                   = {Benedetti, Marcello and Lloyd, Erika and Sack, Stefan and Fiorentini, Mattia},
  Journal                  = {Quantum Science and Technology},
  Year                     = {2019},
  Number                   = {4},
  Pages                    = {043001},
  Volume                   = {4},
  Publisher                = {IOP Publishing},
  URL                      = {https://iopscience.iop.org/article/10.1088/2058-9565/ab4eb5/meta}
}
</code></pre></details>
<details><summary>[Cong et al] <b><a href=https://www.nature.com/articles/s41567-019-0648-8.pdf> Quantum convolutional neural networks. </a></b> <i>Nature Physics.</i> <code><b>[Quantum Neural Networks]</b></code> <code><b>[Quantum Convolutional Neural Networks]</b></code> <code><b>[Quantum Machine Learning]</b></code> <code><i>(Gate Model)</i></code></summary><pre><code>@Article{cong2019quantum,
  Title                    = {Quantum convolutional neural networks},
  Author                   = {Cong, Iris and Choi, Soonwon and Lukin, Mikhail D},
  Journal                  = {Nature Physics},
  Year                     = {2019},
  Number                   = {12},
  Pages                    = {1273--1278},
  Volume                   = {15},
  Publisher                = {Nature Publishing Group},
  URL                      = {https://www.nature.com/articles/s41567-019-0648-8.pdf}
}
</code></pre></details>
<details><summary>[Grant et al] <b><a href=https://quantum-journal.org/papers/q-2019-12-09-214/> An initialization strategy for addressing barren plateaus in parametrized quantum circuits. </a></b> <i>Quantum.</i> <code><b>[Barren Plateaus]</b></code> <code><b>[Parameterized Quantum Circuits]</b></code> <code><b>[Quantum Neural Networks]</b></code> <code><b>[Quantum Machine Learning]</b></code> </summary><pre><code>@article{grant2019initialization,
  title={An initialization strategy for addressing barren plateaus in parametrized quantum circuits},
  author={Grant, Edward and Wossnig, Leonard and Ostaszewski, Mateusz and Benedetti, Marcello},
  journal={Quantum},
  volume={3},
  pages={214},
  year={2019},
  publisher={Verein zur F{\"o}rderung des Open Access Publizierens in den Quantenwissenschaften}
}</code></pre></details>
<details><summary>[Havlíček et al] <b><a href=https://www.nature.com/articles/s41586-019-0980-2> Supervised learning with quantum-enhanced feature spaces. </a></b> <i>Nature.</i> <code><b>[Variational Quantum Algorithms]</b></code> <code><b>[Quantum Support Vector Machine]</b></code> <code><b>[Quantum Classification]</b></code> <code><b>[Quantum Machine Learning]</b></code> <code><i>(Gate Model)</i></code></summary><pre><code>@Article{havlivcek2019supervised,
  Title                    = {Supervised learning with quantum-enhanced feature spaces},
  Author                   = {Havl{\'\i}{\v{c}}ek, Vojt{\v{e}}ch and C{\'o}rcoles, Antonio D and Temme, Kristan and Harrow, Aram W and Kandala, Abhinav and Chow, Jerry M and Gambetta, Jay M},
  Journal                  = {Nature},
  Year                     = {2019},
  Number                   = {7747},
  Pages                    = {209--212},
  Volume                   = {567},
  Publisher                = {Nature Publishing Group},
  URL                      = {https://www.nature.com/articles/s41586-019-0980-2}
}
</code></pre></details>
<details><summary>[Killoran et al] <b><a href=https://journals.aps.org/prresearch/abstract/10.1103/PhysRevResearch.1.033063> Continuous-variable quantum neural networks. </a></b> <i>Physical Review Research.</i> <code><b>[Quantum Neural Networks]</b></code> <code><b>[Quantum Machine Learning]</b></code> <code><i>(Gate Model)</i></code></summary><pre><code>@article{killoran2019continuous,
  title={Continuous-variable quantum neural networks},
  author={Killoran, Nathan and Bromley, Thomas R and Arrazola, Juan Miguel and Schuld, Maria and Quesada, Nicol{\'a}s and Lloyd, Seth},
  journal={Physical Review Research},
  volume={1},
  number={3},
  pages={033063},
  year={2019},
  publisher={APS}
}</code></pre></details>
<details><summary>[Liu et al] <b><a href=https://ieeexplore.ieee.org/document/8631025> A Unitary Weights Based One-Iteration Quantum Perceptron Algorithm for Non-Ideal Training Sets. </a></b> <i>IEEE Access.</i> <code><b>[Quantum Perceptron]</b></code> <code><b>[Quantum Machine Learning]</b></code> <code><i>(Gate Model)</i></code></summary><pre><code>@article{liu2019unitary,
  title={A unitary weights based one-iteration quantum perceptron algorithm for non-ideal training sets},
  author={Liu, Wenjie and Gao, Peipei and Wang, Yuxiang and Yu, Wenbin and Zhang, Maojun},
  journal={IEEE Access},
  volume={7},
  pages={36854--36865},
  year={2019},
  publisher={IEEE}
}</code></pre></details>
<details><summary>[Schuld & Killoran] <b><a href=https://journals.aps.org/prl/abstract/10.1103/PhysRevLett.122.040504> Quantum machine learning in feature Hilbert spaces. </a></b> <i>Physical Review Letters.</i> <code><b>[Quantum Machine Learning]</b></code> <code><b>[Quantum Information and Computing]</b></code> </summary><pre><code>@article{schuld2019quantum,
  title={Quantum machine learning in feature Hilbert spaces},
  author={Schuld, Maria and Killoran, Nathan},
  journal={Physical review letters},
  volume={122},
  number={4},
  pages={040504},
  year={2019},
  publisher={APS}
}</code></pre></details>
<details><summary>[Tacchino et al] <b><a href=https://www.Nature.com/articles/s41534-019-0140-4.pdf> An artificial neuron implemented on an actual quantumprocessor. </a></b> <i>Nature.</i> <code><b>[Quantum Perceptron]</b></code> <code><b>[Quantum Machine Learning]</b></code> <code><b>[Quantum Information and Computing]</b></code> <code><i>(Gate Model)</i></code></summary><pre><code>@article{tacchino2019artificial,
  title={An artificial neuron implemented on an actual quantum processor},
  author={Tacchino, Francesco and Macchiavello, Chiara and Gerace, Dario and Bajoni, Daniele},
  journal={npj Quantum Information},
  volume={5},
  number={1},
  pages={1--8},
  year={2019},
  publisher={Nature Publishing Group}
}</code></pre></details>
<details><summary>[Torrontegui et al] <b><a href=https://iopscience.iop.org/article/10.1209/0295-5075/125/30004/pdf> Unitary quantum perceptron as efficient universal approximator. </a></b> <i>Europhysics Letters.</i> <code><b>[Quantum Perceptron]</b></code> <code><b>[Quantum Machine Learning]</b></code> <code><b>[Quantum Information and Computing]</b></code> </summary><pre><code>@article{torrontegui2019unitary,
  title={Unitary quantum perceptron as efficient universal approximator},
  author={Torrontegui, Erik and Garc{\'\i}a-Ripoll, Juan Jos{\'e}},
  journal={EPL (Europhysics Letters)},
  volume={125},
  number={3},
  pages={30004},
  year={2019},
  publisher={IOP Publishing}
}</code></pre></details>
<details><summary>[Verdon et al] <b><a href=https://arxiv.org/abs/1907.05415> Learning to learn with quantum neural networks via classical neural networks. </a></b> <i>arXiv.</i> <code><b>[Quantum Perceptron]</b></code> <code><b>[Parameterized Quantum Circuits]</b></code> <code><b>[Quantum Neural Networks]</b></code> <code><b>[Quantum Machine Learning]</b></code> <code><b>[Variational Quantum Algorithms]</b></code> </summary><pre><code>@article{verdon2019learning,
  title={Learning to learn with quantum neural networks via classical neural networks},
  author={Verdon, Guillaume and Broughton, Michael and McClean, Jarrod R and Sung, Kevin J and Babbush, Ryan and Jiang, Zhang and Neven, Hartmut and Mohseni, Masoud},
  journal={arXiv preprint arXiv:1907.05415},
  year={2019}
}</code></pre></details>
<details><summary>[Verdon et al] <b><a href=https://arxiv.org/abs/1909.12264> Quantum graph neural networks. </a></b> <i>arXiv.</i> <code><b>[Quantum Neural Networks]</b></code> <code><b>[Quantum Machine Learning]</b></code> <code><i>(Gate Model)</i></code></summary><pre><code>@article{verdon2019quantum,
  title={Quantum graph neural networks},
  author={Verdon, Guillaume and McCourt, Trevor and Luzhnica, Enxhell and Singh, Vikash and Leichenauer, Stefan and Hidary, Jack},
  journal={arXiv preprint arXiv:1909.12264},
  year={2019}
</code></pre></details>
<details><summary>[Wiebe & Wossnig] <b><a href=https://arxiv.org/abs/1905.09902> Generative training of quantum Boltzmann machines with hidden units. </a></b> <i>arXiv.</i> <code><b>[Quantum Boltzmann machines]</b></code> <code><b>[Quantum Machine Learning]</b></code> </summary><pre><code>@article{wiebe2019generative,
  title={Generative training of quantum Boltzmann machines with hidden units},
  author={Wiebe, Nathan and Wossnig, Leonard},
  journal={arXiv preprint arXiv:1905.09902},
  year={2019}
}</code></pre></details>
<details><summary>[Wiersema et al] <b><a href=https://journals.aps.org/pra/abstract/10.1103/PhysRevA.100.020301> Implementing perceptron models with qubits. </a></b> <i>Physical Review A.</i> <code><b>[Quantum Perceptron]</b></code> <code><b>[Quantum Machine Learning]</b></code> <code><b>[Quantum Information and Computing]</b></code> <code><i>(Gate Model)</i></code></summary><pre><code>@article{wiersema2019implementing,
  title={Implementing perceptron models with qubits},
  author={Wiersema, RC and Kappen, HJ},
  journal={Physical Review A},
  volume={100},
  number={2},
  pages={020301},
  year={2019},
  publisher={APS}
}</code></pre></details>
<details><summary>[Zoufal et al] <b><a href=https://www.nature.com/articles/s41534-019-0223-2> Quantum Generative Adversarial Networks for Learning and Loading Random
Distributions. </a></b> <i>npj Quantum Information.</i> <code><b>[Quantum Generative Adversarial Networks]</b></code> <code><b>[Variational Quantum Algorithms]</b></code> <code><b>[Quantum Machine Learning]</b></code> <code><i>(Gate Model)</i></code></summary><pre><code>@article{zoufal2019quantum,
  title={Quantum generative adversarial networks for learning and loading random distributions},
  author={Zoufal, Christa and Lucchi, Aur{\'e}lien and Woerner, Stefan},
  journal={npj Quantum Information},
  volume={5},
  number={1},
  pages={1--9},
  year={2019},
  publisher={Nature Publishing Group}
}</code></pre></details>


### 2018
<details><summary>[Allcock & Zhang] <b><a href=https://academic.oup.com/nsr/article/6/1/26/5222655?login=true> Quantum machine learning. </a></b> <i>National Science Review.</i> <code><b>[Quantum Machine Learning]</b></code> <code><b>[Quantum Boltzmann machines]</b></code> <code><b>[Quantum Generative Adversarial Networks]</b></code> </summary><pre><code>@Article{allcock2019quantum,
  Title                    = {Quantum machine learning},
  Author                   = {Allcock, Jonathan and Zhang, Shengyu},
  Journal                  = {National Science Review},
  Year                     = {2019},
  Number                   = {1},
  Pages                    = {26--28},
  Volume                   = {6},
  Publisher                = {Oxford University Press},
  URL                      = {https://academic.oup.com/nsr/article/6/1/26/5222655?login=true}
}
</code></pre></details>
<details><summary>[Amin et al] <b><a href=https://journals.aps.org/prx/abstract/10.1103/PhysRevX.8.021050> Quantum boltzmann machine. </a></b> <i>Physical Review X.</i> <code><b>[Quantum Boltzmann machines]</b></code> <code><b>[Quantum Machine Learning]</b></code> <code><i>(Gate Model)</i></code> <code><i>(Adiabatic Model)</i></code></summary><pre><code>@Article{amin2018quantum,
  Title                    = {Quantum boltzmann machine},
  Author                   = {Amin, Mohammad H and Andriyash, Evgeny and Rolfe, Jason and Kulchytskyy, Bohdan and Melko, Roger},
  Journal                  = {Physical Review X},
  Year                     = {2018},
  Number                   = {2},
  Pages                    = {021050},
  Volume                   = {8},
  Publisher                = {APS},
  URL                      = {https://journals.aps.org/prx/abstract/10.1103/PhysRevX.8.021050}
}
</code></pre></details>
<details><summary>[Ciliberto et al] <b><a href=https://doi.org/10.1098/rspa.2017.0551> Quantum machine learning: a classical perspective. </a></b> <i>Proceedings of the Royal Society A: Mathematical, Physical and Engineering Sciences.</i> <code><b>[Quantum Machine Learning]</b></code> <code><b>[Quantum Optimization]</b></code> <code><b>[Quantum Neural Networks]</b></code> </summary><pre><code>@Article{ciliberto2018quantum,
  Title                    = {Quantum machine learning: a classical perspective},
  Author                   = {Ciliberto, Carlo and Herbster, Mark and Ialongo, Alessandro Davide and Pontil, Massimiliano and Rocchetto, Andrea and Severini, Simone and Wossnig, Leonard},
  Journal                  = {Proceedings of the Royal Society A: Mathematical, Physical and Engineering Sciences},
  Year                     = {2018},
  Number                   = {2209},
  Pages                    = {20170551},
  Volume                   = {474},
  Publisher                = {The Royal Society Publishing},
  URL                      = {https://doi.org/10.1098/rspa.2017.0551}
}
</code></pre></details>
<details><summary>[Dallaire-Demers & Killoran] <b><a href=https://journals.aps.org/pra/abstract/10.1103/PhysRevA.98.012324> Quantum generative adversarial networks. </a></b> <i>Physical Review A.</i> <code><b>[Quantum Generative Adversarial Networks]</b></code> <code><b>[Quantum Neural Networks]</b></code> <code><b>[Variational Quantum Algorithms]</b></code> <code><b>[Quantum Machine Learning]</b></code> <code><i>(Gate Model)</i></code></summary><pre><code>@article{dallaire2018quantum,
  title={Quantum generative adversarial networks},
  author={Dallaire-Demers, Pierre-Luc and Killoran, Nathan},
  journal={Physical Review A},
  volume={98},
  number={1},
  pages={012324},
  year={2018},
  publisher={APS}
}</code></pre></details>
<details><summary>[Du et al] <b><a href=https://arxiv.org/abs/1809.06056> Implementable quantum classifier for nonlinear data. </a></b> <i>arXiv.</i> <code><b>[Quantum Perceptron]</b></code> <code><b>[Quantum Machine Learning]</b></code> <code><b>[Quantum Classification]</b></code> <code><b>[Quantum Machine Learning]</b></code> <code><i>(Gate Model)</i></code></summary><pre><code>@Article{du2018implementable,
  Title                    = {Implementable quantum classifier for nonlinear data},
  Author                   = {Du, Yuxuan and Hsieh, Min-Hsiu and Liu, Tongliang and Tao, Dacheng},
  Journal                  = {arXiv preprint arXiv:1809.06056},
  Year                     = {2018},
  URL                      = {https://arxiv.org/abs/1809.06056}
}
</code></pre></details>
<details><summary>[Dunjko & Briegel] <b><a href=https://iopscience.iop.org/article/10.1088/1361-6633/aab406/meta> Machine learning & artificial intelligence in the quantum domain: a review of recent progress. </a></b> <i>Reports on Progress in Physics.</i> <code><b>[Quantum Machine Learning]</b></code> <code><b>[Probably Approximately Correct Learning]</b></code> <code><b>[Quantum Information and Computing]</b></code> <code><i>(Gate Model)</i></code> <code><i>(Adiabatic Model)</i></code></summary><pre><code>@Article{dunjko2018machine,
  Title                    = {Machine learning \& artificial intelligence in the quantum domain: a review of recent progress},
  Author                   = {Dunjko, Vedran and Briegel, Hans J},
  Journal                  = {Reports on Progress in Physics},
  Year                     = {2018},
  Number                   = {7},
  Pages                    = {074001},
  Volume                   = {81},
  Publisher                = {IOP Publishing},
  URL                      = {https://iopscience.iop.org/article/10.1088/1361-6633/aab406/meta}
}
</code></pre></details>
<details><summary>[Fahri & Neven] <b><a href=https://arxiv.org/abs/1802.06002> Classification with Quantum Neural Networks on Near Term Processors. </a></b> <i>arXiv.</i> <code><b>[Quantum Neural Networks]</b></code> <code><b>[Quantum Machine Learning]</b></code> <code><b>[Quantum Classification]</b></code> <code><i>(Gate Model)</i></code></summary><pre><code>@Article{farhi2018classification,
  Title                    = {Classification with quantum neural networks on near term processors},
  Author                   = {Farhi, Edward and Neven, Hartmut},
  Journal                  = {arXiv preprint arXiv:1802.06002},
  Year                     = {2018},
  URL                      = {https://arxiv.org/abs/1802.06002}
}
</code></pre></details>
<details><summary>[Grant et al] <b><a href=https://www.nature.com/articles/s41534-018-0116-9> Hierarchical quantum classifiers. </a></b> <i>npj Quantum information.</i> <code><b>[Quantum Neural Networks]</b></code> <code><b>[Quantum Machine Learning]</b></code> <code><b>[Quantum Classification]</b></code> <code><i>(Gate Model)</i></code></summary><pre><code>@Article{grant2018hierarchical,
  Title                    = {Hierarchical quantum classifiers},
  Author                   = {Grant, Edward and Benedetti, Marcello and Cao, Shuxiang and Hallam, Andrew and Lockhart, Joshua and Stojevic, Vid and Green, Andrew G and Severini, Simone},
  Journal                  = {npj Quantum Information},
  Year                     = {2018},
  Number                   = {1},
  Pages                    = {1--8},
  Volume                   = {4},
  Publisher                = {Nature Publishing Group},
  URL                      = {https://www.nature.com/articles/s41534-018-0116-9}
}
</code></pre></details>
<details><summary>[Hu Wei] <b><a href=https://www.scirp.org/pdf/NS_2018031517004350.pdf> Towards a real quantum neuron. </a></b> <i>Natural Science.</i> <code><b>[Quantum Perceptron]</b></code> <code><b>[Quantum Machine Learning]</b></code> <code><i>(Gate Model)</i></code></summary><pre><code>@article{hu2018towards,
  title={Towards a real quantum neuron},
  author={Hu, Wei},
  journal={Natural Science},
  volume={10},
  number={3},
  pages={99--109},
  year={2018},
  publisher={Scientific Research Publishing}
}</code></pre></details>
<details><summary>[McClean et al] <b><a href=https://www.nature.com/articles/s41467-018-07090-4> Barren plateaus in quantum neural network training landscapes. </a></b> <i>Nature communications.</i> <code><b>[Barren Plateaus]</b></code> <code><b>[Parameterized Quantum Circuits]</b></code> <code><b>[Quantum Neural Networks]</b></code> <code><b>[Quantum Machine Learning]</b></code> </summary><pre><code>@article{mcclean2018barren,
  title={Barren plateaus in quantum neural network training landscapes},
  author={McClean, Jarrod R and Boixo, Sergio and Smelyanskiy, Vadim N and Babbush, Ryan and Neven, Hartmut},
  journal={Nature communications},
  volume={9},
  number={1},
  pages={1--6},
  year={2018},
  publisher={Nature Publishing Group}
}</code></pre></details>
<details><summary>[Mitarai et al] <b><a href=https://journals.aps.org/pra/abstract/10.1103/PhysRevA.98.032309> Quantum circuit learning. </a></b> <i>Physical Review A.</i> <code><b>[Parameterized Quantum Circuits]</b></code> <code><b>[Quantum Neural Networks]</b></code> <code><b>[Quantum Machine Learning]</b></code> </summary><pre><code>@article{mitarai2018quantum,
  title={Quantum circuit learning},
  author={Mitarai, Kosuke and Negoro, Makoto and Kitagawa, Masahiro and Fujii, Keisuke},
  journal={Physical Review A},
  volume={98},
  number={3},
  pages={032309},
  year={2018},
  publisher={APS}
}</code></pre></details>
<details><summary>[Moll et al] <b><a href=https://iopscience.iop.org/article/10.1088/2058-9565/aab822/pdf> Quantum optimization using variational algorithms on near-term quantum devices. </a></b> <i>Quantum Science and Technology.</i> <code><b>[Variational Quantum Algorithms]</b></code> <code><b>[Quantum Optimization]</b></code> <code><b>[Quantum Machine Learning]</b></code> <code><b>[Quantum Information and Computing]</b></code> <code><i>(Gate Model)</i></code></summary><pre><code>@Article{moll2018quantum,
  Title                    = {Quantum optimization using variational algorithms on near-term quantum devices},
  Author                   = {Moll, Nikolaj and Barkoutsos, Panagiotis and Bishop, Lev S and Chow, Jerry M and Cross, Andrew and Egger, Daniel J and Filipp, Stefan and Fuhrer, Andreas and Gambetta, Jay M and Ganzhorn, Marc and others},
  Journal                  = {Quantum Science and Technology},
  Year                     = {2018},
  Number                   = {3},
  Pages                    = {030503},
  Volume                   = {3},
  Publisher                = {IOP Publishing},
  URL                      = {https://iopscience.iop.org/article/10.1088/2058-9565/aab822/meta}
}
</code></pre></details>
<details><summary>[Neukart et al] <b><a href=https://www.frontiersin.org/articles/10.3389/fphy.2017.00071/full> Quantum-enhanced reinforcement learning for finite-episode games with discrete state
spaces. </a></b> <i>Frontiers in physics.</i> <code><b>[Quantum Reinforcement Learning]</b></code> <code><b>[Quantum Machine Learning]</b></code> <code><i>(Adiabatic Model)</i></code></summary><pre><code>@article{neukart2018quantum,
  title={Quantum-enhanced reinforcement learning for finite-episode games with discrete state spaces},
  author={Neukart, Florian and Von Dollen, David and Seidel, Christian and Compostella, Gabriele},
  journal={Frontiers in physics},
  volume={5},
  pages={71},
  year={2018},
  publisher={Frontiers}
}</code></pre></details>
<details><summary>[Wiebe et al] <b><a href=https://papers.nips.cc/paper/2016/file/d47268e9db2e9aa3827bba3afb7ff94a-Paper.pdf> Quantum Perceptron Models. </a></b> <i>NIPS .</i> <code><b>[Quantum Perceptron]</b></code> <code><b>[Quantum Machine Learning]</b></code> <code><b>[Quantum Information and Computing]</b></code> <code><i>(Gate Model)</i></code></summary><pre><code>@article{wiebe2016quantum,
  title={Quantum perceptron models},
  author={Wiebe, Nathan and Kapoor, Ashish and Svore, Krysta M},
  journal={arXiv preprint arXiv:1602.04799},
  year={2016}
}</code></pre></details>


### 2017
<details><summary>[Arunachalam & de Wolf] <b><a href=https://dl.acm.org/doi/abs/10.1145/3106700.3106710> Guest column: A survey of quantum learning theory. </a></b> <i>ACM SIGACT News.</i> <code><b>[Quantum Machine Learning]</b></code> <code><b>[Probably Approximately Correct Learning]</b></code> <code><b>[Complexity Theory]</b></code> <code><i>(Gate Model)</i></code></summary><pre><code>@Article{arunachalam2017guest,
  Title                    = {Guest column: A survey of quantum learning theory},
  Author                   = {Arunachalam, Srinivasan and de Wolf, Ronald},
  Journal                  = {ACM SIGACT News},
  Year                     = {2017},
  Number                   = {2},
  Pages                    = {41--67},
  Volume                   = {48},
  Publisher                = {ACM New York, NY, USA},
  URL                      = {https://dl.acm.org/doi/abs/10.1145/3106700.3106710}
}
</code></pre></details>
<details><summary>[Biamonte et al] <b><a href=https://www.Nature.com/articles/Nature23474> Quantum machine learning. </a></b> <i>Nature.</i> <code><b>[Quantum Machine Learning]</b></code> <code><i>(Gate Model)</i></code> <code><i>(Adiabatic Model)</i></code></summary><pre><code>@Article{biamonte2017quantum,
  Title                    = {Quantum machine learning},
  Author                   = {Biamonte, Jacob and Wittek, Peter and Pancotti, Nicola and Rebentrost, Patrick and Wiebe, Nathan and Lloyd, Seth},
  Journal                  = {Nature},
  Year                     = {2017},
  Number                   = {7671},
  Pages                    = {195--202},
  Volume                   = {549},
  Publisher                = {Nature Publishing Group},
  URL                      = {https://www.Nature.com/articles/Nature23474}
}
</code></pre></details>
<details><summary>[Cao et al] <b><a href=https://arxiv.org/pdf/1711.11240.pdf> Quantum Neuron:  an elementary building block for machine learningon quantum computers. </a></b> <i>arXiv.</i> <code><b>[Quantum Perceptron]</b></code> <code><b>[Quantum Information and Computing]</b></code> <code><b>[Quantum Neural Networks]</b></code> <code><b>[Quantum Machine Learning]</b></code> <code><i>(Gate Model)</i></code></summary><pre><code>@Article{cao2017quantum,
  Title                    = {Quantum neuron: an elementary building block for machine learning on quantum computers},
  Author                   = {Cao, Yudong and Guerreschi, Gian Giacomo and Aspuru-Guzik, Al{\'a}n},
  Journal                  = {arXiv preprint arXiv:1711.11240},
  Year                     = {2017},
  URL                      = {https://arxiv.org/pdf/1711.11240.pdf}
}
</code></pre></details>
<details><summary>[Romero et al] <b><a href=https://arxiv.org/pdf/1612.02806.pdf> Quantum autoencoders for efficient compression of quantum data. </a></b> <i>Quantum Science and Technology.</i> <code><b>[Quantum Neural Networks]</b></code> <code><b>[Quantum Autoencoders]</b></code> <code><b>[Quantum Machine Learning]</b></code> <code><i>(Gate Model)</i></code></summary><pre><code>@article{romero2017quantum,
  title={Quantum autoencoders for efficient compression of quantum data},
  author={Romero, Jonathan and Olson, Jonathan P and Aspuru-Guzik, Alan},
  journal={Quantum Science and Technology},
  volume={2},
  number={4},
  pages={045001},
  year={2017},
  publisher={IOP Publishing}
}</code></pre></details>
<details><summary>[Ruan et al] <b><a href=https://idp.springer.com/authorize/casa?redirect_uri=https://link.springer.com/article/10.1007/s10773-017-3514-4&casa_token=CO3V9sMtWRcAAAAA:9hw3GSs7B7yTOJZoSEACk9y4uoIrHM2RuXAeY_ThKvnmvCTLd5lkn-ova5xZ4Do4xVczdIiy_YTuI8JC> Quantum algorithm for k-nearest neighbors classification based on the metric of hamming distance. </a></b> <i>International Journal of Theoretical Physics.</i> <code><b>[Quantum Machine Learning]</b></code> <code><b>[Quantum Classification]</b></code> </summary><pre><code>@article{ruan2017quantum,
  title={Quantum algorithm for k-nearest neighbors classification based on the metric of hamming distance},
  author={Ruan, Yue and Xue, Xiling and Liu, Heng and Tan, Jianing and Li, Xi},
  journal={International Journal of Theoretical Physics},
  volume={56},
  number={11},
  pages={3496--3507},
  year={2017},
  publisher={Springer}
}</code></pre></details>
<details><summary>[Wan et al] <b><a href=https://www.nature.com/articles/s41534-017-0032-4> Quantum generalisation of feedforward neural networks. </a></b> <i>npj Quantum information.</i> <code><b>[Quantum Neural Networks]</b></code> <code><b>[Quantum Autoencoders]</b></code> <code><b>[Quantum Machine Learning]</b></code> <code><i>(Gate Model)</i></code></summary><pre><code>@Article{wan2017quantum,
  Title                    = {Quantum generalisation of feedforward neural networks},
  Author                   = {Wan, Kwok Ho and Dahlsten, Oscar and Kristj{\'a}nsson, Hl{\'e}r and Gardner, Robert and Kim, MS},
  Journal                  = {npj Quantum information},
  Year                     = {2017},
  Number                   = {1},
  Pages                    = {1--8},
  Volume                   = {3},
  Publisher                = {Nature Publishing Group},
  URL                      = {https://www.nature.com/articles/s41534-017-0032-4}
}
</code></pre></details>


### 2016
<details><summary>[Oshurko] <b><a href=https://www.worldscientific.com/doi/abs/10.1142/9781786348210_0010> Quantum machine learning. </a></b> <i>Quantum Computing.</i> <code><b>[Quantum Machine Learning]</b></code> </summary><pre><code>@article{oshurko2016quantum,
  title={Quantum machine learning},
  author={Oshurko, Ievgeniia},
  journal={Quantum Information and Computation Course},
  year={2016}
}</code></pre></details>


### 2015
<details><summary>[Adcock et al] <b><a href=https://arxiv.org/abs/1512.02900> Advances in quantum machine learning. </a></b> <i>arXiv.</i> <code><b>[Quantum Machine Learning]</b></code> <code><b>[Quantum Classification]</b></code> <code><b>[Quantum Clustering]</b></code> <code><b>[Quantum Nearest Neighbors Algorithm]</b></code> <code><b>[Quantum Neural Networks]</b></code> <code><i>(Gate Model)</i></code> <code><i>(Adiabatic Model)</i></code></summary><pre><code>@Article{adcock2015advances,
  Title                    = {Advances in quantum machine learning},
  Author                   = {Adcock, Jeremy and Allen, Euan and Day, Matthew and Frick, Stefan and Hinchliff, Janna and Johnson, Mack and Morley-Short, Sam and Pallister, Sam and Price, Alasdair and Stanisic, Stasja},
  Journal                  = {arXiv preprint arXiv:1512.02900},
  Year                     = {2015},
  URL                      = {https://arxiv.org/abs/1512.02900}
}
</code></pre></details>
<details><summary>[Schuld et al] <b><a href=https://www.tandfonline.com/doi/abs/10.1080/00107514.2014.964942> An introduction to quantum machine learning. </a></b> <i>Contemporary Physics.</i> <code><b>[Quantum Machine Learning]</b></code> <code><i>(Gate Model)</i></code> <code><i>(Adiabatic Model)</i></code></summary><pre><code>@article{schuld2015introduction,
  title={An introduction to quantum machine learning},
  author={Schuld, Maria and Sinayskiy, Ilya and Petruccione, Francesco},
  journal={Contemporary Physics},
  volume={56},
  number={2},
  pages={172--185},
  year={2015},
  publisher={Taylor \& Francis}
}</code></pre></details>
<details><summary>[Schuld et al] <b><a href=https://doi.org/10.1016/j.physleta.2014.11.061> Simulating a perceptron on a quantum computer. </a></b> <i>Physics Letters A.</i> <code><b>[Quantum Perceptron]</b></code> <code><b>[Quantum Machine Learning]</b></code> </summary><pre><code>@article{schuld2015simulating,
  title={Simulating a perceptron on a quantum computer},
  author={Schuld, Maria and Sinayskiy, Ilya and Petruccione, Francesco},
  journal={Physics Letters A},
  volume={379},
  number={7},
  pages={660--663},
  year={2015},
  publisher={Elsevier}
}</code></pre></details>


### 2014
<details><summary>[Farhi et al] <b><a href=https://arxiv.org/abs/1411.4028> A Quantum Approximate Optimization Algorithm. </a></b> <i>arXiv.</i> <code><b>[Variational Quantum Algorithms]</b></code> <code><b>[Parameterized Quantum Circuits]</b></code> <code><b>[Quantum Optimization]</b></code> <code><b>[Quantum Machine Learning]</b></code> <code><b>[]</b></code> <code><i>(Gate Model)</i></code></summary><pre><code>@Article{farhi2014quantum,
  Title                    = {A quantum approximate optimization algorithm},
  Author                   = {Farhi, Edward and Goldstone, Jeffrey and Gutmann, Sam},
  Journal                  = {arXiv preprint arXiv:1411.4028},
  Year                     = {2014},
  URL                      = {https://arxiv.org/abs/1411.4028}
}
</code></pre></details>
<details><summary>[Schuld et al] <b><a href=https://link.springer.com/chapter/10.1007/978-3-319-13560-1_17> Quantum computing for pattern classification. </a></b> <i>Pacific Rim International Conference on Artificial Intelligence.</i> <code><b>[Quantum Machine Learning]</b></code> </summary><pre><code>@inproceedings{schuld2014quantum,
  title={Quantum computing for pattern classification},
  author={Schuld, Maria and Sinayskiy, Ilya and Petruccione, Francesco},
  booktitle={Pacific Rim International Conference on Artificial Intelligence},
  pages={208--220},
  year={2014},
  organization={Springer}
}</code></pre></details>
<details><summary>[Schuld et al] <b><a href=https://link.springer.com/article/10.1007%2Fs11128-014-0809-8> The quest for a Quantum Neural Networt. </a></b> <i>Quantum Information Processing.</i> <code><b>[Quantum Neural Networks]</b></code> <code><b>[Quantum Machine Learning]</b></code> <code><i>(Gate Model)</i></code></summary><pre><code>@article{schuld2014quest,
  title={The quest for a quantum neural network},
  author={Schuld, Maria and Sinayskiy, Ilya and Petruccione, Francesco},
  journal={Quantum Information Processing},
  volume={13},
  number={11},
  pages={2567--2586},
  year={2014},
  publisher={Springer}
}</code></pre></details>
<details><summary>[Wiebe et al] <b><a href=https://dl.acm.org/doi/abs/10.5555/2871393.2871400> Quantum Algorithms for Nearest-Neighbour Methods for Supervised and Unsupervised Learning. </a></b> <i>Quantum Information & Computation.</i> <code><b>[Quantum Nearest Neighbors Algorithm]</b></code> <code><b>[Quantum Classification]</b></code> <code><b>[Quantum Machine Learning]</b></code> <code><i>(Gate Model)</i></code></summary><pre><code>@article{wiebe2015quantum,
  title={Quantum algorithms for nearest-neighbor methods for supervised and unsupervised learning},
  author={Wiebe, Nathan and Kapoor, Ashish and Svore, Krysta M},
  journal={Quantum Information \& Computation},
  volume={15},
  number={3-4},
  pages={316--356},
  year={2015},
  publisher={Rinton Press, Incorporated Paramus, NJ}
}</code></pre></details>


### 2013
<details><summary>[Aïmeur et al] <b><a href=https://link.springer.com/article/10.1007/s10994-012-5316-5> Quantum speed-up for unsupervised learning. </a></b> <i>Machine Learning.</i> <code><b>[Quantum Machine Learning]</b></code> <code><b>[Unsupervised Learning]</b></code> <code><b>[Quantum Clustering]</b></code> <code><i>(Gate Model)</i></code></summary><pre><code>@Article{aimeur2013quantum,
  Title                    = {Quantum speed-up for unsupervised learning},
  Author                   = {A{\"\i}meur, Esma and Brassard, Gilles and Gambs, S{\'e}bastien},
  Journal                  = {Machine Learning},
  Year                     = {2013},
  Number                   = {2},
  Pages                    = {261--287},
  Volume                   = {90},
  Publisher                = {Springer},
  URL                      = {https://link.springer.com/article/10.1007/s10994-012-5316-5}
}
</code></pre></details>
<details><summary>[Lloyd et al] <b><a href=https://arxiv.org/abs/1307.0411> Quantum algorithms for supervised and unsupervised machine learning. </a></b> <i>arXiv.</i> <code><b>[Quantum Machine Learning]</b></code> <code><b>[Quantum Clustering]</b></code> <code><i>(Adiabatic Model)</i></code></summary><pre><code>@article{lloyd2013quantum,
  title={Quantum algorithms for supervised and unsupervised machine learning},
  author={Lloyd, Seth and Mohseni, Masoud and Rebentrost, Patrick},
  journal={arXiv preprint arXiv:1307.0411},
  year={2013}
}</code></pre></details>


### 2012
<details><summary>[Da Silva et al] <b><a href=https://www.sciencedirect.com/science/article/abs/pii/S0925231211004127?via%3Dihub> Classical and superposed learning for quantum weightless neural networks. </a></b> <i>Neurocomputing.</i> <code><b>[Quantum Neural Networks]</b></code> <code><b>[Quantum Machine Learning]</b></code> <code><i>(Gate Model)</i></code></summary><pre><code>@Article{da2012classical,
  Title                    = {Classical and superposed learning for quantum weightless neural networks},
  Author                   = {Da Silva, Adenilton J and De Oliveira, Wilson R and Ludermir, Teresa B},
  Journal                  = {Neurocomputing},
  Year                     = {2012},
  Number                   = {1},
  Pages                    = {52--60},
  Volume                   = {75},
  Publisher                = {Elsevier},
  URL                      = {https://www.sciencedirect.com/science/article/abs/pii/S0925231211004127?via%3Dihub}
}
</code></pre></details>


### 2011
<details><summary>[Panella & Martinelli] <b><a href=https://onlinelibrary.wiley.com/doi/10.1002/cta.619> Neural networks with quantum. </a></b> <i>International Journal of Circuit Theory and Applications.</i> <code><b>[Quantum Neurofuzzy Network]</b></code> <code><b>[Quantum Neural Networks]</b></code> <code><b>[Quantum Machine Learning]</b></code> </summary><pre><code>@article{panella2011neural,
  title={Neural networks with quantum architecture and quantum learning},
  author={Panella, Massimo and Martinelli, Giuseppe},
  journal={International Journal of Circuit Theory and Applications},
  volume={39},
  number={1},
  pages={61--77},
  year={2011},
  publisher={Wiley Online Library}
}</code></pre></details>


### 2010
<details><summary>[Da Silva et al] <b><a href=https://ieeexplore.ieee.org/document/5715204> Superposition Based Learning Algorithm. </a></b> <i>Brazilian Symposium on Neural Networks.</i> <code><b>[Quantum Neural Networks]</b></code> <code><b>[Quantum Machine Learning]</b></code> <code><i>(Gate Model)</i></code></summary><pre><code>@inproceedings{silva2010superposition,
  title={Superposition based learning algorithm},
  author={Silva, Adenilton J and Ludermir, Teresa B and de Oliveira Jr, Wilson R},
  booktitle={2010 Eleventh Brazilian Symposium on Neural Networks},
  pages={1--6},
  year={2010},
  organization={IEEE}
}</code></pre></details>


### 2009
<details><summary>[de Oliveira] <b><a href=https://citeseerx.ist.psu.edu/viewdoc/download?doi=10.1.1.227.4363&rep=rep1&type=pdf> Quantum RAM based neural networks. </a></b> <i>European Symposium on Artificial Neural Network.</i> <code><b>[QRAM]</b></code> <code><b>[Quantum Neural Networks]</b></code> <code><b>[Quantum Machine Learning]</b></code> <code><i>(Gate Model)</i></code></summary><pre><code>@InProceedings{de2009quantum,
  Title                    = {Quantum RAM Based Neural Netoworks.},
  Author                   = {de Oliveira, Wilson Rosa},
  Booktitle                = {ESANN},
  Year                     = {2009},
  Organization             = {Citeseer},
  Pages                    = {331--336},
  Volume                   = {9},
  URL                      = {https://citeseerx.ist.psu.edu/viewdoc/download?doi=10.1.1.227.4363&rep=rep1&type=pdf}
}
</code></pre></details>
<details><summary>[Harrow et al] <b><a href=https://journals.aps.org/prl/abstract/10.1103/PhysRevLett.103.150502> Quantum Algorithm for Linear Systems of Equations. </a></b> <i>Physical Review Letters.</i> <code><b>[Quantum Machine Learning]</b></code> <code><b>[Quantum Speedup-Advantage-Supremacy]</b></code> </summary><pre><code>@Article{harrow2009quantum,
  Title                    = {Quantum algorithm for linear systems of equations},
  Author                   = {Harrow, Aram W and Hassidim, Avinatan and Lloyd, Seth},
  Journal                  = {Physical review letters},
  Year                     = {2009},
  Number                   = {15},
  Pages                    = {150502},
  Volume                   = {103},
  Publisher                = {APS},
  URL                      = {https://journals.aps.org/prl/abstract/10.1103/PhysRevLett.103.150502}
}
</code></pre></details>


### 2008
<details><summary>[Dong et al] <b><a href=https://ieeexplore.ieee.org/document/4579244> Quantum Reinforcement Learning. </a></b> <i>IEEE Transactions on Systems, Man, and Cybernetics, Part B (Cybernetics).</i> <code><b>[Quantum Reinforcement Learning]</b></code> <code><b>[Quantum Machine Learning]</b></code> <code><i>(Gate Model)</i></code></summary><pre><code>@article{dong2008quantum,
  title={Quantum reinforcement learning},
  author={Dong, Daoyi and Chen, Chunlin and Li, Hanxiong and Tarn, Tzyh-Jong},
  journal={IEEE Transactions on Systems, Man, and Cybernetics, Part B (Cybernetics)},
  volume={38},
  number={5},
  pages={1207--1220},
  year={2008},
  publisher={IEEE}
}</code></pre></details>
<details><summary>[Panella & Martinelli] <b><a href=https://ieeexplore.ieee.org/document/4565676> Neurofuzzy networks with nonlinear quantum learning. </a></b> <i>IEEE Transactions on Fuzzy Systems.</i> <code><b>[Quantum Neurofuzzy Network]</b></code> <code><b>[Quantum Neural Networks]</b></code> <code><b>[Quantum Machine Learning]</b></code> </summary><pre><code>@article{panella2008neurofuzzy,
  title={Neurofuzzy networks with nonlinear quantum learning},
  author={Panella, Massimo and Martinelli, Giuseppe},
  journal={IEEE Transactions on Fuzzy Systems},
  volume={17},
  number={3},
  pages={698--710},
  year={2008},
  publisher={IEEE}
}</code></pre></details>


### 2007
<details><summary>[Aïmeur et al] <b><a href=https://dl.acm.org/doi/abs/10.1145/1273496.1273497> Quantum Clustering Algorithms. </a></b> <i>ICML '07.</i> <code><b>[Quantum Machine Learning]</b></code> <code><b>[Unsupervised Learning]</b></code> <code><b>[Quantum Clustering]</b></code> <code><i>(Gate Model)</i></code></summary><pre><code>@inproceedings{aimeur2007quantum,
  title={Quantum clustering algorithms},
  author={A{\"\i}meur, Esma and Brassard, Gilles and Gambs, S{\'e}bastien},
  booktitle={Proceedings of the 24th international conference on machine learning},
  pages={1--8},
  year={2007}
}</code></pre></details>


### 2006
<details><summary>[Aïmeur et al] <b><a href=https://link.springer.com/chapter/10.1007/11766247_37> Machine Learning in a Quantum World. </a></b> <i>Canadian AI 2006.</i> <code><b>[Quantum Machine Learning]</b></code> <code><b>[Unsupervised Learning]</b></code> <code><b>[Quantum Clustering]</b></code> <code><i>(Gate Model)</i></code></summary><pre><code>@InProceedings{aimeur2006machine,
  Title                    = {Machine learning in a quantum world},
  Author                   = {A{\"\i}meur, Esma and Brassard, Gilles and Gambs, S{\'e}bastien},
  Booktitle                = {Conference of the Canadian Society for Computational Studies of Intelligence},
  Year                     = {2006},
  Organization             = {Springer},
  Pages                    = {431--442},
  URL                      = {https://link.springer.com/chapter/10.1007/11766247_37}
}
</code></pre></details>


### 2001
<details><summary>[Altaisky] <b><a href=https://arxiv.org/abs/quant-ph/0107012> Quantum neural network. </a></b> <i>arXiv.</i> <code><b>[Quantum Neural Networks]</b></code> <code><b>[Quantum Perceptron]</b></code> <code><b>[Quantum Machine Learning]</b></code> </summary><pre><code>@Article{altaisky2001quantum,
  Title                    = {Quantum neural network},
  Author                   = {Altaisky, MV},
  Journal                  = {arXiv preprint quant-ph/0107012},
  Year                     = {2001},
  URL                      = {https://arxiv.org/abs/quant-ph/0107012}
}
</code></pre></details>


### 2000
<details><summary>[Narayanan & Menneer] <b><a href=https://www.sciencedirect.com/science/article/abs/pii/S0020025500000554> Quantum artificial neural network architectures and components. </a></b> <i>Information Sciences.</i> <code><b>[Quantum Neural Networks]</b></code> <code><b>[Quantum Machine Learning]</b></code> </summary><pre><code>@article{narayanan2000quantum,
  title={Quantum artificial neural network architectures and components},
  author={Narayanan, Ajit and Menneer, Tammy},
  journal={Information Sciences},
  volume={128},
  number={3-4},
  pages={231--255},
  year={2000},
  publisher={Elsevier}
}</code></pre></details>
<details><summary>[Ventura & Martinez] <b><a href=https://www.sciencedirect.com/science/article/abs/pii/S0020025599001012> Quantum associative memory. </a></b> <i>Information Sciences.</i> <code><b>[Quantum Machine Learning]</b></code> <code><b>[Quantum Information and Computing]</b></code> <code><b>[Quantum Classification]</b></code> </summary><pre><code>@article{ventura2000quantum,
  title={Quantum associative memory},
  author={Ventura, Dan and Martinez, Tony},
  journal={Information Sciences},
  volume={124},
  number={1-4},
  pages={273--296},
  year={2000},
  publisher={Elsevier}
}</code></pre></details>


### 1996
<details><summary>[Behrman et al ] <b><a href=http://citeseerx.ist.psu.edu/viewdoc/download?doi=10.1.1.56.1507&rep=rep1&type=pdf> A quantum dot neural network. </a></b> <i>Workshop on Physics of Computation.</i> <code><b>[Quantum Neural Networks]</b></code> <code><b>[Quantum Machine Learning]</b></code> <code><i>(Gate Model)</i></code></summary><pre><code>@InProceedings{behrman1996quantum,
  Title                    = {A quantum dot neural network},
  Author                   = {Behrman, Elizabeth C and Niemel, John and Steck, James E and Skinner, Steve R},
  Booktitle                = {Proceedings of the 4th Workshop on Physics of Computation},
  Year                     = {1996},
  Pages                    = {22--24},
  URL                      = {http://citeseerx.ist.psu.edu/viewdoc/download?doi=10.1.1.56.1507&rep=rep1&type=pdf}
}
</code></pre></details>


# Quantum Nearest Neighbors Algorithm


### 2020
<details><summary>[Ablayev et al] <b><a href=https://ieeexplore.ieee.org/abstract/document/8935095> On quantum methods for machine learning problems part II: Quantum classification algorithms. </a></b> <i>Big Data Mining and Analytics.</i> <code><b>[Quantum Machine Learning]</b></code> <code><b>[Quantum Classification]</b></code> <code><b>[Quantum Nearest Neighbors Algorithm]</b></code> <code><i>(Gate Model)</i></code></summary><pre><code>@Article{ablayev2020quantum2,
  Title                    = {{On quantum methods for machine learning problems part II: Quantum classification algorithms}},
  Author                   = {Ablayev, Farid and Ablayev, Marat and Huang, Joshua Zhexue and Khadiev, Kamil and Salikhova, Nailya and Wu, Dingming},
  Journal                  = {Big Data Mining and Analytics},
  Year                     = {2020},
  Number                   = {1},
  Pages                    = {56--67},
  Volume                   = {3},
  URL                      = {https://ieeexplore.ieee.org/abstract/document/8935095}
}
</code></pre></details>


### 2015
<details><summary>[Adcock et al] <b><a href=https://arxiv.org/abs/1512.02900> Advances in quantum machine learning. </a></b> <i>arXiv.</i> <code><b>[Quantum Machine Learning]</b></code> <code><b>[Quantum Classification]</b></code> <code><b>[Quantum Clustering]</b></code> <code><b>[Quantum Nearest Neighbors Algorithm]</b></code> <code><b>[Quantum Neural Networks]</b></code> <code><i>(Gate Model)</i></code> <code><i>(Adiabatic Model)</i></code></summary><pre><code>@Article{adcock2015advances,
  Title                    = {Advances in quantum machine learning},
  Author                   = {Adcock, Jeremy and Allen, Euan and Day, Matthew and Frick, Stefan and Hinchliff, Janna and Johnson, Mack and Morley-Short, Sam and Pallister, Sam and Price, Alasdair and Stanisic, Stasja},
  Journal                  = {arXiv preprint arXiv:1512.02900},
  Year                     = {2015},
  URL                      = {https://arxiv.org/abs/1512.02900}
}
</code></pre></details>


### 2014
<details><summary>[Wiebe et al] <b><a href=https://dl.acm.org/doi/abs/10.5555/2871393.2871400> Quantum Algorithms for Nearest-Neighbour Methods for Supervised and Unsupervised Learning. </a></b> <i>Quantum Information & Computation.</i> <code><b>[Quantum Nearest Neighbors Algorithm]</b></code> <code><b>[Quantum Classification]</b></code> <code><b>[Quantum Machine Learning]</b></code> <code><i>(Gate Model)</i></code></summary><pre><code>@article{wiebe2015quantum,
  title={Quantum algorithms for nearest-neighbor methods for supervised and unsupervised learning},
  author={Wiebe, Nathan and Kapoor, Ashish and Svore, Krysta M},
  journal={Quantum Information \& Computation},
  volume={15},
  number={3-4},
  pages={316--356},
  year={2015},
  publisher={Rinton Press, Incorporated Paramus, NJ}
}</code></pre></details>


# Quantum Neural Networks


### 2021
<details><summary>[Cerezo et al] <b><a href=https://www.nature.com/articles/s41467-021-21728-w> Cost Function Dependent Barren Plateaus in Shallow Parametrized Quantum Circuits. </a></b> <i>Nature communications.</i> <code><b>[Barren Plateaus]</b></code> <code><b>[Parameterized Quantum Circuits]</b></code> <code><b>[Quantum Neural Networks]</b></code> <code><b>[Quantum Machine Learning]</b></code> </summary><pre><code>@article{cerezo2021cost,
  title={Cost function dependent barren plateaus in shallow parametrized quantum circuits},
  author={Cerezo, Marco and Sone, Akira and Volkoff, Tyler and Cincio, Lukasz and Coles, Patrick J},
  journal={Nature communications},
  volume={12},
  number={1},
  pages={1--12},
  year={2021},
  publisher={Nature Publishing Group}
}</code></pre></details>
<details><summary>[Chen et al] <b><a href=https://arxiv.org/abs/1805.08654> Universal discriminative quantum neural networks. </a></b> <i>Quantum Machine Intelligence.</i> <code><b>[Quantum Neural Networks]</b></code> <code><b>[Quantum Machine Learning]</b></code> <code><i>(Gate Model)</i></code></summary><pre><code>@Article{chen2021universal,
  Title                    = {Universal discriminative quantum neural networks},
  Author                   = {Chen, Hongxiang and Wossnig, Leonard and Severini, Simone and Neven, Hartmut and Mohseni, Masoud},
  Journal                  = {Quantum Machine Intelligence},
  Year                     = {2021},
  Number                   = {1},
  Pages                    = {1--11},
  Volume                   = {3},
  Publisher                = {Springer},
  URL                      = {https://arxiv.org/abs/1805.08654}
}
</code></pre></details>
<details><summary>[Holmes et al ] <b><a href=https://arxiv.org/abs/2101.02138> Connecting ansatz expressibility to gradient magnitudes and barren plateaus. </a></b> <i>arXiv.</i> <code><b>[Parameterized Quantum Circuits]</b></code> <code><b>[Quantum Neural Networks]</b></code> <code><b>[Barren Plateaus]</b></code> <code><b>[Quantum Machine Learning]</b></code> </summary><pre><code>@article{holmes2021connecting,
  title={Connecting ansatz expressibility to gradient magnitudes and barren plateaus},
  author={Holmes, Zo{\"e} and Sharma, Kunal and Cerezo, M and Coles, Patrick J},
  journal={arXiv preprint arXiv:2101.02138},
  year={2021}
}</code></pre></details>
<details><summary>[Jaderberg et al] <b><a href=https://arxiv.org/abs/2103.14653> Quantum Self-Supervised Learning. </a></b> <i>arXiv.</i> <code><b>[Quantum Neural Networks]</b></code> <code><b>[Quantum Machine Learning]</b></code> <code><i>(Gate Model)</i></code></summary><pre><code>@article{jaderberg2021quantum,
  title={Quantum Self-Supervised Learning},
  author={Jaderberg, Ben and Anderson, Lewis W and Xie, Weidi and Albanie, Samuel and Kiffner, Martin and Jaksch, Dieter},
  journal={arXiv preprint arXiv:2103.14653},
  year={2021}
}</code></pre></details>
<details><summary>[Lubsch et al] <b><a href=https://journals.aps.org/pra/abstract/10.1103/PhysRevA.101.010301> Variational quantum algorithms for nonlinear problems. </a></b> <i>Physical Review A.</i> <code><b>[Nonlinear Problems]</b></code> <code><b>[Partial Differential Equations]</b></code> <code><b>[Quantum Neural Networks]</b></code> <code><b>[Quantum Machine Learning]</b></code> </summary><pre><code>@article{lubasch2020variational,
  title={Variational quantum algorithms for nonlinear problems},
  author={Lubasch, Michael and Joo, Jaewoo and Moinier, Pierre and Kiffner, Martin and Jaksch, Dieter},
  journal={Physical Review A},
  volume={101},
  number={1},
  pages={010301},
  year={2020},
  publisher={APS}
}
</code></pre></details>
<details><summary>[Mangini et al] <b><a href=https://iopscience.iop.org/article/10.1209/0295-5075/134/10002/meta> Quantum computing models for artificial neural networks. </a></b> <i>EPL (Europhysics Letters).</i> <code><b>[Quantum Neural Networks]</b></code> <code><b>[Parameterized Quantum Circuits]</b></code> <code><b>[Variational Quantum Algorithms]</b></code> <code><b>[Quantum Machine Learning]</b></code> </summary><pre><code>@article{mangini2021quantum,
  title={Quantum computing models for artificial neural networks},
  author={Mangini, Stefano and Tacchino, Francesco and Gerace, Dario and Bajoni, Daniele and Macchiavello, Chiara},
  journal={EPL (Europhysics Letters)},
  volume={134},
  number={1},
  pages={10002},
  year={2021},
  publisher={IOP Publishing}
}</code></pre></details>
<details><summary>[Patti et al] <b><a href=https://arxiv.org/abs/2012.12658> Entanglement devised barren plateau mitigation. </a></b> <i>arXiv.</i> <code><b>[Barren Plateaus]</b></code> <code><b>[Parameterized Quantum Circuits]</b></code> <code><b>[Quantum Neural Networks]</b></code> <code><b>[Quantum Machine Learning]</b></code> </summary><pre><code>@article{patti2020entanglement,
  title={Entanglement devised barren plateau mitigation},
  author={Patti, Taylor L and Najafi, Khadijeh and Gao, Xun and Yelin, Susanne F},
  journal={arXiv preprint arXiv:2012.12658},
  year={2020}
}</code></pre></details>
<details><summary>[Perez-Salinas et al] <b><a href=https://arxiv.org/abs/2102.04032> One qubit as a Universal Approximant. </a></b> <i>arXiv.</i> <code><b>[Parameterized Quantum Circuits]</b></code> <code><b>[Quantum Neural Networks]</b></code> <code><b>[Quantum Machine Learning]</b></code> </summary><pre><code>@article{perez2021one,
  title={One qubit as a Universal Approximant},
  author={P{\'e}rez-Salinas, Adri{\'a}n and L{\'o}pez-N{\'u}{\~n}ez, David and Garc{\'\i}a-S{\'a}ez, Artur and Forn-D{\'\i}az, P and Latorre, Jos{\'e} I},
  journal={arXiv preprint arXiv:2102.04032},
  year={2021}
}</code></pre></details>
<details><summary>[Skolik et al] <b><a href=https://link.springer.com/article/10.1007/s42484-020-00036-4> Layerwise learning for quantum neural networks. </a></b> <i>Quantum Machine Intelligence.</i> <code><b>[Barren Plateaus]</b></code> <code><b>[Parameterized Quantum Circuits]</b></code> <code><b>[Quantum Neural Networks]</b></code> <code><b>[Quantum Machine Learning]</b></code> </summary><pre><code>@article{skolik2021layerwise,
  title={Layerwise learning for quantum neural networks},
  author={Skolik, Andrea and McClean, Jarrod R and Mohseni, Masoud and van der Smagt, Patrick and Leib, Martin},
  journal={Quantum Machine Intelligence},
  volume={3},
  number={1},
  pages={1--11},
  year={2021},
  publisher={Springer}
}</code></pre></details>
<details><summary>[Volkoff & Coles] <b><a href=https://arxiv.org/abs/2005.12200> Large gradients via correlation in random parameterized quantum circuits. </a></b> <i>arXiv.</i> <code><b>[Barren Plateaus]</b></code> <code><b>[Parameterized Quantum Circuits]</b></code> <code><b>[Quantum Neural Networks]</b></code> <code><b>[Variational Quantum Algorithms]</b></code> <code><b>[Quantum Machine Learning]</b></code> </summary><pre><code>@article{volkoff2021large,
  title={Large gradients via correlation in random parameterized quantum circuits},
  author={Volkoff, Tyler and Coles, Patrick J},
  journal={Quantum Science and Technology},
  volume={6},
  number={2},
  pages={025008},
  year={2021},
  publisher={IOP Publishing}
}</code></pre></details>


### 2020
<details><summary>[Beer et al] <b><a href=https://www.nature.com/articles/s41467-020-14454-2.pdf> Training deep quantum neural networks. </a></b> <i>Nature communications.</i> <code><b>[Quantum Neural Networks]</b></code> <code><b>[Quantum Perceptron]</b></code> <code><b>[Quantum Machine Learning]</b></code> <code><i>(Gate Model)</i></code></summary><pre><code>@Article{beer2020training,
  Title                    = {Training deep quantum neural networks},
  Author                   = {Beer, Kerstin and Bondarenko, Dmytro and Farrelly, Terry and Osborne, Tobias J and Salzmann, Robert and Scheiermann, Daniel and Wolf, Ramona},
  Journal                  = {Nature communications},
  Year                     = {2020},
  Number                   = {1},
  Pages                    = {1--6},
  Volume                   = {11},
  Publisher                = {Nature Publishing Group},
  URL                      = {https://www.nature.com/articles/s41467-020-14454-2.pdf}
}
</code></pre></details>
<details><summary>[Carolan et al ] <b><a href=https://www.nature.com/articles/s41567-019-0747-6> Variational quantum unsampling on a quantum photonic processor. </a></b> <i>Nature Physics.</i> <code><b>[Variational Quantum Algorithms]</b></code> <code><b>[Quantum Neural Networks]</b></code> </summary><pre><code>@article{carolan2020variational,
  title={Variational quantum unsampling on a quantum photonic processor},
  author={Carolan, Jacques and Mohseni, Masoud and Olson, Jonathan P and Prabhu, Mihika and Chen, Changchen and Bunandar, Darius and Niu, Murphy Yuezhen and Harris, Nicholas C and Wong, Franco NC and Hochberg, Michael and others},
  journal={Nature Physics},
  volume={16},
  number={3},
  pages={322--327},
  year={2020},
  publisher={Nature Publishing Group}
}</code></pre></details>
<details><summary>[Chakraborty et al] <b><a href=https://ieeexplore.ieee.org/document/9137960> An Analytical Review of Quantum Neural Network Models and Relevant Research. </a></b> <i>International Conference on Communication and Electronics Systems.</i> <code><b>[Quantum Neural Networks]</b></code> <code><b>[Quantum Machine Learning]</b></code> </summary><pre><code>@inproceedings{chakraborty2020analytical,
  title={An Analytical Review of Quantum Neural Network Models and Relevant Research},
  author={Chakraborty, Simantini and Das, Tamal and Sutradhar, Saurav and Das, Mrinmoy and Deb, Suman},
  booktitle={2020 5th International Conference on Communication and Electronics Systems (ICCES)},
  pages={1395--1400},
  year={2020},
  organization={IEEE}
}</code></pre></details>
<details><summary>[Gil Vidal & Theis] <b><a href=https://doi.org/10.3389/fphy.2020.00297> Input redundancy for parameterized quantum circuits. </a></b> <i>Frontiers in Physics.</i> <code><b>[Parameterized Quantum Circuits]</b></code> <code><b>[Quantum Neural Networks]</b></code> <code><b>[Quantum Machine Learning]</b></code> </summary><pre><code>@article{gil2020input,
  title={Input redundancy for parameterized quantum circuits},
  author={Gil Vidal, Francisco Javier and Theis, Dirk Oliver},
  journal={Frontiers in Physics},
  volume={8},
  pages={297},
  year={2020},
  publisher={Frontiers}
}</code></pre></details>
<details><summary>[Henderson et al] <b><a href=https://link.springer.com/content/pdf/10.1007/s42484-020-00012-y.pdf> Quanvolutional neural networks: powering image recognition with quantum circuits. </a></b> <i>Quantum Machine Intelligence.</i> <code><b>[Quantum Convolutional Neural Networks]</b></code> <code><b>[Quantum Neural Networks]</b></code> <code><b>[Quantum Machine Learning]</b></code> <code><i>(Gate Model)</i></code></summary><pre><code>@article{henderson2020quanvolutional,
  title={Quanvolutional neural networks: powering image recognition with quantum circuits},
  author={Henderson, Maxwell and Shakya, Samriddhi and Pradhan, Shashindra and Cook, Tristan},
  journal={Quantum Machine Intelligence},
  volume={2},
  number={1},
  pages={1--9},
  year={2020},
  publisher={Springer}
}</code></pre></details>
<details><summary>[Kerenidis et al] <b><a href=https://openreview.net/pdf?id=Hygab1rKDS> Quantum algorithms for deep convolutional neural networks. </a></b> <i>ICLR.</i> <code><b>[Quantum Neural Networks]</b></code> <code><b>[Quantum Convolutional Neural Networks]</b></code> <code><b>[Quantum Machine Learning]</b></code> <code><i>(Gate Model)</i></code></summary><pre><code>@inproceedings{kerenidis2019quantum,
  title={Quantum Algorithms for Deep Convolutional Neural Networks},
  author={Kerenidis, Iordanis and Landman, Jonas and Prakash, Anupam},
  booktitle={International Conference on Learning Representations},
  year={2019}
}</code></pre></details>
<details><summary>[Li et al] <b><a href=https://iopscience.iop.org/article/10.1088/2058-9565/ab9f93> A quantum deep convolutional neural network for image recognition. </a></b> <i>Quantum Science and Technology.</i> <code><b>[Quantum Convolutional Neural Networks]</b></code> <code><b>[Quantum Neural Networks]</b></code> <code><b>[Quantum Machine Learning]</b></code> </summary><pre><code>@article{li2020quantum,
  title={A quantum deep convolutional neural network for image recognition},
  author={Li, YaoChong and Zhou, Ri-Gui and Xu, RuQing and Luo, Jia and Hu, WenWen},
  journal={Quantum Science and Technology},
  volume={5},
  number={4},
  pages={044003},
  year={2020},
  publisher={IOP Publishing}
}</code></pre></details>
<details><summary>[Macaluso et al] <b><a href=https://link.springer.com/content/pdf/10.1007%2F978-3-030-50433-5_45.pdf> A Variational Algorithm for QuantumNeural Networks. </a></b> <i>International Conference on Computational Science.</i> <code><b>[Quantum Neural Networks]</b></code> <code><b>[Quantum Perceptron]</b></code> <code><b>[Quantum Machine Learning]</b></code> <code><i>(Gate Model)</i></code></summary><pre><code>@inproceedings{macaluso2020variational,
  title={A Variational Algorithm for Quantum Neural Networks},
  author={Macaluso, Antonio and Clissa, Luca and Lodi, Stefano and Sartori, Claudio},
  booktitle={International Conference on Computational Science},
  pages={591--604},
  year={2020},
  organization={Springer}
}</code></pre></details>
<details><summary>[Mari et al] <b><a href=https://quantum-journal.org/papers/q-2020-10-09-340/> Transfer learning in hybrid classical-quantum neural networks. </a></b> <i>Quantum.</i> <code><b>[Quantum Neural Networks]</b></code> <code><b>[Quantum Transfer Learning]</b></code> <code><b>[Quantum Machine Learning]</b></code> <code><i>(Gate Model)</i></code></summary><pre><code>@article{mari2020transfer,
  title={Transfer learning in hybrid classical-quantum neural networks},
  author={Mari, Andrea and Bromley, Thomas R and Izaac, Josh and Schuld, Maria and Killoran, Nathan},
  journal={Quantum},
  volume={4},
  pages={340},
  year={2020},
  publisher={Verein zur F{\"o}rderung des Open Access Publizierens in den Quantenwissenschaften}
}</code></pre></details>
<details><summary>[Marrero et al] <b><a href=https://arxiv.org/abs/2010.15968> Entanglement Induced Barren Plateaus. </a></b> <i>arXiv.</i> <code><b>[Barren Plateaus]</b></code> <code><b>[Parameterized Quantum Circuits]</b></code> <code><b>[Quantum Neural Networks]</b></code> <code><b>[Quantum Machine Learning]</b></code> </summary><pre><code>@article{marrero2020entanglement,
  title={Entanglement induced barren plateaus},
  author={Marrero, Carlos Ortiz and Kieferov{\'a}, M{\'a}ria and Wiebe, Nathan},
  journal={arXiv preprint arXiv:2010.15968},
  year={2020}
}</code></pre></details>
<details><summary>[Perez-Salinas et al] <b><a href=https://quantum-journal.org/papers/q-2020-02-06-226/> Data re-uploading for a universal quantum classifier. </a></b> <i>Quantum.</i> <code><b>[Quantum Neural Networks]</b></code> <code><b>[Quantum Classification]</b></code> <code><b>[Quantum Machine Learning]</b></code> <code><i>(Gate Model)</i></code></summary><pre><code>@article{perez2020data,
  title={Data re-uploading for a universal quantum classifier},
  author={P{\'e}rez-Salinas, Adri{\'a}n and Cervera-Lierta, Alba and Gil-Fuster, Elies and Latorre, Jos{\'e} I},
  journal={Quantum},
  volume={4},
  pages={226},
  year={2020},
  publisher={Verein zur F{\"o}rderung des Open Access Publizierens in den Quantenwissenschaften}
}</code></pre></details>
<details><summary>[Pesah et al] <b><a href=https://arxiv.org/abs/2011.02966> Absence of Barren Plateaus in Quantum Convolutional Neural Networks. </a></b> <i>arXiv.</i> <code><b>[Barren Plateaus]</b></code> <code><b>[Quantum Convolutional Neural Networks]</b></code> <code><b>[Parameterized Quantum Circuits]</b></code> <code><b>[Quantum Neural Networks]</b></code> <code><b>[Quantum Machine Learning]</b></code> </summary><pre><code>@article{pesah2020absence,
  title={Absence of barren plateaus in quantum convolutional neural networks},
  author={Pesah, Arthur and Cerezo, M and Wang, Samson and Volkoff, Tyler and Sornborger, Andrew T and Coles, Patrick J},
  journal={arXiv preprint arXiv:2011.02966},
  year={2020}
}</code></pre></details>
<details><summary>[Schuld et al] <b><a href=https://journals.aps.org/pra/abstract/10.1103/PhysRevA.101.032308> Circuit-centric quantum classifiers. </a></b> <i>Physical Review A.</i> <code><b>[Quantum Neural Networks]</b></code> <code><b>[Quantum Classification]</b></code> <code><b>[Quantum Machine Learning]</b></code> <code><i>(Gate Model)</i></code></summary><pre><code>@article{schuld2020circuit,
  title={Circuit-centric quantum classifiers},
  author={Schuld, Maria and Bocharov, Alex and Svore, Krysta M and Wiebe, Nathan},
  journal={Physical Review A},
  volume={101},
  number={3},
  pages={032308},
  year={2020},
  publisher={APS}
}</code></pre></details>
<details><summary>[Sharma et al] <b><a href=https://arxiv.org/abs/2005.12458> Trainability of Dissipative Perceptron-Based Quantum Neural Networks. </a></b> <i>arXiv.</i> <code><b>[Parameterized Quantum Circuits]</b></code> <code><b>[Quantum Neural Networks]</b></code> <code><b>[Quantum Machine Learning]</b></code> <code><b>[Barren Plateaus]</b></code> </summary><pre><code>@article{sharma2020trainability,
  title={Trainability of dissipative perceptron-based quantum neural networks},
  author={Sharma, Kunal and Cerezo, Marco and Cincio, Lukasz and Coles, Patrick J},
  journal={arXiv preprint arXiv:2005.12458},
  year={2020}
}</code></pre></details>
<details><summary>[Tacchino et al] <b><a href=https://ieeexplore.ieee.org/abstract/document/9364892> Variational Learning for Quantum Artificial Neural Networks. </a></b> <i>IEEE International Conference on Quantum Computing and Engineering.</i> <code><b>[Parameterized Quantum Circuits]</b></code> <code><b>[Quantum Neural Networks]</b></code> <code><b>[Quantum Machine Learning]</b></code> </summary><pre><code>@inproceedings{tacchino2020variational,
  title={Variational learning for quantum artificial neural networks},
  author={Tacchino, Francesco and Barkoutsos, Panagiotis Kl and Macchiavello, Chiara and Gerace, Dario and Tavernelli, Ivano and Bajoni, Daniele},
  booktitle={2020 IEEE International Conference on Quantum Computing and Engineering (QCE)},
  pages={130--136},
  year={2020},
  organization={IEEE}
}</code></pre></details>


### 2019
<details><summary>[Cong et al] <b><a href=https://www.nature.com/articles/s41567-019-0648-8.pdf> Quantum convolutional neural networks. </a></b> <i>Nature Physics.</i> <code><b>[Quantum Neural Networks]</b></code> <code><b>[Quantum Convolutional Neural Networks]</b></code> <code><b>[Quantum Machine Learning]</b></code> <code><i>(Gate Model)</i></code></summary><pre><code>@Article{cong2019quantum,
  Title                    = {Quantum convolutional neural networks},
  Author                   = {Cong, Iris and Choi, Soonwon and Lukin, Mikhail D},
  Journal                  = {Nature Physics},
  Year                     = {2019},
  Number                   = {12},
  Pages                    = {1273--1278},
  Volume                   = {15},
  Publisher                = {Nature Publishing Group},
  URL                      = {https://www.nature.com/articles/s41567-019-0648-8.pdf}
}
</code></pre></details>
<details><summary>[Grant et al] <b><a href=https://quantum-journal.org/papers/q-2019-12-09-214/> An initialization strategy for addressing barren plateaus in parametrized quantum circuits. </a></b> <i>Quantum.</i> <code><b>[Barren Plateaus]</b></code> <code><b>[Parameterized Quantum Circuits]</b></code> <code><b>[Quantum Neural Networks]</b></code> <code><b>[Quantum Machine Learning]</b></code> </summary><pre><code>@article{grant2019initialization,
  title={An initialization strategy for addressing barren plateaus in parametrized quantum circuits},
  author={Grant, Edward and Wossnig, Leonard and Ostaszewski, Mateusz and Benedetti, Marcello},
  journal={Quantum},
  volume={3},
  pages={214},
  year={2019},
  publisher={Verein zur F{\"o}rderung des Open Access Publizierens in den Quantenwissenschaften}
}</code></pre></details>
<details><summary>[Kamruzzaman et al] <b><a href=https://link.springer.com/chapter/10.1007/978-3-030-12385-7_24> Quantum Deep Learning Neural Networks. </a></b> <i>FICC 2019.</i> <code><b>[Quantum Neural Networks]</b></code> </summary><pre><code>@inproceedings{kamruzzaman2019quantum,
  title={Quantum deep learning neural networks},
  author={Kamruzzaman, Abu and Alhwaiti, Yousef and Leider, Avery and Tappert, Charles C},
  booktitle={Future of Information and Communication Conference},
  pages={299--311},
  year={2019},
  organization={Springer}
}</code></pre></details>
<details><summary>[Killoran et al] <b><a href=https://journals.aps.org/prresearch/abstract/10.1103/PhysRevResearch.1.033063> Continuous-variable quantum neural networks. </a></b> <i>Physical Review Research.</i> <code><b>[Quantum Neural Networks]</b></code> <code><b>[Quantum Machine Learning]</b></code> <code><i>(Gate Model)</i></code></summary><pre><code>@article{killoran2019continuous,
  title={Continuous-variable quantum neural networks},
  author={Killoran, Nathan and Bromley, Thomas R and Arrazola, Juan Miguel and Schuld, Maria and Quesada, Nicol{\'a}s and Lloyd, Seth},
  journal={Physical Review Research},
  volume={1},
  number={3},
  pages={033063},
  year={2019},
  publisher={APS}
}</code></pre></details>
<details><summary>[Verdon et al] <b><a href=https://arxiv.org/abs/1907.05415> Learning to learn with quantum neural networks via classical neural networks. </a></b> <i>arXiv.</i> <code><b>[Quantum Perceptron]</b></code> <code><b>[Parameterized Quantum Circuits]</b></code> <code><b>[Quantum Neural Networks]</b></code> <code><b>[Quantum Machine Learning]</b></code> <code><b>[Variational Quantum Algorithms]</b></code> </summary><pre><code>@article{verdon2019learning,
  title={Learning to learn with quantum neural networks via classical neural networks},
  author={Verdon, Guillaume and Broughton, Michael and McClean, Jarrod R and Sung, Kevin J and Babbush, Ryan and Jiang, Zhang and Neven, Hartmut and Mohseni, Masoud},
  journal={arXiv preprint arXiv:1907.05415},
  year={2019}
}</code></pre></details>
<details><summary>[Verdon et al] <b><a href=https://arxiv.org/abs/1909.12264> Quantum graph neural networks. </a></b> <i>arXiv.</i> <code><b>[Quantum Neural Networks]</b></code> <code><b>[Quantum Machine Learning]</b></code> <code><i>(Gate Model)</i></code></summary><pre><code>@article{verdon2019quantum,
  title={Quantum graph neural networks},
  author={Verdon, Guillaume and McCourt, Trevor and Luzhnica, Enxhell and Singh, Vikash and Leichenauer, Stefan and Hidary, Jack},
  journal={arXiv preprint arXiv:1909.12264},
  year={2019}
</code></pre></details>


### 2018
<details><summary>[Ciliberto et al] <b><a href=https://doi.org/10.1098/rspa.2017.0551> Quantum machine learning: a classical perspective. </a></b> <i>Proceedings of the Royal Society A: Mathematical, Physical and Engineering Sciences.</i> <code><b>[Quantum Machine Learning]</b></code> <code><b>[Quantum Optimization]</b></code> <code><b>[Quantum Neural Networks]</b></code> </summary><pre><code>@Article{ciliberto2018quantum,
  Title                    = {Quantum machine learning: a classical perspective},
  Author                   = {Ciliberto, Carlo and Herbster, Mark and Ialongo, Alessandro Davide and Pontil, Massimiliano and Rocchetto, Andrea and Severini, Simone and Wossnig, Leonard},
  Journal                  = {Proceedings of the Royal Society A: Mathematical, Physical and Engineering Sciences},
  Year                     = {2018},
  Number                   = {2209},
  Pages                    = {20170551},
  Volume                   = {474},
  Publisher                = {The Royal Society Publishing},
  URL                      = {https://doi.org/10.1098/rspa.2017.0551}
}
</code></pre></details>
<details><summary>[Dallaire-Demers & Killoran] <b><a href=https://journals.aps.org/pra/abstract/10.1103/PhysRevA.98.012324> Quantum generative adversarial networks. </a></b> <i>Physical Review A.</i> <code><b>[Quantum Generative Adversarial Networks]</b></code> <code><b>[Quantum Neural Networks]</b></code> <code><b>[Variational Quantum Algorithms]</b></code> <code><b>[Quantum Machine Learning]</b></code> <code><i>(Gate Model)</i></code></summary><pre><code>@article{dallaire2018quantum,
  title={Quantum generative adversarial networks},
  author={Dallaire-Demers, Pierre-Luc and Killoran, Nathan},
  journal={Physical Review A},
  volume={98},
  number={1},
  pages={012324},
  year={2018},
  publisher={APS}
}</code></pre></details>
<details><summary>[Fahri & Neven] <b><a href=https://arxiv.org/abs/1802.06002> Classification with Quantum Neural Networks on Near Term Processors. </a></b> <i>arXiv.</i> <code><b>[Quantum Neural Networks]</b></code> <code><b>[Quantum Machine Learning]</b></code> <code><b>[Quantum Classification]</b></code> <code><i>(Gate Model)</i></code></summary><pre><code>@Article{farhi2018classification,
  Title                    = {Classification with quantum neural networks on near term processors},
  Author                   = {Farhi, Edward and Neven, Hartmut},
  Journal                  = {arXiv preprint arXiv:1802.06002},
  Year                     = {2018},
  URL                      = {https://arxiv.org/abs/1802.06002}
}
</code></pre></details>
<details><summary>[Grant et al] <b><a href=https://www.nature.com/articles/s41534-018-0116-9> Hierarchical quantum classifiers. </a></b> <i>npj Quantum information.</i> <code><b>[Quantum Neural Networks]</b></code> <code><b>[Quantum Machine Learning]</b></code> <code><b>[Quantum Classification]</b></code> <code><i>(Gate Model)</i></code></summary><pre><code>@Article{grant2018hierarchical,
  Title                    = {Hierarchical quantum classifiers},
  Author                   = {Grant, Edward and Benedetti, Marcello and Cao, Shuxiang and Hallam, Andrew and Lockhart, Joshua and Stojevic, Vid and Green, Andrew G and Severini, Simone},
  Journal                  = {npj Quantum Information},
  Year                     = {2018},
  Number                   = {1},
  Pages                    = {1--8},
  Volume                   = {4},
  Publisher                = {Nature Publishing Group},
  URL                      = {https://www.nature.com/articles/s41534-018-0116-9}
}
</code></pre></details>
<details><summary>[McClean et al] <b><a href=https://www.nature.com/articles/s41467-018-07090-4> Barren plateaus in quantum neural network training landscapes. </a></b> <i>Nature communications.</i> <code><b>[Barren Plateaus]</b></code> <code><b>[Parameterized Quantum Circuits]</b></code> <code><b>[Quantum Neural Networks]</b></code> <code><b>[Quantum Machine Learning]</b></code> </summary><pre><code>@article{mcclean2018barren,
  title={Barren plateaus in quantum neural network training landscapes},
  author={McClean, Jarrod R and Boixo, Sergio and Smelyanskiy, Vadim N and Babbush, Ryan and Neven, Hartmut},
  journal={Nature communications},
  volume={9},
  number={1},
  pages={1--6},
  year={2018},
  publisher={Nature Publishing Group}
}</code></pre></details>
<details><summary>[Mitarai et al] <b><a href=https://journals.aps.org/pra/abstract/10.1103/PhysRevA.98.032309> Quantum circuit learning. </a></b> <i>Physical Review A.</i> <code><b>[Parameterized Quantum Circuits]</b></code> <code><b>[Quantum Neural Networks]</b></code> <code><b>[Quantum Machine Learning]</b></code> </summary><pre><code>@article{mitarai2018quantum,
  title={Quantum circuit learning},
  author={Mitarai, Kosuke and Negoro, Makoto and Kitagawa, Masahiro and Fujii, Keisuke},
  journal={Physical Review A},
  volume={98},
  number={3},
  pages={032309},
  year={2018},
  publisher={APS}
}</code></pre></details>


### 2017
<details><summary>[Cao et al] <b><a href=https://arxiv.org/pdf/1711.11240.pdf> Quantum Neuron:  an elementary building block for machine learningon quantum computers. </a></b> <i>arXiv.</i> <code><b>[Quantum Perceptron]</b></code> <code><b>[Quantum Information and Computing]</b></code> <code><b>[Quantum Neural Networks]</b></code> <code><b>[Quantum Machine Learning]</b></code> <code><i>(Gate Model)</i></code></summary><pre><code>@Article{cao2017quantum,
  Title                    = {Quantum neuron: an elementary building block for machine learning on quantum computers},
  Author                   = {Cao, Yudong and Guerreschi, Gian Giacomo and Aspuru-Guzik, Al{\'a}n},
  Journal                  = {arXiv preprint arXiv:1711.11240},
  Year                     = {2017},
  URL                      = {https://arxiv.org/pdf/1711.11240.pdf}
}
</code></pre></details>
<details><summary>[Romero et al] <b><a href=https://arxiv.org/pdf/1612.02806.pdf> Quantum autoencoders for efficient compression of quantum data. </a></b> <i>Quantum Science and Technology.</i> <code><b>[Quantum Neural Networks]</b></code> <code><b>[Quantum Autoencoders]</b></code> <code><b>[Quantum Machine Learning]</b></code> <code><i>(Gate Model)</i></code></summary><pre><code>@article{romero2017quantum,
  title={Quantum autoencoders for efficient compression of quantum data},
  author={Romero, Jonathan and Olson, Jonathan P and Aspuru-Guzik, Alan},
  journal={Quantum Science and Technology},
  volume={2},
  number={4},
  pages={045001},
  year={2017},
  publisher={IOP Publishing}
}</code></pre></details>
<details><summary>[Wan et al] <b><a href=https://www.nature.com/articles/s41534-017-0032-4> Quantum generalisation of feedforward neural networks. </a></b> <i>npj Quantum information.</i> <code><b>[Quantum Neural Networks]</b></code> <code><b>[Quantum Autoencoders]</b></code> <code><b>[Quantum Machine Learning]</b></code> <code><i>(Gate Model)</i></code></summary><pre><code>@Article{wan2017quantum,
  Title                    = {Quantum generalisation of feedforward neural networks},
  Author                   = {Wan, Kwok Ho and Dahlsten, Oscar and Kristj{\'a}nsson, Hl{\'e}r and Gardner, Robert and Kim, MS},
  Journal                  = {npj Quantum information},
  Year                     = {2017},
  Number                   = {1},
  Pages                    = {1--8},
  Volume                   = {3},
  Publisher                = {Nature Publishing Group},
  URL                      = {https://www.nature.com/articles/s41534-017-0032-4}
}
</code></pre></details>


### 2015
<details><summary>[Adcock et al] <b><a href=https://arxiv.org/abs/1512.02900> Advances in quantum machine learning. </a></b> <i>arXiv.</i> <code><b>[Quantum Machine Learning]</b></code> <code><b>[Quantum Classification]</b></code> <code><b>[Quantum Clustering]</b></code> <code><b>[Quantum Nearest Neighbors Algorithm]</b></code> <code><b>[Quantum Neural Networks]</b></code> <code><i>(Gate Model)</i></code> <code><i>(Adiabatic Model)</i></code></summary><pre><code>@Article{adcock2015advances,
  Title                    = {Advances in quantum machine learning},
  Author                   = {Adcock, Jeremy and Allen, Euan and Day, Matthew and Frick, Stefan and Hinchliff, Janna and Johnson, Mack and Morley-Short, Sam and Pallister, Sam and Price, Alasdair and Stanisic, Stasja},
  Journal                  = {arXiv preprint arXiv:1512.02900},
  Year                     = {2015},
  URL                      = {https://arxiv.org/abs/1512.02900}
}
</code></pre></details>


### 2014
<details><summary>[Schuld et al] <b><a href=https://link.springer.com/article/10.1007%2Fs11128-014-0809-8> The quest for a Quantum Neural Networt. </a></b> <i>Quantum Information Processing.</i> <code><b>[Quantum Neural Networks]</b></code> <code><b>[Quantum Machine Learning]</b></code> <code><i>(Gate Model)</i></code></summary><pre><code>@article{schuld2014quest,
  title={The quest for a quantum neural network},
  author={Schuld, Maria and Sinayskiy, Ilya and Petruccione, Francesco},
  journal={Quantum Information Processing},
  volume={13},
  number={11},
  pages={2567--2586},
  year={2014},
  publisher={Springer}
}</code></pre></details>


### 2012
<details><summary>[Da Silva et al] <b><a href=https://www.sciencedirect.com/science/article/abs/pii/S0925231211004127?via%3Dihub> Classical and superposed learning for quantum weightless neural networks. </a></b> <i>Neurocomputing.</i> <code><b>[Quantum Neural Networks]</b></code> <code><b>[Quantum Machine Learning]</b></code> <code><i>(Gate Model)</i></code></summary><pre><code>@Article{da2012classical,
  Title                    = {Classical and superposed learning for quantum weightless neural networks},
  Author                   = {Da Silva, Adenilton J and De Oliveira, Wilson R and Ludermir, Teresa B},
  Journal                  = {Neurocomputing},
  Year                     = {2012},
  Number                   = {1},
  Pages                    = {52--60},
  Volume                   = {75},
  Publisher                = {Elsevier},
  URL                      = {https://www.sciencedirect.com/science/article/abs/pii/S0925231211004127?via%3Dihub}
}
</code></pre></details>


### 2011
<details><summary>[Panella & Martinelli] <b><a href=https://onlinelibrary.wiley.com/doi/10.1002/cta.619> Neural networks with quantum. </a></b> <i>International Journal of Circuit Theory and Applications.</i> <code><b>[Quantum Neurofuzzy Network]</b></code> <code><b>[Quantum Neural Networks]</b></code> <code><b>[Quantum Machine Learning]</b></code> </summary><pre><code>@article{panella2011neural,
  title={Neural networks with quantum architecture and quantum learning},
  author={Panella, Massimo and Martinelli, Giuseppe},
  journal={International Journal of Circuit Theory and Applications},
  volume={39},
  number={1},
  pages={61--77},
  year={2011},
  publisher={Wiley Online Library}
}</code></pre></details>


### 2010
<details><summary>[Da Silva et al] <b><a href=https://ieeexplore.ieee.org/document/5715204> Superposition Based Learning Algorithm. </a></b> <i>Brazilian Symposium on Neural Networks.</i> <code><b>[Quantum Neural Networks]</b></code> <code><b>[Quantum Machine Learning]</b></code> <code><i>(Gate Model)</i></code></summary><pre><code>@inproceedings{silva2010superposition,
  title={Superposition based learning algorithm},
  author={Silva, Adenilton J and Ludermir, Teresa B and de Oliveira Jr, Wilson R},
  booktitle={2010 Eleventh Brazilian Symposium on Neural Networks},
  pages={1--6},
  year={2010},
  organization={IEEE}
}</code></pre></details>


### 2009
<details><summary>[de Oliveira] <b><a href=https://citeseerx.ist.psu.edu/viewdoc/download?doi=10.1.1.227.4363&rep=rep1&type=pdf> Quantum RAM based neural networks. </a></b> <i>European Symposium on Artificial Neural Network.</i> <code><b>[QRAM]</b></code> <code><b>[Quantum Neural Networks]</b></code> <code><b>[Quantum Machine Learning]</b></code> <code><i>(Gate Model)</i></code></summary><pre><code>@InProceedings{de2009quantum,
  Title                    = {Quantum RAM Based Neural Netoworks.},
  Author                   = {de Oliveira, Wilson Rosa},
  Booktitle                = {ESANN},
  Year                     = {2009},
  Organization             = {Citeseer},
  Pages                    = {331--336},
  Volume                   = {9},
  URL                      = {https://citeseerx.ist.psu.edu/viewdoc/download?doi=10.1.1.227.4363&rep=rep1&type=pdf}
}
</code></pre></details>


### 2008
<details><summary>[Panella & Martinelli] <b><a href=https://ieeexplore.ieee.org/document/4565676> Neurofuzzy networks with nonlinear quantum learning. </a></b> <i>IEEE Transactions on Fuzzy Systems.</i> <code><b>[Quantum Neurofuzzy Network]</b></code> <code><b>[Quantum Neural Networks]</b></code> <code><b>[Quantum Machine Learning]</b></code> </summary><pre><code>@article{panella2008neurofuzzy,
  title={Neurofuzzy networks with nonlinear quantum learning},
  author={Panella, Massimo and Martinelli, Giuseppe},
  journal={IEEE Transactions on Fuzzy Systems},
  volume={17},
  number={3},
  pages={698--710},
  year={2008},
  publisher={IEEE}
}</code></pre></details>


### 2003
<details><summary>[Ricks & Ventura] <b><a href=https://dl.acm.org/doi/abs/10.5555/2981345.2981472> Training a Quantum Neural Network. </a></b> <i>NIPS.</i> <code><b>[Quantum Neural Networks]</b></code> </summary><pre><code>@article{ricks2003training,
  title={Training a quantum neural network},
  author={Ricks, Bob and Ventura, Dan},
  journal={Advances in neural information processing systems},
  volume={16},
  pages={1019--1026},
  year={2003}
}</code></pre></details>


### 2001
<details><summary>[Altaisky] <b><a href=https://arxiv.org/abs/quant-ph/0107012> Quantum neural network. </a></b> <i>arXiv.</i> <code><b>[Quantum Neural Networks]</b></code> <code><b>[Quantum Perceptron]</b></code> <code><b>[Quantum Machine Learning]</b></code> </summary><pre><code>@Article{altaisky2001quantum,
  Title                    = {Quantum neural network},
  Author                   = {Altaisky, MV},
  Journal                  = {arXiv preprint quant-ph/0107012},
  Year                     = {2001},
  URL                      = {https://arxiv.org/abs/quant-ph/0107012}
}
</code></pre></details>


### 2000
<details><summary>[Narayanan & Menneer] <b><a href=https://www.sciencedirect.com/science/article/abs/pii/S0020025500000554> Quantum artificial neural network architectures and components. </a></b> <i>Information Sciences.</i> <code><b>[Quantum Neural Networks]</b></code> <code><b>[Quantum Machine Learning]</b></code> </summary><pre><code>@article{narayanan2000quantum,
  title={Quantum artificial neural network architectures and components},
  author={Narayanan, Ajit and Menneer, Tammy},
  journal={Information Sciences},
  volume={128},
  number={3-4},
  pages={231--255},
  year={2000},
  publisher={Elsevier}
}</code></pre></details>


### 1996
<details><summary>[Behrman et al ] <b><a href=http://citeseerx.ist.psu.edu/viewdoc/download?doi=10.1.1.56.1507&rep=rep1&type=pdf> A quantum dot neural network. </a></b> <i>Workshop on Physics of Computation.</i> <code><b>[Quantum Neural Networks]</b></code> <code><b>[Quantum Machine Learning]</b></code> <code><i>(Gate Model)</i></code></summary><pre><code>@InProceedings{behrman1996quantum,
  Title                    = {A quantum dot neural network},
  Author                   = {Behrman, Elizabeth C and Niemel, John and Steck, James E and Skinner, Steve R},
  Booktitle                = {Proceedings of the 4th Workshop on Physics of Computation},
  Year                     = {1996},
  Pages                    = {22--24},
  URL                      = {http://citeseerx.ist.psu.edu/viewdoc/download?doi=10.1.1.56.1507&rep=rep1&type=pdf}
}
</code></pre></details>


# Quantum Optimization


### 2021
<details><summary>[Amaro et al.] <b><a href=https://arxiv.org/abs/2106.10055> Filtering variational quantum algorithms for combinatorial optimization. </a></b> <i>arXiv.</i> <code><b>[Variational Quantum Algorithms]</b></code> <code><b>[Quantum Optimization]</b></code> <code><b>[Parameterized Quantum Circuits]</b></code> </summary><pre><code>@article{amaro2021filtering,
  title={Filtering variational quantum algorithms for combinatorial optimization},
  author={Amaro, David and Modica, Carlo and Rosenkranz, Matthias and Fiorentini, Mattia and Benedetti, Marcello and Lubasch, Michael},
  journal={Quantum Science and Technology},
  year={2021},
  publisher={IOP Publishing}
}
</code></pre></details>
<details><summary>[Bharti et al] <b><a href=https://arxiv.org/abs/2101.08448> Noisy intermediate-scale quantum (NISQ) algorithms. </a></b> <i>arXiv.</i> <code><b>[Variational Quantum Algorithms]</b></code> <code><b>[Parameterized Quantum Circuits]</b></code> <code><b>[Quantum Optimization]</b></code> <code><b>[Quantum Machine Learning]</b></code> <code><i>(Gate Model)</i></code> <code><i>(Adiabatic Model)</i></code></summary><pre><code>@article{bharti2021noisy,
  title={Noisy intermediate-scale quantum (NISQ) algorithms},
  author={Bharti, Kishor and Cervera-Lierta, Alba and Kyaw, Thi Ha and Haug, Tobias and Alperin-Lea, Sumner and Anand, Abhinav and Degroote, Matthias and Heimonen, Hermanni and Kottmann, Jakob S and Menke, Tim and others},
  journal={arXiv preprint arXiv:2101.08448},
  year={2021}
}</code></pre></details>
<details><summary>[Egger et al] <b><a href=https://quantum-journal.org/papers/q-2021-06-17-479/> Warm-starting quantum optimization. </a></b> <i>Quantum.</i> <code><b>[Quantum Optimization]</b></code> <code><b>[Parameterized Quantum Circuits]</b></code> <code><b>[Quantum Machine Learning]</b></code> </summary><pre><code>@article{egger2021warm,
  title={Warm-starting quantum optimization},
  author={Egger, Daniel J and Mare{\v{c}}ek, Jakub and Woerner, Stefan},
  journal={Quantum},
  volume={5},
  pages={479},
  year={2021},
  publisher={Verein zur F{\"o}rderung des Open Access Publizierens in den Quantenwissenschaften}
}</code></pre></details>


### 2020
<details><summary>[Barkoutsos et al] <b><a href=https://quantum-journal.org/papers/q-2020-04-20-256/> Improving Variational Quantum Optimization Using CVaR. </a></b> <i>Quantum.</i> <code><b>[Variational Quantum Algorithms]</b></code> <code><b>[Quantum Optimization]</b></code> <code><b>[Quantum Machine Learning]</b></code> </summary><pre><code>@article{barkoutsos2020improving,
  title={Improving variational quantum optimization using cvar},
  author={Barkoutsos, Panagiotis Kl and Nannicini, Giacomo and Robert, Anton and Tavernelli, Ivano and Woerner, Stefan},
  journal={Quantum},
  volume={4},
  pages={256},
  year={2020},
  publisher={Verein zur F{\"o}rderung des Open Access Publizierens in den Quantenwissenschaften}
}</code></pre></details>


### 2018
<details><summary>[Ciliberto et al] <b><a href=https://doi.org/10.1098/rspa.2017.0551> Quantum machine learning: a classical perspective. </a></b> <i>Proceedings of the Royal Society A: Mathematical, Physical and Engineering Sciences.</i> <code><b>[Quantum Machine Learning]</b></code> <code><b>[Quantum Optimization]</b></code> <code><b>[Quantum Neural Networks]</b></code> </summary><pre><code>@Article{ciliberto2018quantum,
  Title                    = {Quantum machine learning: a classical perspective},
  Author                   = {Ciliberto, Carlo and Herbster, Mark and Ialongo, Alessandro Davide and Pontil, Massimiliano and Rocchetto, Andrea and Severini, Simone and Wossnig, Leonard},
  Journal                  = {Proceedings of the Royal Society A: Mathematical, Physical and Engineering Sciences},
  Year                     = {2018},
  Number                   = {2209},
  Pages                    = {20170551},
  Volume                   = {474},
  Publisher                = {The Royal Society Publishing},
  URL                      = {https://doi.org/10.1098/rspa.2017.0551}
}
</code></pre></details>
<details><summary>[Moll et al] <b><a href=https://iopscience.iop.org/article/10.1088/2058-9565/aab822/pdf> Quantum optimization using variational algorithms on near-term quantum devices. </a></b> <i>Quantum Science and Technology.</i> <code><b>[Variational Quantum Algorithms]</b></code> <code><b>[Quantum Optimization]</b></code> <code><b>[Quantum Machine Learning]</b></code> <code><b>[Quantum Information and Computing]</b></code> <code><i>(Gate Model)</i></code></summary><pre><code>@Article{moll2018quantum,
  Title                    = {Quantum optimization using variational algorithms on near-term quantum devices},
  Author                   = {Moll, Nikolaj and Barkoutsos, Panagiotis and Bishop, Lev S and Chow, Jerry M and Cross, Andrew and Egger, Daniel J and Filipp, Stefan and Fuhrer, Andreas and Gambetta, Jay M and Ganzhorn, Marc and others},
  Journal                  = {Quantum Science and Technology},
  Year                     = {2018},
  Number                   = {3},
  Pages                    = {030503},
  Volume                   = {3},
  Publisher                = {IOP Publishing},
  URL                      = {https://iopscience.iop.org/article/10.1088/2058-9565/aab822/meta}
}
</code></pre></details>


### 2014
<details><summary>[Farhi et al] <b><a href=https://arxiv.org/abs/1411.4028> A Quantum Approximate Optimization Algorithm. </a></b> <i>arXiv.</i> <code><b>[Variational Quantum Algorithms]</b></code> <code><b>[Parameterized Quantum Circuits]</b></code> <code><b>[Quantum Optimization]</b></code> <code><b>[Quantum Machine Learning]</b></code> <code><b>[]</b></code> <code><i>(Gate Model)</i></code></summary><pre><code>@Article{farhi2014quantum,
  Title                    = {A quantum approximate optimization algorithm},
  Author                   = {Farhi, Edward and Goldstone, Jeffrey and Gutmann, Sam},
  Journal                  = {arXiv preprint arXiv:1411.4028},
  Year                     = {2014},
  URL                      = {https://arxiv.org/abs/1411.4028}
}
</code></pre></details>


### 2010
<details><summary>[Altshuler et al] <b><a href=https://www.pnas.org/content/107/28/12446> Anderson localization makes adiabatic quantum optimization fail. </a></b> <i>PNAS.</i> <code><b>[Adiabatic Quantum Computing]</b></code> <code><b>[Quantum Optimization]</b></code> <code><i>(Adiabatic Model)</i></code></summary><pre><code>@Article{altshuler2010anderson,
  Title                    = {Anderson localization makes adiabatic quantum optimization fail},
  Author                   = {Altshuler, Boris and Krovi, Hari and Roland, J{\'e}r{\'e}mie},
  Journal                  = {Proceedings of the National Academy of Sciences},
  Year                     = {2010},
  Number                   = {28},
  Pages                    = {12446--12450},
  Volume                   = {107},
  Publisher                = {National Acad Sciences},
  URL                      = {https://www.pnas.org/content/107/28/12446}
}
</code></pre></details>


### 1989
<details><summary>[Apolloni et al] <b><a href=https://www.sciencedirect.com/science/article/pii/0304414989900409?via%3Dihub> Quantum stochastic optimization. </a></b> <i>Stochastic Processes and their Applications.</i> <code><b>[Adiabatic Quantum Computing]</b></code> <code><b>[Quantum Annealing]</b></code> <code><b>[Quantum Optimization]</b></code> <code><i>(Adiabatic Model)</i></code></summary><pre><code>@Article{apolloni1989quantum,
  Title                    = {Quantum stochastic optimization},
  Author                   = {Apolloni, Bruno and Carvalho, C and De Falco, Diego},
  Journal                  = {Stochastic Processes and their Applications},
  Year                     = {1989},
  Number                   = {2},
  Pages                    = {233--244},
  Volume                   = {33},
  Publisher                = {Elsevier},
  URL                      = {https://www.sciencedirect.com/science/article/pii/0304414989900409?via%3Dihub}
}
</code></pre></details>


# Quantum Perceptron


### 2021
<details><summary>[Ban et al] <b><a href=https://www.Nature.com/articles/s41598-021-85208-3> Speeding up quantum perceptron via shortcuts to adiabaticity. </a></b> <i>Scientific reports.</i> <code><b>[Quantum Perceptron]</b></code> <code><b>[Quantum Machine Learning]</b></code> <code><i>(Adiabatic Model)</i></code></summary><pre><code>@Article{ban2021speeding,
  Title                    = {Speeding up quantum perceptron via shortcuts to adiabaticity},
  Author                   = {Ban, Yue and Chen, Xi and Torrontegui, E and Solano, Enrique and Casanova, Jorge},
  Journal                  = {Scientific reports},
  Year                     = {2021},
  Number                   = {1},
  Pages                    = {1--8},
  Volume                   = {11},
  Publisher                = {Nature Publishing Group},
  URL                      = {https://www.Nature.com/articles/s41598-021-85208-3}
}
</code></pre></details>


### 2020
<details><summary>[Beer et al] <b><a href=https://www.nature.com/articles/s41467-020-14454-2.pdf> Training deep quantum neural networks. </a></b> <i>Nature communications.</i> <code><b>[Quantum Neural Networks]</b></code> <code><b>[Quantum Perceptron]</b></code> <code><b>[Quantum Machine Learning]</b></code> <code><i>(Gate Model)</i></code></summary><pre><code>@Article{beer2020training,
  Title                    = {Training deep quantum neural networks},
  Author                   = {Beer, Kerstin and Bondarenko, Dmytro and Farrelly, Terry and Osborne, Tobias J and Salzmann, Robert and Scheiermann, Daniel and Wolf, Ramona},
  Journal                  = {Nature communications},
  Year                     = {2020},
  Number                   = {1},
  Pages                    = {1--6},
  Volume                   = {11},
  Publisher                = {Nature Publishing Group},
  URL                      = {https://www.nature.com/articles/s41467-020-14454-2.pdf}
}
</code></pre></details>
<details><summary>[Macaluso et al] <b><a href=https://link.springer.com/content/pdf/10.1007%2F978-3-030-50433-5_45.pdf> A Variational Algorithm for QuantumNeural Networks. </a></b> <i>International Conference on Computational Science.</i> <code><b>[Quantum Neural Networks]</b></code> <code><b>[Quantum Perceptron]</b></code> <code><b>[Quantum Machine Learning]</b></code> <code><i>(Gate Model)</i></code></summary><pre><code>@inproceedings{macaluso2020variational,
  title={A Variational Algorithm for Quantum Neural Networks},
  author={Macaluso, Antonio and Clissa, Luca and Lodi, Stefano and Sartori, Claudio},
  booktitle={International Conference on Computational Science},
  pages={591--604},
  year={2020},
  organization={Springer}
}</code></pre></details>


### 2019
<details><summary>[Liu et al] <b><a href=https://ieeexplore.ieee.org/document/8631025> A Unitary Weights Based One-Iteration Quantum Perceptron Algorithm for Non-Ideal Training Sets. </a></b> <i>IEEE Access.</i> <code><b>[Quantum Perceptron]</b></code> <code><b>[Quantum Machine Learning]</b></code> <code><i>(Gate Model)</i></code></summary><pre><code>@article{liu2019unitary,
  title={A unitary weights based one-iteration quantum perceptron algorithm for non-ideal training sets},
  author={Liu, Wenjie and Gao, Peipei and Wang, Yuxiang and Yu, Wenbin and Zhang, Maojun},
  journal={IEEE Access},
  volume={7},
  pages={36854--36865},
  year={2019},
  publisher={IEEE}
}</code></pre></details>
<details><summary>[Tacchino et al] <b><a href=https://www.Nature.com/articles/s41534-019-0140-4.pdf> An artificial neuron implemented on an actual quantumprocessor. </a></b> <i>Nature.</i> <code><b>[Quantum Perceptron]</b></code> <code><b>[Quantum Machine Learning]</b></code> <code><b>[Quantum Information and Computing]</b></code> <code><i>(Gate Model)</i></code></summary><pre><code>@article{tacchino2019artificial,
  title={An artificial neuron implemented on an actual quantum processor},
  author={Tacchino, Francesco and Macchiavello, Chiara and Gerace, Dario and Bajoni, Daniele},
  journal={npj Quantum Information},
  volume={5},
  number={1},
  pages={1--8},
  year={2019},
  publisher={Nature Publishing Group}
}</code></pre></details>
<details><summary>[Torrontegui et al] <b><a href=https://iopscience.iop.org/article/10.1209/0295-5075/125/30004/pdf> Unitary quantum perceptron as efficient universal approximator. </a></b> <i>Europhysics Letters.</i> <code><b>[Quantum Perceptron]</b></code> <code><b>[Quantum Machine Learning]</b></code> <code><b>[Quantum Information and Computing]</b></code> </summary><pre><code>@article{torrontegui2019unitary,
  title={Unitary quantum perceptron as efficient universal approximator},
  author={Torrontegui, Erik and Garc{\'\i}a-Ripoll, Juan Jos{\'e}},
  journal={EPL (Europhysics Letters)},
  volume={125},
  number={3},
  pages={30004},
  year={2019},
  publisher={IOP Publishing}
}</code></pre></details>
<details><summary>[Verdon et al] <b><a href=https://arxiv.org/abs/1907.05415> Learning to learn with quantum neural networks via classical neural networks. </a></b> <i>arXiv.</i> <code><b>[Quantum Perceptron]</b></code> <code><b>[Parameterized Quantum Circuits]</b></code> <code><b>[Quantum Neural Networks]</b></code> <code><b>[Quantum Machine Learning]</b></code> <code><b>[Variational Quantum Algorithms]</b></code> </summary><pre><code>@article{verdon2019learning,
  title={Learning to learn with quantum neural networks via classical neural networks},
  author={Verdon, Guillaume and Broughton, Michael and McClean, Jarrod R and Sung, Kevin J and Babbush, Ryan and Jiang, Zhang and Neven, Hartmut and Mohseni, Masoud},
  journal={arXiv preprint arXiv:1907.05415},
  year={2019}
}</code></pre></details>
<details><summary>[Wiersema et al] <b><a href=https://journals.aps.org/pra/abstract/10.1103/PhysRevA.100.020301> Implementing perceptron models with qubits. </a></b> <i>Physical Review A.</i> <code><b>[Quantum Perceptron]</b></code> <code><b>[Quantum Machine Learning]</b></code> <code><b>[Quantum Information and Computing]</b></code> <code><i>(Gate Model)</i></code></summary><pre><code>@article{wiersema2019implementing,
  title={Implementing perceptron models with qubits},
  author={Wiersema, RC and Kappen, HJ},
  journal={Physical Review A},
  volume={100},
  number={2},
  pages={020301},
  year={2019},
  publisher={APS}
}</code></pre></details>


### 2018
<details><summary>[Du et al] <b><a href=https://arxiv.org/abs/1809.06056> Implementable quantum classifier for nonlinear data. </a></b> <i>arXiv.</i> <code><b>[Quantum Perceptron]</b></code> <code><b>[Quantum Machine Learning]</b></code> <code><b>[Quantum Classification]</b></code> <code><b>[Quantum Machine Learning]</b></code> <code><i>(Gate Model)</i></code></summary><pre><code>@Article{du2018implementable,
  Title                    = {Implementable quantum classifier for nonlinear data},
  Author                   = {Du, Yuxuan and Hsieh, Min-Hsiu and Liu, Tongliang and Tao, Dacheng},
  Journal                  = {arXiv preprint arXiv:1809.06056},
  Year                     = {2018},
  URL                      = {https://arxiv.org/abs/1809.06056}
}
</code></pre></details>
<details><summary>[Hu Wei] <b><a href=https://www.scirp.org/pdf/NS_2018031517004350.pdf> Towards a real quantum neuron. </a></b> <i>Natural Science.</i> <code><b>[Quantum Perceptron]</b></code> <code><b>[Quantum Machine Learning]</b></code> <code><i>(Gate Model)</i></code></summary><pre><code>@article{hu2018towards,
  title={Towards a real quantum neuron},
  author={Hu, Wei},
  journal={Natural Science},
  volume={10},
  number={3},
  pages={99--109},
  year={2018},
  publisher={Scientific Research Publishing}
}</code></pre></details>
<details><summary>[Wiebe et al] <b><a href=https://papers.nips.cc/paper/2016/file/d47268e9db2e9aa3827bba3afb7ff94a-Paper.pdf> Quantum Perceptron Models. </a></b> <i>NIPS .</i> <code><b>[Quantum Perceptron]</b></code> <code><b>[Quantum Machine Learning]</b></code> <code><b>[Quantum Information and Computing]</b></code> <code><i>(Gate Model)</i></code></summary><pre><code>@article{wiebe2016quantum,
  title={Quantum perceptron models},
  author={Wiebe, Nathan and Kapoor, Ashish and Svore, Krysta M},
  journal={arXiv preprint arXiv:1602.04799},
  year={2016}
}</code></pre></details>


### 2017
<details><summary>[Cao et al] <b><a href=https://arxiv.org/pdf/1711.11240.pdf> Quantum Neuron:  an elementary building block for machine learningon quantum computers. </a></b> <i>arXiv.</i> <code><b>[Quantum Perceptron]</b></code> <code><b>[Quantum Information and Computing]</b></code> <code><b>[Quantum Neural Networks]</b></code> <code><b>[Quantum Machine Learning]</b></code> <code><i>(Gate Model)</i></code></summary><pre><code>@Article{cao2017quantum,
  Title                    = {Quantum neuron: an elementary building block for machine learning on quantum computers},
  Author                   = {Cao, Yudong and Guerreschi, Gian Giacomo and Aspuru-Guzik, Al{\'a}n},
  Journal                  = {arXiv preprint arXiv:1711.11240},
  Year                     = {2017},
  URL                      = {https://arxiv.org/pdf/1711.11240.pdf}
}
</code></pre></details>


### 2015
<details><summary>[Schuld et al] <b><a href=https://doi.org/10.1016/j.physleta.2014.11.061> Simulating a perceptron on a quantum computer. </a></b> <i>Physics Letters A.</i> <code><b>[Quantum Perceptron]</b></code> <code><b>[Quantum Machine Learning]</b></code> </summary><pre><code>@article{schuld2015simulating,
  title={Simulating a perceptron on a quantum computer},
  author={Schuld, Maria and Sinayskiy, Ilya and Petruccione, Francesco},
  journal={Physics Letters A},
  volume={379},
  number={7},
  pages={660--663},
  year={2015},
  publisher={Elsevier}
}</code></pre></details>


### 2001
<details><summary>[Altaisky] <b><a href=https://arxiv.org/abs/quant-ph/0107012> Quantum neural network. </a></b> <i>arXiv.</i> <code><b>[Quantum Neural Networks]</b></code> <code><b>[Quantum Perceptron]</b></code> <code><b>[Quantum Machine Learning]</b></code> </summary><pre><code>@Article{altaisky2001quantum,
  Title                    = {Quantum neural network},
  Author                   = {Altaisky, MV},
  Journal                  = {arXiv preprint quant-ph/0107012},
  Year                     = {2001},
  URL                      = {https://arxiv.org/abs/quant-ph/0107012}
}
</code></pre></details>


# Quantum Speedup-Advantage-Supremacy


### 2021
<details><summary>[Liu et al] <b><a href=https://www.nature.com/articles/s41567-021-01287-z> A rigorous and robust quantum speed-up in supervised machine learning. </a></b> <i>Nature Physics.</i> <code><b>[Quantum Machine Learning]</b></code> <code><b>[Quantum Classification]</b></code> <code><b>[Quantum Speedup-Advantage-Supremacy]</b></code> </summary><pre><code>@article{liu2021rigorous,
  title={A rigorous and robust quantum speed-up in supervised machine learning},
  author={Liu, Yunchao and Arunachalam, Srinivasan and Temme, Kristan},
  journal={Nature Physics},
  pages={1--5},
  year={2021},
  publisher={Nature Publishing Group}
}</code></pre></details>


### 2020
<details><summary>[Duan & Guo] <b><a href=https://www.sciencedirect.com/science/article/pii/S037596012030462X> A survey on HHL algorithm: From theory to application in quantum machine learning. </a></b> <i>Physics Letters A.</i> <code><b>[Quantum Machine Learning]</b></code> <code><b>[Quantum Speedup-Advantage-Supremacy]</b></code> </summary><pre><code>@Article{duan1998probabilistic,
  Title                    = {Probabilistic cloning and identification of linearly independent quantum states},
  Author                   = {Duan, Lu-Ming and Guo, Guang-Can},
  Journal                  = {Physical review letters},
  Year                     = {1998},
  Number                   = {22},
  Pages                    = {4999},
  Volume                   = {80},
  Publisher                = {APS},
  URL                      = {https://www.sciencedirect.com/science/article/pii/S037596012030462X}
}
</code></pre></details>


### 2019
<details><summary>[Arute et al] <b><a href=https://www.Nature.com/articles/s41586-019-1666-5> Quantum supremacy using a programmable superconducting processor. </a></b> <i>Nature.</i> <code><b>[Quantum Speedup-Advantage-Supremacy]</b></code> <code><b>[Superconducting Qubits]</b></code> <code><i>(Gate Model)</i></code></summary><pre><code>@Article{arute2019quantum,
  Title                    = {Quantum supremacy using a programmable superconducting processor},
  Author                   = {Arute, Frank and Arya, Kunal and Babbush, Ryan and Bacon, Dave and Bardin, Joseph C and Barends, Rami and Biswas, Rupak and Boixo, Sergio and Brandao, Fernando GSL and Buell, David A and others},
  Journal                  = {Nature},
  Year                     = {2019},
  Number                   = {7779},
  Pages                    = {505--510},
  Volume                   = {574},
  Publisher                = {Nature Publishing Group},
  URL                      = {https://www.Nature.com/articles/s41586-019-1666-5}
}
</code></pre></details>
<details><summary>[Brandão et al] <b><a href=https://arxiv.org/abs/1710.02581> Quantum SDP Solvers: Large Speed-ups, Optimality, and Applications to Quantum Learning. </a></b> <i>arXiv.</i> <code><b>[Quantum Speedup-Advantage-Supremacy]</b></code> <code><b>[Semidefinite programming]</b></code> </summary><pre><code>@Article{brandao2019QuantumSDP,
  Title                    = {Quantum SDP Solvers: Large Speed-ups, Optimality, and Applications to Quantum Learning},
  Author                   = {Brand{\~a}o, Fernando GSL and Kalev, Amir and Li, Tongyang and Lin, Cedric Yen-Yu and Svore, Krysta M and Wu, Xiaodi},
  Journal                  = {arXiv preprint arXiv:1710.02581},
  Year                     = {2019},
  Primaryclass             = {quant-ph},
  URL                      = {https://arxiv.org/abs/1710.02581}
}
</code></pre></details>


### 2018
<details><summary>[Boixo et al] <b><a href=https://www.nature.com/articles/s41567-018-0124-x> Characterizing quantum supremacy in near-term devices. </a></b> <i>Nature Physics.</i> <code><b>[Quantum Speedup-Advantage-Supremacy]</b></code> </summary><pre><code>@Article{boixo2018characterizing,
  Title                    = {Characterizing quantum supremacy in near-term devices},
  Author                   = {Boixo, Sergio and Isakov, Sergei V and Smelyanskiy, Vadim N and Babbush, Ryan and Ding, Nan and Jiang, Zhang and Bremner, Michael J and Martinis, John M and Neven, Hartmut},
  Journal                  = {Nature Physics},
  Year                     = {2018},
  Number                   = {6},
  Pages                    = {595--600},
  Volume                   = {14},
  Publisher                = {Nature Publishing Group},
  URL                      = {https://www.nature.com/articles/s41567-018-0124-x}
}
</code></pre></details>


### 2017
<details><summary>[Bremner et al] <b><a href=https://doi.org/10.22331/q-2017-04-25-8> Achieving quantum supremacy with sparse and noisy commuting quantum computations. </a></b> <i>Quantum.</i> <code><b>[Quantum Speedup-Advantage-Supremacy]</b></code> <code><i>(Gate Model)</i></code> <code><i>(Adiabatic Model)</i></code></summary><pre><code>@Article{bremner2017achieving,
  Title                    = {Achieving quantum supremacy with sparse and noisy commuting quantum computations},
  Author                   = {Bremner, Michael J and Montanaro, Ashley and Shepherd, Dan J},
  Journal                  = {Quantum},
  Year                     = {2017},
  Pages                    = {8},
  Volume                   = {1},
  DOI                      = {https://doi.org/10.22331/q-2017-04-25-8},
  Publisher                = {Verein zur F{\"o}rderung des Open Access Publizierens in den Quantenwissenschaften}
}
</code></pre></details>
<details><summary>[Mandra' et al] <b><a href=https://iopscience.iop.org/article/10.1088/2058-9565/aa7877> The pitfalls of planar spin-glass benchmarks: raising the bar for quantum annealers (again). </a></b> <i>Quantum Science and Technology.</i> <code><b>[Quantum Annealing]</b></code> <code><b>[Quantum Speedup-Advantage-Supremacy]</b></code> <code><i>(Adiabatic Model)</i></code></summary><pre><code>@article{mandra2017pitfalls,
  title={The pitfalls of planar spin-glass benchmarks: raising the bar for quantum annealers (again)},
  author={Mandra, Salvatore and Katzgraber, Helmut G and Thomas, Creighton},
  journal={Quantum Science and Technology},
  volume={2},
  number={3},
  pages={038501},
  year={2017},
  publisher={IOP Publishing}
}</code></pre></details>


### 2016
<details><summary>[Batle et al] <b><a href=https://link.springer.com/article/10.1007/s11128-016-1324-x> Do multipartite correlations speed up adiabatic quantum computation or quantum annealing?. </a></b> <i>Quantum Information Processing.</i> <code><b>[Adiabatic Quantum Computing]</b></code> <code><b>[Quantum Annealing]</b></code> <code><b>[Quantum Speedup-Advantage-Supremacy]</b></code> <code><i>(Adiabatic Model)</i></code></summary><pre><code>@Article{batle2016multipartite,
  Title                    = {Do multipartite correlations speed up adiabatic quantum computation or quantum annealing?},
  Author                   = {Batle, Josep and Ooi, CH Raymond and Farouk, Ahmed and Abutalib, M and Abdalla, S},
  Journal                  = {Quantum Information Processing},
  Year                     = {2016},
  Number                   = {8},
  Pages                    = {3081--3099},
  Volume                   = {15},
  Publisher                = {Springer},
  URL                      = {https://link.springer.com/article/10.1007/s11128-016-1324-x}
}
</code></pre></details>
<details><summary>[Mandra' et al] <b><a href=https://journals.aps.org/pra/abstract/10.1103/PhysRevA.94.022337> Strengths and weaknesses of weak-strong cluster problems: A detailed overview of state-of-the-art classical heuristics versus quantum approaches. </a></b> <i>Physical Review A.</i> <code><b>[Quantum Annealing]</b></code> <code><b>[Quantum Speedup-Advantage-Supremacy]</b></code> <code><i>(Adiabatic Model)</i></code></summary><pre><code>@article{mandra2016strengths,
  title={Strengths and weaknesses of weak-strong cluster problems: A detailed overview of state-of-the-art classical heuristics versus quantum approaches},
  author={Mandra, Salvatore and Zhu, Zheng and Wang, Wenlong and Perdomo-Ortiz, Alejandro and Katzgraber, Helmut G},
  journal={Physical Review A},
  volume={94},
  number={2},
  pages={022337},
  year={2016},
  publisher={APS}
}</code></pre></details>


### 2015
<details><summary>[Aaronson ] <b><a href=https://www.nature.com/articles/nphys3272> Quantum Machine Learning Algorithms: Read the Fine Print. </a></b> <i>Nature Physics.</i> <code><b>[Quantum Speedup-Advantage-Supremacy]</b></code> </summary><pre><code>@Article{aaronson2015read,
  Title                    = {Read the fine print},
  Author                   = {Aaronson, Scott},
  Journal                  = {Nature Physics},
  Year                     = {2015},
  Number                   = {4},
  Pages                    = {291--293},
  Volume                   = {11},
  Publisher                = {Nature Publishing Group},
  URL                      = {https://www.nature.com/articles/nphys3272}
}
</code></pre></details>
<details><summary>[Itay ert al] <b><a href=https://journals.aps.org/pra/abstract/10.1103/PhysRevA.92.042325> Probing for quantum speedup in spin-glass problems with planted solutions. </a></b> <i>Physical Review A.</i> <code><b>[Quantum Annealing]</b></code> <code><b>[Quantum Speedup-Advantage-Supremacy]</b></code> <code><i>(Adiabatic Model)</i></code></summary><pre><code>@article{hen2015probing,
  title={Probing for quantum speedup in spin-glass problems with planted solutions},
  author={Hen, Itay and Job, Joshua and Albash, Tameem and R{\o}nnow, Troels F and Troyer, Matthias and Lidar, Daniel A},
  journal={Physical Review A},
  volume={92},
  number={4},
  pages={042325},
  year={2015},
  publisher={APS}
}</code></pre></details>
<details><summary>[Katzgraber et al] <b><a href=https://journals.aps.org/prx/abstract/10.1103/PhysRevX.5.031026> Seeking Quantum Speedup Through Spin Glasses: The Good, the Bad, and the Ugly. </a></b> <i>Physical Review X.</i> <code><b>[Quantum Annealing]</b></code> <code><b>[Quantum Speedup-Advantage-Supremacy]</b></code> <code><i>(Adiabatic Model)</i></code></summary><pre><code>@article{katzgraber2015seeking,
  title={Seeking quantum speedup through spin glasses: The good, the bad, and the ugly},
  author={Katzgraber, Helmut G and Hamze, Firas and Zhu, Zheng and Ochoa, Andrew J and Munoz-Bauza, Humberto},
  journal={Physical Review X},
  volume={5},
  number={3},
  pages={031026},
  year={2015},
  publisher={APS}
}</code></pre></details>


### 2014
<details><summary>[Rønnow et al] <b><a href=https://arxiv.org/pdf/1401.2910.pdf> Defining and detecting quantum speedup. </a></b> <i>Science.</i> <code><b>[Quantum Information and Computing]</b></code> <code><b>[Quantum Speedup-Advantage-Supremacy]</b></code> <code><i>(Adiabatic Model)</i></code></summary><pre><code>@article{ronnow2014defining,
  title={Defining and detecting quantum speedup},
  author={R{\o}nnow, Troels F and Wang, Zhihui and Job, Joshua and Boixo, Sergio and Isakov, Sergei V and Wecker, David and Martinis, John M and Lidar, Daniel A and Troyer, Matthias},
  journal={science},
  volume={345},
  number={6195},
  pages={420--424},
  year={2014},
  publisher={American Association for the Advancement of Science}
}</code></pre></details>


### 2011
<details><summary>[Aaronson and Arkhipov] <b><a href=https://dl.acm.org/doi/pdf/10.1145/1993636.1993682> The computational complexity of linear optics. </a></b> <i>Annual ACM symposium on Theory of computing.</i> <code><b>[Quantum Speedup-Advantage-Supremacy]</b></code> <code><b>[Complexity Theory]</b></code> </summary><pre><code>@InProceedings{aaronson2011computational,
  Title                    = {The computational complexity of linear optics},
  Author                   = {Aaronson, Scott and Arkhipov, Alex},
  Booktitle                = {Proceedings of the forty-third annual ACM symposium on Theory of computing},
  Year                     = {2011},
  Pages                    = {333--342},
  Publisher                = {Association for Computing Machinery},
  URL                      = {https://dl.acm.org/doi/pdf/10.1145/1993636.1993682}
}
</code></pre></details>
<details><summary>[Bremner et al] <b><a href=https://doi.org/10.1098/rspa.2010.0301> Classical simulation of commuting quantum computations implies collapse of the polynomial hierarchy. </a></b> <i>Proceedings of the Royal Society A: Mathematical, Physical and Engineering Sciences.</i> <code><b>[Quantum Speedup-Advantage-Supremacy]</b></code> <code><b>[Quantum Information and Computing]</b></code> <code><i>(Gate Model)</i></code></summary><pre><code>@Article{bremner2011classical,
  Title                    = {Classical simulation of commuting quantum computations implies collapse of the polynomial hierarchy},
  Author                   = {Bremner, Michael J and Jozsa, Richard and Shepherd, Dan J},
  Journal                  = {Proceedings of the Royal Society A: Mathematical, Physical and Engineering Sciences},
  Year                     = {2011},
  Number                   = {2126},
  Pages                    = {459--472},
  Volume                   = {467},
  Publisher                = {The Royal Society Publishing},
  URL                      = {https://doi.org/10.1098/rspa.2010.0301}
}
</code></pre></details>


### 2009
<details><summary>[Harrow et al] <b><a href=https://journals.aps.org/prl/abstract/10.1103/PhysRevLett.103.150502> Quantum Algorithm for Linear Systems of Equations. </a></b> <i>Physical Review Letters.</i> <code><b>[Quantum Machine Learning]</b></code> <code><b>[Quantum Speedup-Advantage-Supremacy]</b></code> </summary><pre><code>@Article{harrow2009quantum,
  Title                    = {Quantum algorithm for linear systems of equations},
  Author                   = {Harrow, Aram W and Hassidim, Avinatan and Lloyd, Seth},
  Journal                  = {Physical review letters},
  Year                     = {2009},
  Number                   = {15},
  Pages                    = {150502},
  Volume                   = {103},
  Publisher                = {APS},
  URL                      = {https://journals.aps.org/prl/abstract/10.1103/PhysRevLett.103.150502}
}
</code></pre></details>


### 1999
<details><summary>[Abrams &Lloyd] <b><a href=https://journals.aps.org/prl/abstract/10.1103/PhysRevLett.83.5162> Quantum algorithm providing exponential speed increase for finding eigenvalues and eigenvectors. </a></b> <i>Physical Review Letters.</i> <code><b>[Quantum Speedup-Advantage-Supremacy]</b></code> <code><i>(Gate Model)</i></code></summary><pre><code>@Article{abrams1999quantum,
  Title                    = {Quantum algorithm providing exponential speed increase for finding eigenvalues and eigenvectors},
  Author                   = {Abrams, Daniel S and Lloyd, Seth},
  Journal                  = {Physical review letters},
  Year                     = {1999},
  Number                   = {24},
  Pages                    = {5162},
  Volume                   = {83},
  Publisher                = {APS},
  URL                      = {https://journals.aps.org/prl/abstract/10.1103/PhysRevLett.83.5162}
}
</code></pre></details>


### 1997
<details><summary>[Bernstein & Vazirani] <b><a href=https://epubs.siam.org/doi/abs/10.1137/S0097539796300921> Quantum complexity theory. </a></b> <i>SIAM Journal on computing.</i> <code><b>[Quantum Information and Computing]</b></code> <code><b>[Quantum Speedup-Advantage-Supremacy]</b></code> </summary><pre><code>@article{bernstein1997quantum,
  title={Quantum complexity theory},
  author={Bernstein, Ethan and Vazirani, Umesh},
  journal={SIAM Journal on computing},
  volume={26},
  number={5},
  pages={1411--1473},
  year={1997},
  publisher={SIAM}
}</code></pre></details>
<details><summary>[Simon] <b><a href=https://epubs.siam.org/doi/10.1137/S0097539796298637> On the Power of Quantum Computation. </a></b> <i>SIAM Journal on Computing.</i> <code><b>[Quantum Information and Computing]</b></code> <code><b>[Quantum Speedup-Advantage-Supremacy]</b></code> </summary><pre><code>@article{simon1997power,
  title={On the power of quantum computation},
  author={Simon, Daniel R},
  journal={SIAM journal on computing},
  volume={26},
  number={5},
  pages={1474--1483},
  year={1997},
  publisher={SIAM}
}</code></pre></details>


### 1992
<details><summary>[Deutsch & Josza] <b><a href=https://royalsocietypublishing.org/doi/10.1098/rspa.1992.0167> Rapid solution of problems by quantum computation. </a></b> <i>Proceedings of the Royal Society A.</i> <code><b>[Quantum Information and Computing]</b></code> <code><b>[Quantum Speedup-Advantage-Supremacy]</b></code> </summary><pre><code>@Article{deutsch1992rapid,
  Title                    = {Rapid solution of problems by quantum computation},
  Author                   = {Deutsch, David and Jozsa, Richard},
  Journal                  = {Proceedings of the Royal Society of London. Series A: Mathematical and Physical Sciences},
  Year                     = {1992},
  Number                   = {1907},
  Pages                    = {553--558},
  Volume                   = {439},
  Publisher                = {The Royal Society London},
  URL                      = {https://royalsocietypublishing.org/doi/10.1098/rspa.1992.0167}
}
</code></pre></details>


# Simulated Annealing


### 2005
<details><summary>[Battaglia et al] <b><a href=https://link.springer.com/chapter/10.1007%2F11526216_7> Deterministic and stochastic quantum annealing approaches. </a></b> <i>Lecture Notes in Physics .</i> <code><b>[Adiabatic Quantum Computing]</b></code> <code><b>[Quantum Annealing]</b></code> <code><b>[Simulated Annealing]</b></code> <code><i>(Adiabatic Model)</i></code></summary><pre><code>@InCollection{battaglia2005deterministic,
  Title                    = {Deterministic and stochastic quantum annealing approaches},
  Author                   = {Battaglia, Demian and Stella, Lorenzo and Zagordi, Osvaldo and Santoro, Giuseppe E and Tosatti, Erio},
  Booktitle                = {Quantum Annealing and Other Optimization Methods},
  Publisher                = {Springer},
  Year                     = {2005},
  Pages                    = {171--206},
  URL                      = {https://link.springer.com/chapter/10.1007%2F11526216_7}
}
</code></pre></details>
<details><summary>[Battaglia et al] <b><a href=https://journals.aps.org/pre/abstract/10.1103/PhysRevE.71.066707> Optimization by Quantum Annealing: Lessons from hard 3-SAT cases,. </a></b> <i>Physical Review E.</i> <code><b>[Adiabatic Quantum Computing]</b></code> <code><b>[Quantum Annealing]</b></code> <code><b>[Simulated Annealing]</b></code> <code><b>[Quantum Monte Carlo]</b></code> <code><i>(Adiabatic Model)</i></code></summary><pre><code>@Article{battaglia2005optimization,
  Title                    = {Optimization by quantum annealing: Lessons from hard satisfiability problems},
  Author                   = {Battaglia, Demian A and Santoro, Giuseppe E and Tosatti, Erio},
  Journal                  = {Physical Review E},
  Year                     = {2005},
  Number                   = {6},
  Pages                    = {066707},
  Volume                   = {71},
  Publisher                = {APS},
  URL                      = {https://journals.aps.org/pre/abstract/10.1103/PhysRevE.71.066707}
}
</code></pre></details>


### 1983
<details><summary>[Kirkpatrick et al] <b><a href=http://wexler.free.fr/library/files/kirkpatrick%20(1983)%20optimization%20by%20simulated%20annealing.pdf> Optimization by Simulated Annealing. </a></b> <i>Science.</i> <code><b>[Simulated Annealing]</b></code> </summary><pre><code>@article{kirkpatrick1983optimization,
  title={Optimization by simulated annealing},
  author={Kirkpatrick, Scott and Gelatt, C Daniel and Vecchi, Mario P},
  journal={science},
  volume={220},
  number={4598},
  pages={671--680},
  year={1983},
  publisher={American association for the advancement of science}
}</code></pre></details>


# Superconducting Qubits


### 2020
<details><summary>[Kjaergaard et al] <b><a href=https://www.annualreviews.org/doi/pdf/10.1146/annurev-conmatphys-031119-050605> Superconducting Qubits:
Current State of Play. </a></b> <i>Annual review of condensed matter physics.</i> <code><b>[Quantum Information and Computing]</b></code> <code><b>[Physical Realization of Qubits]</b></code> <code><b>[Superconducting Qubits]</b></code> </summary><pre><code>@article{kjaergaard2020superconducting,
  title={Superconducting qubits: Current state of play},
  author={Kjaergaard, Morten and Schwartz, Mollie E and Braum{\"u}ller, Jochen and Krantz, Philip and Wang, Joel I-J and Gustavsson, Simon and Oliver, William D},
  journal={Annual Review of Condensed Matter Physics},
  volume={11},
  pages={369--395},
  year={2020},
  publisher={Annual Reviews}
}</code></pre></details>


### 2019
<details><summary>[Arute et al] <b><a href=https://www.Nature.com/articles/s41586-019-1666-5> Quantum supremacy using a programmable superconducting processor. </a></b> <i>Nature.</i> <code><b>[Quantum Speedup-Advantage-Supremacy]</b></code> <code><b>[Superconducting Qubits]</b></code> <code><i>(Gate Model)</i></code></summary><pre><code>@Article{arute2019quantum,
  Title                    = {Quantum supremacy using a programmable superconducting processor},
  Author                   = {Arute, Frank and Arya, Kunal and Babbush, Ryan and Bacon, Dave and Bardin, Joseph C and Barends, Rami and Biswas, Rupak and Boixo, Sergio and Brandao, Fernando GSL and Buell, David A and others},
  Journal                  = {Nature},
  Year                     = {2019},
  Number                   = {7779},
  Pages                    = {505--510},
  Volume                   = {574},
  Publisher                = {Nature Publishing Group},
  URL                      = {https://www.Nature.com/articles/s41586-019-1666-5}
}
</code></pre></details>
<details><summary>[Krantz et al] <b><a href=https://aip.scitation.org/doi/full/10.1063/1.5089550> A quantum engineer's guide to superconducting qubits. </a></b> <i>Applied Physics Reviews .</i> <code><b>[Physical Realization of Qubits]</b></code> <code><b>[Superconducting Qubits]</b></code> </summary><pre><code>@article{krantz2019quantum,
  title={A quantum engineer's guide to superconducting qubits},
  author={Krantz, Philip and Kjaergaard, Morten and Yan, Fei and Orlando, Terry P and Gustavsson, Simon and Oliver, William D},
  journal={Applied Physics Reviews},
  volume={6},
  number={2},
  pages={021318},
  year={2019},
  publisher={AIP Publishing LLC}
}</code></pre></details>


### 2017
<details><summary>[Gambetta et al] <b><a href=https://www.nature.com/articles/s41534-016-0004-0> Building logical qubits in a superconducting quantum computing system. </a></b> <i>Physical Review Letters.</i> <code><b>[Physical Realization of Qubits]</b></code> <code><b>[Superconducting Qubits]</b></code> </summary><pre><code>@Article{gambetta2017building,
  Title                    = {Building logical qubits in a superconducting quantum computing system},
  Author                   = {Gambetta, Jay M and Chow, Jerry M and Steffen, Matthias},
  Journal                  = {npj Quantum Information},
  Year                     = {2017},
  Number                   = {1},
  Pages                    = {1--7},
  Volume                   = {3},
  Publisher                = {Nature Publishing Group},
  URL                      = {https://www.nature.com/articles/s41534-016-0004-0}
}
</code></pre></details>


### 2014
<details><summary>[Bunyk et al] <b><a href=https://ieeexplore.ieee.org/document/6802426> Architectural considerations in the design of a superconducting quantum annealing processor. </a></b> <i>IEEE Transactions on Applied Superconductivity.</i> <code><b>[Quantum Annealing]</b></code> <code><b>[Quantum Information and Computing]</b></code> <code><b>[QPU]</b></code> <code><b>[Superconducting Qubits]</b></code> <code><i>(Adiabatic Model)</i></code></summary><pre><code>@Article{bunyk2014architectural,
  Title                    = {Architectural considerations in the design of a superconducting quantum annealing processor},
  Author                   = {Bunyk, Paul I and Hoskinson, Emile M and Johnson, Mark W and Tolkacheva, Elena and Altomare, Fabio and Berkley, Andrew J and Harris, Richard and Hilton, Jeremy P and Lanting, Trevor and Przybysz, Anthony J and others},
  Journal                  = {IEEE Transactions on Applied Superconductivity},
  Year                     = {2014},
  Number                   = {4},
  Pages                    = {1--10},
  Volume                   = {24},
  Publisher                = {IEEE},
  URL                      = {https://ieeexplore.ieee.org/document/6802426}
}
</code></pre></details>


### 2013
<details><summary>[Devoret & Schoelkopf] <b><a href=https://science.sciencemag.org/content/339/6124/1169.abstract> Superconducting Circuits for Quantum Information: An Outlook. </a></b> <i>Science.</i> <code><b>[Quantum Information and Computing]</b></code> <code><b>[Superconducting Qubits]</b></code> <code><i>(Gate Model)</i></code></summary><pre><code>@Article{devoret2013superconducting,
  Title                    = {Superconducting circuits for quantum information: an outlook},
  Author                   = {Devoret, Michel H and Schoelkopf, Robert J},
  Journal                  = {Science},
  Year                     = {2013},
  Number                   = {6124},
  Pages                    = {1169--1174},
  Volume                   = {339},
  Publisher                = {American Association for the Advancement of Science},
  URL                      = {https://science.sciencemag.org/content/339/6124/1169.abstract}
}
</code></pre></details>


### 2010
<details><summary>[Harris et al] <b><a href=https://journals.aps.org/prb/abstract/10.1103/PhysRevB.82.024511> Experimental investigation of an eight-qubit unit cell in a superconducting optimization processor. </a></b> <i>Physical Review B.</i> <code><b>[Quantum Annealing]</b></code> <code><b>[Superconducting Qubits]</b></code> <code><i>(Adiabatic Model)</i></code></summary><pre><code>@article{harris2010experimental,
  title={Experimental investigation of an eight-qubit unit cell in a superconducting optimization processor},
  author={Harris, Richard and Johnson, Mark W and Lanting, T and Berkley, AJ and Johansson, J and Bunyk, P and Tolkacheva, E and Ladizinsky, E and Ladizinsky, N and Oh, T and others},
  journal={Physical Review B},
  volume={82},
  number={2},
  pages={024511},
  year={2010},
  publisher={APS}
}</code></pre></details>


# Unsupervised Learning


### 2013
<details><summary>[Aïmeur et al] <b><a href=https://link.springer.com/article/10.1007/s10994-012-5316-5> Quantum speed-up for unsupervised learning. </a></b> <i>Machine Learning.</i> <code><b>[Quantum Machine Learning]</b></code> <code><b>[Unsupervised Learning]</b></code> <code><b>[Quantum Clustering]</b></code> <code><i>(Gate Model)</i></code></summary><pre><code>@Article{aimeur2013quantum,
  Title                    = {Quantum speed-up for unsupervised learning},
  Author                   = {A{\"\i}meur, Esma and Brassard, Gilles and Gambs, S{\'e}bastien},
  Journal                  = {Machine Learning},
  Year                     = {2013},
  Number                   = {2},
  Pages                    = {261--287},
  Volume                   = {90},
  Publisher                = {Springer},
  URL                      = {https://link.springer.com/article/10.1007/s10994-012-5316-5}
}
</code></pre></details>


### 2007
<details><summary>[Aïmeur et al] <b><a href=https://dl.acm.org/doi/abs/10.1145/1273496.1273497> Quantum Clustering Algorithms. </a></b> <i>ICML '07.</i> <code><b>[Quantum Machine Learning]</b></code> <code><b>[Unsupervised Learning]</b></code> <code><b>[Quantum Clustering]</b></code> <code><i>(Gate Model)</i></code></summary><pre><code>@inproceedings{aimeur2007quantum,
  title={Quantum clustering algorithms},
  author={A{\"\i}meur, Esma and Brassard, Gilles and Gambs, S{\'e}bastien},
  booktitle={Proceedings of the 24th international conference on machine learning},
  pages={1--8},
  year={2007}
}</code></pre></details>


### 2006
<details><summary>[Aïmeur et al] <b><a href=https://link.springer.com/chapter/10.1007/11766247_37> Machine Learning in a Quantum World. </a></b> <i>Canadian AI 2006.</i> <code><b>[Quantum Machine Learning]</b></code> <code><b>[Unsupervised Learning]</b></code> <code><b>[Quantum Clustering]</b></code> <code><i>(Gate Model)</i></code></summary><pre><code>@InProceedings{aimeur2006machine,
  Title                    = {Machine learning in a quantum world},
  Author                   = {A{\"\i}meur, Esma and Brassard, Gilles and Gambs, S{\'e}bastien},
  Booktitle                = {Conference of the Canadian Society for Computational Studies of Intelligence},
  Year                     = {2006},
  Organization             = {Springer},
  Pages                    = {431--442},
  URL                      = {https://link.springer.com/chapter/10.1007/11766247_37}
}
</code></pre></details>


# Variational Quantum Algorithms


### 2021
<details><summary>[Amaro et al.] <b><a href=https://arxiv.org/abs/2106.10055> Filtering variational quantum algorithms for combinatorial optimization. </a></b> <i>arXiv.</i> <code><b>[Variational Quantum Algorithms]</b></code> <code><b>[Quantum Optimization]</b></code> <code><b>[Parameterized Quantum Circuits]</b></code> </summary><pre><code>@article{amaro2021filtering,
  title={Filtering variational quantum algorithms for combinatorial optimization},
  author={Amaro, David and Modica, Carlo and Rosenkranz, Matthias and Fiorentini, Mattia and Benedetti, Marcello and Lubasch, Michael},
  journal={Quantum Science and Technology},
  year={2021},
  publisher={IOP Publishing}
}
</code></pre></details>
<details><summary>[Bharti et al] <b><a href=https://arxiv.org/abs/2101.08448> Noisy intermediate-scale quantum (NISQ) algorithms. </a></b> <i>arXiv.</i> <code><b>[Variational Quantum Algorithms]</b></code> <code><b>[Parameterized Quantum Circuits]</b></code> <code><b>[Quantum Optimization]</b></code> <code><b>[Quantum Machine Learning]</b></code> <code><i>(Gate Model)</i></code> <code><i>(Adiabatic Model)</i></code></summary><pre><code>@article{bharti2021noisy,
  title={Noisy intermediate-scale quantum (NISQ) algorithms},
  author={Bharti, Kishor and Cervera-Lierta, Alba and Kyaw, Thi Ha and Haug, Tobias and Alperin-Lea, Sumner and Anand, Abhinav and Degroote, Matthias and Heimonen, Hermanni and Kottmann, Jakob S and Menke, Tim and others},
  journal={arXiv preprint arXiv:2101.08448},
  year={2021}
}</code></pre></details>
<details><summary>[Mangini et al] <b><a href=https://iopscience.iop.org/article/10.1209/0295-5075/134/10002/meta> Quantum computing models for artificial neural networks. </a></b> <i>EPL (Europhysics Letters).</i> <code><b>[Quantum Neural Networks]</b></code> <code><b>[Parameterized Quantum Circuits]</b></code> <code><b>[Variational Quantum Algorithms]</b></code> <code><b>[Quantum Machine Learning]</b></code> </summary><pre><code>@article{mangini2021quantum,
  title={Quantum computing models for artificial neural networks},
  author={Mangini, Stefano and Tacchino, Francesco and Gerace, Dario and Bajoni, Daniele and Macchiavello, Chiara},
  journal={EPL (Europhysics Letters)},
  volume={134},
  number={1},
  pages={10002},
  year={2021},
  publisher={IOP Publishing}
}</code></pre></details>
<details><summary>[Plekhanov et al] <b><a href=https://arxiv.org/abs/2109.03687> Variational quantum amplitude estimation. </a></b> <i>arXiv.</i> <code><b>[Variational Quantum Algorithms]</b></code> <code><b>[Amplitude estimation]</b></code> <code><b>[Parameterized Quantum Circuits]</b></code> </summary><pre><code>@article{plekhanov2021variational,
  title={Variational quantum amplitude estimation},
  author={Plekhanov, Kirill and Rosenkranz, Matthias and Fiorentini, Mattia and Lubasch, Michael},
  journal={arXiv preprint arXiv:2109.03687},
  year={2021}
}
</code></pre></details>
<details><summary>[Schuld et al] <b><a href=https://journals.aps.org/pra/abstract/10.1103/PhysRevA.103.032430> Effect of data encoding on the expressive power of variational quantum-machine-learning models. </a></b> <i>Physical Review A.</i> <code><b>[Parameterized Quantum Circuits]</b></code> <code><b>[Variational Quantum Algorithms]</b></code> <code><b>[Quantum Machine Learning]</b></code> </summary><pre><code>@article{schuld2021effect,
  title={Effect of data encoding on the expressive power of variational quantum-machine-learning models},
  author={Schuld, Maria and Sweke, Ryan and Meyer, Johannes Jakob},
  journal={Physical Review A},
  volume={103},
  number={3},
  pages={032430},
  year={2021},
  publisher={APS}
}</code></pre></details>
<details><summary>[Volkoff & Coles] <b><a href=https://arxiv.org/abs/2005.12200> Large gradients via correlation in random parameterized quantum circuits. </a></b> <i>arXiv.</i> <code><b>[Barren Plateaus]</b></code> <code><b>[Parameterized Quantum Circuits]</b></code> <code><b>[Quantum Neural Networks]</b></code> <code><b>[Variational Quantum Algorithms]</b></code> <code><b>[Quantum Machine Learning]</b></code> </summary><pre><code>@article{volkoff2021large,
  title={Large gradients via correlation in random parameterized quantum circuits},
  author={Volkoff, Tyler and Coles, Patrick J},
  journal={Quantum Science and Technology},
  volume={6},
  number={2},
  pages={025008},
  year={2021},
  publisher={IOP Publishing}
}</code></pre></details>


### 2020
<details><summary>[Barkoutsos et al] <b><a href=https://quantum-journal.org/papers/q-2020-04-20-256/> Improving Variational Quantum Optimization Using CVaR. </a></b> <i>Quantum.</i> <code><b>[Variational Quantum Algorithms]</b></code> <code><b>[Quantum Optimization]</b></code> <code><b>[Quantum Machine Learning]</b></code> </summary><pre><code>@article{barkoutsos2020improving,
  title={Improving variational quantum optimization using cvar},
  author={Barkoutsos, Panagiotis Kl and Nannicini, Giacomo and Robert, Anton and Tavernelli, Ivano and Woerner, Stefan},
  journal={Quantum},
  volume={4},
  pages={256},
  year={2020},
  publisher={Verein zur F{\"o}rderung des Open Access Publizierens in den Quantenwissenschaften}
}</code></pre></details>
<details><summary>[Carolan et al ] <b><a href=https://www.nature.com/articles/s41567-019-0747-6> Variational quantum unsampling on a quantum photonic processor. </a></b> <i>Nature Physics.</i> <code><b>[Variational Quantum Algorithms]</b></code> <code><b>[Quantum Neural Networks]</b></code> </summary><pre><code>@article{carolan2020variational,
  title={Variational quantum unsampling on a quantum photonic processor},
  author={Carolan, Jacques and Mohseni, Masoud and Olson, Jonathan P and Prabhu, Mihika and Chen, Changchen and Bunandar, Darius and Niu, Murphy Yuezhen and Harris, Nicholas C and Wong, Franco NC and Hochberg, Michael and others},
  journal={Nature Physics},
  volume={16},
  number={3},
  pages={322--327},
  year={2020},
  publisher={Nature Publishing Group}
}</code></pre></details>
<details><summary>[Cerezo et al] <b><a href=https://arxiv.org/pdf/2012.09265.pdf> Variational Quantum Algorithms. </a></b> <i>arXiv.</i> <code><b>[Variational Quantum Algorithms]</b></code> <code><b>[Quantum Machine Learning]</b></code> <code><i>(Gate Model)</i></code> <code><i>(Adiabatic Model)</i></code></summary><pre><code>@Article{cerezo2020variational,
  Title                    = {Variational quantum algorithms},
  Author                   = {Cerezo, Marco and Arrasmith, Andrew and Babbush, Ryan and Benjamin, Simon C and Endo, Suguru and Fujii, Keisuke and McClean, Jarrod R and Mitarai, Kosuke and Yuan, Xiao and Cincio, Lukasz and others},
  Journal                  = {arXiv preprint arXiv:2012.09265},
  Year                     = {2020},
  URL                      = {https://arxiv.org/pdf/2012.09265.pdf}
}
</code></pre></details>
<details><summary>[Wang et al] <b><a href=https://arxiv.org/abs/2007.14384> Noise-Induced Barren Plateaus in Variational Quantum Algorithms. </a></b> <i>arXiv.</i> <code><b>[Barren Plateaus]</b></code> <code><b>[Parameterized Quantum Circuits]</b></code> <code><b>[Variational Quantum Algorithms]</b></code> <code><b>[Quantum Machine Learning]</b></code> </summary><pre><code>@article{wang2020noise,
  title={Noise-induced barren plateaus in variational quantum algorithms},
  author={Wang, Samson and Fontana, Enrico and Cerezo, Marco and Sharma, Kunal and Sone, Akira and Cincio, Lukasz and Coles, Patrick J},
  journal={arXiv preprint arXiv:2007.14384},
  year={2020}
}</code></pre></details>


### 2019
<details><summary>[Benedetti et al] <b><a href=https://iopscience.iop.org/article/10.1088/2058-9565/ab4eb5/meta> Parameterized quantum circuits as machine learning models. </a></b> <i>Quantum Science and Technology.</i> <code><b>[Variational Quantum Algorithms]</b></code> <code><b>[Quantum Machine Learning]</b></code> </summary><pre><code>@Article{benedetti2019parameterized,
  Title                    = {Parameterized quantum circuits as machine learning models},
  Author                   = {Benedetti, Marcello and Lloyd, Erika and Sack, Stefan and Fiorentini, Mattia},
  Journal                  = {Quantum Science and Technology},
  Year                     = {2019},
  Number                   = {4},
  Pages                    = {043001},
  Volume                   = {4},
  Publisher                = {IOP Publishing},
  URL                      = {https://iopscience.iop.org/article/10.1088/2058-9565/ab4eb5/meta}
}
</code></pre></details>
<details><summary>[Havlíček et al] <b><a href=https://www.nature.com/articles/s41586-019-0980-2> Supervised learning with quantum-enhanced feature spaces. </a></b> <i>Nature.</i> <code><b>[Variational Quantum Algorithms]</b></code> <code><b>[Quantum Support Vector Machine]</b></code> <code><b>[Quantum Classification]</b></code> <code><b>[Quantum Machine Learning]</b></code> <code><i>(Gate Model)</i></code></summary><pre><code>@Article{havlivcek2019supervised,
  Title                    = {Supervised learning with quantum-enhanced feature spaces},
  Author                   = {Havl{\'\i}{\v{c}}ek, Vojt{\v{e}}ch and C{\'o}rcoles, Antonio D and Temme, Kristan and Harrow, Aram W and Kandala, Abhinav and Chow, Jerry M and Gambetta, Jay M},
  Journal                  = {Nature},
  Year                     = {2019},
  Number                   = {7747},
  Pages                    = {209--212},
  Volume                   = {567},
  Publisher                = {Nature Publishing Group},
  URL                      = {https://www.nature.com/articles/s41586-019-0980-2}
}
</code></pre></details>
<details><summary>[Sim et al] <b><a href=https://onlinelibrary.wiley.com/doi/10.1002/qute.201900070> Expressibility and Entangling Capability of Parameterized Quantum Circuits for Hybrid Quantum-Classical Algorithms. </a></b> <i>Advanced Quantum Technologies.</i> <code><b>[Variational Quantum Algorithms]</b></code> <code><b>[Parameterized Quantum Circuits]</b></code> <code><i>(Gate Model)</i></code></summary><pre><code>@article{sim2019expressibility,
  title={Expressibility and entangling capability of parameterized quantum circuits for hybrid quantum-classical algorithms},
  author={Sim, Sukin and Johnson, Peter D and Aspuru-Guzik, Al{\'a}n},
  journal={Advanced Quantum Technologies},
  volume={2},
  number={12},
  pages={1900070},
  year={2019},
  publisher={Wiley Online Library}
}</code></pre></details>
<details><summary>[Verdon et al] <b><a href=https://arxiv.org/abs/1907.05415> Learning to learn with quantum neural networks via classical neural networks. </a></b> <i>arXiv.</i> <code><b>[Quantum Perceptron]</b></code> <code><b>[Parameterized Quantum Circuits]</b></code> <code><b>[Quantum Neural Networks]</b></code> <code><b>[Quantum Machine Learning]</b></code> <code><b>[Variational Quantum Algorithms]</b></code> </summary><pre><code>@article{verdon2019learning,
  title={Learning to learn with quantum neural networks via classical neural networks},
  author={Verdon, Guillaume and Broughton, Michael and McClean, Jarrod R and Sung, Kevin J and Babbush, Ryan and Jiang, Zhang and Neven, Hartmut and Mohseni, Masoud},
  journal={arXiv preprint arXiv:1907.05415},
  year={2019}
}</code></pre></details>
<details><summary>[Wang et al ] <b><a href=https://journals.aps.org/prl/abstract/10.1103/PhysRevLett.122.140504> Accelerated Variational Quantum Eigensolver. </a></b> <i>Physical review letters.</i> <code><b>[Variational Quantum Algorithms]</b></code> </summary><pre><code>@article{wang2019accelerated,
  title={Accelerated variational quantum eigensolver},
  author={Wang, Daochen and Higgott, Oscar and Brierley, Stephen},
  journal={Physical review letters},
  volume={122},
  number={14},
  pages={140504},
  year={2019},
  publisher={APS}
}</code></pre></details>
<details><summary>[Zoufal et al] <b><a href=https://www.nature.com/articles/s41534-019-0223-2> Quantum Generative Adversarial Networks for Learning and Loading Random
Distributions. </a></b> <i>npj Quantum Information.</i> <code><b>[Quantum Generative Adversarial Networks]</b></code> <code><b>[Variational Quantum Algorithms]</b></code> <code><b>[Quantum Machine Learning]</b></code> <code><i>(Gate Model)</i></code></summary><pre><code>@article{zoufal2019quantum,
  title={Quantum generative adversarial networks for learning and loading random distributions},
  author={Zoufal, Christa and Lucchi, Aur{\'e}lien and Woerner, Stefan},
  journal={npj Quantum Information},
  volume={5},
  number={1},
  pages={1--9},
  year={2019},
  publisher={Nature Publishing Group}
}</code></pre></details>


### 2018
<details><summary>[Dallaire-Demers & Killoran] <b><a href=https://journals.aps.org/pra/abstract/10.1103/PhysRevA.98.012324> Quantum generative adversarial networks. </a></b> <i>Physical Review A.</i> <code><b>[Quantum Generative Adversarial Networks]</b></code> <code><b>[Quantum Neural Networks]</b></code> <code><b>[Variational Quantum Algorithms]</b></code> <code><b>[Quantum Machine Learning]</b></code> <code><i>(Gate Model)</i></code></summary><pre><code>@article{dallaire2018quantum,
  title={Quantum generative adversarial networks},
  author={Dallaire-Demers, Pierre-Luc and Killoran, Nathan},
  journal={Physical Review A},
  volume={98},
  number={1},
  pages={012324},
  year={2018},
  publisher={APS}
}</code></pre></details>
<details><summary>[Moll et al] <b><a href=https://iopscience.iop.org/article/10.1088/2058-9565/aab822/pdf> Quantum optimization using variational algorithms on near-term quantum devices. </a></b> <i>Quantum Science and Technology.</i> <code><b>[Variational Quantum Algorithms]</b></code> <code><b>[Quantum Optimization]</b></code> <code><b>[Quantum Machine Learning]</b></code> <code><b>[Quantum Information and Computing]</b></code> <code><i>(Gate Model)</i></code></summary><pre><code>@Article{moll2018quantum,
  Title                    = {Quantum optimization using variational algorithms on near-term quantum devices},
  Author                   = {Moll, Nikolaj and Barkoutsos, Panagiotis and Bishop, Lev S and Chow, Jerry M and Cross, Andrew and Egger, Daniel J and Filipp, Stefan and Fuhrer, Andreas and Gambetta, Jay M and Ganzhorn, Marc and others},
  Journal                  = {Quantum Science and Technology},
  Year                     = {2018},
  Number                   = {3},
  Pages                    = {030503},
  Volume                   = {3},
  Publisher                = {IOP Publishing},
  URL                      = {https://iopscience.iop.org/article/10.1088/2058-9565/aab822/meta}
}
</code></pre></details>


### 2017
<details><summary>[Kandala et al ] <b><a href=https://www.nature.com/articles/nature23879?sf114016447=1> Hardware-efficient variational quantum eigensolver for small molecules and quantum magnets. </a></b> <i>Nature.</i> <code><b>[Variational Quantum Algorithms]</b></code> </summary><pre><code>@article{kandala2017hardware,
  title={Hardware-efficient variational quantum eigensolver for small molecules and quantum magnets},
  author={Kandala, Abhinav and Mezzacapo, Antonio and Temme, Kristan and Takita, Maika and Brink, Markus and Chow, Jerry M and Gambetta, Jay M},
  journal={Nature},
  volume={549},
  number={7671},
  pages={242--246},
  year={2017},
  publisher={Nature Publishing Group}
}</code></pre></details>


### 2014
<details><summary>[Farhi et al] <b><a href=https://arxiv.org/abs/1411.4028> A Quantum Approximate Optimization Algorithm. </a></b> <i>arXiv.</i> <code><b>[Variational Quantum Algorithms]</b></code> <code><b>[Parameterized Quantum Circuits]</b></code> <code><b>[Quantum Optimization]</b></code> <code><b>[Quantum Machine Learning]</b></code> <code><b>[]</b></code> <code><i>(Gate Model)</i></code></summary><pre><code>@Article{farhi2014quantum,
  Title                    = {A quantum approximate optimization algorithm},
  Author                   = {Farhi, Edward and Goldstone, Jeffrey and Gutmann, Sam},
  Journal                  = {arXiv preprint arXiv:1411.4028},
  Year                     = {2014},
  URL                      = {https://arxiv.org/abs/1411.4028}
}
</code></pre></details>
<details><summary>[Peruzzo et al] <b><a href=https://www.nature.com/articles/ncomms5213> A variational eigenvalue solver on a photonic quantum processor. </a></b> <i>Nature communications.</i> <code><b>[Variational Quantum Algorithms]</b></code> </summary><pre><code>@article{peruzzo2014variational,
  title={A variational eigenvalue solver on a photonic quantum processor},
  author={Peruzzo, Alberto and McClean, Jarrod and Shadbolt, Peter and Yung, Man-Hong and Zhou, Xiao-Qi and Love, Peter J and Aspuru-Guzik, Al{\'a}n and O’brien, Jeremy L},
  journal={Nature communications},
  volume={5},
  number={1},
  pages={1--7},
  year={2014},
  publisher={Nature Publishing Group}
}</code></pre></details>
